# Individual Project (MSc) - Techniques for Compressing Pruned Feedforward Neural Networks: A Comparative Study

**By Melany Satkunarajah** (S210010900), melany.satkunarajah@city.ac.uk

This file has the full code for the project.



## Importing Relevant Modules
Importing the relevant modules that are required for this notebook.

In [ ]:
#mount to google drive account
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
#import required modules
import os
import re
import copy
import time
import math
import torch
import json
import itertools

import numpy as np
import pandas as pd
from torch import nn
import seaborn as sns
from tqdm import tqdm


import torch.optim as optim
from torch.optim import Adam
from datetime import datetime
import matplotlib.pyplot as plt
import torch.nn.functional as F
import torch.nn.utils.prune as prune
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader, random_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import mean_absolute_error, mean_squared_error

from sklearn.decomposition import PCA
from sklearn.decomposition import FastICA
from sklearn.decomposition import SparsePCA
from sklearn.preprocessing import StandardScaler

## UC Irvine Machine Learning Repository Datasets

In this section, we are going to review/prepare the dataset for our dimensionality reduction task, and split it into training and testing sets.


The dataset was obtained from UC Irvine Machine Learning Repository:

*   Online News Popularity - https://archive.ics.uci.edu/dataset/332/online+news+popularity

### Review/Preprocessing of the Dataset
In this section, we are going to review and prepare the dataset for our analysis.

In [ ]:
#import csv file using pandas
#data_one = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/IP_MSC/UCI_code/datasets/OnlineNewsPopularity.csv', encoding='utf-8')
!wget "https://archive.ics.uci.edu/static/public/332/online+news+popularity.zip"
!unzip online+news+popularity.zip
!ls OnlineNewsPopularity
data_one = pd.read_csv('OnlineNewsPopularity/OnlineNewsPopularity.csv')

In [ ]:
# Displaying the first few rows of the DataFrame
data_one.head()

In [ ]:
# Displaying the text representation of the DataFrame shape
print("Data_One Shape: ({} rows, {} columns)".format(data_one.shape[0],
                                                      data_one.shape[1]))

In [ ]:
# Display concise information about the Dataset
data_one.info()

Removing the leading spaces from the feature names as we can see output above that there are additional leading spaces in front of the feature names.

In [ ]:
# Checking and remoivng leading spaces from feature names
data_one.columns = data_one.columns.str.strip()

Here is a full list of feature attributes with details of this dataset:
1.   **url** -  URL of the article (non-predictive)
2.   **timedelta** - Days between the article publication and the dataset acquisition (non-predictive)
3.   **n_tokens_title** -  Number of words in the title
4.   **n_tokens_content** - Number of words in the content
5. **n_unique_tokens** - Rate of unique words in the content
6. **n_non_stop_words** - Rate of non-stop words in the content
7. **n_non_stop_unique_tokens** - Rate of unique non-stop words in the content
8. **num_hrefs** - Number of links
9. **num_self_hrefs** - Number of links to other articles published by Mashable
10. **num_imgs** - Number of images
11. **num_videos** - Number of videos
12. **average_token_length** - Average length of the words in the content
13. **num_keywords** - Number of keywords in the metadata
14. **data_channel_is_lifestyle** - Is data channel 'Lifestyle'?
15. **data_channel_is_entertainment** - Is data channel 'Entertainment'?
16. **data_channel_is_bus** - Is data channel 'Business'?
17. **data_channel_is_socmed** - Is data channel 'Social Media'?
18. **data_channel_is_tech** - Is data channel 'Tech'?
19. **data_channel_is_world** - Is data channel 'World'?
20. **kw_min_min** - Worst keyword (min. shares)
21. **kw_max_min** - Worst keyword (max. shares)
22. **kw_avg_min** - Worst keyword (avg. shares)
23. **kw_min_max** - Best keyword (min. shares)
24. **kw_max_max** - Best keyword (max. shares)
25. **kw_avg_max** - Best keyword (avg. shares)
26. **kw_min_avg** - Avg. keyword (min. shares)
27. **kw_max_avg** - Avg. keyword (max. shares)
28. **kw_avg_avg** - Avg. keyword (avg. shares)
29. **self_reference_min_shares** - Min. shares of referenced articles in Mashable
30. **self_reference_max_shares** - Max. shares of referenced articles in Mashable
31. **self_reference_avg_sharess** - Avg. shares of referenced articles in Mashable
32. **weekday_is_monday** - Was the article published on a Monday?
33. **weekday_is_tuesday** - Was the article published on a Tuesday?
34. **weekday_is_wednesday** - Was the article published on a Wednesday?
35. **weekday_is_thursday** - Was the article published on a Thursday?
36. **weekday_is_friday** - Was the article published on a Friday?
37. **weekday_is_saturday** - Was the article published on a Saturday?
38. **weekday_is_sunday** - Was the article published on a Sunday?
39. **is_weekend** - Was the article published on the weekend?
40. **LDA_00** - Closeness to LDA topic 0
41. **LDA_01** - Closeness to LDA topic 1
42. **LDA_02** - Closeness to LDA topic 2
43. **LDA_03** - Closeness to LDA topic 3
44. **LDA_04** - Closeness to LDA topic 4
45. **global_subjectivity** - Text subjectivity
46. **global_sentiment_polarity** - Text sentiment polarity
47. **global_rate_positive_words** - Rate of positive words in the content
48. **global_rate_negative_words** - Rate of negative words in the content
49. **rate_positive_words** - Rate of positive words among non-neutral tokens
50. **rate_negative_words** - Rate of negative words among non-neutral tokens
51. **avg_positive_polarity** - Avg. polarity of positive words
52. **min_positive_polarity** - Min. polarity of positive words
53. **max_positive_polarity** - Max. polarity of positive words
54. **avg_negative_polarity** - Avg. polarity of negative  words
55. **min_negative_polarity** - Min. polarity of negative  words
56. **max_negative_polarity** - Max. polarity of negative  words
57. **title_subjectivity** - Title subjectivity
58. **title_sentiment_polarity** - Title polarity
59. **abs_title_subjectivity** - Absolute subjectivity level
60. **abs_title_sentiment_polarity** - Absolute polarity level
61. **shares** - Number of shares (target)

Reference for obtaining feature attribute description:
https://archive.ics.uci.edu/dataset/332/online+news+popularity


In [ ]:
# Checking for missing values
missing_values = data_one.isnull().sum()

# Displaying the count of missing values for each column
print("Missing Values:\n", missing_values)

# Checking for duplicate rows
duplicate_rows = data_one.duplicated().sum()

# Displaying the count of duplicate rows
print("\nDuplicate Rows:", duplicate_rows)



*   No duplicate or missing values in this dataset

After reviewing the columns, it is ideal to drop the non-predictive attricbutes, which is in our case:
*   url
*   timedelta

The non-predictive attributes are listed here: https://archive.ics.uci.edu/dataset/332/online+news+popularity





In [ ]:
# Remove specified columns from the DataFrame
columns_to_drop = ['url', 'timedelta']
data_one.drop(columns=columns_to_drop, inplace=True)
data_one.head(n=4)

These are the binary columns in this dataset:
*   data_channel_is_lifestyle
*   data_channel_is_entertainment
*   data_channel_is_bus
*   data_channel_is_socmed
*   data_channel_is_tech
*   data_channel_is_world
*   weekday_is_monday
*   weekday_is_tuesday
*   weekday_is_wednesday
*   weekday_is_thursday
*   weekday_is_friday
*   weekday_is_saturday
*   weekday_is_sunday
*   is_weekend

We have changed the type from float to boolean, so it gives us a true or false output instead.

In [ ]:
# Identify binary columns
binary_columns = [col for col in data_one.columns if data_one[col].nunique() == 2]

# Display the binary columns
print("Binary Columns:")
print(binary_columns)

In [ ]:
# Convert binary columns to boolean type
data_one[binary_columns] = data_one[binary_columns].astype(bool)

In [ ]:
# check if the changes have been applied in a random column
data_one['weekday_is_friday']

In [ ]:
# Display the first 4 rows of the modified DataFrame
data_one.head(n=4)

Reviewing the table above, you can see that the following features have discrete values:
*   n_tokens_title
*   n_tokens_content
*   num_hrefs
*   num_self_hrefs
*   num_imgs
*   num_videos
*   num_keywords

Instead of saving them as floats, we will convert them to an integer.

In [ ]:
# list of feature attributes that have discrete values
integer_features = ['n_tokens_title', 'n_tokens_content', 'num_hrefs',
                    'num_self_hrefs', 'num_imgs', 'num_videos', 'num_keywords']

# changing these features into integer values
data_one[integer_features] = data_one[integer_features].astype('int64')

In [ ]:
# checking the columns after amendents made
print(data_one.dtypes)

After we have made the changes for the feature types, and removed the two non-predictive columns. Let's look at the descriptive statistical measure for this dataset.

In [ ]:
# Displaying descriptive statistics of the DataFrame
data_one.describe()

In [ ]:
# Current DataFrame shape
print("Data_One Shape: ({} rows, {} columns)".format(data_one.shape[0],
                                                      data_one.shape[1]))

**'n_tokens_content'** represents the number of words in the news articles and some some articles have inputs with a minimum value of 0 indicate articles with no content which add no meaning for sharing activity of the article. So we are going to remove them.

In [ ]:
# Identify rows where 'n_tokens_content' is 0
no_words_indices = data_one[data_one['n_tokens_content'] == 0].index

# Print the number of news items with no words
print('Number of News Articles with no words:', no_words_indices.size)

# Drop rows where 'n_tokens_content' is 0
data_one = data_one[data_one['n_tokens_content'] != 0]

In [ ]:
# Current DataFrame shape
print("Data_One Shape: ({} rows, {} columns)".format(data_one.shape[0],
                                                      data_one.shape[1]))

In [ ]:
# Calculate the correlation matrix
correlation_matrix = data_one.corr()

# Create a heatmap using seaborn to visualize the correlation matrix
sns.heatmap(correlation_matrix, cmap="coolwarm")

# Display the heatmap
plt.show()

In [ ]:
# Displaying histograms for each numerical column in the DataFrame
data_one.hist(figsize=(20, 20))
plt.show()

**Target Value Distribution**


The target variable of this dataset is the 'shares' column, which is the number of shares.

In [ ]:
plt.clf()

# Set the random seed for reproducibility
np.random.seed(1)

# Set the plotting style
plt.style.use('default')

# Create a subplot within the figure
plt.subplot()

# Plot a histogram for the 'shares' column
data_one['shares'].hist(bins=10, range=(1, 8000))

# Set the title and axis labels for the plot
plt.title('Histogram of Shares of Online Articles')
plt.xlabel('Shares')
plt.ylabel('Count')

plt.show()

In [ ]:
# output target variable distribution
data_one['shares'].describe()

To determine if an article is popular, we look at how many times it has been shared online. Using the median number of shares in the dataset as a benchmark.

If an article has been shared 1400 or more times, it is considered popular.

If an article has been shared less than 1400 times, it is considered less popular.

We will create a new column, popularity, to determine the popularity of the article based on the number of shares, and then we can do that throughout the week.

In [ ]:
# checking the numbers of words in the title and content of the article
# Create a figure with two subplots
fig, axs = plt.subplots(2)

# Plot a boxplot for 'n_tokens_title' in the first subplot
sns.boxplot(data=data_one, x='n_tokens_title', ax=axs[0])

# Plot a boxplot for 'n_tokens_content' in the second subplot
sns.boxplot(data=data_one, x='n_tokens_content', ax=axs[1])

# title
fig.suptitle('Number of Words in Title against Content')

#### Identifying and Removing Outliers

In [ ]:
# Get numerical columns within the dataset
num_cols = data_one.select_dtypes(['int64', 'float64']).columns

# Calculate the number of rows and columns for the grid
num_plots = len(num_cols)
num_rows = math.ceil(num_plots / 3)
num_cols_subplot = min(num_plots, 3)

# Set up the grid for boxplots
fig, axs = plt.subplots(num_rows, num_cols_subplot, figsize=(20, num_rows * 4))

# Flattening the grid
axs = axs.flatten()

# Plot boxplots in the grid
for i, column in enumerate(num_cols):
    sns.boxplot(x=data_one[column], ax=axs[i])
    axs[i].set_title(f'Boxplot of {column}')

# Hide empty subplots
for i in range(num_plots, len(axs)):
    axs[i].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# Identify and print outliers for each numerical column
for column in num_cols:
    q1 = data_one[column].quantile(0.25)  # First Quartile
    q3 = data_one[column].quantile(0.75)  # Third Quartile
    IQR = q3 - q1  # Interquartile Range

    llimit = q1 - 1.5 * IQR  # Lower Limit
    ulimit = q3 + 1.5 * IQR  # Upper Limit

    # Identify outliers
    outliers = data_one[(data_one[column] < llimit) | (data_one[column] > ulimit)]

    # Print information about outliers
    print(f'Column: {column}')
    print(f'Number of outliers in "{column}": {len(outliers)}')
    print(f'Lower Limit: {llimit}')
    print(f'Upper Limit: {ulimit}')
    print(f'Interquartile Range (IQR): {IQR}')
    print('-' * 30)  # Separator

In [ ]:
# Createing a copy of the original dataset
data_one_no_outliers = data_one.copy()

# Identify and remove outliers for each numerical column
for column in num_cols:
    q1 = data_one[column].quantile(0.25)  # First Quartile
    q3 = data_one[column].quantile(0.75)  # Third Quartile
    IQR = q3 - q1  # Interquartile Range

    llimit = q1 - 1.5 * IQR  # Lower Limit
    ulimit = q3 + 1.5 * IQR  # Upper Limit

    # Remove outliers and update the copy
    data_one_no_outliers = data_one_no_outliers[
        (data_one_no_outliers[column] >= llimit) & (data_one_no_outliers[column] <= ulimit)
    ]

# Display the shape of the original and outlier-removed datasets
print("Original Dataset Shape:", data_one.shape)
print("Outlier-Removed Dataset Shape:", data_one_no_outliers.shape)

In [ ]:
# Current DataFrame shape after removing outliers
print("Data_One Shape after Removing Outliers: ({} rows, {} columns)".format(data_one_no_outliers.shape[0],
                                                                             data_one_no_outliers.shape[1]))

In [ ]:
data_one_no_outliers.describe()

### Splitting into Training and Testing Set

**Dataset One with Outliers**

This is the original dataset including the outliers.

In [ ]:
# features are stored in X and the target ('shares') is stored in y
X0 = data_one.drop('shares', axis=1)  # Features
y0 = data_one['shares']  # Target variable

# Split the dataset into training and testing sets
X0_train, X0_test, y0_train, y0_test = train_test_split(X0, y0, test_size=0.2,
                                                        random_state=42)

# Print the shapes of the resulting sets
print("X0_train shape:", X0_train.shape)
print("X0_test shape:", X0_test.shape)
print("y0_train shape:", y0_train.shape)
print("y0_test shape:", y0_test.shape)

**Dataset_One without Outliers**

This is the original dataset excluding the outliers.

In [ ]:
# features are stored in X and the target ('shares') is stored in y
X = data_one_no_outliers.drop('shares', axis=1)  # Features
y = data_one_no_outliers['shares']  # Target variable

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2,
                                                    random_state=42)

# Print the shapes of the resulting sets
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

## Feedforward Neural Network for Regression

Below we have defined the feedforward neural network

In [ ]:
class RegressionNN(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size):
        super(RegressionNN, self).__init__()

        # Define fully connected layers with ReLU activations
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.fc3 = nn.Linear(hidden_size2, output_size)

        # Initialize the weights using Xavier initialization
        self.init_weights()

    def init_weights(self):
        # Xavier initialization for each layer
        for layer in [self.fc1, self.fc2, self.fc3]:
            nn.init.xavier_normal_(layer.weight)

    def forward(self, x):
        # Forward pass through the network
        x = F.relu(self.fc1(x))
        #x = self.dropout(x)
        x = F.relu(self.fc2(x))
        #x = self.dropout(x)
        x = self.fc3(x)
        return x

# reference used: https://www.geeksforgeeks.org/select-the-right-weight-for-deep-neural-network-in-pytorch/
# https://pytorch.org/docs/stable/nn.init.html
# https://machinelearningmastery.com/choose-an-activation-function-for-deep-learning/#:~:text=If%20your%20problem%20is%20a,%3A%20One%20node%2C%20linear%20activation.
# https://medium.com/@gidim/part-2-selecting-the-right-weight-initialization-for-your-deep-neural-network-cc27cf2d5e56

**Moving the model to the specified device (GPU or CPU)**


In [ ]:
torch.cuda.is_available()
# ref: https://stackoverflow.com/questions/50560395/how-to-install-cuda-in-google-colab-gpus
# Output would be True if Pytorch is using GPU otherwise it would be False.

# defining training device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

### Scaling the features

Before we train the RegressionNN network that we have defined above, we need to double check the types of our training and testing sets. So we can convert and scale them appropriately for our regression task.

In [ ]:
# Checking the type of training and testing set
print("Types for Dataset with outliers")
print(type(y0_test))
print(type(y0_train))
print(type(X0_train))
print(type(X0_test))
print()
print("Types for Dataset without outliers")
print(type(y_test))
print(type(y_train))
print(type(X_train))
print(type(X_test))

In [ ]:
def preprocess_data(X_train, X_test, y_train, y_test, batch_size=64, device='cpu'):
    # Step 1: Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Step 2: Converting to PyTorch tensors
    X_train_tensor = torch.FloatTensor(X_train_scaled).to(device)
    X_test_tensor = torch.FloatTensor(X_test_scaled).to(device)
    y_train_tensor = torch.FloatTensor(y_train.values).view(-1, 1).to(device)
    y_test_tensor = torch.FloatTensor(y_test.values).view(-1, 1).to(device)

    # Step 3: Createing DataLoader for training data
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    # Step 4: Creating DataLoader for testing data
    test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, test_loader

In [ ]:
# Using the preprocess_data function on the dataset with outliers
train_loader0, test_loader0 = preprocess_data(X0_train, X0_test,y0_train, y0_test,
                                              batch_size=64,
                                              device='cuda' if torch.cuda.is_available() else 'cpu')

# Print the shapes of tensors
for batch in train_loader0:
    X0_batch, y0_batch = batch
    print("X0_batch shape:", X0_batch.shape)
    print("y0_batch shape:", y0_batch.shape)
    break

In [ ]:
# Using the preprocess_data function on the dataset without outliers
train_loader, test_loader = preprocess_data(X_train, X_test, y_train, y_test,
                                            batch_size=64,
                                            device='cuda' if torch.cuda.is_available() else 'cpu')

# Print the shapes of tensors
for batch in train_loader:
    X_batch, y_batch = batch
    print("X_batch shape:", X_batch.shape)
    print("y_batch shape:", y_batch.shape)
    break

In [ ]:
# PCA on the dataset without outliers
pca = PCA()
pca.fit(X_train)
cumulative_variance = np.cumsum(pca.explained_variance_ratio_)
num_components = np.argmax(cumulative_variance >= 0.95) + 1  # 95% threshold

plt.plot(range(1, len(pca.explained_variance_ratio_) + 1),
         cumulative_variance, marker='o')
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance')
plt.show()

## Training and Testing of the Feedford Neural Network

Before we train the RegressionNN network that we have defined above, we need to double check the types of our training and testing sets. So we can convert them appropriately for our regression task.

In [ ]:
# reference: https://discuss.pytorch.org/t/output-evaluation-loss-after-every-n-batches-instead-of-epochs-with-pytorch/116619
def OG_train_and_validate(train_loader, test_loader, input_size, hidden_size1, hidden_size2,
                       output_size, learning_rate, epochs):
    # Creating an object of the RegressionNN class
    regression_net = RegressionNN(input_size, hidden_size1,
                                  hidden_size2, output_size)

    # Move the model to CUDA if available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    regression_net = regression_net.to(device)

    # Defining Mean Squared Error loss function
    criterion = nn.MSELoss()

    # Defining the Adam optimizer
    optimizer = torch.optim.Adam(regression_net.parameters(), lr=learning_rate)

    # List to store training and validation losses
    training_losses = []
    validation_losses = []

    # Training loop
    for epoch in range(epochs):
        # Set the model to training mode
        regression_net.train()

        # Initializing total loss and total MAE
        total_loss = 0.0
        total_mae = 0.0

        # List to store all outputs during training
        all_outputs = []

        # Iterating over training data in batches
        for batch in train_loader:
            inputs = batch[0].to(device)
            targets = batch[1].squeeze().to(device)

            # Forward pass
            outputs = regression_net(inputs)

            # Calculating the MSE loss
            mse_loss = criterion(outputs.view(-1), targets.view(-1))

            # Calculating the MAE loss
            mae_loss = nn.L1Loss()(outputs.view(-1), targets.view(-1))

            # Backward and optimize
            optimizer.zero_grad()
            mse_loss.backward()

            # Store outputs for later printing
            all_outputs.extend(outputs.view(-1).tolist())

            # Gradient Clipping
            # reference used: https://stackoverflow.com/questions/54716377/how-to-do-gradient-clipping-in-pytorch
            torch.nn.utils.clip_grad_norm_(regression_net.parameters(),
                                           max_norm=1.0)

            optimizer.step()

            # Update total loss
            total_loss += mse_loss.item()

            # Update total MAE
            total_mae += mae_loss.item()

        # Calculating average loss and MAE for the epoch
        average_loss = total_loss / len(train_loader)
        average_mae = total_mae / len(train_loader)

        # Storing training loss
        training_losses.append(average_loss)

        # Printing the metrics for each epoch
        print(f'Epoch [{epoch + 1}/{epochs}], MSE Loss (Train): {average_loss:.3f}, MAE Loss (Train): {average_mae:.3f}')

        # Validation
        regression_net.eval()
        with torch.no_grad():
            validation_loss, validation_mae = 0.0, 0.0

            # Iterate over validation data in batches
            for batch in test_loader:
                val_inputs = batch[0].to(device)
                val_targets = batch[1].squeeze().to(device)

                val_outputs = regression_net(val_inputs)
                validation_loss += criterion(val_outputs.view(-1),
                                             val_targets.view(-1)).item()
                validation_mae += nn.L1Loss()(val_outputs.view(-1),
                                              val_targets.view(-1)).item()

            # Calculate average validation loss and MAE
            average_validation_loss = validation_loss / len(test_loader)
            average_validation_mae = validation_mae / len(test_loader)

            # Store validation loss for later plotting
            validation_losses.append(average_validation_loss)

            # Print the validation metrics
            print(f'Validation Loss: {average_validation_loss:.3f}, Validation MAE: {average_validation_mae:.3f}')

    # Plotting
    epochs_range = range(1, epochs + 1)
    plt.plot(epochs_range, training_losses, label='Training Loss')
    plt.plot(epochs_range, validation_losses, label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

    return regression_net, training_losses, validation_losses

References used for the training and testing loop above:

https://discuss.pytorch.org/t/output-evaluation-loss-after-every-n-batches-instead-of-epochs-with-pytorch/116619

In [ ]:
# function to do grid search
# rerence: https://towardsdatascience.com/how-to-write-your-grid-search-function-in-python-43ad0da97522
def grid_search(train_loader, test_loader, input_size, output_size,
                learning_rates, batch_sizes, hidden_sizes1, hidden_sizes2, epochs):

    # Initialize variables to store the best model, its loss, and information
    best_model = None
    best_loss = float('inf')
    best_model_info = {}

    # Iterate over hyperparameter combinations
    for lr in learning_rates:
        for batch_size in batch_sizes:
            for hidden_size1 in hidden_sizes1:
                for hidden_size2 in hidden_sizes2:
                    print(f"\nTraining for lr={lr}, batch_size={batch_size}, hidden_size1={hidden_size1}, hidden_size2={hidden_size2}")

                    # Create and train the model
                    model, training_losses, validation_losses = OG_train_and_validate(
                        train_loader, test_loader, input_size,
                        hidden_size1, hidden_size2, output_size,
                        learning_rate=lr, epochs=epochs
                    )

                    # Checking if the current model has the lowest validation loss
                    if validation_losses[-1] < best_loss:
                        best_loss = validation_losses[-1]
                        best_model = model

                        # Save the best model's state_dict to a file
                        torch.save(best_model.state_dict(), 'best_model.pth')

                        # Save the hyperparameters of the best model
                        best_model_info = {
                            'input_size': input_size,
                            'hidden_size1': hidden_size1,
                            'hidden_size2': hidden_size2,
                            'output_size': output_size,
                            'learning_rate': lr,
                            'epochs': epochs,
                            'batch_size': batch_size
                        }

    # Displaying information about the best model
    print("\nBest Model Information:")
    print(f"Input Size: {best_model_info['input_size']}")
    print(f"Hidden Size 1: {best_model_info['hidden_size1']}")
    print(f"Hidden Size 2: {best_model_info['hidden_size2']}")
    print(f"Output Size: {best_model_info['output_size']}")
    print(f"Learning Rate: {best_model_info['learning_rate']}")
    print(f"Epochs: {best_model_info['epochs']}")
    print(f"Batch Size: {best_model_info['batch_size']}")
    print(f"Best Validation Loss: {best_loss:.3f}")

    # Return the best model and its information as a directory
    return  {'best_model': best_model, 'best_model_info': best_model_info}

#### Dataset One with Outliers

In [ ]:
# Set your input sizes
input_size = X0_train.shape[1]  # Number of input features
hidden_size1 = 128
hidden_size2 = 64
output_size = 1  # For regression
epochs = 200
batch_size = 64
learning_rate = 0.001

# Call train_and_validate function for the dataset with outliers
trained_model0, training_losses0, validation_losses0 = OG_train_and_validate(
    train_loader0, test_loader0, input_size, hidden_size1, hidden_size2,
    output_size,learning_rate=learning_rate, epochs=epochs
    )

#### Dataset One without Outliers

In [ ]:
# Set your input sizes
input_size = X_train.shape[1]  # Number of input features
hidden_size1 = 128
hidden_size2 = 64
output_size = 1  # For regression
epochs = 200
batch_size = 64
learning_rate = 0.001

# Call train_and_validate function for the dataset wihtout outliers
trained_model, training_losses, validation_losses = OG_train_and_validate(
    train_loader, test_loader, input_size, hidden_size1, hidden_size2,
    output_size,learning_rate=learning_rate, epochs=epochs
    )

##### Grid Search to find the best Model

From this point onwards, we will use the dataset without outliers, as shown in the training above. This dataset had better training and validation losses than the dataset with outliers.

In [ ]:
# Hyperparameters for grid search
learning_rates = [0.001, 0.01, 0.1]
batch_sizes = [64, 128]
hidden_sizes1 = [64, 128, 256]
hidden_sizes2 = [32, 64, 128]
epochs = 200

# Perform grid search using the dataset without outliers
best_model, best_model_info = grid_search(train_loader, test_loader,
                                          input_size=X_train.shape[1],
                                          output_size=1,
                                          learning_rates=learning_rates,
                                          batch_sizes=batch_sizes,
                                          hidden_sizes1=hidden_sizes1,
                                          hidden_sizes2=hidden_sizes2,
                                          epochs=epochs)

#### Saving the Trained Model

In [ ]:
# Save the best model's state_dict to Google Drive
model_save_path_drive = '/content/gdrive/My Drive/Colab Notebooks/IP_MSC/UCI_code/best_model.pth'
best_model_path = '/content/best_model.pth'
!cp "$best_model_path" "$model_save_path_drive"

# Save the best model info to Google Drive
model_info_save_path_drive = '/content/gdrive/My Drive/Colab Notebooks/IP_MSC/UCI_code/best_model_info.json'
with open(model_info_save_path_drive, 'w') as json_file:
    json.dump(best_model_info, json_file)

In [ ]:
# hyperparameters for best model
input_size = X_train.shape[1]  # Number of input features
hidden_size1 = 64
hidden_size2 = 32
output_size = 1  # For regression
epochs = 200
batch_size = 64
learning_rate = 0.001

In [ ]:
# Call train_and_validate function to show the training of the best model
# This is not being used for anything else but just for visualisation
trained_model_b, training_losses_b, validation_losses_b = OG_train_and_validate(
    train_loader, test_loader, input_size, hidden_size1, hidden_size2,
    output_size,learning_rate=learning_rate, epochs=epochs
    )

## **Load the Trained Model**

In this section, we are going to load the trained model and review quickly the number of parameters and the structure of it.

In [ ]:
# hyperparameters for best model
input_size = 58  # Number of input features
hidden_size1 = 64
hidden_size2 = 32
output_size = 1  # For regression
epochs = 200
batch_size = 64
learning_rate = 0.001

In [ ]:
#Load the saved model from google drive
model = RegressionNN(input_size,hidden_size1, hidden_size2, output_size)
model_path = '/content/gdrive/My Drive/Colab Notebooks/IP_MSC/UCI_code/best_model.pth'
model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
model = model.to(device)

In [ ]:
# Apply training mode before applying pruning on the loaded model
#model.train()

In [ ]:
# Put the model in evaluation mode
#model.eval()

In [ ]:
#function to obtain the model size
# reference: https://discuss.pytorch.org/t/how-do-i-check-the-number-of-parameters-of-a-model/4325
# https://wandb.ai/wandb_fc/tips/reports/How-To-Calculate-Number-of-Model-Parameters-for-PyTorch-and-TensorFlow-Models--VmlldzoyMDYyNzIx
model_size = sum(p.numel() for p in model.parameters())
print(f"Loaded model size: {model_size} parameters")

The funcation, `check_model_stats`, below takes in a neural network model as input. It checks the sparsity of specific layers within the model, calculating the ratio of zero-valued weights to the total number of weights in each parameter tensor. The function then prints the layer name, parameter name, sparsity value, shape, and original weights of the parameter tensor. This information helps assess the impact of pruning techniques on the model's parameters, particularly regarding the sparsity introduced after pruning.

In [ ]:
# defining a function to check model stats for the orignal and pruned model
def check_model_stats(model):
    # Define global variables for parameters to check
    parameters_to_check = [
        (model.fc1, 'weight'),
        (model.fc2, 'weight'),
        (model.fc3, 'weight'),
    ]

    # Print sparsity and other information
    print("Stats for each parameter:")
    for layer, parameter_name in parameters_to_check:
        parameter_tensor = getattr(layer, parameter_name)
        sparsity_value = torch.sum(parameter_tensor == 0).item() / parameter_tensor.numel()
        print(f"{layer.__class__.__name__}.{parameter_name}:")
        print(f"Sparsity: {sparsity_value:.4f}")
        print(f"Shape: {parameter_tensor.shape}")
        print(f"Original weights:")
        print(parameter_tensor)

In [ ]:
check_model_stats(model)

The function `get_zero_nonzero_values` analyzes the distribution of zero and non-zero values in the weight parameters of a neural network model. It extracts these values and updates counters, then prints the shape of each weight parameter and additional statistics such as minimum, maximum, mean, and standard deviation if there are non-zero values. Finally, the function prints the counts of zero and non-zero values and returns the lists containing these values.

In [ ]:
def get_zero_nonzero_values(model, epsilon=1e-8):
    zero_values = []
    nonzero_values = []

    zero_count = 0
    nonzero_count = 0

    for name, param in model.named_parameters():
        if 'weight' in name:  # Assuming you are interested in pruning weights
            # Convert the parameter tensor to a flat 1D tensor
            flat_param = param.flatten()

            # Use boolean masks to identify zero and non-zero values
            zero_mask = (torch.abs(flat_param) <= epsilon)
            nonzero_mask = ~zero_mask

            # Extract zero and non-zero values using boolean masks
            zero_values.extend(flat_param[zero_mask].tolist())
            nonzero_values.extend(flat_param[nonzero_mask].tolist())

            # Update counts
            zero_count += torch.sum(zero_mask).item()
            nonzero_count += torch.sum(nonzero_mask).item()

            # Print shape of the parameter tensor
            print(f"Parameter '{name}' Shape: {param.shape}")

            # Print additional information about non-zero values
            if torch.sum(nonzero_mask).item() > 0:
                print(f"Non-Zero Values ({name}):")
                print(f"  Min: {torch.min(flat_param[nonzero_mask]).item()}")
                print(f"  Max: {torch.max(flat_param[nonzero_mask]).item()}")
                print(f"  Mean: {torch.mean(flat_param[nonzero_mask]).item()}")
                print(f"  Std: {torch.std(flat_param[nonzero_mask]).item()}\n")

    print("\nCount of Zero Values:", zero_count)
    print("Count of Non-Zero Values:", nonzero_count)

    return zero_values, nonzero_values

In [ ]:
zero_values, nonzero_values = get_zero_nonzero_values(model)

## Pruning of the Trained Feedforward Neural Network

In this section, we are going to apply pruning on the already trained model and see how it affects the model when pruning is applied at different thresholds.

References used:

https://towardsdatascience.com/how-to-prune-neural-networks-with-pytorch-ebef60316b91

### Global Sparsity Function

The sparsity function calculates a PyTorch model's global sparsity by dividing the number of zero-valued parameters by the total number of parameters.

Here's a step-by-step explanation of the code:
1. **a, b = 0., 0.** - Set variables 'a' and 'b' to zero. Use 'a' to count total parameters and 'b' to count zero-valued parameters.
2. **for p in model.parameters()** - Iterating over the parameters of the provided model, which are the learnable weights and biases of its layers.
3. **a += p.numel()** - To add the number of elements in the parameter tensor 'p' to the variable 'a', use the numel() function.
4. **b += (p == 0).sum() ** - To count the number of zeros in a tensor called "p", use "p == 0" to create a binary tensor and then use the "sum()" function to count the True values. Add the result to the variable "b".
5. **return b / a** - Calculate the ratio of zero-valued parameters to the total number of parameters to determine the global sparsity of the model.

Code obtained for the global sparity function:
https://github.com/WongKinYiu/yolov7/blob/main/utils/torch_utils.py

Martin, M. Local and global processing: The role of sparsity. Memory & Cognition 7, 476–484 (1979). https://doi.org/10.3758/BF03198264

In [ ]:
# Code obtained from
# https://github.com/WongKinYiu/yolov7/blob/main/utils/torch_utils.py
def sparsity(model):
    # Return global model sparsity
    a, b = 0., 0.
    for p in model.parameters():
        a += p.numel()
        b += (p == 0).sum()
    return b / a

### Local Pruning of the Trained Model

This code below defines a local pruning function for a PyTorch neural network model. The function reduces the sparsity of the model by removing smaller connections.

It calculates the threshold for pruning and generates a binary mask to indicate which weights should be pruned. The actual pruning is performed using PyTorch's `prune.custom_from_mask` method, which applies the generated mask to the weights of the linear layer.  This mask is applied to the weights of the linear layer and the pruned model is saved.

Finally, the function prints the global sparsity of the pruned model and its new size.

Reference:
https://pytorch.org/docs/stable/generated/torch.nn.utils.prune.custom_from_mask.html

In [ ]:
# defining the local pruning function
def apply_local_prune(model,input_size, hidden_size1, hidden_size2,
                output_size, pruning_amount):
    # Create a new instance of the model
    pruned_model = RegressionNN(input_size, hidden_size1,
                                hidden_size2, output_size)

    # Load the parameters from the original model
    relevant_state_dict = {key: value for key, value in model.state_dict().items() if key in pruned_model.state_dict()}
    pruned_model.load_state_dict(model.state_dict())

    for name, module in pruned_model.named_children():
        if isinstance(module, nn.Linear):
            # Calculate the threshold for pruning based on the pruning amount
             # reference: https://stackoverflow.com/questions/61269052/remove-smallest-p-numbers-from-multiple-numpy-arrays
            threshold = np.percentile(np.abs(module.weight.cpu().detach().numpy()),
                                      100 * pruning_amount)

            # Generate a custom mask based on the threshold
            # Referemce: https://stackoverflow.com/questions/61629395/how-to-prune-weights-less-than-a-threshold-in-pytorch
            custom_mask = torch.abs(module.weight) > threshold
            custom_mask = custom_mask.to(torch.float32)

            # Apply pruning using the custom mask
            prune.custom_from_mask(module, name='weight', mask=custom_mask)

            # Calculate sparsity for this layer
            sparsity = torch.sum(custom_mask == 0).item() / custom_mask.numel()
            print(f"Sparsity in {name}: {sparsity:.2%}")

    # Save the pruned model with the pruning amount in the name
    pruning_amount_int = int(pruning_amount * 100)
    pruned_model_name = f'pruned_local_{pruning_amount_int}.pth'
    torch.save(pruned_model.state_dict(), pruned_model_name)

    return pruned_model


The function `compare_weights` compares weight matrices between an original model and a pruned model in the neural network. It iterates through the layers, ensuring that each linear layer's weight matrices match and prints a message if they don't.

In [ ]:
# comparing the weights of the trained and pruned model
def compare_weights(original_model, pruned_model):
    print("Comparing weights between original and pruned models:")

    # Iterate through the layers of the models
    for original_layer, pruned_layer in zip(original_model.children(), pruned_model.children()):
        if isinstance(original_layer, nn.Linear):
            original_weights = original_layer.weight.cpu().detach().numpy()
            pruned_weights = pruned_layer.weight.cpu().detach().numpy()

            # Check the shape of the weight matrices match
            assert original_weights.shape == pruned_weights.shape, "Shape mismatch in weight matrices"

            # Check the pruned weights are different from the original weights
            if not np.array_equal(original_weights, pruned_weights):
                layer_name = original_layer.__class__.__name__
                print(f"Weights in {layer_name} layer are different.")

**Pruning Amount 0.6**

In [ ]:
# Apply local pruning for pruning amount 0.5
pruning_amount = 0.6
pruned_local1 = apply_local_prune(model, input_size, hidden_size1,
                                  hidden_size2, output_size, pruning_amount)



# Compare the weights of the loaded and pruned model
compare_weights(model, pruned_local1)

**Pruning Amount 0.7**

In [ ]:
# Apply local pruning for pruning amount 0.7
pruning_amount = 0.7
pruned_local2 = apply_local_prune(model, input_size, hidden_size1,
                                  hidden_size2, output_size, pruning_amount)

# Compare the weights of the loaded and pruned model
compare_weights(model, pruned_local2)

**Pruning Amount 0.8**

In [ ]:
# Apply local pruning for pruning amount 0.8
pruning_amount = 0.8
pruned_local3 = apply_local_prune(model, input_size, hidden_size1,
                                  hidden_size2, output_size, pruning_amount)

# Compare the weights of the loaded and pruned model
compare_weights(model, pruned_local3)

**Pruning Amount 0.9**

In [ ]:
# Apply local pruning for pruning amount 0.9
pruning_amount = 0.9
pruned_local4 = apply_local_prune(model, input_size, hidden_size1,
                                  hidden_size2, output_size, pruning_amount)

# Compare the weights of the loaded and pruned model
compare_weights(model, pruned_local4)

In [ ]:
# Apply local pruning for pruning amount 0.8
pruning_amount = 0.95
pruned_local5 = apply_local_prune(model, input_size, hidden_size1,
                                  hidden_size2, output_size, pruning_amount)

# Compare the weights of the loaded and pruned model
compare_weights(model, pruned_local5)

### Layer-Specific Local Pruning

The code below defines a local pruning function that selectively prunes weights in fully-connected (linear) layers based on specified layer-wise sparsity values in the loaded model. The function prints a message indicating the start of the pruning process and iterates through each child module of the model, checking if the module is a linear layer. If a non-zero sparsity value is specified for the layer, the function proceeds with pruning. The function applies pruning to the linear layer using the prune.custom_from_mask method and prints the global sparsity of the pruned model and its new size.

Reference used:

https://towardsdatascience.com/how-to-prune-neural-networks-with-pytorch-ebef60316b91

In [ ]:
# function that removes characters that are not allowed in filenames
# reference: https://docs.python.org/3/library/re.html
def sanitize_for_filename(s):
    return re.sub(r'[^\w\-_.]', '_', s)

In [ ]:
# function for pruning per layer
# reference: https://leimao.github.io/blog/PyTorch-Pruning/
# defining the local pruning function with layer sparsity
def local_prune_layer_sparsity(model, layer_sparsity_dict):
    # Create a new instance of the model
    pruned_model = RegressionNN(input_size, hidden_size1, hidden_size2, output_size)

    # Load the parameters from the original model to the new model, filtering out irrelevant keys
    relevant_state_dict = {key: value for key, value in model.state_dict().items() if key in pruned_model.state_dict()}
    pruned_model.load_state_dict(model.state_dict())

    for name, module in pruned_model.named_children():
        if isinstance(module, nn.Linear) and name in layer_sparsity_dict:
            # Check if the layer has a specified sparsity in layer_sparsity_dict
            layer_sparsity = layer_sparsity_dict[name]

            if layer_sparsity > 0.0:
                # Calculate the threshold for pruning based on the layer sparsity
                # reference: https://stackoverflow.com/questions/63582590/why-do-we-call-detach-before-calling-numpy-on-a-pytorch-tensor
                threshold = np.percentile(np.abs(module.weight.cpu().detach().numpy()),
                                          100 * layer_sparsity)

                # Generate a custom mask based on the threshold
                custom_mask = torch.abs(module.weight) > threshold
                custom_mask = custom_mask.to(torch.float32)

                # Apply pruning using the custom mask
                prune.custom_from_mask(module, name='weight', mask=custom_mask)

    # Transfer the pruned model's state to the original model
    #model.load_state_dict(pruned_model.state_dict(), strict=False)

    # Convert layer sparsity values to integers for better readability in filenames
    layer_sparsity_int_dict = {k: int(v * 100) for k, v in layer_sparsity_dict.items()}

    # Generate a filename-friendly representation of the layer sparsity dictionary
    filename_friendly_repr = '_'.join(f'{sanitize_for_filename(k)}_{v}' for k, v in layer_sparsity_int_dict.items())

    # Save the pruned model with the layer sparsity values in the name
    pruned_model_name = f'pruned_local_layer_sparsity_{filename_friendly_repr}.pth'
    torch.save(pruned_model.state_dict(), pruned_model_name)

    return pruned_model

The `layer_sparsity_dict` allows you to set different levels of sparsity for each layer, which should add up to the desired overall sparsity of the model. The values for sparsity range from 0 (no pruning) to 1 (complete pruning).

For instance, let's say for the three layers in our model called *fc1*, *fc2*, and *fc3*, and we want to reduce the overall size of the trained model by 70%. To achieve this, we have to assign sparsity values of 0.2, 0.3, and 0.2 respectively to each of the three layers. The sum of these values (0.2 + 0.3 + 0.2) equals 0.7, which achieves the desired overall sparsity of 70% for the entire model. We can adjust these values based on specific requirements for each layer.

**Pruning Amount 0.6**

In [ ]:
# Specify layer-wise sparsity as a dictionary - resulting pruning amount 0.6
layer_sparsity_dict_06 = {'fc1': 0.2, 'fc2': 0.2, 'fc3': 0.2}

# Apply local pruning
pruned_sparse1 = local_prune_layer_sparsity(model, layer_sparsity_dict_06)

#compare weights of the pruned and loaded model
compare_weights(model, pruned_sparse1)

**Pruning Amount 0.7**

In [ ]:
# Specify layer-wise sparsity as a dictionary - resulting pruning amount 0.7
layer_sparsity_dict_07 = {'fc1': 0.2, 'fc2': 0.3, 'fc3': 0.2}

# Apply local pruning
pruned_sparse2 = local_prune_layer_sparsity(model, layer_sparsity_dict_07)

#compare weights of the pruned and loaded model
compare_weights(model, pruned_sparse2)

**Pruning Amount 0.8**

In [ ]:
# Specify layer-wise sparsity as a dictionary - resulting pruning amount 0.8
layer_sparsity_dict_08 = {'fc1': 0.25, 'fc2': 0.25, 'fc3': 0.3}

# Apply local pruning
pruned_sparse3 = local_prune_layer_sparsity(model, layer_sparsity_dict_08)

#compare weights of the pruned and loaded model
compare_weights(model, pruned_sparse3)

**Pruning Amount 0.9**

In [ ]:
# Specify layer-wise sparsity as a dictionary - resulting pruning amount 0.9
layer_sparsity_dict_09 = {'fc1': 0.3, 'fc2': 0.3, 'fc3': 0.3}

# Apply local pruning
pruned_sparse4 = local_prune_layer_sparsity(model, layer_sparsity_dict_09)

#compare weights of the pruned and loaded model
compare_weights(model, pruned_sparse4)

In [ ]:
# Specify layer-wise sparsity as a dictionary - resulting pruning amount 0.95
layer_sparsity_dict_095 = {'fc1': 0.31, 'fc2': 0.31, 'fc3': 0.33}

# Apply local pruning
pruned_sparse5 = local_prune_layer_sparsity(model, layer_sparsity_dict_095)

#compare weights of the pruned and loaded model
compare_weights(model, pruned_sparse5)

## Principal Component Analysis (PCA) on a Pruned Model

In this section, our goal is to reduce the number of input features on the first layer of the locally pruned model, fc1, by applying PCA compression. To achieve this, we will use a typical approach that involves applying PCA in a way that compresses the dimensionality of the input space while preserving the essence of the transformation that the original weights provided. This approach modifies the architecture of the neural network differently from reducing the number of input features across the layers.

Reference:
https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html

In [ ]:
# Define the apply_pca_and_reshape function
def apply_pca_and_reshape(weights, n_components):
  # apply PCA
    pca = PCA(n_components=n_components)
    # reducing the output featurures and transforming them
    transformed_weights = pca.fit_transform(weights.T).T
    return transformed_weights

In [ ]:
# Modifying the RegressionNN to accommodate the reduced dimensions
class CompressedRegressionNN(nn.Module):
    def __init__(self, input_size, pca_fc1_out_features,
                 pca_fc2_out_features, output_size):
        super(CompressedRegressionNN, self).__init__()

        # Adjusted first layer with reduced output features
        self.fc1 = nn.Linear(input_size, pca_fc1_out_features)

        # Adjusted second layer with reduced output features
        self.fc2 = nn.Linear(pca_fc1_out_features, pca_fc2_out_features)

        # Third layer remains unchanged
        self.fc3 = nn.Linear(pca_fc2_out_features, output_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
# Original models hidden_size1, hidden_size2, input and output sizes
input_size = X_train.shape[1] # input features = 58
output_size = 1  # Output features for the third layer
hidden_size1 = 64
hidden_size2 = 32

### Functions to Train and Validate Compressed Models

Testing the compressed model without training.

In [ ]:
# function to evaluate the model without training
# model - model to evaluate
# test_loader - DataLoader for the test dataset
# device (torch.device) - device to run the evaluation on ('cuda' or 'cpu')
def evaluate_model(model, test_loader, device):

    model.to(device)
    model.eval()  # Set the model to evaluation mode

    criterion = nn.MSELoss()  # Initialize the loss function

    total_loss = 0.0
    total_mae = 0.0
    samples = 0

    with torch.no_grad():  # No gradient calculations needed
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)

            outputs = model(inputs)  # Make predictions

            loss = criterion(outputs, targets)
            total_loss += loss.item() * inputs.size(0)  # Accumulate loss

            mae = torch.mean(torch.abs(outputs - targets)).item()
            total_mae += mae * inputs.size(0)  # Accumulate MAE

            samples += inputs.size(0)  # Keep track of total samples

    average_loss = total_loss / samples
    average_mae = total_mae / samples

    return average_loss, average_mae



```
average_loss, average_mae = evaluate_model(PCA_local_05, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')
```



Training and Validating the PCA compressed model.

In [ ]:
# Function to train and validate the model
# Reference: https://discuss.pytorch.org/t/output-evaluation-loss-after-every-n-batches-instead-of-epochs-with-pytorch/116619
def train_and_validate(model, train_loader, test_loader, learning_rate,
                       epochs, max_grad_norm=None):

    # Move the model to CUDA if available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    # Defining Mean Squared Error loss function and optimizer
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    # Lists to store losses
    training_losses = []
    validation_losses = []

    # Training loop
    for epoch in range(epochs):
        model.train()  # Set the model to training mode

        total_loss = 0.0
        total_mae = 0.0

        # Training
        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.squeeze().to(device)

            # Forward pass
            outputs = model(inputs)
            mse_loss = criterion(outputs.view(-1), targets.view(-1))
            mae_loss = nn.L1Loss()(outputs.view(-1), targets.view(-1))

            # Backward and optimize
            optimizer.zero_grad()
            mse_loss.backward()

            # Gradient Clipping
            if max_grad_norm is not None:
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=max_grad_norm)

            optimizer.step()

            total_loss += mse_loss.item()
            total_mae += mae_loss.item()

        average_loss = total_loss / len(train_loader)
        average_mae = total_mae / len(train_loader)
        training_losses.append(average_loss)
        print(f'Epoch [{epoch + 1}/{epochs}], MSE Loss (Train): {average_loss:.3f}, MAE Loss (Train): {average_mae:.3f}')

        # Validation
        model.eval()
        validation_loss, validation_mae = 0.0, 0.0
        with torch.no_grad():
            for inputs, targets in test_loader:
                inputs, targets = inputs.to(device), targets.squeeze().to(device)
                outputs = model(inputs)
                validation_loss += criterion(outputs.view(-1), targets.view(-1)).item()
                validation_mae += nn.L1Loss()(outputs.view(-1), targets.view(-1)).item()

        average_validation_loss = validation_loss / len(test_loader)
        average_validation_mae = validation_mae / len(test_loader)
        validation_losses.append(average_validation_loss)
        print(f'Validation Loss: {average_validation_loss:.3f}, Validation MAE: {average_validation_mae:.3f}')

    # Plotting losses
    plt.plot(range(1, epochs + 1), training_losses, label='Training Loss')
    plt.plot(range(1, epochs + 1), validation_losses, label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

    return model, training_losses, validation_losses



```
trained_model, training_losses, validation_losses = train_and_validate(
    model=PCA_local_05, train_loader=train_loader,
    test_loader=test_loader, learning_rate=0.001, epochs=200
)
```



This function transfers weights (and biases) from a pruned model to another. It takes the source model (pruned model) and the target model as arguments. It applies existing masks to the weights, computes pruned weights, and updates corresponding layers in the target model.

In [ ]:
# Copying pruned weights (and biases) from a source model to a target model
# source_model (torch.nn.Module): The model from which to transfer weights.
# target_model (torch.nn.Module): The model to which weights will be transferred.
# reference: https://jimmy-shen.medium.com/pytorch-freeze-part-of-the-layers-4554105e03a6
def transfer_pruned_weights(source_model, target_model):
    for name, module in source_model.named_children():
        if isinstance(module, nn.Linear):
            # Retrieve the original weight and mask
            orig_weight = module.weight_orig if hasattr(module, 'weight_orig') else module.weight
            mask = module.weight_mask if hasattr(module, 'weight_mask') else torch.ones_like(module.weight)

            # Apply the mask to get the pruned weight
            # https://stackoverflow.com/questions/69311857/why-doesnt-torch-pruning-actually-remove-filters-or-weights
            pruned_weight = orig_weight * mask

            # Manually update the weights in the target model
            target_layer = getattr(target_model, name)
            target_layer.weight = nn.Parameter(pruned_weight)

            # Transfer the bias if it exists
            if module.bias is not None:
              # https://stackoverflow.com/questions/73228490/how-to-set-specific-values-for-the-weight-and-bias-in-a-neural-net
                target_layer.bias = nn.Parameter(module.bias.data)

**Checking the weight shape of the model**

In [ ]:
# weight shape of the loaded model
print("fc1 weight shape:", model.fc1.weight.shape)
print("fc2 weight shape:", model.fc2.weight.shape)
print("fc3 weight shape:", model.fc3.weight.shape)

### PCA on Locally Pruned Models

#### PCA on the Pruned Model, `pruned_local1`, with Pruning Amount=0.6

###### fc1 out_features=50 and fc2 out_features=30

In [ ]:
# create copy of the locally pruned model
temp_l1_5030 = RegressionNN(input_size, hidden_size1,
                               hidden_size2, output_size)
transfer_pruned_weights(pruned_local1, temp_l1_5030)

In [ ]:
# weight shape of the loaded pruned model
print("fc1 weight shape:", temp_l1_5030.fc1.weight.shape)
print("fc2 weight shape:", temp_l1_5030.fc2.weight.shape)
print("fc3 weight shape:", temp_l1_5030.fc3.weight.shape)

In [ ]:
# Extract weights from the pruned model
original_fc1_weights = temp_l1_5030.fc1.weight.data.cpu().numpy()

# Apply PCA to fc1 weights
transformed_fc1_weights_l1_5030 = apply_pca_and_reshape(original_fc1_weights, 50)

# Update fc2 in the pruned model to match new fc1 out_features
temp_l1_5030.fc2 = nn.Linear(50, hidden_size2)

# Extract fc2 weights after updating fc2 in the pruned model
original_fc2_weights = temp_l1_5030.fc2.weight.data.cpu().numpy()

# Apply PCA to fc2 weights
transformed_fc2_weights_l1_5030 = apply_pca_and_reshape(original_fc2_weights, 30)

# Instantiate the compressed model
PCA_local1_5030 = CompressedRegressionNN(58, 50, 30, 1)

# Assign the PCA-transformed weights
PCA_local1_5030.fc1.weight.data = torch.from_numpy(transformed_fc1_weights_l1_5030).float().view(50, 58)
PCA_local1_5030.fc2.weight.data = torch.from_numpy(transformed_fc2_weights_l1_5030).float().view(30, 50)

# Reset biases
PCA_local1_5030.fc1.bias.data.fill_(0)
PCA_local1_5030.fc2.bias.data.fill_(0)
PCA_local1_5030.fc3.bias.data.fill_(0)

# Validate the weight shapes
print("Transformed fc1 weight shape:", PCA_local1_5030.fc1.weight.shape)
print("Transformed fc2 weight shape:", PCA_local1_5030.fc2.weight.shape)
print("Transformed fc3 weight shape:", PCA_local1_5030.fc3.weight.shape)
print()
print(PCA_local1_5030)

In [ ]:
average_loss, average_mae = evaluate_model(PCA_local1_5030, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=PCA_local1_5030, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

###### fc1 out_features=50 and fc2 out_features=10

In [ ]:
# create copy of the locally pruned model
temp_l1_5010 = RegressionNN(input_size, hidden_size1,
                               hidden_size2, output_size)
transfer_pruned_weights(pruned_local1, temp_l1_5010)

In [ ]:
# weight shape of the loaded pruned model
print("fc1 weight shape:", temp_l1_5010.fc1.weight.shape)
print("fc2 weight shape:", temp_l1_5010.fc2.weight.shape)
print("fc3 weight shape:", temp_l1_5010.fc3.weight.shape)

In [ ]:
# Extract weights from the pruned model
original_fc1_weights_l1_5010 = temp_l1_5010.fc1.weight.data.cpu().numpy()

# Apply PCA to fc1 weights
transformed_fc1_weights_l1_5010 = apply_pca_and_reshape(original_fc1_weights_l1_5010,
                                                        50)

# Update fc2 in the pruned model to match new fc1 out_features
temp_l1_5010.fc2 = nn.Linear(50, hidden_size2)

# Extract fc2 weights after updating fc2 in the pruned model
original_fc2_weights_l1_5010 = temp_l1_5010.fc2.weight.data.cpu().numpy()

# Apply PCA to fc2 weights
transformed_fc2_weights_l1_5010 = apply_pca_and_reshape(original_fc2_weights_l1_5010,
                                                        10)

# Instantiate the compressed model
PCA_local1_5010 = CompressedRegressionNN(58, 50, 10, 1)

# Assign the PCA-transformed weights
PCA_local1_5010.fc1.weight.data = torch.from_numpy(transformed_fc1_weights_l1_5010).float().view(50, 58)
PCA_local1_5010.fc2.weight.data = torch.from_numpy(transformed_fc2_weights_l1_5010).float().view(10, 50)

# Reset biases
PCA_local1_5010.fc1.bias.data.fill_(0)
PCA_local1_5010.fc2.bias.data.fill_(0)
PCA_local1_5010.fc3.bias.data.fill_(0)

# Validate the weight shapes
print("Transformed fc1 weight shape:", PCA_local1_5010.fc1.weight.shape)
print("Transformed fc2 weight shape:", PCA_local1_5010.fc2.weight.shape)
print("Transformed fc3 weight shape:", PCA_local1_5010.fc3.weight.shape)
print()
print(PCA_local1_5010)

In [ ]:
average_loss, average_mae = evaluate_model(PCA_local1_5010, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=PCA_local1_5010, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

###### fc1 out_features=50 and fc2 out_features=5

In [ ]:
# create copy of the locally pruned model
temp_l1_505 = RegressionNN(input_size, hidden_size1,
                               hidden_size2, output_size)
transfer_pruned_weights(pruned_local1, temp_l1_505)

In [ ]:
# weight shape of the loaded pruned model
print("fc1 weight shape:", temp_l1_505.fc1.weight.shape)
print("fc2 weight shape:", temp_l1_505.fc2.weight.shape)
print("fc3 weight shape:", temp_l1_505.fc3.weight.shape)

In [ ]:
# Extract weights from the pruned model
original_fc1_weights_l1_505 = temp_l1_505.fc1.weight.data.cpu().numpy()

# Apply PCA to fc1 weights
transformed_fc1_weights_l1_505 = apply_pca_and_reshape(original_fc1_weights_l1_505,
                                                       50)

# Update fc2 in the pruned model to match new fc1 out_features
temp_l1_505.fc2 = nn.Linear(50, hidden_size2)

# Extract fc2 weights after updating fc2 in the pruned model
original_fc2_weights_l1_505 = temp_l1_505.fc2.weight.data.cpu().numpy()

# Apply PCA to fc2 weights
transformed_fc2_weights_l1_505 = apply_pca_and_reshape(original_fc2_weights_l1_505,
                                                       5)

# Instantiate the compressed model
PCA_local1_505 = CompressedRegressionNN(58, 50, 5, 1)

# Assign the PCA-transformed weights
PCA_local1_505.fc1.weight.data = torch.from_numpy(transformed_fc1_weights_l1_505).float().view(50, 58)
PCA_local1_505.fc2.weight.data = torch.from_numpy(transformed_fc2_weights_l1_505).float().view(5, 50)

# Reset biases
PCA_local1_505.fc1.bias.data.fill_(0)
PCA_local1_505.fc2.bias.data.fill_(0)
PCA_local1_505.fc3.bias.data.fill_(0)

# Validate the weight shapes
print("Transformed fc1 weight shape:", PCA_local1_505.fc1.weight.shape)
print("Transformed fc2 weight shape:", PCA_local1_505.fc2.weight.shape)
print("Transformed fc3 weight shape:", PCA_local1_505.fc3.weight.shape)
print()
print(PCA_local1_505)

In [ ]:
average_loss, average_mae = evaluate_model(PCA_local1_505, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=PCA_local1_505, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

###### fc1 out_features=30 and fc2 out_features=10

In [ ]:
# create copy of the locally pruned model
temp_l1_3010 = RegressionNN(input_size, hidden_size1,
                               hidden_size2, output_size)
transfer_pruned_weights(pruned_local1, temp_l1_3010)

In [ ]:
# weight shape of the loaded pruned model
print("fc1 weight shape:", temp_l1_3010.fc1.weight.shape)
print("fc2 weight shape:", temp_l1_3010.fc2.weight.shape)
print("fc3 weight shape:", temp_l1_3010.fc3.weight.shape)

In [ ]:
# Extract weights from the pruned model
original_fc1_weights_l1_3010 = temp_l1_3010.fc1.weight.data.cpu().numpy()

# Apply PCA to fc1 weights
transformed_fc1_weights_l1_3010 = apply_pca_and_reshape(original_fc1_weights_l1_3010,
                                                        30)

# Update fc2 in the pruned model to match new fc1 out_features
temp_l1_3010.fc2 = nn.Linear(30, hidden_size2)

# Extract fc2 weights after updating fc2 in the pruned model
original_fc2_weights_l1_3010 = temp_l1_3010.fc2.weight.data.cpu().numpy()

# Apply PCA to fc2 weights
transformed_fc2_weights_l1_3010 = apply_pca_and_reshape(original_fc2_weights_l1_3010,
                                                        10)

# Instantiate the compressed model
PCA_local1_3010 = CompressedRegressionNN(58, 30, 10, 1)

# Assign the PCA-transformed weights
PCA_local1_3010.fc1.weight.data = torch.from_numpy(transformed_fc1_weights_l1_3010).float().view(30, 58)
PCA_local1_3010.fc2.weight.data = torch.from_numpy(transformed_fc2_weights_l1_3010).float().view(10, 30)

# Reset biases
PCA_local1_3010.fc1.bias.data.fill_(0)
PCA_local1_3010.fc2.bias.data.fill_(0)
PCA_local1_3010.fc3.bias.data.fill_(0)

# Validate the weight shapes
print("Transformed fc1 weight shape:", PCA_local1_3010.fc1.weight.shape)
print("Transformed fc2 weight shape:", PCA_local1_3010.fc2.weight.shape)
print("Transformed fc3 weight shape:", PCA_local1_3010.fc3.weight.shape)
print()
print(PCA_local1_3010)

In [ ]:
average_loss, average_mae = evaluate_model(PCA_local1_3010, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=PCA_local1_3010, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

###### fc1 out_features=30 and fc2 out_features=5

In [ ]:
# create copy of the locally pruned model
temp_l1_305 = RegressionNN(input_size, hidden_size1,
                           hidden_size2, output_size)
transfer_pruned_weights(pruned_local1, temp_l1_305)

In [ ]:
# weight shape of the loaded pruned model
print("fc1 weight shape:", temp_l1_305.fc1.weight.shape)
print("fc2 weight shape:", temp_l1_305.fc2.weight.shape)
print("fc3 weight shape:", temp_l1_305.fc3.weight.shape)

In [ ]:
# Extract weights from the pruned model
original_fc1_weights_l1_305 = temp_l1_305.fc1.weight.data.cpu().numpy()

# Apply PCA to fc1 weights
transformed_fc1_weights_l1_305 = apply_pca_and_reshape(original_fc1_weights_l1_305,
                                                       30)

# Update fc2 in the pruned model to match new fc1 out_features
temp_l1_305.fc2 = nn.Linear(30, hidden_size2)

# Extract fc2 weights after updating fc2 in the pruned model
original_fc2_weights_l1_305 = temp_l1_305.fc2.weight.data.cpu().numpy()

# Apply PCA to fc2 weights
transformed_fc2_weights_l1_305 = apply_pca_and_reshape(original_fc2_weights_l1_305,
                                                        5)

# Instantiate the compressed model
PCA_local1_305 = CompressedRegressionNN(58, 30, 5, 1)

# Assign the PCA-transformed weights
PCA_local1_305.fc1.weight.data = torch.from_numpy(transformed_fc1_weights_l1_305).float().view(30, 58)
PCA_local1_305.fc2.weight.data = torch.from_numpy(transformed_fc2_weights_l1_305).float().view(5, 30)

# Reset biases
PCA_local1_305.fc1.bias.data.fill_(0)
PCA_local1_305.fc2.bias.data.fill_(0)
PCA_local1_305.fc3.bias.data.fill_(0)

# Validate the weight shapes
print("Transformed fc1 weight shape:", PCA_local1_305.fc1.weight.shape)
print("Transformed fc2 weight shape:", PCA_local1_305.fc2.weight.shape)
print("Transformed fc3 weight shape:", PCA_local1_305.fc3.weight.shape)
print()
print(PCA_local1_305)

In [ ]:
average_loss, average_mae = evaluate_model(PCA_local1_305, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=PCA_local1_305, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

###### fc1 out_features=10 and fc2 out_features=5

In [ ]:
# create copy of the locally pruned model
temp_l1_105 = RegressionNN(input_size, hidden_size1,
                               hidden_size2, output_size)
transfer_pruned_weights(pruned_local1, temp_l1_105)

In [ ]:
# weight shape of the loaded pruned model
print("fc1 weight shape:", temp_l1_105.fc1.weight.shape)
print("fc2 weight shape:", temp_l1_105.fc2.weight.shape)
print("fc3 weight shape:", temp_l1_105.fc3.weight.shape)

In [ ]:
# Extract weights from the pruned model
original_fc1_weights_l1_105 = temp_l1_105.fc1.weight.data.cpu().numpy()

# Apply PCA to fc1 weights
transformed_fc1_weights_l1_105 = apply_pca_and_reshape(original_fc1_weights_l1_105,
                                                       10)

# Update fc2 in the pruned model to match new fc1 out_features
temp_l1_105.fc2 = nn.Linear(10, hidden_size2)

# Extract fc2 weights after updating fc2 in the pruned model
original_fc2_weights_l1_105 = temp_l1_105.fc2.weight.data.cpu().numpy()

# Apply PCA to fc2 weights
transformed_fc2_weights_l1_105 = apply_pca_and_reshape(original_fc2_weights_l1_105,
                                                        5)

# Instantiate the compressed model
PCA_local1_105 = CompressedRegressionNN(58, 10, 5, 1)

# Assign the PCA-transformed weights
PCA_local1_105.fc1.weight.data = torch.from_numpy(transformed_fc1_weights_l1_105).float().view(10, 58)
PCA_local1_105.fc2.weight.data = torch.from_numpy(transformed_fc2_weights_l1_105).float().view(5, 10)

# Reset biases
PCA_local1_105.fc1.bias.data.fill_(0)
PCA_local1_105.fc2.bias.data.fill_(0)
PCA_local1_105.fc3.bias.data.fill_(0)

# Validate the weight shapes
print("Transformed fc1 weight shape:", PCA_local1_105.fc1.weight.shape)
print("Transformed fc2 weight shape:", PCA_local1_105.fc2.weight.shape)
print("Transformed fc3 weight shape:", PCA_local1_105.fc3.weight.shape)
print()
print(PCA_local1_105)

In [ ]:
average_loss, average_mae = evaluate_model(PCA_local1_105, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=PCA_local1_105, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

#### PCA on the Pruned Model, `pruned_local2`, with Pruning Amount=0.7

###### fc1 out_features=50 and fc2 out_features=30

In [ ]:
# create copy of the locally pruned model
temp_l2_5030 = RegressionNN(input_size, hidden_size1,
                               hidden_size2, output_size)
transfer_pruned_weights(pruned_local2, temp_l2_5030)

In [ ]:
# weight shape of the loaded pruned model
print("fc1 weight shape:", temp_l2_5030.fc1.weight.shape)
print("fc2 weight shape:", temp_l2_5030.fc2.weight.shape)
print("fc3 weight shape:", temp_l2_5030.fc3.weight.shape)

In [ ]:
# Extract weights from the pruned model
original_fc1_weights_l2_5030 = temp_l2_5030.fc1.weight.data.cpu().numpy()

# Apply PCA to fc1 weights
transformed_fc1_weights_l2_5030 = apply_pca_and_reshape(original_fc1_weights_l2_5030,
                                                        50)

# Update fc2 in the pruned model to match new fc1 out_features
temp_l2_5030.fc2 = nn.Linear(50, hidden_size2)

# Extract fc2 weights after updating fc2 in the pruned model
original_fc2_weights_l2_5030 = temp_l2_5030.fc2.weight.data.cpu().numpy()

# Apply PCA to fc2 weights
transformed_fc2_weights_l2_5030 = apply_pca_and_reshape(original_fc2_weights_l2_5030,
                                                        30)

# Instantiate the compressed model
PCA_local2_5030 = CompressedRegressionNN(58, 50, 30, 1)

# Assign the PCA-transformed weights
PCA_local2_5030.fc1.weight.data = torch.from_numpy(transformed_fc1_weights_l2_5030).float().view(50, 58)
PCA_local2_5030.fc2.weight.data = torch.from_numpy(transformed_fc2_weights_l2_5030).float().view(30, 50)

# Reset biases
PCA_local2_5030.fc1.bias.data.fill_(0)
PCA_local2_5030.fc2.bias.data.fill_(0)
PCA_local2_5030.fc3.bias.data.fill_(0)

# Validate the weight shapes
print("Transformed fc1 weight shape:", PCA_local2_5030.fc1.weight.shape)
print("Transformed fc2 weight shape:", PCA_local2_5030.fc2.weight.shape)
print("Transformed fc3 weight shape:", PCA_local2_5030.fc3.weight.shape)
print()
print(PCA_local2_5030)

In [ ]:
average_loss, average_mae = evaluate_model(PCA_local2_5030, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=PCA_local2_5030, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

###### fc1 out_features=50 and fc2 out_features=10

In [ ]:
# create copy of the locally pruned model
temp_l2_5010 = RegressionNN(input_size, hidden_size1,
                               hidden_size2, output_size)
transfer_pruned_weights(pruned_local2, temp_l2_5010)

In [ ]:
# weight shape of the loaded pruned model
print("fc1 weight shape:", temp_l2_5010.fc1.weight.shape)
print("fc2 weight shape:", temp_l2_5010.fc2.weight.shape)
print("fc3 weight shape:", temp_l2_5010.fc3.weight.shape)

In [ ]:
# Extract weights from the pruned model
original_fc1_weights_l2_5010 = temp_l2_5010.fc1.weight.data.cpu().numpy()

# Apply PCA to fc1 weights
transformed_fc1_weights_l2_5010 = apply_pca_and_reshape(original_fc1_weights_l2_5010,
                                                        50)

# Update fc2 in the pruned model to match new fc1 out_features
temp_l2_5010.fc2 = nn.Linear(50, hidden_size2)

# Extract fc2 weights after updating fc2 in the pruned model
original_fc2_weights_l2_5010 = temp_l2_5010.fc2.weight.data.cpu().numpy()

# Apply PCA to fc2 weights
transformed_fc2_weights_l2_5010 = apply_pca_and_reshape(original_fc2_weights_l2_5010,
                                                        10)

# Instantiate the compressed model
PCA_local2_5010 = CompressedRegressionNN(58, 50, 10, 1)

# Assign the PCA-transformed weights
PCA_local2_5010.fc1.weight.data = torch.from_numpy(transformed_fc1_weights_l2_5010).float().view(50, 58)
PCA_local2_5010.fc2.weight.data = torch.from_numpy(transformed_fc2_weights_l2_5010).float().view(10, 50)

# Reset biases
PCA_local2_5010.fc1.bias.data.fill_(0)
PCA_local2_5010.fc2.bias.data.fill_(0)
PCA_local2_5010.fc3.bias.data.fill_(0)

# Validate the weight shapes
print("Transformed fc1 weight shape:", PCA_local2_5010.fc1.weight.shape)
print("Transformed fc2 weight shape:", PCA_local2_5010.fc2.weight.shape)
print("Transformed fc3 weight shape:", PCA_local2_5010.fc3.weight.shape)
print()
print(PCA_local2_5010)

In [ ]:
average_loss, average_mae = evaluate_model(PCA_local2_5010, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=PCA_local2_5010, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

###### fc1 out_features=50 and fc2 out_features=5

In [ ]:
# create copy of the locally pruned model
temp_l2_505 = RegressionNN(input_size, hidden_size1,
                               hidden_size2, output_size)
transfer_pruned_weights(pruned_local2, temp_l2_505)

In [ ]:
# weight shape of the loaded pruned model
print("fc1 weight shape:", temp_l2_505.fc1.weight.shape)
print("fc2 weight shape:", temp_l2_505.fc2.weight.shape)
print("fc3 weight shape:", temp_l2_505.fc3.weight.shape)

In [ ]:
# Extract weights from the pruned model
original_fc1_weights_l2_505 = temp_l2_505.fc1.weight.data.cpu().numpy()

# Apply PCA to fc1 weights
transformed_fc1_weights_l2_505 = apply_pca_and_reshape(original_fc1_weights_l2_505,
                                                       50)

# Update fc2 in the pruned model to match new fc1 out_features
temp_l2_505.fc2 = nn.Linear(50, hidden_size2)

# Extract fc2 weights after updating fc2 in the pruned model
original_fc2_weights_l2_505 = temp_l2_505.fc2.weight.data.cpu().numpy()

# Apply PCA to fc2 weights
transformed_fc2_weights_l2_505 = apply_pca_and_reshape(original_fc2_weights_l2_505,
                                                       5)

# Instantiate the compressed model
PCA_local2_505 = CompressedRegressionNN(58, 50, 5, 1)

# Assign the PCA-transformed weights
PCA_local2_505.fc1.weight.data = torch.from_numpy(transformed_fc1_weights_l2_505).float().view(50, 58)
PCA_local2_505.fc2.weight.data = torch.from_numpy(transformed_fc2_weights_l2_505).float().view(5, 50)

# Reset biases
PCA_local2_505.fc1.bias.data.fill_(0)
PCA_local2_505.fc2.bias.data.fill_(0)
PCA_local2_505.fc3.bias.data.fill_(0)

# Validate the weight shapes
print("Transformed fc1 weight shape:", PCA_local2_505.fc1.weight.shape)
print("Transformed fc2 weight shape:", PCA_local2_505.fc2.weight.shape)
print("Transformed fc3 weight shape:", PCA_local2_505.fc3.weight.shape)
print()
print(PCA_local2_505)

In [ ]:
average_loss, average_mae = evaluate_model(PCA_local2_505, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=PCA_local2_505, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

###### fc1 out_features=30 and fc2 out_features=10

In [ ]:
# create copy of the locally pruned model
temp_l2_3010 = RegressionNN(input_size, hidden_size1,
                               hidden_size2, output_size)
transfer_pruned_weights(pruned_local2, temp_l2_3010)

In [ ]:
# weight shape of the loaded pruned model
print("fc1 weight shape:", temp_l2_3010.fc1.weight.shape)
print("fc2 weight shape:", temp_l2_3010.fc2.weight.shape)
print("fc3 weight shape:", temp_l2_3010.fc3.weight.shape)

In [ ]:
# Extract weights from the pruned model
original_fc1_weights_l2_3010 = temp_l2_3010.fc1.weight.data.cpu().numpy()

# Apply PCA to fc1 weights
transformed_fc1_weights_l2_3010 = apply_pca_and_reshape(original_fc1_weights_l2_3010,
                                                        30)

# Update fc2 in the pruned model to match new fc1 out_features
temp_l2_3010.fc2 = nn.Linear(30, hidden_size2)

# Extract fc2 weights after updating fc2 in the pruned model
original_fc2_weights_l2_3010 = temp_l2_3010.fc2.weight.data.cpu().numpy()

# Apply PCA to fc2 weights
transformed_fc2_weights_l2_3010 = apply_pca_and_reshape(original_fc2_weights_l2_3010,
                                                        10)

# Instantiate the compressed model
PCA_local2_3010 = CompressedRegressionNN(58, 30, 10, 1)

# Assign the PCA-transformed weights
PCA_local2_3010.fc1.weight.data = torch.from_numpy(transformed_fc1_weights_l2_3010).float().view(30, 58)
PCA_local2_3010.fc2.weight.data = torch.from_numpy(transformed_fc2_weights_l2_3010).float().view(10, 30)

# Reset biases
PCA_local2_3010.fc1.bias.data.fill_(0)
PCA_local2_3010.fc2.bias.data.fill_(0)
PCA_local2_3010.fc3.bias.data.fill_(0)

# Validate the weight shapes
print("Transformed fc1 weight shape:", PCA_local2_3010.fc1.weight.shape)
print("Transformed fc2 weight shape:", PCA_local2_3010.fc2.weight.shape)
print("Transformed fc3 weight shape:", PCA_local2_3010.fc3.weight.shape)
print()
print(PCA_local2_3010)

In [ ]:
average_loss, average_mae = evaluate_model(PCA_local2_3010, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=PCA_local2_3010, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

###### fc1 out_features=30 and fc2 out_features=5

In [ ]:
# create copy of the locally pruned model
temp_l2_305 = RegressionNN(input_size, hidden_size1,
                           hidden_size2, output_size)
transfer_pruned_weights(pruned_local2, temp_l2_305)

In [ ]:
# weight shape of the loaded pruned model
print("fc1 weight shape:", temp_l2_305.fc1.weight.shape)
print("fc2 weight shape:", temp_l2_305.fc2.weight.shape)
print("fc3 weight shape:", temp_l2_305.fc3.weight.shape)

In [ ]:
# Extract weights from the pruned model
original_fc1_weights_l2_305 = temp_l2_305.fc1.weight.data.cpu().numpy()

# Apply PCA to fc1 weights
transformed_fc1_weights_l2_305 = apply_pca_and_reshape(original_fc1_weights_l2_305,
                                                       30)

# Update fc2 in the pruned model to match new fc1 out_features
temp_l2_305.fc2 = nn.Linear(30, hidden_size2)

# Extract fc2 weights after updating fc2 in the pruned model
original_fc2_weights_l2_305 = temp_l2_305.fc2.weight.data.cpu().numpy()

# Apply PCA to fc2 weights
transformed_fc2_weights_l2_305 = apply_pca_and_reshape(original_fc2_weights_l2_305,
                                                        5)

# Instantiate the compressed model
PCA_local2_305 = CompressedRegressionNN(58, 30, 5, 1)

# Assign the PCA-transformed weights
PCA_local2_305.fc1.weight.data = torch.from_numpy(transformed_fc1_weights_l2_305).float().view(30, 58)
PCA_local2_305.fc2.weight.data = torch.from_numpy(transformed_fc2_weights_l2_305).float().view(5, 30)

# Reset biases
PCA_local2_305.fc1.bias.data.fill_(0)
PCA_local2_305.fc2.bias.data.fill_(0)
PCA_local2_305.fc3.bias.data.fill_(0)

# Validate the weight shapes
print("Transformed fc1 weight shape:", PCA_local2_305.fc1.weight.shape)
print("Transformed fc2 weight shape:", PCA_local2_305.fc2.weight.shape)
print("Transformed fc3 weight shape:", PCA_local2_305.fc3.weight.shape)
print()
print(PCA_local2_305)

In [ ]:
average_loss, average_mae = evaluate_model(PCA_local2_305, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=PCA_local2_305, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

###### fc1 out_features=10 and fc2 out_features=5

In [ ]:
# create copy of the locally pruned model
temp_l2_105 = RegressionNN(input_size, hidden_size1,
                               hidden_size2, output_size)
transfer_pruned_weights(pruned_local2, temp_l2_105)

In [ ]:
# weight shape of the loaded pruned model
print("fc1 weight shape:", temp_l2_105.fc1.weight.shape)
print("fc2 weight shape:", temp_l2_105.fc2.weight.shape)
print("fc3 weight shape:", temp_l2_105.fc3.weight.shape)

In [ ]:
# Extract weights from the pruned model
original_fc1_weights_l2_105 = temp_l2_105.fc1.weight.data.cpu().numpy()

# Apply PCA to fc1 weights
transformed_fc1_weights_l2_105 = apply_pca_and_reshape(original_fc1_weights_l2_105,
                                                       10)

# Update fc2 in the pruned model to match new fc1 out_features
temp_l2_105.fc2 = nn.Linear(10, hidden_size2)

# Extract fc2 weights after updating fc2 in the pruned model
original_fc2_weights_l1_105 = temp_l2_105.fc2.weight.data.cpu().numpy()

# Apply PCA to fc2 weights
transformed_fc2_weights_l2_105 = apply_pca_and_reshape(original_fc2_weights_l1_105,
                                                        5)

# Instantiate the compressed model
PCA_local2_105 = CompressedRegressionNN(58, 10, 5, 1)

# Assign the PCA-transformed weights
PCA_local2_105.fc1.weight.data = torch.from_numpy(transformed_fc1_weights_l2_105).float().view(10, 58)
PCA_local2_105.fc2.weight.data = torch.from_numpy(transformed_fc2_weights_l2_105).float().view(5, 10)

# Reset biases
PCA_local2_105.fc1.bias.data.fill_(0)
PCA_local2_105.fc2.bias.data.fill_(0)
PCA_local2_105.fc3.bias.data.fill_(0)

# Validate the weight shapes
print("Transformed fc1 weight shape:", PCA_local2_105.fc1.weight.shape)
print("Transformed fc2 weight shape:", PCA_local2_105.fc2.weight.shape)
print("Transformed fc3 weight shape:", PCA_local2_105.fc3.weight.shape)
print()
print(PCA_local2_105)

In [ ]:
average_loss, average_mae = evaluate_model(PCA_local2_105, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=PCA_local2_105, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

#### PCA on the Pruned Model, `pruned_local3`, with Pruning Amount=0.8

###### fc1 out_features=50 and fc2 out_features=30

In [ ]:
# create copy of the locally pruned model
temp_l3_5030 = RegressionNN(input_size, hidden_size1,
                               hidden_size2, output_size)
transfer_pruned_weights(pruned_local3, temp_l3_5030)

In [ ]:
# weight shape of the loaded pruned model
print("fc1 weight shape:", temp_l3_5030.fc1.weight.shape)
print("fc2 weight shape:", temp_l3_5030.fc2.weight.shape)
print("fc3 weight shape:", temp_l3_5030.fc3.weight.shape)

In [ ]:
# Extract weights from the pruned model
original_fc1_weights_l3_5030 = temp_l2_5030.fc1.weight.data.cpu().numpy()

# Apply PCA to fc1 weights
transformed_fc1_weights_l2_5030 = apply_pca_and_reshape(original_fc1_weights_l3_5030,
                                                        50)

# Update fc2 in the pruned model to match new fc1 out_features
temp_l2_5030.fc2 = nn.Linear(50, hidden_size2)

# Extract fc2 weights after updating fc2 in the pruned model
original_fc2_weights_l3_5030 = temp_l2_5030.fc2.weight.data.cpu().numpy()

# Apply PCA to fc2 weights
transformed_fc2_weights_l2_5030 = apply_pca_and_reshape(original_fc2_weights_l3_5030,
                                                        30)

# Instantiate the compressed model
PCA_local3_5030 = CompressedRegressionNN(58, 50, 30, 1)

# Assign the PCA-transformed weights
PCA_local3_5030.fc1.weight.data = torch.from_numpy(transformed_fc1_weights_l2_5030).float().view(50, 58)
PCA_local3_5030.fc2.weight.data = torch.from_numpy(transformed_fc2_weights_l2_5030).float().view(30, 50)

# Reset biases
PCA_local3_5030.fc1.bias.data.fill_(0)
PCA_local3_5030.fc2.bias.data.fill_(0)
PCA_local3_5030.fc3.bias.data.fill_(0)

# Validate the weight shapes
print("Transformed fc1 weight shape:", PCA_local3_5030.fc1.weight.shape)
print("Transformed fc2 weight shape:", PCA_local3_5030.fc2.weight.shape)
print("Transformed fc3 weight shape:", PCA_local3_5030.fc3.weight.shape)
print()
print(PCA_local3_5030)

In [ ]:
average_loss, average_mae = evaluate_model(PCA_local3_5030, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=PCA_local3_5030, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

###### fc1 out_features=50 and fc2 out_features=10

In [ ]:
# create copy of the locally pruned model
temp_l3_5010 = RegressionNN(input_size, hidden_size1,
                               hidden_size2, output_size)
transfer_pruned_weights(pruned_local3, temp_l3_5010)

In [ ]:
# weight shape of the loaded pruned model
print("fc1 weight shape:", temp_l3_5010.fc1.weight.shape)
print("fc2 weight shape:", temp_l3_5010.fc2.weight.shape)
print("fc3 weight shape:", temp_l3_5010.fc3.weight.shape)

In [ ]:
# Extract weights from the pruned model
original_fc1_weights_l3_5010 = temp_l3_5010.fc1.weight.data.cpu().numpy()

# Apply PCA to fc1 weights
transformed_fc1_weights_l3_5010 = apply_pca_and_reshape(original_fc1_weights_l3_5010,
                                                        50)

# Update fc2 in the pruned model to match new fc1 out_features
temp_l3_5010.fc2 = nn.Linear(50, hidden_size2)

# Extract fc2 weights after updating fc2 in the pruned model
original_fc2_weights_l3_5010 = temp_l3_5010.fc2.weight.data.cpu().numpy()

# Apply PCA to fc2 weights
transformed_fc2_weights_l3_5010 = apply_pca_and_reshape(original_fc2_weights_l3_5010,
                                                        10)

# Instantiate the compressed model
PCA_local3_5010 = CompressedRegressionNN(58, 50, 10, 1)

# Assign the PCA-transformed weights
PCA_local3_5010.fc1.weight.data = torch.from_numpy(transformed_fc1_weights_l3_5010).float().view(50, 58)
PCA_local3_5010.fc2.weight.data = torch.from_numpy(transformed_fc2_weights_l3_5010).float().view(10, 50)

# Reset biases
PCA_local3_5010.fc1.bias.data.fill_(0)
PCA_local3_5010.fc2.bias.data.fill_(0)
PCA_local3_5010.fc3.bias.data.fill_(0)

# Validate the weight shapes
print("Transformed fc1 weight shape:", PCA_local3_5010.fc1.weight.shape)
print("Transformed fc2 weight shape:", PCA_local3_5010.fc2.weight.shape)
print("Transformed fc3 weight shape:", PCA_local3_5010.fc3.weight.shape)
print()
print(PCA_local3_5010)

In [ ]:
average_loss, average_mae = evaluate_model(PCA_local3_5010, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=PCA_local3_5010, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

###### fc1 out_features=50 and fc2 out_features=5

In [ ]:
# create copy of the locally pruned model
temp_l3_505 = RegressionNN(input_size, hidden_size1,
                               hidden_size2, output_size)
transfer_pruned_weights(pruned_local3, temp_l3_505)

In [ ]:
# weight shape of the loaded pruned model
print("fc1 weight shape:", temp_l3_505.fc1.weight.shape)
print("fc2 weight shape:", temp_l3_505.fc2.weight.shape)
print("fc3 weight shape:", temp_l3_505.fc3.weight.shape)

In [ ]:
# Extract weights from the pruned model
original_fc1_weights_l3_505 = temp_l3_505.fc1.weight.data.cpu().numpy()

# Apply PCA to fc1 weights
transformed_fc1_weights_l3_505 = apply_pca_and_reshape(original_fc1_weights_l3_505,
                                                       50)

# Update fc2 in the pruned model to match new fc1 out_features
temp_l3_505.fc2 = nn.Linear(50, hidden_size2)

# Extract fc2 weights after updating fc2 in the pruned model
original_fc2_weights_l3_505 = temp_l3_505.fc2.weight.data.cpu().numpy()

# Apply PCA to fc2 weights
transformed_fc2_weights_l3_505 = apply_pca_and_reshape(original_fc2_weights_l3_505,
                                                       5)

# Instantiate the compressed model
PCA_local3_505 = CompressedRegressionNN(58, 50, 5, 1)

# Assign the PCA-transformed weights
PCA_local3_505.fc1.weight.data = torch.from_numpy(transformed_fc1_weights_l3_505).float().view(50, 58)
PCA_local3_505.fc2.weight.data = torch.from_numpy(transformed_fc2_weights_l3_505).float().view(5, 50)

# Reset biases
PCA_local3_505.fc1.bias.data.fill_(0)
PCA_local3_505.fc2.bias.data.fill_(0)
PCA_local3_505.fc3.bias.data.fill_(0)

# Validate the weight shapes
print("Transformed fc1 weight shape:", PCA_local3_505.fc1.weight.shape)
print("Transformed fc2 weight shape:", PCA_local3_505.fc2.weight.shape)
print("Transformed fc3 weight shape:", PCA_local3_505.fc3.weight.shape)
print()
print(PCA_local3_505)

In [ ]:
average_loss, average_mae = evaluate_model(PCA_local3_505, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=PCA_local3_505, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

###### fc1 out_features=30 and fc2 out_features=10

In [ ]:
# create copy of the locally pruned model
temp_l3_3010 = RegressionNN(input_size, hidden_size1,
                               hidden_size2, output_size)
transfer_pruned_weights(pruned_local3, temp_l3_3010)

In [ ]:
# weight shape of the loaded pruned model
print("fc1 weight shape:", temp_l3_3010.fc1.weight.shape)
print("fc2 weight shape:", temp_l3_3010.fc2.weight.shape)
print("fc3 weight shape:", temp_l3_3010.fc3.weight.shape)

In [ ]:
# Extract weights from the pruned model
original_fc1_weights_l3_3010 = temp_l3_3010.fc1.weight.data.cpu().numpy()

# Apply PCA to fc1 weights
transformed_fc1_weights_l3_3010 = apply_pca_and_reshape(original_fc1_weights_l3_3010,
                                                        30)

# Update fc2 in the pruned model to match new fc1 out_features
temp_l3_3010.fc2 = nn.Linear(30, hidden_size2)

# Extract fc2 weights after updating fc2 in the pruned model
original_fc2_weights_l3_3010 = temp_l3_3010.fc2.weight.data.cpu().numpy()

# Apply PCA to fc2 weights
transformed_fc2_weights_l3_3010 = apply_pca_and_reshape(original_fc2_weights_l3_3010,
                                                        10)

# Instantiate the compressed model
PCA_local3_3010 = CompressedRegressionNN(58, 30, 10, 1)

# Assign the PCA-transformed weights
PCA_local3_3010.fc1.weight.data = torch.from_numpy(transformed_fc1_weights_l3_3010).float().view(30, 58)
PCA_local3_3010.fc2.weight.data = torch.from_numpy(transformed_fc2_weights_l3_3010).float().view(10, 30)

# Reset biases
PCA_local3_3010.fc1.bias.data.fill_(0)
PCA_local3_3010.fc2.bias.data.fill_(0)
PCA_local3_3010.fc3.bias.data.fill_(0)

# Validate the weight shapes
print("Transformed fc1 weight shape:", PCA_local3_3010.fc1.weight.shape)
print("Transformed fc2 weight shape:", PCA_local3_3010.fc2.weight.shape)
print("Transformed fc3 weight shape:", PCA_local3_3010.fc3.weight.shape)
print()
print(PCA_local3_3010)

In [ ]:
average_loss, average_mae = evaluate_model(PCA_local3_3010, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=PCA_local3_3010, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

###### fc1 out_features=30 and fc2 out_features=5

In [ ]:
# create copy of the locally pruned model
temp_l3_305 = RegressionNN(input_size, hidden_size1,
                           hidden_size2, output_size)
transfer_pruned_weights(pruned_local3, temp_l3_305)

In [ ]:
# weight shape of the loaded pruned model
print("fc1 weight shape:", temp_l3_305.fc1.weight.shape)
print("fc2 weight shape:", temp_l3_305.fc2.weight.shape)
print("fc3 weight shape:", temp_l3_305.fc3.weight.shape)

In [ ]:
# Extract weights from the pruned model
original_fc1_weights_l3_305 = temp_l3_305.fc1.weight.data.cpu().numpy()

# Apply PCA to fc1 weights
transformed_fc1_weights_l3_305 = apply_pca_and_reshape(original_fc1_weights_l3_305,
                                                       30)

# Update fc2 in the pruned model to match new fc1 out_features
temp_l3_305.fc2 = nn.Linear(30, hidden_size2)

# Extract fc2 weights after updating fc2 in the pruned model
original_fc2_weights_l3_305 = temp_l3_305.fc2.weight.data.cpu().numpy()

# Apply PCA to fc2 weights
transformed_fc2_weights_l3_305 = apply_pca_and_reshape(original_fc2_weights_l3_305,
                                                        5)

# Instantiate the compressed model
PCA_local3_305 = CompressedRegressionNN(58, 30, 5, 1)

# Assign the PCA-transformed weights
PCA_local3_305.fc1.weight.data = torch.from_numpy(transformed_fc1_weights_l3_305).float().view(30, 58)
PCA_local3_305.fc2.weight.data = torch.from_numpy(transformed_fc2_weights_l3_305).float().view(5, 30)

# Reset biases
PCA_local3_305.fc1.bias.data.fill_(0)
PCA_local3_305.fc2.bias.data.fill_(0)
PCA_local3_305.fc3.bias.data.fill_(0)

# Validate the weight shapes
print("Transformed fc1 weight shape:", PCA_local3_305.fc1.weight.shape)
print("Transformed fc2 weight shape:", PCA_local3_305.fc2.weight.shape)
print("Transformed fc3 weight shape:", PCA_local3_305.fc3.weight.shape)
print()
print(PCA_local3_305)

In [ ]:
average_loss, average_mae = evaluate_model(PCA_local3_305, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=PCA_local3_305, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

###### fc1 out_features=10 and fc2 out_features=5

In [ ]:
# create copy of the locally pruned model
temp_l3_105 = RegressionNN(input_size, hidden_size1,
                               hidden_size2, output_size)
transfer_pruned_weights(pruned_local3, temp_l3_105)

In [ ]:
# weight shape of the loaded pruned model
print("fc1 weight shape:", temp_l3_105.fc1.weight.shape)
print("fc2 weight shape:", temp_l3_105.fc2.weight.shape)
print("fc3 weight shape:", temp_l3_105.fc3.weight.shape)

In [ ]:
# Extract weights from the pruned model
original_fc1_weights_l3_105 = temp_l3_105.fc1.weight.data.cpu().numpy()

# Apply PCA to fc1 weights
transformed_fc1_weights_l3_105 = apply_pca_and_reshape(original_fc1_weights_l3_105,
                                                       10)

# Update fc2 in the pruned model to match new fc1 out_features
temp_l3_105.fc2 = nn.Linear(10, hidden_size2)

# Extract fc2 weights after updating fc2 in the pruned model
original_fc2_weights_l3_105 = temp_l3_105.fc2.weight.data.cpu().numpy()

# Apply PCA to fc2 weights
transformed_fc2_weights_l3_105 = apply_pca_and_reshape(original_fc2_weights_l3_105,
                                                        5)

# Instantiate the compressed model
PCA_local3_105 = CompressedRegressionNN(58, 10, 5, 1)

# Assign the PCA-transformed weights
PCA_local3_105.fc1.weight.data = torch.from_numpy(transformed_fc1_weights_l3_105).float().view(10, 58)
PCA_local3_105.fc2.weight.data = torch.from_numpy(transformed_fc2_weights_l3_105).float().view(5, 10)

# Reset biases
PCA_local3_105.fc1.bias.data.fill_(0)
PCA_local3_105.fc2.bias.data.fill_(0)
PCA_local3_105.fc3.bias.data.fill_(0)

# Validate the weight shapes
print("Transformed fc1 weight shape:", PCA_local3_105.fc1.weight.shape)
print("Transformed fc2 weight shape:", PCA_local3_105.fc2.weight.shape)
print("Transformed fc3 weight shape:", PCA_local3_105.fc3.weight.shape)
print()
print(PCA_local3_105)

In [ ]:
average_loss, average_mae = evaluate_model(PCA_local3_105, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=PCA_local3_105, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

#### PCA on the Pruned Model, `pruned_local4`, with Pruning Amount=0.9

###### fc1 out_features=50 and fc2 out_features=30

In [ ]:
# create copy of the locally pruned model
temp_l4_5030 = RegressionNN(input_size, hidden_size1,
                               hidden_size2, output_size)
transfer_pruned_weights(pruned_local4, temp_l4_5030)

In [ ]:
# weight shape of the loaded pruned model
print("fc1 weight shape:", temp_l4_5030.fc1.weight.shape)
print("fc2 weight shape:", temp_l4_5030.fc2.weight.shape)
print("fc3 weight shape:", temp_l4_5030.fc3.weight.shape)

In [ ]:
# Extract weights from the pruned model
original_fc1_weights_l4_5030 = temp_l4_5030.fc1.weight.data.cpu().numpy()

# Apply PCA to fc1 weights
transformed_fc1_weights_l4_5030 = apply_pca_and_reshape(original_fc1_weights_l4_5030,
                                                        50)

# Update fc2 in the pruned model to match new fc1 out_features
temp_l4_5030.fc2 = nn.Linear(50, hidden_size2)

# Extract fc2 weights after updating fc2 in the pruned model
original_fc2_weights_l4_5030 = temp_l4_5030.fc2.weight.data.cpu().numpy()

# Apply PCA to fc2 weights
transformed_fc2_weights_l4_5030 = apply_pca_and_reshape(original_fc2_weights_l4_5030,
                                                        30)

# Instantiate the compressed model
PCA_local4_5030 = CompressedRegressionNN(58, 50, 30, 1)

# Assign the PCA-transformed weights
PCA_local4_5030.fc1.weight.data = torch.from_numpy(transformed_fc1_weights_l4_5030).float().view(50, 58)
PCA_local4_5030.fc2.weight.data = torch.from_numpy(transformed_fc2_weights_l4_5030).float().view(30, 50)

# Reset biases
PCA_local4_5030.fc1.bias.data.fill_(0)
PCA_local4_5030.fc2.bias.data.fill_(0)
PCA_local4_5030.fc3.bias.data.fill_(0)

# Validate the weight shapes
print("Transformed fc1 weight shape:", PCA_local4_5030.fc1.weight.shape)
print("Transformed fc2 weight shape:", PCA_local4_5030.fc2.weight.shape)
print("Transformed fc3 weight shape:", PCA_local4_5030.fc3.weight.shape)
print()
print(PCA_local4_5030)

In [ ]:
average_loss, average_mae = evaluate_model(PCA_local4_5030, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=PCA_local4_5030, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

###### fc1 out_features=50 and fc2 out_features=10

In [ ]:
# create copy of the locally pruned model
temp_l4_5010 = RegressionNN(input_size, hidden_size1,
                               hidden_size2, output_size)
transfer_pruned_weights(pruned_local4, temp_l4_5010)

In [ ]:
# weight shape of the loaded pruned model
print("fc1 weight shape:", temp_l4_5010.fc1.weight.shape)
print("fc2 weight shape:", temp_l4_5010.fc2.weight.shape)
print("fc3 weight shape:", temp_l4_5010.fc3.weight.shape)

In [ ]:
# Extract weights from the pruned model
original_fc1_weights_l4_5010 = temp_l4_5010.fc1.weight.data.cpu().numpy()

# Apply PCA to fc1 weights
transformed_fc1_weights_l4_5010 = apply_pca_and_reshape(original_fc1_weights_l4_5010,
                                                        50)

# Update fc2 in the pruned model to match new fc1 out_features
temp_l4_5010.fc2 = nn.Linear(50, hidden_size2)

# Extract fc2 weights after updating fc2 in the pruned model
original_fc2_weights_l4_5010 = temp_l4_5010.fc2.weight.data.cpu().numpy()

# Apply PCA to fc2 weights
transformed_fc2_weights_l4_5010 = apply_pca_and_reshape(original_fc2_weights_l4_5010,
                                                        10)

# Instantiate the compressed model
PCA_local4_5010 = CompressedRegressionNN(58, 50, 10, 1)

# Assign the PCA-transformed weights
PCA_local4_5010.fc1.weight.data = torch.from_numpy(transformed_fc1_weights_l4_5010).float().view(50, 58)
PCA_local4_5010.fc2.weight.data = torch.from_numpy(transformed_fc2_weights_l4_5010).float().view(10, 50)

# Reset biases
PCA_local4_5010.fc1.bias.data.fill_(0)
PCA_local4_5010.fc2.bias.data.fill_(0)
PCA_local4_5010.fc3.bias.data.fill_(0)

# Validate the weight shapes
print("Transformed fc1 weight shape:", PCA_local4_5010.fc1.weight.shape)
print("Transformed fc2 weight shape:", PCA_local4_5010.fc2.weight.shape)
print("Transformed fc3 weight shape:", PCA_local4_5010.fc3.weight.shape)
print()
print(PCA_local4_5010)

In [ ]:
average_loss, average_mae = evaluate_model(PCA_local4_5010, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=PCA_local4_5010, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

###### fc1 out_features=50 and fc2 out_features=5

In [ ]:
# create copy of the locally pruned model
temp_l4_505 = RegressionNN(input_size, hidden_size1,
                               hidden_size2, output_size)
transfer_pruned_weights(pruned_local4, temp_l4_505)

In [ ]:
# weight shape of the loaded pruned model
print("fc1 weight shape:", temp_l4_505.fc1.weight.shape)
print("fc2 weight shape:", temp_l4_505.fc2.weight.shape)
print("fc3 weight shape:", temp_l4_505.fc3.weight.shape)

In [ ]:
# Extract weights from the pruned model
original_fc1_weights_l4_505 = temp_l4_505.fc1.weight.data.cpu().numpy()

# Apply PCA to fc1 weights
transformed_fc1_weights_l4_505 = apply_pca_and_reshape(original_fc1_weights_l4_505,
                                                       50)

# Update fc2 in the pruned model to match new fc1 out_features
temp_l4_505.fc2 = nn.Linear(50, hidden_size2)

# Extract fc2 weights after updating fc2 in the pruned model
original_fc2_weights_l4_505 = temp_l4_505.fc2.weight.data.cpu().numpy()

# Apply PCA to fc2 weights
transformed_fc2_weights_l4_505 = apply_pca_and_reshape(original_fc2_weights_l4_505,
                                                       5)

# Instantiate the compressed model
PCA_local4_505 = CompressedRegressionNN(58, 50, 5, 1)

# Assign the PCA-transformed weights
PCA_local4_505.fc1.weight.data = torch.from_numpy(transformed_fc1_weights_l4_505).float().view(50, 58)
PCA_local4_505.fc2.weight.data = torch.from_numpy(transformed_fc2_weights_l4_505).float().view(5, 50)

# Reset biases
PCA_local4_505.fc1.bias.data.fill_(0)
PCA_local4_505.fc2.bias.data.fill_(0)
PCA_local4_505.fc3.bias.data.fill_(0)

# Validate the weight shapes
print("Transformed fc1 weight shape:", PCA_local4_505.fc1.weight.shape)
print("Transformed fc2 weight shape:", PCA_local4_505.fc2.weight.shape)
print("Transformed fc3 weight shape:", PCA_local4_505.fc3.weight.shape)
print()
print(PCA_local4_505)

In [ ]:
average_loss, average_mae = evaluate_model(PCA_local4_505, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=PCA_local4_505, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

###### fc1 out_features=30 and fc2 out_features=10

In [ ]:
# create copy of the locally pruned model
temp_l4_3010 = RegressionNN(input_size, hidden_size1,
                               hidden_size2, output_size)
transfer_pruned_weights(pruned_local4, temp_l4_3010)

In [ ]:
# weight shape of the loaded pruned model
print("fc1 weight shape:", temp_l4_3010.fc1.weight.shape)
print("fc2 weight shape:", temp_l4_3010.fc2.weight.shape)
print("fc3 weight shape:", temp_l4_3010.fc3.weight.shape)

In [ ]:
# Extract weights from the pruned model
original_fc1_weights_l4_3010 = temp_l4_3010.fc1.weight.data.cpu().numpy()

# Apply PCA to fc1 weights
transformed_fc1_weights_l4_3010 = apply_pca_and_reshape(original_fc1_weights_l4_3010,
                                                        30)

# Update fc2 in the pruned model to match new fc1 out_features
temp_l4_3010.fc2 = nn.Linear(30, hidden_size2)

# Extract fc2 weights after updating fc2 in the pruned model
original_fc2_weights_l4_3010 = temp_l4_3010.fc2.weight.data.cpu().numpy()

# Apply PCA to fc2 weights
transformed_fc2_weights_l4_3010 = apply_pca_and_reshape(original_fc2_weights_l4_3010,
                                                        10)

# Instantiate the compressed model
PCA_local4_3010 = CompressedRegressionNN(58, 30, 10, 1)

# Assign the PCA-transformed weights
PCA_local4_3010.fc1.weight.data = torch.from_numpy(transformed_fc1_weights_l4_3010).float().view(30, 58)
PCA_local4_3010.fc2.weight.data = torch.from_numpy(transformed_fc2_weights_l4_3010).float().view(10, 30)

# Reset biases
PCA_local4_3010.fc1.bias.data.fill_(0)
PCA_local4_3010.fc2.bias.data.fill_(0)
PCA_local4_3010.fc3.bias.data.fill_(0)

# Validate the weight shapes
print("Transformed fc1 weight shape:", PCA_local4_3010.fc1.weight.shape)
print("Transformed fc2 weight shape:", PCA_local4_3010.fc2.weight.shape)
print("Transformed fc3 weight shape:", PCA_local4_3010.fc3.weight.shape)
print()
print(PCA_local4_3010)

In [ ]:
average_loss, average_mae = evaluate_model(PCA_local4_3010, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=PCA_local4_3010, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

###### fc1 out_features=30 and fc2 out_features=5

In [ ]:
# create copy of the locally pruned model
temp_l4_305 = RegressionNN(input_size, hidden_size1,
                           hidden_size2, output_size)
transfer_pruned_weights(pruned_local4, temp_l4_305)

In [ ]:
# weight shape of the loaded pruned model
print("fc1 weight shape:", temp_l4_305.fc1.weight.shape)
print("fc2 weight shape:", temp_l4_305.fc2.weight.shape)
print("fc3 weight shape:", temp_l4_305.fc3.weight.shape)

In [ ]:
# Extract weights from the pruned model
original_fc1_weights_l4_305 = temp_l4_305.fc1.weight.data.cpu().numpy()

# Apply PCA to fc1 weights
transformed_fc1_weights_l4_305 = apply_pca_and_reshape(original_fc1_weights_l4_305,
                                                       30)

# Update fc2 in the pruned model to match new fc1 out_features
temp_l4_305.fc2 = nn.Linear(30, hidden_size2)

# Extract fc2 weights after updating fc2 in the pruned model
original_fc2_weights_l4_305 = temp_l4_305.fc2.weight.data.cpu().numpy()

# Apply PCA to fc2 weights
transformed_fc2_weights_l4_305 = apply_pca_and_reshape(original_fc2_weights_l4_305,
                                                        5)

# Instantiate the compressed model
PCA_local4_305 = CompressedRegressionNN(58, 30, 5, 1)

# Assign the PCA-transformed weights
PCA_local4_305.fc1.weight.data = torch.from_numpy(transformed_fc1_weights_l4_305).float().view(30, 58)
PCA_local4_305.fc2.weight.data = torch.from_numpy(transformed_fc2_weights_l4_305).float().view(5, 30)

# Reset biases
PCA_local4_305.fc1.bias.data.fill_(0)
PCA_local4_305.fc2.bias.data.fill_(0)
PCA_local4_305.fc3.bias.data.fill_(0)

# Validate the weight shapes
print("Transformed fc1 weight shape:", PCA_local4_305.fc1.weight.shape)
print("Transformed fc2 weight shape:", PCA_local4_305.fc2.weight.shape)
print("Transformed fc3 weight shape:", PCA_local4_305.fc3.weight.shape)
print()
print(PCA_local4_305)

In [ ]:
average_loss, average_mae = evaluate_model(PCA_local4_305, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=PCA_local4_305, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

###### fc1 out_features=10 and fc2 out_features=5

In [ ]:
# create copy of the locally pruned model
temp_l4_105 = RegressionNN(input_size, hidden_size1,
                               hidden_size2, output_size)
transfer_pruned_weights(pruned_local4, temp_l4_105)

In [ ]:
# weight shape of the loaded pruned model
print("fc1 weight shape:", temp_l4_105.fc1.weight.shape)
print("fc2 weight shape:", temp_l4_105.fc2.weight.shape)
print("fc3 weight shape:", temp_l4_105.fc3.weight.shape)

In [ ]:
# Extract weights from the pruned model
original_fc1_weights_l4_105 = temp_l4_105.fc1.weight.data.cpu().numpy()

# Apply PCA to fc1 weights
transformed_fc1_weights_l4_105 = apply_pca_and_reshape(original_fc1_weights_l4_105,
                                                       10)

# Update fc2 in the pruned model to match new fc1 out_features
temp_l4_105.fc2 = nn.Linear(10, hidden_size2)

# Extract fc2 weights after updating fc2 in the pruned model
original_fc2_weights_l4_105 = temp_l4_105.fc2.weight.data.cpu().numpy()

# Apply PCA to fc2 weights
transformed_fc2_weights_l4_105 = apply_pca_and_reshape(original_fc2_weights_l4_105,
                                                        5)

# Instantiate the compressed model
PCA_local4_105 = CompressedRegressionNN(58, 10, 5, 1)

# Assign the PCA-transformed weights
PCA_local4_105.fc1.weight.data = torch.from_numpy(transformed_fc1_weights_l4_105).float().view(10, 58)
PCA_local4_105.fc2.weight.data = torch.from_numpy(transformed_fc2_weights_l4_105).float().view(5, 10)

# Reset biases
PCA_local4_105.fc1.bias.data.fill_(0)
PCA_local4_105.fc2.bias.data.fill_(0)
PCA_local4_105.fc3.bias.data.fill_(0)

# Validate the weight shapes
print("Transformed fc1 weight shape:", PCA_local4_105.fc1.weight.shape)
print("Transformed fc2 weight shape:", PCA_local4_105.fc2.weight.shape)
print("Transformed fc3 weight shape:", PCA_local4_105.fc3.weight.shape)
print()
print(PCA_local4_105)

In [ ]:
average_loss, average_mae = evaluate_model(PCA_local4_105, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=PCA_local4_105, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

#### PCA on the Pruned Model, `pruned_local5`, with Pruning Amount=0.95

###### fc1 out_features=50 and fc2 out_features=30

In [ ]:
# create copy of the locally pruned model
temp_l5_5030 = RegressionNN(input_size, hidden_size1,
                               hidden_size2, output_size)
transfer_pruned_weights(pruned_local5, temp_l5_5030)

In [ ]:
# weight shape of the loaded pruned model
print("fc1 weight shape:", temp_l5_5030.fc1.weight.shape)
print("fc2 weight shape:", temp_l5_5030.fc2.weight.shape)
print("fc3 weight shape:", temp_l5_5030.fc3.weight.shape)

In [ ]:
# Extract weights from the pruned model
original_fc1_weights_l5_5030 = temp_l5_5030.fc1.weight.data.cpu().numpy()

# Apply PCA to fc1 weights
transformed_fc1_weights_l5_5030 = apply_pca_and_reshape(original_fc1_weights_l5_5030,
                                                        50)

# Update fc2 in the pruned model to match new fc1 out_features
temp_l5_5030.fc2 = nn.Linear(50, hidden_size2)

# Extract fc2 weights after updating fc2 in the pruned model
original_fc2_weights_l5_5030 = temp_l5_5030.fc2.weight.data.cpu().numpy()

# Apply PCA to fc2 weights
transformed_fc2_weights_l5_5030 = apply_pca_and_reshape(original_fc2_weights_l5_5030,
                                                        30)

# Instantiate the compressed model
PCA_local5_5030 = CompressedRegressionNN(58, 50, 30, 1)

# Assign the PCA-transformed weights
PCA_local5_5030.fc1.weight.data = torch.from_numpy(transformed_fc1_weights_l5_5030).float().view(50, 58)
PCA_local5_5030.fc2.weight.data = torch.from_numpy(transformed_fc2_weights_l5_5030).float().view(30, 50)

# Reset biases
PCA_local5_5030.fc1.bias.data.fill_(0)
PCA_local5_5030.fc2.bias.data.fill_(0)
PCA_local5_5030.fc3.bias.data.fill_(0)

# Validate the weight shapes
print("Transformed fc1 weight shape:", PCA_local5_5030.fc1.weight.shape)
print("Transformed fc2 weight shape:", PCA_local5_5030.fc2.weight.shape)
print("Transformed fc3 weight shape:", PCA_local5_5030.fc3.weight.shape)
print()
print(PCA_local5_5030)

In [ ]:
average_loss, average_mae = evaluate_model(PCA_local5_5030, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=PCA_local5_5030, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

###### fc1 out_features=50 and fc2 out_features=10

In [ ]:
# create copy of the locally pruned model
temp_l5_5010 = RegressionNN(input_size, hidden_size1,
                               hidden_size2, output_size)
transfer_pruned_weights(pruned_local5, temp_l5_5010)

In [ ]:
# weight shape of the loaded pruned model
print("fc1 weight shape:", temp_l5_5010.fc1.weight.shape)
print("fc2 weight shape:", temp_l5_5010.fc2.weight.shape)
print("fc3 weight shape:", temp_l5_5010.fc3.weight.shape)

In [ ]:
# Extract weights from the pruned model
original_fc1_weights_l5_5010 = temp_l5_5010.fc1.weight.data.cpu().numpy()

# Apply PCA to fc1 weights
transformed_fc1_weights_l5_5010 = apply_pca_and_reshape(original_fc1_weights_l5_5010,
                                                        50)

# Update fc2 in the pruned model to match new fc1 out_features
temp_l5_5010.fc2 = nn.Linear(50, hidden_size2)

# Extract fc2 weights after updating fc2 in the pruned model
original_fc2_weights_l5_5010 = temp_l5_5010.fc2.weight.data.cpu().numpy()

# Apply PCA to fc2 weights
transformed_fc2_weights_l5_5010 = apply_pca_and_reshape(original_fc2_weights_l5_5010,
                                                        10)

# Instantiate the compressed model
PCA_local5_5010 = CompressedRegressionNN(58, 50, 10, 1)

# Assign the PCA-transformed weights
PCA_local5_5010.fc1.weight.data = torch.from_numpy(transformed_fc1_weights_l5_5010).float().view(50, 58)
PCA_local5_5010.fc2.weight.data = torch.from_numpy(transformed_fc2_weights_l5_5010).float().view(10, 50)

# Reset biases
PCA_local5_5010.fc1.bias.data.fill_(0)
PCA_local5_5010.fc2.bias.data.fill_(0)
PCA_local5_5010.fc3.bias.data.fill_(0)

# Validate the weight shapes
print("Transformed fc1 weight shape:", PCA_local5_5010.fc1.weight.shape)
print("Transformed fc2 weight shape:", PCA_local5_5010.fc2.weight.shape)
print("Transformed fc3 weight shape:", PCA_local5_5010.fc3.weight.shape)
print()
print(PCA_local5_5010)

In [ ]:
average_loss, average_mae = evaluate_model(PCA_local5_5010, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=PCA_local5_5010, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

###### fc1 out_features=50 and fc2 out_features=5

In [ ]:
# create copy of the locally pruned model
temp_l5_505 = RegressionNN(input_size, hidden_size1,
                               hidden_size2, output_size)
transfer_pruned_weights(pruned_local5, temp_l5_505)

In [ ]:
# weight shape of the loaded pruned model
print("fc1 weight shape:", temp_l5_505.fc1.weight.shape)
print("fc2 weight shape:", temp_l5_505.fc2.weight.shape)
print("fc3 weight shape:", temp_l5_505.fc3.weight.shape)

In [ ]:
# Extract weights from the pruned model
original_fc1_weights_l5_505 = temp_l5_505.fc1.weight.data.cpu().numpy()

# Apply PCA to fc1 weights
transformed_fc1_weights_l5_505 = apply_pca_and_reshape(original_fc1_weights_l5_505,
                                                       50)

# Update fc2 in the pruned model to match new fc1 out_features
temp_l5_505.fc2 = nn.Linear(50, hidden_size2)

# Extract fc2 weights after updating fc2 in the pruned model
original_fc2_weights_l5_505 = temp_l5_505.fc2.weight.data.cpu().numpy()

# Apply PCA to fc2 weights
transformed_fc2_weights_l5_505 = apply_pca_and_reshape(original_fc2_weights_l5_505,
                                                       5)

# Instantiate the compressed model
PCA_local5_505 = CompressedRegressionNN(58, 50, 5, 1)

# Assign the PCA-transformed weights
PCA_local5_505.fc1.weight.data = torch.from_numpy(transformed_fc1_weights_l5_505).float().view(50, 58)
PCA_local5_505.fc2.weight.data = torch.from_numpy(transformed_fc2_weights_l5_505).float().view(5, 50)

# Reset biases
PCA_local5_505.fc1.bias.data.fill_(0)
PCA_local5_505.fc2.bias.data.fill_(0)
PCA_local5_505.fc3.bias.data.fill_(0)

# Validate the weight shapes
print("Transformed fc1 weight shape:", PCA_local5_505.fc1.weight.shape)
print("Transformed fc2 weight shape:", PCA_local5_505.fc2.weight.shape)
print("Transformed fc3 weight shape:", PCA_local5_505.fc3.weight.shape)
print()
print(PCA_local5_505)

In [ ]:
average_loss, average_mae = evaluate_model(PCA_local5_505, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=PCA_local5_505, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

###### fc1 out_features=30 and fc2 out_features=10

In [ ]:
# create copy of the locally pruned model
temp_l5_3010 = RegressionNN(input_size, hidden_size1,
                               hidden_size2, output_size)
transfer_pruned_weights(pruned_local5, temp_l5_3010)

In [ ]:
# weight shape of the loaded pruned model
print("fc1 weight shape:", temp_l5_3010.fc1.weight.shape)
print("fc2 weight shape:", temp_l5_3010.fc2.weight.shape)
print("fc3 weight shape:", temp_l5_3010.fc3.weight.shape)

In [ ]:
# Extract weights from the pruned model
original_fc1_weights_l5_3010 = temp_l5_3010.fc1.weight.data.cpu().numpy()

# Apply PCA to fc1 weights
transformed_fc1_weights_l5_3010 = apply_pca_and_reshape(original_fc1_weights_l5_3010,
                                                        30)

# Update fc2 in the pruned model to match new fc1 out_features
temp_l5_3010.fc2 = nn.Linear(30, hidden_size2)

# Extract fc2 weights after updating fc2 in the pruned model
original_fc2_weights_l5_3010 = temp_l5_3010.fc2.weight.data.cpu().numpy()

# Apply PCA to fc2 weights
transformed_fc2_weights_l5_3010 = apply_pca_and_reshape(original_fc2_weights_l5_3010,
                                                        10)

# Instantiate the compressed model
PCA_local5_3010 = CompressedRegressionNN(58, 30, 10, 1)

# Assign the PCA-transformed weights
PCA_local5_3010.fc1.weight.data = torch.from_numpy(transformed_fc1_weights_l5_3010).float().view(30, 58)
PCA_local5_3010.fc2.weight.data = torch.from_numpy(transformed_fc2_weights_l5_3010).float().view(10, 30)

# Reset biases
PCA_local5_3010.fc1.bias.data.fill_(0)
PCA_local5_3010.fc2.bias.data.fill_(0)
PCA_local5_3010.fc3.bias.data.fill_(0)

# Validate the weight shapes
print("Transformed fc1 weight shape:", PCA_local5_3010.fc1.weight.shape)
print("Transformed fc2 weight shape:", PCA_local5_3010.fc2.weight.shape)
print("Transformed fc3 weight shape:", PCA_local5_3010.fc3.weight.shape)
print()
print(PCA_local5_3010)

In [ ]:
average_loss, average_mae = evaluate_model(PCA_local5_3010, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=PCA_local5_3010, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

###### fc1 out_features=30 and fc2 out_features=5

In [ ]:
# create copy of the locally pruned model
temp_l5_305 = RegressionNN(input_size, hidden_size1,
                           hidden_size2, output_size)
transfer_pruned_weights(pruned_local5, temp_l5_305)

In [ ]:
# weight shape of the loaded pruned model
print("fc1 weight shape:", temp_l5_305.fc1.weight.shape)
print("fc2 weight shape:", temp_l5_305.fc2.weight.shape)
print("fc3 weight shape:", temp_l5_305.fc3.weight.shape)

In [ ]:
# Extract weights from the pruned model
original_fc1_weights_l5_305 = temp_l5_305.fc1.weight.data.cpu().numpy()

# Apply PCA to fc1 weights
transformed_fc1_weights_l5_305 = apply_pca_and_reshape(original_fc1_weights_l5_305,
                                                       30)

# Update fc2 in the pruned model to match new fc1 out_features
temp_l5_305.fc2 = nn.Linear(30, hidden_size2)

# Extract fc2 weights after updating fc2 in the pruned model
original_fc2_weights_l5_305 = temp_l5_305.fc2.weight.data.cpu().numpy()

# Apply PCA to fc2 weights
transformed_fc2_weights_l5_305 = apply_pca_and_reshape(original_fc2_weights_l5_305,
                                                        5)

# Instantiate the compressed model
PCA_local5_305 = CompressedRegressionNN(58, 30, 5, 1)

# Assign the PCA-transformed weights
PCA_local5_305.fc1.weight.data = torch.from_numpy(transformed_fc1_weights_l5_305).float().view(30, 58)
PCA_local5_305.fc2.weight.data = torch.from_numpy(transformed_fc2_weights_l5_305).float().view(5, 30)

# Reset biases
PCA_local5_305.fc1.bias.data.fill_(0)
PCA_local5_305.fc2.bias.data.fill_(0)
PCA_local5_305.fc3.bias.data.fill_(0)

# Validate the weight shapes
print("Transformed fc1 weight shape:", PCA_local5_305.fc1.weight.shape)
print("Transformed fc2 weight shape:", PCA_local5_305.fc2.weight.shape)
print("Transformed fc3 weight shape:", PCA_local5_305.fc3.weight.shape)
print()
print(PCA_local5_305)

In [ ]:
average_loss, average_mae = evaluate_model(PCA_local5_305, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=PCA_local5_305, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

###### fc1 out_features=10 and fc2 out_features=5

In [ ]:
# create copy of the locally pruned model
temp_l5_105 = RegressionNN(input_size, hidden_size1,
                               hidden_size2, output_size)
transfer_pruned_weights(pruned_local5, temp_l5_105)

In [ ]:
# weight shape of the loaded pruned model
print("fc1 weight shape:", temp_l5_105.fc1.weight.shape)
print("fc2 weight shape:", temp_l5_105.fc2.weight.shape)
print("fc3 weight shape:", temp_l5_105.fc3.weight.shape)

In [ ]:
# Extract weights from the pruned model
original_fc1_weights_l5_105 = temp_l5_105.fc1.weight.data.cpu().numpy()

# Apply PCA to fc1 weights
transformed_fc1_weights_l5_105 = apply_pca_and_reshape(original_fc1_weights_l5_105,
                                                       10)

# Update fc2 in the pruned model to match new fc1 out_features
temp_l5_105.fc2 = nn.Linear(10, hidden_size2)

# Extract fc2 weights after updating fc2 in the pruned model
original_fc2_weights_l5_105 = temp_l5_105.fc2.weight.data.cpu().numpy()

# Apply PCA to fc2 weights
transformed_fc2_weights_l5_105 = apply_pca_and_reshape(original_fc2_weights_l5_105,
                                                        5)

# Instantiate the compressed model
PCA_local5_105 = CompressedRegressionNN(58, 10, 5, 1)

# Assign the PCA-transformed weights
PCA_local5_105.fc1.weight.data = torch.from_numpy(transformed_fc1_weights_l5_105).float().view(10, 58)
PCA_local5_105.fc2.weight.data = torch.from_numpy(transformed_fc2_weights_l5_105).float().view(5, 10)

# Reset biases
PCA_local5_105.fc1.bias.data.fill_(0)
PCA_local5_105.fc2.bias.data.fill_(0)
PCA_local5_105.fc3.bias.data.fill_(0)

# Validate the weight shapes
print("Transformed fc1 weight shape:", PCA_local5_105.fc1.weight.shape)
print("Transformed fc2 weight shape:", PCA_local5_105.fc2.weight.shape)
print("Transformed fc3 weight shape:", PCA_local5_105.fc3.weight.shape)
print()
print(PCA_local5_105)

In [ ]:
average_loss, average_mae = evaluate_model(PCA_local5_105, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=PCA_local5_105, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

### PCA on Layer-Wise Locally Pruned Models

#### PCA on the Pruned Model, `pruned_sparse1`, with Pruning Amount=0.6

###### fc1 out_features=50 and fc2 out_features=30

In [ ]:
# create copy of the locally pruned model
temp_sl1_5030 = RegressionNN(input_size, hidden_size1,
                               hidden_size2, output_size)
transfer_pruned_weights(pruned_sparse1, temp_sl1_5030)

In [ ]:
# weight shape of the loaded pruned model
print("fc1 weight shape:", temp_sl1_5030.fc1.weight.shape)
print("fc2 weight shape:", temp_sl1_5030.fc2.weight.shape)
print("fc3 weight shape:", temp_sl1_5030.fc3.weight.shape)

In [ ]:
# Extract weights from the pruned model
original_fc1_weights_sl1_5030 = temp_sl1_5030.fc1.weight.data.cpu().numpy()

# Apply PCA to fc1 weights
transformed_fc1_weights_sl1_5030 = apply_pca_and_reshape(original_fc1_weights_sl1_5030,
                                                        50)

# Update fc2 in the pruned model to match new fc1 out_features
temp_sl1_5030.fc2 = nn.Linear(50, hidden_size2)

# Extract fc2 weights after updating fc2 in the pruned model
original_fc2_weights_sl1_5030 = temp_sl1_5030.fc2.weight.data.cpu().numpy()

# Apply PCA to fc2 weights
transformed_fc2_weights_sl1_5030 = apply_pca_and_reshape(original_fc2_weights_sl1_5030,
                                                        30)

# Instantiate the compressed model
PCA_sparse1_5030 = CompressedRegressionNN(58, 50, 30, 1)

# Assign the PCA-transformed weights
PCA_sparse1_5030.fc1.weight.data = torch.from_numpy(transformed_fc1_weights_sl1_5030).float().view(50, 58)
PCA_sparse1_5030.fc2.weight.data = torch.from_numpy(transformed_fc2_weights_sl1_5030).float().view(30, 50)

# Reset biases
PCA_sparse1_5030.fc1.bias.data.fill_(0)
PCA_sparse1_5030.fc2.bias.data.fill_(0)
PCA_sparse1_5030.fc3.bias.data.fill_(0)

# Validate the weight shapes
print("Transformed fc1 weight shape:", PCA_sparse1_5030.fc1.weight.shape)
print("Transformed fc2 weight shape:", PCA_sparse1_5030.fc2.weight.shape)
print("Transformed fc3 weight shape:", PCA_sparse1_5030.fc3.weight.shape)
print()
print(PCA_sparse1_5030)

In [ ]:
average_loss, average_mae = evaluate_model(PCA_sparse1_5030, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=PCA_sparse1_5030, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

###### fc1 out_features=50 and fc2 out_features=10

In [ ]:
# create copy of the locally pruned model
temp_sl1_5010 = RegressionNN(input_size, hidden_size1,
                               hidden_size2, output_size)
transfer_pruned_weights(pruned_sparse1, temp_sl1_5010)

In [ ]:
# weight shape of the loaded pruned model
print("fc1 weight shape:", temp_sl1_5010.fc1.weight.shape)
print("fc2 weight shape:", temp_sl1_5010.fc2.weight.shape)
print("fc3 weight shape:", temp_sl1_5010.fc3.weight.shape)

In [ ]:
# Extract weights from the pruned model
original_fc1_weights_sl1_5010 = temp_sl1_5010.fc1.weight.data.cpu().numpy()

# Apply PCA to fc1 weights
transformed_fc1_weights_sl1_5010 = apply_pca_and_reshape(original_fc1_weights_sl1_5010,
                                                        50)

# Update fc2 in the pruned model to match new fc1 out_features
temp_sl1_5010.fc2 = nn.Linear(50, hidden_size2)

# Extract fc2 weights after updating fc2 in the pruned model
original_fc2_weights_sl1_5010 = temp_sl1_5010.fc2.weight.data.cpu().numpy()

# Apply PCA to fc2 weights
transformed_fc2_weights_sl1_5010 = apply_pca_and_reshape(original_fc2_weights_sl1_5010,
                                                        10)

# Instantiate the compressed model
PCA_sparse1_5010 = CompressedRegressionNN(58, 50, 10, 1)

# Assign the PCA-transformed weights
PCA_sparse1_5010.fc1.weight.data = torch.from_numpy(transformed_fc1_weights_sl1_5010).float().view(50, 58)
PCA_sparse1_5010.fc2.weight.data = torch.from_numpy(transformed_fc2_weights_sl1_5010).float().view(10, 50)

# Reset biases
PCA_sparse1_5010.fc1.bias.data.fill_(0)
PCA_sparse1_5010.fc2.bias.data.fill_(0)
PCA_sparse1_5010.fc3.bias.data.fill_(0)

# Validate the weight shapes
print("Transformed fc1 weight shape:", PCA_sparse1_5010.fc1.weight.shape)
print("Transformed fc2 weight shape:", PCA_sparse1_5010.fc2.weight.shape)
print("Transformed fc3 weight shape:", PCA_sparse1_5010.fc3.weight.shape)
print()
print(PCA_sparse1_5010)

In [ ]:
average_loss, average_mae = evaluate_model(PCA_sparse1_5010, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=PCA_sparse1_5010, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

###### fc1 out_features=50 and fc2 out_features=5

In [ ]:
# create copy of the locally pruned model
temp_sl1_505 = RegressionNN(input_size, hidden_size1,
                               hidden_size2, output_size)
transfer_pruned_weights(pruned_sparse1, temp_sl1_505)

In [ ]:
# weight shape of the loaded pruned model
print("fc1 weight shape:", temp_sl1_505.fc1.weight.shape)
print("fc2 weight shape:", temp_sl1_505.fc2.weight.shape)
print("fc3 weight shape:", temp_sl1_505.fc3.weight.shape)

In [ ]:
# Extract weights from the pruned model
original_fc1_weights_sl1_505 = temp_sl1_505.fc1.weight.data.cpu().numpy()

# Apply PCA to fc1 weights
transformed_fc1_weights_sl1_505 = apply_pca_and_reshape(original_fc1_weights_sl1_505,
                                                       50)

# Update fc2 in the pruned model to match new fc1 out_features
temp_sl1_505.fc2 = nn.Linear(50, hidden_size2)

# Extract fc2 weights after updating fc2 in the pruned model
original_fc2_weights_sl1_505 = temp_sl1_505.fc2.weight.data.cpu().numpy()

# Apply PCA to fc2 weights
transformed_fc2_weights_sl1_505 = apply_pca_and_reshape(original_fc2_weights_sl1_505,
                                                       5)

# Instantiate the compressed model
PCA_sparse1_505 = CompressedRegressionNN(58, 50, 5, 1)

# Assign the PCA-transformed weights
PCA_sparse1_505.fc1.weight.data = torch.from_numpy(transformed_fc1_weights_sl1_505).float().view(50, 58)
PCA_sparse1_505.fc2.weight.data = torch.from_numpy(transformed_fc2_weights_sl1_505).float().view(5, 50)

# Reset biases
PCA_sparse1_505.fc1.bias.data.fill_(0)
PCA_sparse1_505.fc2.bias.data.fill_(0)
PCA_sparse1_505.fc3.bias.data.fill_(0)

# Validate the weight shapes
print("Transformed fc1 weight shape:", PCA_sparse1_505.fc1.weight.shape)
print("Transformed fc2 weight shape:", PCA_sparse1_505.fc2.weight.shape)
print("Transformed fc3 weight shape:", PCA_sparse1_505.fc3.weight.shape)
print()
print(PCA_sparse1_505)

In [ ]:
average_loss, average_mae = evaluate_model(PCA_sparse1_505, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=PCA_sparse1_505, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

###### fc1 out_features=30 and fc2 out_features=10

In [ ]:
# create copy of the locally pruned model
temp_sl1_3010 = RegressionNN(input_size, hidden_size1,
                               hidden_size2, output_size)
transfer_pruned_weights(pruned_sparse1, temp_sl1_3010)

In [ ]:
# weight shape of the loaded pruned model
print("fc1 weight shape:", temp_l1_3010.fc1.weight.shape)
print("fc2 weight shape:", temp_l1_3010.fc2.weight.shape)
print("fc3 weight shape:", temp_l1_3010.fc3.weight.shape)

In [ ]:
# Extract weights from the pruned model
original_fc1_weights_sl1_3010 = temp_l1_3010.fc1.weight.data.cpu().numpy()

# Apply PCA to fc1 weights
transformed_fc1_weights_sl1_3010 = apply_pca_and_reshape(original_fc1_weights_sl1_3010,
                                                        30)

# Update fc2 in the pruned model to match new fc1 out_features
temp_l1_3010.fc2 = nn.Linear(30, hidden_size2)

# Extract fc2 weights after updating fc2 in the pruned model
original_fc2_weights_sl1_3010 = temp_l1_3010.fc2.weight.data.cpu().numpy()

# Apply PCA to fc2 weights
transformed_fc2_weights_sl1_3010 = apply_pca_and_reshape(original_fc2_weights_sl1_3010,
                                                        10)

# Instantiate the compressed model
PCA_sparse1_3010 = CompressedRegressionNN(58, 30, 10, 1)

# Assign the PCA-transformed weights
PCA_sparse1_3010.fc1.weight.data = torch.from_numpy(transformed_fc1_weights_sl1_3010).float().view(30, 58)
PCA_sparse1_3010.fc2.weight.data = torch.from_numpy(transformed_fc2_weights_sl1_3010).float().view(10, 30)

# Reset biases
PCA_sparse1_3010.fc1.bias.data.fill_(0)
PCA_sparse1_3010.fc2.bias.data.fill_(0)
PCA_sparse1_3010.fc3.bias.data.fill_(0)

# Validate the weight shapes
print("Transformed fc1 weight shape:", PCA_sparse1_3010.fc1.weight.shape)
print("Transformed fc2 weight shape:", PCA_sparse1_3010.fc2.weight.shape)
print("Transformed fc3 weight shape:", PCA_sparse1_3010.fc3.weight.shape)
print()
print(PCA_sparse1_3010)

In [ ]:
average_loss, average_mae = evaluate_model(PCA_sparse1_3010, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=PCA_sparse1_3010, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

###### fc1 out_features=30 and fc2 out_features=5

In [ ]:
# create copy of the locally pruned model
temp_sl1_305 = RegressionNN(input_size, hidden_size1,
                           hidden_size2, output_size)
transfer_pruned_weights(pruned_sparse1, temp_sl1_305)

In [ ]:
# weight shape of the loaded pruned model
print("fc1 weight shape:", temp_sl1_305.fc1.weight.shape)
print("fc2 weight shape:", temp_sl1_305.fc2.weight.shape)
print("fc3 weight shape:", temp_sl1_305.fc3.weight.shape)

In [ ]:
# Extract weights from the pruned model
original_fc1_weights_sl1_305 = temp_sl1_305.fc1.weight.data.cpu().numpy()

# Apply PCA to fc1 weights
transformed_fc1_weights_sl1_305 = apply_pca_and_reshape(original_fc1_weights_sl1_305,
                                                       30)

# Update fc2 in the pruned model to match new fc1 out_features
temp_sl1_305.fc2 = nn.Linear(30, hidden_size2)

# Extract fc2 weights after updating fc2 in the pruned model
original_fc2_weights_sl1_305 = temp_sl1_305.fc2.weight.data.cpu().numpy()

# Apply PCA to fc2 weights
transformed_fc2_weights_sl1_305 = apply_pca_and_reshape(original_fc2_weights_sl1_305,
                                                        5)

# Instantiate the compressed model
PCA_sparse1_305 = CompressedRegressionNN(58, 30, 5, 1)

# Assign the PCA-transformed weights
PCA_sparse1_305.fc1.weight.data = torch.from_numpy(transformed_fc1_weights_sl1_305).float().view(30, 58)
PCA_sparse1_305.fc2.weight.data = torch.from_numpy(transformed_fc2_weights_sl1_305).float().view(5, 30)

# Reset biases
PCA_sparse1_305.fc1.bias.data.fill_(0)
PCA_sparse1_305.fc2.bias.data.fill_(0)
PCA_sparse1_305.fc3.bias.data.fill_(0)

# Validate the weight shapes
print("Transformed fc1 weight shape:", PCA_sparse1_305.fc1.weight.shape)
print("Transformed fc2 weight shape:", PCA_sparse1_305.fc2.weight.shape)
print("Transformed fc3 weight shape:", PCA_sparse1_305.fc3.weight.shape)
print()
print(PCA_sparse1_305)

In [ ]:
average_loss, average_mae = evaluate_model(PCA_sparse1_305, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=PCA_sparse1_305, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

###### fc1 out_features=10 and fc2 out_features=5

In [ ]:
# create copy of the locally pruned model
temp_sl1_105 = RegressionNN(input_size, hidden_size1,
                               hidden_size2, output_size)
transfer_pruned_weights(pruned_sparse1, temp_sl1_105)

In [ ]:
# weight shape of the loaded pruned model
print("fc1 weight shape:", temp_sl1_105.fc1.weight.shape)
print("fc2 weight shape:", temp_sl1_105.fc2.weight.shape)
print("fc3 weight shape:", temp_sl1_105.fc3.weight.shape)

In [ ]:
# Extract weights from the pruned model
original_fc1_weights_sl1_105 = temp_sl1_105.fc1.weight.data.cpu().numpy()

# Apply PCA to fc1 weights
transformed_fc1_weights_sl1_105 = apply_pca_and_reshape(original_fc1_weights_sl1_105,
                                                       10)

# Update fc2 in the pruned model to match new fc1 out_features
temp_sl1_105.fc2 = nn.Linear(10, hidden_size2)

# Extract fc2 weights after updating fc2 in the pruned model
original_fc2_weights_sl1_105 = temp_sl1_105.fc2.weight.data.cpu().numpy()

# Apply PCA to fc2 weights
transformed_fc2_weights_sl1_105 = apply_pca_and_reshape(original_fc2_weights_sl1_105,
                                                        5)

# Instantiate the compressed model
PCA_sparse1_105 = CompressedRegressionNN(58, 10, 5, 1)

# Assign the PCA-transformed weights
PCA_sparse1_105.fc1.weight.data = torch.from_numpy(transformed_fc1_weights_sl1_105).float().view(10, 58)
PCA_sparse1_105.fc2.weight.data = torch.from_numpy(transformed_fc2_weights_sl1_105).float().view(5, 10)

# Reset biases
PCA_sparse1_105.fc1.bias.data.fill_(0)
PCA_sparse1_105.fc2.bias.data.fill_(0)
PCA_sparse1_105.fc3.bias.data.fill_(0)

# Validate the weight shapes
print("Transformed fc1 weight shape:", PCA_sparse1_105.fc1.weight.shape)
print("Transformed fc2 weight shape:", PCA_sparse1_105.fc2.weight.shape)
print("Transformed fc3 weight shape:", PCA_sparse1_105.fc3.weight.shape)
print()
print(PCA_sparse1_105)

In [ ]:
average_loss, average_mae = evaluate_model(PCA_sparse1_105, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=PCA_sparse1_105, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

#### PCA on the Pruned Model, `pruned_sparse2`, with Pruning Amount=0.7

###### fc1 out_features=50 and fc2 out_features=30

In [ ]:
# create copy of the locally pruned model
temp_sl2_5030 = RegressionNN(input_size, hidden_size1,
                               hidden_size2, output_size)
transfer_pruned_weights(pruned_sparse2, temp_sl2_5030)

In [ ]:
# weight shape of the loaded pruned model
print("fc1 weight shape:", temp_sl2_5030.fc1.weight.shape)
print("fc2 weight shape:", temp_sl2_5030.fc2.weight.shape)
print("fc3 weight shape:", temp_sl2_5030.fc3.weight.shape)

In [ ]:
# Extract weights from the pruned model
original_fc1_weights_sl2_5030 = temp_sl2_5030.fc1.weight.data.cpu().numpy()

# Apply PCA to fc1 weights
transformed_fc1_weights_sl2_5030 = apply_pca_and_reshape(original_fc1_weights_sl2_5030,
                                                        50)

# Update fc2 in the pruned model to match new fc1 out_features
temp_sl2_5030.fc2 = nn.Linear(50, hidden_size2)

# Extract fc2 weights after updating fc2 in the pruned model
original_fc2_weights_sl2_5030 = temp_sl2_5030.fc2.weight.data.cpu().numpy()

# Apply PCA to fc2 weights
transformed_fc2_weights_sl2_5030 = apply_pca_and_reshape(original_fc2_weights_sl2_5030,
                                                        30)

# Instantiate the compressed model
PCA_sparse2_5030 = CompressedRegressionNN(58, 50, 30, 1)

# Assign the PCA-transformed weights
PCA_sparse2_5030.fc1.weight.data = torch.from_numpy(transformed_fc1_weights_sl2_5030).float().view(50, 58)
PCA_sparse2_5030.fc2.weight.data = torch.from_numpy(transformed_fc2_weights_sl2_5030).float().view(30, 50)

# Reset biases
PCA_sparse2_5030.fc1.bias.data.fill_(0)
PCA_sparse2_5030.fc2.bias.data.fill_(0)
PCA_sparse2_5030.fc3.bias.data.fill_(0)

# Validate the weight shapes
print("Transformed fc1 weight shape:", PCA_sparse2_5030.fc1.weight.shape)
print("Transformed fc2 weight shape:", PCA_sparse2_5030.fc2.weight.shape)
print("Transformed fc3 weight shape:", PCA_sparse2_5030.fc3.weight.shape)
print()
print(PCA_sparse2_5030)

In [ ]:
average_loss, average_mae = evaluate_model(PCA_sparse2_5030, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=PCA_sparse2_5030, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

###### fc1 out_features=50 and fc2 out_features=10

In [ ]:
# create copy of the locally pruned model
temp_sl2_5010 = RegressionNN(input_size, hidden_size1,
                               hidden_size2, output_size)
transfer_pruned_weights(pruned_sparse2, temp_sl2_5010)

In [ ]:
# weight shape of the loaded pruned model
print("fc1 weight shape:", temp_sl2_5010.fc1.weight.shape)
print("fc2 weight shape:", temp_sl2_5010.fc2.weight.shape)
print("fc3 weight shape:", temp_sl2_5010.fc3.weight.shape)

In [ ]:
# Extract weights from the pruned model
original_fc1_weights_sl2_5010 = temp_sl2_5010.fc1.weight.data.cpu().numpy()

# Apply PCA to fc1 weights
transformed_fc1_weights_sl2_5010 = apply_pca_and_reshape(original_fc1_weights_sl2_5010,
                                                        50)

# Update fc2 in the pruned model to match new fc1 out_features
temp_sl2_5010.fc2 = nn.Linear(50, hidden_size2)

# Extract fc2 weights after updating fc2 in the pruned model
original_fc2_weights_sl2_5010 = temp_sl2_5010.fc2.weight.data.cpu().numpy()

# Apply PCA to fc2 weights
transformed_fc2_weights_sl2_5010 = apply_pca_and_reshape(original_fc2_weights_sl2_5010,
                                                        10)

# Instantiate the compressed model
PCA_sparse2_5010 = CompressedRegressionNN(58, 50, 10, 1)

# Assign the PCA-transformed weights
PCA_sparse2_5010.fc1.weight.data = torch.from_numpy(transformed_fc1_weights_sl2_5010).float().view(50, 58)
PCA_sparse2_5010.fc2.weight.data = torch.from_numpy(transformed_fc2_weights_sl2_5010).float().view(10, 50)

# Reset biases
PCA_sparse2_5010.fc1.bias.data.fill_(0)
PCA_sparse2_5010.fc2.bias.data.fill_(0)
PCA_sparse2_5010.fc3.bias.data.fill_(0)

# Validate the weight shapes
print("Transformed fc1 weight shape:", PCA_sparse2_5010.fc1.weight.shape)
print("Transformed fc2 weight shape:", PCA_sparse2_5010.fc2.weight.shape)
print("Transformed fc3 weight shape:", PCA_sparse2_5010.fc3.weight.shape)
print()
print(PCA_sparse2_5010)

In [ ]:
average_loss, average_mae = evaluate_model(PCA_sparse2_5010, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=PCA_sparse2_5010, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

###### fc1 out_features=50 and fc2 out_features=5

In [ ]:
# create copy of the locally pruned model
temp_sl2_505 = RegressionNN(input_size, hidden_size1,
                               hidden_size2, output_size)
transfer_pruned_weights(pruned_sparse2, temp_sl2_505)

In [ ]:
# weight shape of the loaded pruned model
print("fc1 weight shape:", temp_sl2_505.fc1.weight.shape)
print("fc2 weight shape:", temp_sl2_505.fc2.weight.shape)
print("fc3 weight shape:", temp_sl2_505.fc3.weight.shape)

In [ ]:
# Extract weights from the pruned model
original_fc1_weights_sl2_505 = temp_sl2_505.fc1.weight.data.cpu().numpy()

# Apply PCA to fc1 weights
transformed_fc1_weights_sl2_505 = apply_pca_and_reshape(original_fc1_weights_sl2_505,
                                                       50)

# Update fc2 in the pruned model to match new fc1 out_features
temp_sl2_505.fc2 = nn.Linear(50, hidden_size2)

# Extract fc2 weights after updating fc2 in the pruned model
original_fc2_weights_sl2_505 = temp_sl2_505.fc2.weight.data.cpu().numpy()

# Apply PCA to fc2 weights
transformed_fc2_weights_sl2_505 = apply_pca_and_reshape(original_fc2_weights_sl2_505,
                                                       5)

# Instantiate the compressed model
PCA_sparse2_505 = CompressedRegressionNN(58, 50, 5, 1)

# Assign the PCA-transformed weights
PCA_sparse2_505.fc1.weight.data = torch.from_numpy(transformed_fc1_weights_sl2_505).float().view(50, 58)
PCA_sparse2_505.fc2.weight.data = torch.from_numpy(transformed_fc2_weights_sl2_505).float().view(5, 50)

# Reset biases
PCA_sparse2_505.fc1.bias.data.fill_(0)
PCA_sparse2_505.fc2.bias.data.fill_(0)
PCA_sparse2_505.fc3.bias.data.fill_(0)

# Validate the weight shapes
print("Transformed fc1 weight shape:", PCA_sparse2_505.fc1.weight.shape)
print("Transformed fc2 weight shape:", PCA_sparse2_505.fc2.weight.shape)
print("Transformed fc3 weight shape:", PCA_sparse2_505.fc3.weight.shape)
print()
print(PCA_sparse2_505)

In [ ]:
average_loss, average_mae = evaluate_model(PCA_sparse2_505, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=PCA_sparse2_505, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

###### fc1 out_features=30 and fc2 out_features=10

In [ ]:
# create copy of the locally pruned model
temp_sl2_3010 = RegressionNN(input_size, hidden_size1,
                               hidden_size2, output_size)
transfer_pruned_weights(pruned_sparse2, temp_sl2_3010)

In [ ]:
# weight shape of the loaded pruned model
print("fc1 weight shape:", temp_sl2_3010.fc1.weight.shape)
print("fc2 weight shape:", temp_sl2_3010.fc2.weight.shape)
print("fc3 weight shape:", temp_sl2_3010.fc3.weight.shape)

In [ ]:
# Extract weights from the pruned model
original_fc1_weights_sl2_3010 = temp_sl2_3010.fc1.weight.data.cpu().numpy()

# Apply PCA to fc1 weights
transformed_fc1_weights_sl2_3010 = apply_pca_and_reshape(original_fc1_weights_sl2_3010,
                                                        30)

# Update fc2 in the pruned model to match new fc1 out_features
temp_sl2_3010.fc2 = nn.Linear(30, hidden_size2)

# Extract fc2 weights after updating fc2 in the pruned model
original_fc2_weights_sl2_3010 = temp_sl2_3010.fc2.weight.data.cpu().numpy()

# Apply PCA to fc2 weights
transformed_fc2_weights_sl2_3010 = apply_pca_and_reshape(original_fc2_weights_sl2_3010,
                                                        10)

# Instantiate the compressed model
PCA_sparse2_3010 = CompressedRegressionNN(58, 30, 10, 1)

# Assign the PCA-transformed weights
PCA_sparse2_3010.fc1.weight.data = torch.from_numpy(transformed_fc1_weights_sl2_3010).float().view(30, 58)
PCA_sparse2_3010.fc2.weight.data = torch.from_numpy(transformed_fc2_weights_sl2_3010).float().view(10, 30)

# Reset biases
PCA_sparse2_3010.fc1.bias.data.fill_(0)
PCA_sparse2_3010.fc2.bias.data.fill_(0)
PCA_sparse2_3010.fc3.bias.data.fill_(0)

# Validate the weight shapes
print("Transformed fc1 weight shape:", PCA_sparse2_3010.fc1.weight.shape)
print("Transformed fc2 weight shape:", PCA_sparse2_3010.fc2.weight.shape)
print("Transformed fc3 weight shape:", PCA_sparse2_3010.fc3.weight.shape)
print()
print(PCA_sparse2_3010)

In [ ]:
average_loss, average_mae = evaluate_model(PCA_sparse2_3010, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=PCA_sparse2_3010, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

###### fc1 out_features=30 and fc2 out_features=5

In [ ]:
# create copy of the locally pruned model
temp_sl2_305 = RegressionNN(input_size, hidden_size1,
                           hidden_size2, output_size)
transfer_pruned_weights(pruned_sparse2, temp_sl2_305)

In [ ]:
# weight shape of the loaded pruned model
print("fc1 weight shape:", temp_sl2_305.fc1.weight.shape)
print("fc2 weight shape:", temp_sl2_305.fc2.weight.shape)
print("fc3 weight shape:", temp_sl2_305.fc3.weight.shape)

In [ ]:
# Extract weights from the pruned model
original_fc1_weights_sl2_305 = temp_sl2_305.fc1.weight.data.cpu().numpy()

# Apply PCA to fc1 weights
transformed_fc1_weights_sl2_305 = apply_pca_and_reshape(original_fc1_weights_sl2_305,
                                                       30)

# Update fc2 in the pruned model to match new fc1 out_features
temp_sl2_305.fc2 = nn.Linear(30, hidden_size2)

# Extract fc2 weights after updating fc2 in the pruned model
original_fc2_weights_sl2_305 = temp_sl2_305.fc2.weight.data.cpu().numpy()

# Apply PCA to fc2 weights
transformed_fc2_weights_sl2_305 = apply_pca_and_reshape(original_fc2_weights_sl2_305,
                                                        5)

# Instantiate the compressed model
PCA_sparse2_305 = CompressedRegressionNN(58, 30, 5, 1)

# Assign the PCA-transformed weights
PCA_sparse2_305.fc1.weight.data = torch.from_numpy(transformed_fc1_weights_sl2_305).float().view(30, 58)
PCA_sparse2_305.fc2.weight.data = torch.from_numpy(transformed_fc2_weights_sl2_305).float().view(5, 30)

# Reset biases
PCA_sparse2_305.fc1.bias.data.fill_(0)
PCA_sparse2_305.fc2.bias.data.fill_(0)
PCA_sparse2_305.fc3.bias.data.fill_(0)

# Validate the weight shapes
print("Transformed fc1 weight shape:", PCA_sparse2_305.fc1.weight.shape)
print("Transformed fc2 weight shape:", PCA_sparse2_305.fc2.weight.shape)
print("Transformed fc3 weight shape:", PCA_sparse2_305.fc3.weight.shape)
print()
print(PCA_sparse2_305)

In [ ]:
average_loss, average_mae = evaluate_model(PCA_sparse2_305, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=PCA_sparse2_305, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

###### fc1 out_features=10 and fc2 out_features=5

In [ ]:
# create copy of the locally pruned model
temp_sl2_105 = RegressionNN(input_size, hidden_size1,
                               hidden_size2, output_size)
transfer_pruned_weights(pruned_sparse2, temp_sl2_105)

In [ ]:
# weight shape of the loaded pruned model
print("fc1 weight shape:", temp_sl2_105.fc1.weight.shape)
print("fc2 weight shape:", temp_sl2_105.fc2.weight.shape)
print("fc3 weight shape:", temp_sl2_105.fc3.weight.shape)

In [ ]:
# Extract weights from the pruned model
original_fc1_weights_sl2_105 = temp_sl2_105.fc1.weight.data.cpu().numpy()

# Apply PCA to fc1 weights
transformed_fc1_weights_sl2_105 = apply_pca_and_reshape(original_fc1_weights_sl2_105,
                                                       10)

# Update fc2 in the pruned model to match new fc1 out_features
temp_sl2_105.fc2 = nn.Linear(10, hidden_size2)

# Extract fc2 weights after updating fc2 in the pruned model
original_fc2_weights_sl2_105 = temp_sl2_105.fc2.weight.data.cpu().numpy()

# Apply PCA to fc2 weights
transformed_fc2_weights_sl2_105 = apply_pca_and_reshape(original_fc2_weights_sl2_105,
                                                        5)

# Instantiate the compressed model
PCA_sparse2_105 = CompressedRegressionNN(58, 10, 5, 1)

# Assign the PCA-transformed weights
PCA_sparse2_105.fc1.weight.data = torch.from_numpy(transformed_fc1_weights_sl2_105).float().view(10, 58)
PCA_sparse2_105.fc2.weight.data = torch.from_numpy(transformed_fc2_weights_sl2_105).float().view(5, 10)

# Reset biases
PCA_sparse2_105.fc1.bias.data.fill_(0)
PCA_sparse2_105.fc2.bias.data.fill_(0)
PCA_sparse2_105.fc3.bias.data.fill_(0)

# Validate the weight shapes
print("Transformed fc1 weight shape:", PCA_sparse2_105.fc1.weight.shape)
print("Transformed fc2 weight shape:", PCA_sparse2_105.fc2.weight.shape)
print("Transformed fc3 weight shape:", PCA_sparse2_105.fc3.weight.shape)
print()
print(PCA_sparse2_105)

In [ ]:
average_loss, average_mae = evaluate_model(PCA_sparse2_105, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=PCA_sparse2_105, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

#### PCA on the Pruned Model, `pruned_sparse3`, with Pruning Amount=0.8

###### fc1 out_features=50 and fc2 out_features=30

In [ ]:
# create copy of the locally pruned model
temp_sl3_5030 = RegressionNN(input_size, hidden_size1,
                               hidden_size2, output_size)
transfer_pruned_weights(pruned_sparse3, temp_sl3_5030)

In [ ]:
# weight shape of the loaded pruned model
print("fc1 weight shape:", temp_sl3_5030.fc1.weight.shape)
print("fc2 weight shape:", temp_sl3_5030.fc2.weight.shape)
print("fc3 weight shape:", temp_sl3_5030.fc3.weight.shape)

In [ ]:
# Extract weights from the pruned model
original_fc1_weights_sl3_5030 = temp_sl3_5030.fc1.weight.data.cpu().numpy()

# Apply PCA to fc1 weights
transformed_fc1_weights_sl3_5030 = apply_pca_and_reshape(original_fc1_weights_sl3_5030,
                                                        50)

# Update fc2 in the pruned model to match new fc1 out_features
temp_sl3_5030.fc2 = nn.Linear(50, hidden_size2)

# Extract fc2 weights after updating fc2 in the pruned model
original_fc2_weights_sl3_5030 = temp_sl3_5030.fc2.weight.data.cpu().numpy()

# Apply PCA to fc2 weights
transformed_fc2_weights_sl3_5030 = apply_pca_and_reshape(original_fc2_weights_sl3_5030,
                                                        30)

# Instantiate the compressed model
PCA_sparse3_5030 = CompressedRegressionNN(58, 50, 30, 1)

# Assign the PCA-transformed weights
PCA_sparse3_5030.fc1.weight.data = torch.from_numpy(transformed_fc1_weights_sl3_5030).float().view(50, 58)
PCA_sparse3_5030.fc2.weight.data = torch.from_numpy(transformed_fc2_weights_sl3_5030).float().view(30, 50)

# Reset biases
PCA_sparse3_5030.fc1.bias.data.fill_(0)
PCA_sparse3_5030.fc2.bias.data.fill_(0)
PCA_sparse3_5030.fc3.bias.data.fill_(0)

# Validate the weight shapes
print("Transformed fc1 weight shape:", PCA_sparse3_5030.fc1.weight.shape)
print("Transformed fc2 weight shape:", PCA_sparse3_5030.fc2.weight.shape)
print("Transformed fc3 weight shape:", PCA_sparse3_5030.fc3.weight.shape)
print()
print(PCA_sparse3_5030)

In [ ]:
average_loss, average_mae = evaluate_model(PCA_sparse3_5030, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=PCA_sparse3_5030, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

###### fc1 out_features=50 and fc2 out_features=10

In [ ]:
# create copy of the locally pruned model
temp_sl3_5010 = RegressionNN(input_size, hidden_size1,
                               hidden_size2, output_size)
transfer_pruned_weights(pruned_sparse3, temp_sl3_5010)

In [ ]:
# weight shape of the loaded pruned model
print("fc1 weight shape:", temp_sl3_5010.fc1.weight.shape)
print("fc2 weight shape:", temp_sl3_5010.fc2.weight.shape)
print("fc3 weight shape:", temp_sl3_5010.fc3.weight.shape)

In [ ]:
# Extract weights from the pruned model
original_fc1_weights_sl3_5010 = temp_sl3_5010.fc1.weight.data.cpu().numpy()

# Apply PCA to fc1 weights
transformed_fc1_weights_sl3_5010 = apply_pca_and_reshape(original_fc1_weights_sl3_5010,
                                                        50)

# Update fc2 in the pruned model to match new fc1 out_features
temp_sl3_5010.fc2 = nn.Linear(50, hidden_size2)

# Extract fc2 weights after updating fc2 in the pruned model
original_fc2_weights_sl3_5010 = temp_sl3_5010.fc2.weight.data.cpu().numpy()

# Apply PCA to fc2 weights
transformed_fc2_weights_sl3_5010 = apply_pca_and_reshape(original_fc2_weights_sl3_5010,
                                                        10)

# Instantiate the compressed model
PCA_sparse3_5010 = CompressedRegressionNN(58, 50, 10, 1)

# Assign the PCA-transformed weights
PCA_sparse3_5010.fc1.weight.data = torch.from_numpy(transformed_fc1_weights_sl3_5010).float().view(50, 58)
PCA_sparse3_5010.fc2.weight.data = torch.from_numpy(transformed_fc2_weights_sl3_5010).float().view(10, 50)

# Reset biases
PCA_sparse3_5010.fc1.bias.data.fill_(0)
PCA_sparse3_5010.fc2.bias.data.fill_(0)
PCA_sparse3_5010.fc3.bias.data.fill_(0)

# Validate the weight shapes
print("Transformed fc1 weight shape:", PCA_sparse3_5010.fc1.weight.shape)
print("Transformed fc2 weight shape:", PCA_sparse3_5010.fc2.weight.shape)
print("Transformed fc3 weight shape:", PCA_sparse3_5010.fc3.weight.shape)
print()
print(PCA_sparse3_5010)

In [ ]:
average_loss, average_mae = evaluate_model(PCA_sparse3_5010, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=PCA_sparse3_5010, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

###### fc1 out_features=50 and fc2 out_features=5

In [ ]:
# create copy of the locally pruned model
temp_sl3_505 = RegressionNN(input_size, hidden_size1,
                               hidden_size2, output_size)
transfer_pruned_weights(pruned_sparse3, temp_sl3_505)

In [ ]:
# weight shape of the loaded pruned model
print("fc1 weight shape:", temp_sl3_505.fc1.weight.shape)
print("fc2 weight shape:", temp_sl3_505.fc2.weight.shape)
print("fc3 weight shape:", temp_sl3_505.fc3.weight.shape)

In [ ]:
# Extract weights from the pruned model
original_fc1_weights_sl3_505 = temp_sl3_505.fc1.weight.data.cpu().numpy()

# Apply PCA to fc1 weights
transformed_fc1_weights_sl3_505 = apply_pca_and_reshape(original_fc1_weights_sl3_505,
                                                       50)

# Update fc2 in the pruned model to match new fc1 out_features
temp_sl3_505.fc2 = nn.Linear(50, hidden_size2)

# Extract fc2 weights after updating fc2 in the pruned model
original_fc2_weights_sl3_505 = temp_sl3_505.fc2.weight.data.cpu().numpy()

# Apply PCA to fc2 weights
transformed_fc2_weights_sl3_505 = apply_pca_and_reshape(original_fc2_weights_sl3_505,
                                                       5)

# Instantiate the compressed model
PCA_sparse3_505 = CompressedRegressionNN(58, 50, 5, 1)

# Assign the PCA-transformed weights
PCA_sparse3_505.fc1.weight.data = torch.from_numpy(transformed_fc1_weights_sl3_505).float().view(50, 58)
PCA_sparse3_505.fc2.weight.data = torch.from_numpy(transformed_fc2_weights_sl3_505).float().view(5, 50)

# Reset biases
PCA_sparse3_505.fc1.bias.data.fill_(0)
PCA_sparse3_505.fc2.bias.data.fill_(0)
PCA_sparse3_505.fc3.bias.data.fill_(0)

# Validate the weight shapes
print("Transformed fc1 weight shape:", PCA_sparse3_505.fc1.weight.shape)
print("Transformed fc2 weight shape:", PCA_sparse3_505.fc2.weight.shape)
print("Transformed fc3 weight shape:", PCA_sparse3_505.fc3.weight.shape)
print()
print(PCA_sparse3_505)

In [ ]:
average_loss, average_mae = evaluate_model(PCA_sparse3_505, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=PCA_sparse3_505, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

###### fc1 out_features=30 and fc2 out_features=10

In [ ]:
# create copy of the locally pruned model
temp_sl3_3010 = RegressionNN(input_size, hidden_size1,
                               hidden_size2, output_size)
transfer_pruned_weights(pruned_sparse3, temp_sl3_3010)

In [ ]:
# weight shape of the loaded pruned model
print("fc1 weight shape:", temp_sl3_3010.fc1.weight.shape)
print("fc2 weight shape:", temp_sl3_3010.fc2.weight.shape)
print("fc3 weight shape:", temp_sl3_3010.fc3.weight.shape)

In [ ]:
# Extract weights from the pruned model
original_fc1_weights_sl3_3010 = temp_sl3_3010.fc1.weight.data.cpu().numpy()

# Apply PCA to fc1 weights
transformed_fc1_weights_sl3_3010 = apply_pca_and_reshape(original_fc1_weights_sl3_3010,
                                                        30)

# Update fc2 in the pruned model to match new fc1 out_features
temp_sl3_3010.fc2 = nn.Linear(30, hidden_size2)

# Extract fc2 weights after updating fc2 in the pruned model
original_fc2_weights_sl3_3010 = temp_sl3_3010.fc2.weight.data.cpu().numpy()

# Apply PCA to fc2 weights
transformed_fc2_weights_sl3_3010 = apply_pca_and_reshape(original_fc2_weights_sl3_3010,
                                                        10)

# Instantiate the compressed model
PCA_sparse3_3010 = CompressedRegressionNN(58, 30, 10, 1)

# Assign the PCA-transformed weights
PCA_sparse3_3010.fc1.weight.data = torch.from_numpy(transformed_fc1_weights_sl3_3010).float().view(30, 58)
PCA_sparse3_3010.fc2.weight.data = torch.from_numpy(transformed_fc2_weights_sl3_3010).float().view(10, 30)

# Reset biases
PCA_sparse3_3010.fc1.bias.data.fill_(0)
PCA_sparse3_3010.fc2.bias.data.fill_(0)
PCA_sparse3_3010.fc3.bias.data.fill_(0)

# Validate the weight shapes
print("Transformed fc1 weight shape:", PCA_sparse3_3010.fc1.weight.shape)
print("Transformed fc2 weight shape:", PCA_sparse3_3010.fc2.weight.shape)
print("Transformed fc3 weight shape:", PCA_sparse3_3010.fc3.weight.shape)
print()
print(PCA_sparse3_3010)

In [ ]:
average_loss, average_mae = evaluate_model(PCA_sparse3_3010, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=PCA_sparse3_3010, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

###### fc1 out_features=30 and fc2 out_features=5

In [ ]:
# create copy of the locally pruned model
temp_sl3_305 = RegressionNN(input_size, hidden_size1,
                           hidden_size2, output_size)
transfer_pruned_weights(pruned_sparse3, temp_sl3_305)

In [ ]:
# weight shape of the loaded pruned model
print("fc1 weight shape:", temp_sl3_305.fc1.weight.shape)
print("fc2 weight shape:", temp_sl3_305.fc2.weight.shape)
print("fc3 weight shape:", temp_sl3_305.fc3.weight.shape)

In [ ]:
# Extract weights from the pruned model
original_fc1_weights_sl3_305 = temp_sl3_305.fc1.weight.data.cpu().numpy()

# Apply PCA to fc1 weights
transformed_fc1_weights_sl3_305 = apply_pca_and_reshape(original_fc1_weights_sl3_305,
                                                       30)

# Update fc2 in the pruned model to match new fc1 out_features
temp_sl3_305.fc2 = nn.Linear(30, hidden_size2)

# Extract fc2 weights after updating fc2 in the pruned model
original_fc2_weights_sl3_305 = temp_sl3_305.fc2.weight.data.cpu().numpy()

# Apply PCA to fc2 weights
transformed_fc2_weights_sl3_305 = apply_pca_and_reshape(original_fc2_weights_sl3_305,
                                                        5)

# Instantiate the compressed model
PCA_sparse3_305 = CompressedRegressionNN(58, 30, 5, 1)

# Assign the PCA-transformed weights
PCA_sparse3_305.fc1.weight.data = torch.from_numpy(transformed_fc1_weights_sl3_305).float().view(30, 58)
PCA_sparse3_305.fc2.weight.data = torch.from_numpy(transformed_fc2_weights_sl3_305).float().view(5, 30)

# Reset biases
PCA_sparse3_305.fc1.bias.data.fill_(0)
PCA_sparse3_305.fc2.bias.data.fill_(0)
PCA_sparse3_305.fc3.bias.data.fill_(0)

# Validate the weight shapes
print("Transformed fc1 weight shape:", PCA_sparse3_305.fc1.weight.shape)
print("Transformed fc2 weight shape:", PCA_sparse3_305.fc2.weight.shape)
print("Transformed fc3 weight shape:", PCA_sparse3_305.fc3.weight.shape)
print()
print(PCA_sparse3_305)

In [ ]:
average_loss, average_mae = evaluate_model(PCA_sparse3_305, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=PCA_sparse3_305, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

###### fc1 out_features=10 and fc2 out_features=5

In [ ]:
# create copy of the locally pruned model
temp_sl3_105 = RegressionNN(input_size, hidden_size1,
                               hidden_size2, output_size)
transfer_pruned_weights(pruned_sparse3, temp_sl3_105)

In [ ]:
# weight shape of the loaded pruned model
print("fc1 weight shape:", temp_sl3_105.fc1.weight.shape)
print("fc2 weight shape:", temp_sl3_105.fc2.weight.shape)
print("fc3 weight shape:", temp_sl3_105.fc3.weight.shape)

In [ ]:
# Extract weights from the pruned model
original_fc1_weights_sl3_105 = temp_sl3_105.fc1.weight.data.cpu().numpy()

# Apply PCA to fc1 weights
transformed_fc1_weights_sl3_105 = apply_pca_and_reshape(original_fc1_weights_sl3_105,
                                                       10)

# Update fc2 in the pruned model to match new fc1 out_features
temp_sl3_105.fc2 = nn.Linear(10, hidden_size2)

# Extract fc2 weights after updating fc2 in the pruned model
original_fc2_weights_sl3_105 = temp_sl3_105.fc2.weight.data.cpu().numpy()

# Apply PCA to fc2 weights
transformed_fc2_weights_sl3_105 = apply_pca_and_reshape(original_fc2_weights_sl3_105,
                                                        5)

# Instantiate the compressed model
PCA_sparse3_105 = CompressedRegressionNN(58, 10, 5, 1)

# Assign the PCA-transformed weights
PCA_sparse3_105.fc1.weight.data = torch.from_numpy(transformed_fc1_weights_sl3_105).float().view(10, 58)
PCA_sparse3_105.fc2.weight.data = torch.from_numpy(transformed_fc2_weights_sl3_105).float().view(5, 10)

# Reset biases
PCA_sparse3_105.fc1.bias.data.fill_(0)
PCA_sparse3_105.fc2.bias.data.fill_(0)
PCA_sparse3_105.fc3.bias.data.fill_(0)

# Validate the weight shapes
print("Transformed fc1 weight shape:", PCA_sparse3_105.fc1.weight.shape)
print("Transformed fc2 weight shape:", PCA_sparse3_105.fc2.weight.shape)
print("Transformed fc3 weight shape:", PCA_sparse3_105.fc3.weight.shape)
print()
print(PCA_sparse3_105)

In [ ]:
average_loss, average_mae = evaluate_model(PCA_sparse3_105, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=PCA_sparse3_105, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

#### PCA on the Pruned Model, `pruned_sparse4`, with Pruning Amount=0.9

###### fc1 out_features=50 and fc2 out_features=30

In [ ]:
# create copy of the locally pruned model
temp_sl4_5030 = RegressionNN(input_size, hidden_size1,
                               hidden_size2, output_size)
transfer_pruned_weights(pruned_sparse4, temp_sl4_5030)

In [ ]:
# weight shape of the loaded pruned model
print("fc1 weight shape:", temp_sl4_5030.fc1.weight.shape)
print("fc2 weight shape:", temp_sl4_5030.fc2.weight.shape)
print("fc3 weight shape:", temp_sl4_5030.fc3.weight.shape)

In [ ]:
# Extract weights from the pruned model
original_fc1_weights_sl4_5030 = temp_sl4_5030.fc1.weight.data.cpu().numpy()

# Apply PCA to fc1 weights
transformed_fc1_weights_sl4_5030 = apply_pca_and_reshape(original_fc1_weights_sl4_5030,
                                                        50)

# Update fc2 in the pruned model to match new fc1 out_features
temp_sl4_5030.fc2 = nn.Linear(50, hidden_size2)

# Extract fc2 weights after updating fc2 in the pruned model
original_fc2_weights_sl4_5030 = temp_sl4_5030.fc2.weight.data.cpu().numpy()

# Apply PCA to fc2 weights
transformed_fc2_weights_sl4_5030 = apply_pca_and_reshape(original_fc2_weights_sl4_5030,
                                                        30)

# Instantiate the compressed model
PCA_sparse4_5030 = CompressedRegressionNN(58, 50, 30, 1)

# Assign the PCA-transformed weights
PCA_sparse4_5030.fc1.weight.data = torch.from_numpy(transformed_fc1_weights_sl4_5030).float().view(50, 58)
PCA_sparse4_5030.fc2.weight.data = torch.from_numpy(transformed_fc2_weights_sl4_5030).float().view(30, 50)

# Reset biases
PCA_sparse4_5030.fc1.bias.data.fill_(0)
PCA_sparse4_5030.fc2.bias.data.fill_(0)
PCA_sparse4_5030.fc3.bias.data.fill_(0)

# Validate the weight shapes
print("Transformed fc1 weight shape:", PCA_sparse4_5030.fc1.weight.shape)
print("Transformed fc2 weight shape:", PCA_sparse4_5030.fc2.weight.shape)
print("Transformed fc3 weight shape:", PCA_sparse4_5030.fc3.weight.shape)
print()
print(PCA_sparse4_5030)

In [ ]:
average_loss, average_mae = evaluate_model(PCA_sparse4_5030, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=PCA_sparse4_5030, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

###### fc1 out_features=50 and fc2 out_features=10

In [ ]:
# create copy of the locally pruned model
temp_sl4_5010 = RegressionNN(input_size, hidden_size1,
                               hidden_size2, output_size)
transfer_pruned_weights(pruned_sparse4, temp_sl4_5010)

In [ ]:
# weight shape of the loaded pruned model
print("fc1 weight shape:", temp_sl4_5010.fc1.weight.shape)
print("fc2 weight shape:", temp_sl4_5010.fc2.weight.shape)
print("fc3 weight shape:", temp_sl4_5010.fc3.weight.shape)

In [ ]:
# Extract weights from the pruned model
original_fc1_weights_sl4_5010 = temp_sl4_5010.fc1.weight.data.cpu().numpy()

# Apply PCA to fc1 weights
transformed_fc1_weights_sl4_5010 = apply_pca_and_reshape(original_fc1_weights_sl4_5010,
                                                        50)

# Update fc2 in the pruned model to match new fc1 out_features
temp_sl4_5010.fc2 = nn.Linear(50, hidden_size2)

# Extract fc2 weights after updating fc2 in the pruned model
original_fc2_weights_sl4_5010 = temp_sl4_5010.fc2.weight.data.cpu().numpy()

# Apply PCA to fc2 weights
transformed_fc2_weights_sl4_5010 = apply_pca_and_reshape(original_fc2_weights_sl4_5010,
                                                        10)

# Instantiate the compressed model
PCA_sparse4_5010 = CompressedRegressionNN(58, 50, 10, 1)

# Assign the PCA-transformed weights
PCA_sparse4_5010.fc1.weight.data = torch.from_numpy(transformed_fc1_weights_sl4_5010).float().view(50, 58)
PCA_sparse4_5010.fc2.weight.data = torch.from_numpy(transformed_fc2_weights_sl4_5010).float().view(10, 50)

# Reset biases
PCA_sparse4_5010.fc1.bias.data.fill_(0)
PCA_sparse4_5010.fc2.bias.data.fill_(0)
PCA_sparse4_5010.fc3.bias.data.fill_(0)

# Validate the weight shapes
print("Transformed fc1 weight shape:", PCA_sparse4_5010.fc1.weight.shape)
print("Transformed fc2 weight shape:", PCA_sparse4_5010.fc2.weight.shape)
print("Transformed fc3 weight shape:", PCA_sparse4_5010.fc3.weight.shape)
print()
print(PCA_sparse4_5010)

In [ ]:
average_loss, average_mae = evaluate_model(PCA_sparse4_5010, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=PCA_sparse4_5010, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

###### fc1 out_features=50 and fc2 out_features=5

In [ ]:
# create copy of the locally pruned model
temp_sl4_505 = RegressionNN(input_size, hidden_size1,
                               hidden_size2, output_size)
transfer_pruned_weights(pruned_sparse4, temp_sl4_505)

In [ ]:
# weight shape of the loaded pruned model
print("fc1 weight shape:", temp_sl4_505.fc1.weight.shape)
print("fc2 weight shape:", temp_sl4_505.fc2.weight.shape)
print("fc3 weight shape:", temp_sl4_505.fc3.weight.shape)

In [ ]:
# Extract weights from the pruned model
original_fc1_weights_sl4_505 = temp_sl4_505.fc1.weight.data.cpu().numpy()

# Apply PCA to fc1 weights
transformed_fc1_weights_sl4_505 = apply_pca_and_reshape(original_fc1_weights_sl4_505,
                                                       50)

# Update fc2 in the pruned model to match new fc1 out_features
temp_sl4_505.fc2 = nn.Linear(50, hidden_size2)

# Extract fc2 weights after updating fc2 in the pruned model
original_fc2_weights_sl4_505 = temp_sl4_505.fc2.weight.data.cpu().numpy()

# Apply PCA to fc2 weights
transformed_fc2_weights_sl4_505 = apply_pca_and_reshape(original_fc2_weights_sl4_505,
                                                       5)

# Instantiate the compressed model
PCA_sparse4_505 = CompressedRegressionNN(58, 50, 5, 1)

# Assign the PCA-transformed weights
PCA_sparse4_505.fc1.weight.data = torch.from_numpy(transformed_fc1_weights_sl4_505).float().view(50, 58)
PCA_sparse4_505.fc2.weight.data = torch.from_numpy(transformed_fc2_weights_sl4_505).float().view(5, 50)

# Reset biases
PCA_sparse4_505.fc1.bias.data.fill_(0)
PCA_sparse4_505.fc2.bias.data.fill_(0)
PCA_sparse4_505.fc3.bias.data.fill_(0)

# Validate the weight shapes
print("Transformed fc1 weight shape:", PCA_sparse4_505.fc1.weight.shape)
print("Transformed fc2 weight shape:", PCA_sparse4_505.fc2.weight.shape)
print("Transformed fc3 weight shape:", PCA_sparse4_505.fc3.weight.shape)
print()
print(PCA_sparse4_505)

In [ ]:
average_loss, average_mae = evaluate_model(PCA_sparse4_505, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=PCA_sparse4_505, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

###### fc1 out_features=30 and fc2 out_features=10

In [ ]:
# create copy of the locally pruned model
temp_sl4_3010 = RegressionNN(input_size, hidden_size1,
                               hidden_size2, output_size)
transfer_pruned_weights(pruned_sparse4, temp_sl4_3010)

In [ ]:
# weight shape of the loaded pruned model
print("fc1 weight shape:", temp_sl4_3010.fc1.weight.shape)
print("fc2 weight shape:", temp_sl4_3010.fc2.weight.shape)
print("fc3 weight shape:", temp_sl4_3010.fc3.weight.shape)

In [ ]:
# Extract weights from the pruned model
original_fc1_weights_sl4_3010 = temp_sl4_3010.fc1.weight.data.cpu().numpy()

# Apply PCA to fc1 weights
transformed_fc1_weights_sl4_3010 = apply_pca_and_reshape(original_fc1_weights_sl4_3010,
                                                        30)

# Update fc2 in the pruned model to match new fc1 out_features
temp_sl4_3010.fc2 = nn.Linear(30, hidden_size2)

# Extract fc2 weights after updating fc2 in the pruned model
original_fc2_weights_sl4_3010 = temp_sl4_3010.fc2.weight.data.cpu().numpy()

# Apply PCA to fc2 weights
transformed_fc2_weights_sl4_3010 = apply_pca_and_reshape(original_fc2_weights_sl4_3010,
                                                        10)

# Instantiate the compressed model
PCA_sparse4_3010 = CompressedRegressionNN(58, 30, 10, 1)

# Assign the PCA-transformed weights
PCA_sparse4_3010.fc1.weight.data = torch.from_numpy(transformed_fc1_weights_sl4_3010).float().view(30, 58)
PCA_sparse4_3010.fc2.weight.data = torch.from_numpy(transformed_fc2_weights_sl4_3010).float().view(10, 30)

# Reset biases
PCA_sparse4_3010.fc1.bias.data.fill_(0)
PCA_sparse4_3010.fc2.bias.data.fill_(0)
PCA_sparse4_3010.fc3.bias.data.fill_(0)

# Validate the weight shapes
print("Transformed fc1 weight shape:", PCA_sparse4_3010.fc1.weight.shape)
print("Transformed fc2 weight shape:", PCA_sparse4_3010.fc2.weight.shape)
print("Transformed fc3 weight shape:", PCA_sparse4_3010.fc3.weight.shape)
print()
print(PCA_sparse4_3010)

In [ ]:
average_loss, average_mae = evaluate_model(PCA_sparse4_3010, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=PCA_sparse4_3010, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

###### fc1 out_features=30 and fc2 out_features=5

In [ ]:
# create copy of the locally pruned model
temp_sl4_305 = RegressionNN(input_size, hidden_size1,
                           hidden_size2, output_size)
transfer_pruned_weights(pruned_sparse4, temp_sl4_305)

In [ ]:
# weight shape of the loaded pruned model
print("fc1 weight shape:", temp_sl4_305.fc1.weight.shape)
print("fc2 weight shape:", temp_sl4_305.fc2.weight.shape)
print("fc3 weight shape:", temp_sl4_305.fc3.weight.shape)

In [ ]:
# Extract weights from the pruned model
original_fc1_weights_sl4_305 = temp_sl4_305.fc1.weight.data.cpu().numpy()

# Apply PCA to fc1 weights
transformed_fc1_weights_sl4_305 = apply_pca_and_reshape(original_fc1_weights_sl4_305,
                                                       30)

# Update fc2 in the pruned model to match new fc1 out_features
temp_sl4_305.fc2 = nn.Linear(30, hidden_size2)

# Extract fc2 weights after updating fc2 in the pruned model
original_fc2_weights_sl4_305 = temp_sl4_305.fc2.weight.data.cpu().numpy()

# Apply PCA to fc2 weights
transformed_fc2_weights_sl4_305 = apply_pca_and_reshape(original_fc2_weights_sl4_305,
                                                        5)

# Instantiate the compressed model
PCA_sparse4_305 = CompressedRegressionNN(58, 30, 5, 1)

# Assign the PCA-transformed weights
PCA_sparse4_305.fc1.weight.data = torch.from_numpy(transformed_fc1_weights_sl4_305).float().view(30, 58)
PCA_sparse4_305.fc2.weight.data = torch.from_numpy(transformed_fc2_weights_sl4_305).float().view(5, 30)

# Reset biases
PCA_sparse4_305.fc1.bias.data.fill_(0)
PCA_sparse4_305.fc2.bias.data.fill_(0)
PCA_sparse4_305.fc3.bias.data.fill_(0)

# Validate the weight shapes
print("Transformed fc1 weight shape:", PCA_sparse4_305.fc1.weight.shape)
print("Transformed fc2 weight shape:", PCA_sparse4_305.fc2.weight.shape)
print("Transformed fc3 weight shape:", PCA_sparse4_305.fc3.weight.shape)
print()
print(PCA_sparse4_305)

In [ ]:
average_loss, average_mae = evaluate_model(PCA_sparse4_305, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=PCA_sparse4_305, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

###### fc1 out_features=10 and fc2 out_features=5

In [ ]:
# create copy of the locally pruned model
temp_sl4_105 = RegressionNN(input_size, hidden_size1,
                               hidden_size2, output_size)
transfer_pruned_weights(pruned_sparse4, temp_sl4_105)

In [ ]:
# weight shape of the loaded pruned model
print("fc1 weight shape:", temp_sl4_105.fc1.weight.shape)
print("fc2 weight shape:", temp_sl4_105.fc2.weight.shape)
print("fc3 weight shape:", temp_sl4_105.fc3.weight.shape)

In [ ]:
# Extract weights from the pruned model
original_fc1_weights_sl4_105 = temp_sl4_105.fc1.weight.data.cpu().numpy()

# Apply PCA to fc1 weights
transformed_fc1_weights_sl4_105 = apply_pca_and_reshape(original_fc1_weights_sl4_105,
                                                       10)

# Update fc2 in the pruned model to match new fc1 out_features
temp_sl4_105.fc2 = nn.Linear(10, hidden_size2)

# Extract fc2 weights after updating fc2 in the pruned model
original_fc2_weights_sl4_105 = temp_sl4_105.fc2.weight.data.cpu().numpy()

# Apply PCA to fc2 weights
transformed_fc2_weights_sl4_105 = apply_pca_and_reshape(original_fc2_weights_sl4_105,
                                                        5)

# Instantiate the compressed model
PCA_sparse4_105 = CompressedRegressionNN(58, 10, 5, 1)

# Assign the PCA-transformed weights
PCA_sparse4_105.fc1.weight.data = torch.from_numpy(transformed_fc1_weights_sl4_105).float().view(10, 58)
PCA_sparse4_105.fc2.weight.data = torch.from_numpy(transformed_fc2_weights_sl4_105).float().view(5, 10)

# Reset biases
PCA_sparse4_105.fc1.bias.data.fill_(0)
PCA_sparse4_105.fc2.bias.data.fill_(0)
PCA_sparse4_105.fc3.bias.data.fill_(0)

# Validate the weight shapes
print("Transformed fc1 weight shape:", PCA_sparse4_105.fc1.weight.shape)
print("Transformed fc2 weight shape:", PCA_sparse4_105.fc2.weight.shape)
print("Transformed fc3 weight shape:", PCA_sparse4_105.fc3.weight.shape)
print()
print(PCA_sparse4_105)

In [ ]:
average_loss, average_mae = evaluate_model(PCA_sparse4_105, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=PCA_sparse4_105, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

#### PCA on the Pruned Model, `pruned_sparse5`, with Pruning Amount=0.95

###### fc1 out_features=50 and fc2 out_features=30

In [ ]:
# create copy of the locally pruned model
temp_sl5_5030 = RegressionNN(input_size, hidden_size1,
                               hidden_size2, output_size)
transfer_pruned_weights(pruned_sparse5, temp_sl5_5030)

In [ ]:
# weight shape of the loaded pruned model
print("fc1 weight shape:", temp_sl5_5030.fc1.weight.shape)
print("fc2 weight shape:", temp_sl5_5030.fc2.weight.shape)
print("fc3 weight shape:", temp_sl5_5030.fc3.weight.shape)

In [ ]:
# Extract weights from the pruned model
original_fc1_weights_sl5_5030 = temp_sl5_5030.fc1.weight.data.cpu().numpy()

# Apply PCA to fc1 weights
transformed_fc1_weights_sl5_5030 = apply_pca_and_reshape(original_fc1_weights_sl5_5030,
                                                        50)

# Update fc2 in the pruned model to match new fc1 out_features
temp_sl5_5030.fc2 = nn.Linear(50, hidden_size2)

# Extract fc2 weights after updating fc2 in the pruned model
original_fc2_weights_sl5_5030 = temp_sl5_5030.fc2.weight.data.cpu().numpy()

# Apply PCA to fc2 weights
transformed_fc2_weights_sl5_5030 = apply_pca_and_reshape(original_fc2_weights_sl5_5030,
                                                        30)

# Instantiate the compressed model
PCA_sparse5_5030 = CompressedRegressionNN(58, 50, 30, 1)

# Assign the PCA-transformed weights
PCA_sparse5_5030.fc1.weight.data = torch.from_numpy(transformed_fc1_weights_sl5_5030).float().view(50, 58)
PCA_sparse5_5030.fc2.weight.data = torch.from_numpy(transformed_fc2_weights_sl5_5030).float().view(30, 50)

# Reset biases
PCA_sparse5_5030.fc1.bias.data.fill_(0)
PCA_sparse5_5030.fc2.bias.data.fill_(0)
PCA_sparse5_5030.fc3.bias.data.fill_(0)

# Validate the weight shapes
print("Transformed fc1 weight shape:", PCA_sparse5_5030.fc1.weight.shape)
print("Transformed fc2 weight shape:", PCA_sparse5_5030.fc2.weight.shape)
print("Transformed fc3 weight shape:", PCA_sparse5_5030.fc3.weight.shape)
print()
print(PCA_sparse5_5030)

In [ ]:
average_loss, average_mae = evaluate_model(PCA_sparse5_5030, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=PCA_sparse5_5030, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

###### fc1 out_features=50 and fc2 out_features=10

In [ ]:
# create copy of the locally pruned model
temp_sl5_5010 = RegressionNN(input_size, hidden_size1,
                               hidden_size2, output_size)
transfer_pruned_weights(pruned_sparse5, temp_sl5_5010)

In [ ]:
# weight shape of the loaded pruned model
print("fc1 weight shape:", temp_sl5_5010.fc1.weight.shape)
print("fc2 weight shape:", temp_sl5_5010.fc2.weight.shape)
print("fc3 weight shape:", temp_sl5_5010.fc3.weight.shape)

In [ ]:
# Extract weights from the pruned model
original_fc1_weights_sl5_5010 = temp_sl5_5010.fc1.weight.data.cpu().numpy()

# Apply PCA to fc1 weights
transformed_fc1_weights_sl5_5010 = apply_pca_and_reshape(original_fc1_weights_sl5_5010,
                                                        50)

# Update fc2 in the pruned model to match new fc1 out_features
temp_sl5_5010.fc2 = nn.Linear(50, hidden_size2)

# Extract fc2 weights after updating fc2 in the pruned model
original_fc2_weights_sl5_5010 = temp_sl5_5010.fc2.weight.data.cpu().numpy()

# Apply PCA to fc2 weights
transformed_fc2_weights_sl5_5010 = apply_pca_and_reshape(original_fc2_weights_sl5_5010,
                                                        10)

# Instantiate the compressed model
PCA_sparse5_5010 = CompressedRegressionNN(58, 50, 10, 1)

# Assign the PCA-transformed weights
PCA_sparse5_5010.fc1.weight.data = torch.from_numpy(transformed_fc1_weights_sl5_5010).float().view(50, 58)
PCA_sparse5_5010.fc2.weight.data = torch.from_numpy(transformed_fc2_weights_sl5_5010).float().view(10, 50)

# Reset biases
PCA_sparse5_5010.fc1.bias.data.fill_(0)
PCA_sparse5_5010.fc2.bias.data.fill_(0)
PCA_sparse5_5010.fc3.bias.data.fill_(0)

# Validate the weight shapes
print("Transformed fc1 weight shape:", PCA_sparse5_5010.fc1.weight.shape)
print("Transformed fc2 weight shape:", PCA_sparse5_5010.fc2.weight.shape)
print("Transformed fc3 weight shape:", PCA_sparse5_5010.fc3.weight.shape)
print()
print(PCA_sparse5_5010)

In [ ]:
average_loss, average_mae = evaluate_model(PCA_sparse5_5010, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=PCA_sparse5_5010, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

###### fc1 out_features=50 and fc2 out_features=5

In [ ]:
# create copy of the locally pruned model
temp_sl5_505 = RegressionNN(input_size, hidden_size1,
                               hidden_size2, output_size)
transfer_pruned_weights(pruned_sparse5, temp_sl5_505)

In [ ]:
# weight shape of the loaded pruned model
print("fc1 weight shape:", temp_sl5_505.fc1.weight.shape)
print("fc2 weight shape:", temp_sl5_505.fc2.weight.shape)
print("fc3 weight shape:", temp_sl5_505.fc3.weight.shape)

In [ ]:
# Extract weights from the pruned model
original_fc1_weights_sl5_505 = temp_sl5_505.fc1.weight.data.cpu().numpy()

# Apply PCA to fc1 weights
transformed_fc1_weights_sl5_505 = apply_pca_and_reshape(original_fc1_weights_sl5_505,
                                                       50)

# Update fc2 in the pruned model to match new fc1 out_features
temp_sl5_505.fc2 = nn.Linear(50, hidden_size2)

# Extract fc2 weights after updating fc2 in the pruned model
original_fc2_weights_sl5_505 = temp_sl5_505.fc2.weight.data.cpu().numpy()

# Apply PCA to fc2 weights
transformed_fc2_weights_sl5_505 = apply_pca_and_reshape(original_fc2_weights_sl5_505,
                                                       5)

# Instantiate the compressed model
PCA_sparse5_505 = CompressedRegressionNN(58, 50, 5, 1)

# Assign the PCA-transformed weights
PCA_sparse5_505.fc1.weight.data = torch.from_numpy(transformed_fc1_weights_sl5_505).float().view(50, 58)
PCA_sparse5_505.fc2.weight.data = torch.from_numpy(transformed_fc2_weights_sl5_505).float().view(5, 50)

# Reset biases
PCA_sparse5_505.fc1.bias.data.fill_(0)
PCA_sparse5_505.fc2.bias.data.fill_(0)
PCA_sparse5_505.fc3.bias.data.fill_(0)

# Validate the weight shapes
print("Transformed fc1 weight shape:", PCA_sparse5_505.fc1.weight.shape)
print("Transformed fc2 weight shape:", PCA_sparse5_505.fc2.weight.shape)
print("Transformed fc3 weight shape:", PCA_sparse5_505.fc3.weight.shape)
print()
print(PCA_sparse5_505)

In [ ]:
average_loss, average_mae = evaluate_model(PCA_sparse5_505, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=PCA_sparse5_505, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

###### fc1 out_features=30 and fc2 out_features=10

In [ ]:
# create copy of the locally pruned model
temp_sl5_3010 = RegressionNN(input_size, hidden_size1,
                               hidden_size2, output_size)
transfer_pruned_weights(pruned_sparse5, temp_sl5_3010)

In [ ]:
# weight shape of the loaded pruned model
print("fc1 weight shape:", temp_sl5_3010.fc1.weight.shape)
print("fc2 weight shape:", temp_sl5_3010.fc2.weight.shape)
print("fc3 weight shape:", temp_sl5_3010.fc3.weight.shape)

In [ ]:
# Extract weights from the pruned model
original_fc1_weights_sl5_3010 = temp_sl5_3010.fc1.weight.data.cpu().numpy()

# Apply PCA to fc1 weights
transformed_fc1_weights_sl5_3010 = apply_pca_and_reshape(original_fc1_weights_sl5_3010,
                                                        30)

# Update fc2 in the pruned model to match new fc1 out_features
temp_sl5_3010.fc2 = nn.Linear(30, hidden_size2)

# Extract fc2 weights after updating fc2 in the pruned model
original_fc2_weights_sl5_3010 = temp_sl5_3010.fc2.weight.data.cpu().numpy()

# Apply PCA to fc2 weights
transformed_fc2_weights_sl5_3010 = apply_pca_and_reshape(original_fc2_weights_sl5_3010,
                                                        10)

# Instantiate the compressed model
PCA_sparse5_3010 = CompressedRegressionNN(58, 30, 10, 1)

# Assign the PCA-transformed weights
PCA_sparse5_3010.fc1.weight.data = torch.from_numpy(transformed_fc1_weights_sl5_3010).float().view(30, 58)
PCA_sparse5_3010.fc2.weight.data = torch.from_numpy(transformed_fc2_weights_sl5_3010).float().view(10, 30)

# Reset biases
PCA_sparse5_3010.fc1.bias.data.fill_(0)
PCA_sparse5_3010.fc2.bias.data.fill_(0)
PCA_sparse5_3010.fc3.bias.data.fill_(0)

# Validate the weight shapes
print("Transformed fc1 weight shape:", PCA_sparse5_3010.fc1.weight.shape)
print("Transformed fc2 weight shape:", PCA_sparse5_3010.fc2.weight.shape)
print("Transformed fc3 weight shape:", PCA_sparse5_3010.fc3.weight.shape)
print()
print(PCA_sparse5_3010)

In [ ]:
average_loss, average_mae = evaluate_model(PCA_sparse5_3010, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=PCA_sparse5_3010, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

###### fc1 out_features=30 and fc2 out_features=5

In [ ]:
# create copy of the locally pruned model
temp_sl5_305 = RegressionNN(input_size, hidden_size1,
                           hidden_size2, output_size)
transfer_pruned_weights(pruned_sparse5, temp_sl5_305)

In [ ]:
# weight shape of the loaded pruned model
print("fc1 weight shape:", temp_sl5_305.fc1.weight.shape)
print("fc2 weight shape:", temp_sl5_305.fc2.weight.shape)
print("fc3 weight shape:", temp_sl5_305.fc3.weight.shape)

In [ ]:
# Extract weights from the pruned model
original_fc1_weights_sl5_305 = temp_sl5_305.fc1.weight.data.cpu().numpy()

# Apply PCA to fc1 weights
transformed_fc1_weights_sl5_305 = apply_pca_and_reshape(original_fc1_weights_sl5_305,
                                                       30)

# Update fc2 in the pruned model to match new fc1 out_features
temp_sl5_305.fc2 = nn.Linear(30, hidden_size2)

# Extract fc2 weights after updating fc2 in the pruned model
original_fc2_weights_sl5_305 = temp_sl5_305.fc2.weight.data.cpu().numpy()

# Apply PCA to fc2 weights
transformed_fc2_weights_sl5_305 = apply_pca_and_reshape(original_fc2_weights_sl5_305,
                                                        5)

# Instantiate the compressed model
PCA_sparse5_305 = CompressedRegressionNN(58, 30, 5, 1)

# Assign the PCA-transformed weights
PCA_sparse5_305.fc1.weight.data = torch.from_numpy(transformed_fc1_weights_sl5_305).float().view(30, 58)
PCA_sparse5_305.fc2.weight.data = torch.from_numpy(transformed_fc2_weights_sl5_305).float().view(5, 30)

# Reset biases
PCA_sparse5_305.fc1.bias.data.fill_(0)
PCA_sparse5_305.fc2.bias.data.fill_(0)
PCA_sparse5_305.fc3.bias.data.fill_(0)

# Validate the weight shapes
print("Transformed fc1 weight shape:", PCA_sparse5_305.fc1.weight.shape)
print("Transformed fc2 weight shape:", PCA_sparse5_305.fc2.weight.shape)
print("Transformed fc3 weight shape:", PCA_sparse5_305.fc3.weight.shape)
print()
print(PCA_sparse5_305)

In [ ]:
average_loss, average_mae = evaluate_model(PCA_sparse5_305, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=PCA_sparse5_305, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

###### fc1 out_features=10 and fc2 out_features=5

In [ ]:
# create copy of the locally pruned model
temp_sl5_105 = RegressionNN(input_size, hidden_size1,
                               hidden_size2, output_size)
transfer_pruned_weights(pruned_sparse5, temp_sl5_105)

In [ ]:
# weight shape of the loaded pruned model
print("fc1 weight shape:", temp_sl5_105.fc1.weight.shape)
print("fc2 weight shape:", temp_sl5_105.fc2.weight.shape)
print("fc3 weight shape:", temp_sl5_105.fc3.weight.shape)

In [ ]:
# Extract weights from the pruned model
original_fc1_weights_sl5_105 = temp_sl5_105.fc1.weight.data.cpu().numpy()

# Apply PCA to fc1 weights
transformed_fc1_weights_sl5_105 = apply_pca_and_reshape(original_fc1_weights_sl5_105,
                                                       10)

# Update fc2 in the pruned model to match new fc1 out_features
temp_sl5_105.fc2 = nn.Linear(10, hidden_size2)

# Extract fc2 weights after updating fc2 in the pruned model
original_fc2_weights_sl5_105 = temp_sl5_105.fc2.weight.data.cpu().numpy()

# Apply PCA to fc2 weights
transformed_fc2_weights_sl5_105 = apply_pca_and_reshape(original_fc2_weights_sl5_105,
                                                        5)

# Instantiate the compressed model
PCA_sparse5_105 = CompressedRegressionNN(58, 10, 5, 1)

# Assign the PCA-transformed weights
PCA_sparse5_105.fc1.weight.data = torch.from_numpy(transformed_fc1_weights_sl5_105).float().view(10, 58)
PCA_sparse5_105.fc2.weight.data = torch.from_numpy(transformed_fc2_weights_sl5_105).float().view(5, 10)

# Reset biases
PCA_sparse5_105.fc1.bias.data.fill_(0)
PCA_sparse5_105.fc2.bias.data.fill_(0)
PCA_sparse5_105.fc3.bias.data.fill_(0)

# Validate the weight shapes
print("Transformed fc1 weight shape:", PCA_sparse5_105.fc1.weight.shape)
print("Transformed fc2 weight shape:", PCA_sparse5_105.fc2.weight.shape)
print("Transformed fc3 weight shape:", PCA_sparse5_105.fc3.weight.shape)
print()
print(PCA_sparse5_105)

In [ ]:
average_loss, average_mae = evaluate_model(PCA_sparse5_105, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=PCA_sparse5_105, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

## Sparse Principal Components Analysis (SparsePCA) on a Pruned Model

In [ ]:
# function to apply SparsePCA on the input features, in_features
# alpha (float) - high value to ensure more sparsity
def apply_sparse_pca(layer, n_components, alpha=1.0):

    # Extract the layer's weights and transposing them
    weights = layer.weight.data.cpu().numpy().T

    # Initialize the SparsePCA
    # reference: https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.SparsePCA.html#sklearn.decomposition.SparsePCA
    sparse_pca = SparsePCA(n_components=n_components, alpha=alpha, random_state=42)

    # Fit the SparsePCA model on the transposed weights and transforming them
    transformed_weights = sparse_pca.fit_transform(weights)

    # Transpose the transformed weights back to their original orientation
    return transformed_weights.T

### SparsePCA on Locally Pruned Model

#### SparsePCA on the Pruned Model, `pruned_local1`, with Pruning Amount=0.6

###### no_components = 50

In [ ]:
# create copy of the model
sparsePCA_l1_temp = RegressionNN(input_size, hidden_size1,
                                         hidden_size2, output_size)
transfer_pruned_weights(pruned_local1, sparsePCA_l1_temp)

In [ ]:
# Apply SparsePCA to fc1 and fc2 of the pruned model
l1_50_transformed_fc1_weights = apply_sparse_pca(sparsePCA_l1_temp.fc1,
                                           n_components=50, alpha=0.5)

# the adjusted model
fc1_out_features = 50
fc2_out_features = 32
SparsePCA_l1_50 = CompressedRegressionNN(input_size, fc1_out_features,
                                          fc2_out_features, output_size)

# Update the adjusted model with SparsePCA transformed weights
SparsePCA_l1_50.fc1.weight.data = torch.FloatTensor(l1_50_transformed_fc1_weights).to(sparsePCA_l1_temp.fc1.weight.device)

# Reset biases to zero because they are not specifically adjusted post-PCA
SparsePCA_l1_50.fc1.bias.data.fill_(0)
SparsePCA_l1_50.fc2.bias.data.fill_(0)
SparsePCA_l1_50.fc3.bias.data.fill_(0)

# testing without training
average_loss, average_mae = evaluate_model(SparsePCA_l1_50, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=SparsePCA_l1_50, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

##### no_components = 30

In [ ]:
# create copy of the model
sparsePCA_l13_temp = RegressionNN(input_size, hidden_size1,
                                         hidden_size2, output_size)
transfer_pruned_weights(pruned_local1, sparsePCA_l13_temp)

# Apply SparsePCA to fc1 and fc2 of the pruned model
l1_30_transformed_fc1_weights = apply_sparse_pca(sparsePCA_l13_temp.fc1,
                                           n_components=30, alpha=0.5)

# the adjusted model
fc1_out_features = 30
fc2_out_features = 32
SparsePCA_l1_30 = CompressedRegressionNN(input_size, fc1_out_features,
                                          fc2_out_features, output_size)

# Update the adjusted model with SparsePCA l1_30_transformed weights
SparsePCA_l1_30.fc1.weight.data = torch.FloatTensor(l1_30_transformed_fc1_weights).to(sparsePCA_l13_temp.fc1.weight.device)

# Reset biases to zero because they are not specifically adjusted post-PCA
SparsePCA_l1_30.fc1.bias.data.fill_(0)
SparsePCA_l1_30.fc2.bias.data.fill_(0)
SparsePCA_l1_30.fc3.bias.data.fill_(0)

# testing without training
average_loss, average_mae = evaluate_model(SparsePCA_l1_30, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=SparsePCA_l1_30, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

##### no_components = 10

In [ ]:
# create copy of the model
sparsePCA_l110_temp = RegressionNN(input_size, hidden_size1,
                                         hidden_size2, output_size)
transfer_pruned_weights(pruned_local1, sparsePCA_l110_temp)

# Apply SparsePCA to fc1 and fc2 of the pruned model
l1_10_transformed_fc1_weights = apply_sparse_pca(sparsePCA_l110_temp.fc1,
                                           n_components=10, alpha=0.5)

# the adjusted model
fc1_out_features = 10
fc2_out_features = 32
SparsePCA_l1_10 = CompressedRegressionNN(input_size, fc1_out_features,
                                          fc2_out_features, output_size)

# Update the adjusted model with SparsePCA l1_10_transformed weights
SparsePCA_l1_10.fc1.weight.data = torch.FloatTensor(l1_10_transformed_fc1_weights).to(sparsePCA_l110_temp.fc1.weight.device)

# Reset biases to zero because they are not specifically adjusted post-PCA
SparsePCA_l1_10.fc1.bias.data.fill_(0)
SparsePCA_l1_10.fc2.bias.data.fill_(0)
SparsePCA_l1_10.fc3.bias.data.fill_(0)

# testing without training
average_loss, average_mae = evaluate_model(SparsePCA_l1_10, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=SparsePCA_l1_10, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

#### SparsePCA on the Pruned Model, `pruned_local2`, with Pruning Amount=0.7

##### no_components = 50

In [ ]:
# create copy of the model
sparse_l2_temp = RegressionNN(input_size, hidden_size1,
                                         hidden_size2, output_size)
transfer_pruned_weights(pruned_local2, sparse_l2_temp)

In [ ]:
# Apply SparsePCA to fc1 and fc2 of the pruned model
l2_50_transformed_fc1_weights = apply_sparse_pca(sparse_l2_temp.fc1,
                                           n_components=50, alpha=0.5)

# the adjusted model
fc1_out_features = 50
fc2_out_features = 32
SparsePCA_l2_50 = CompressedRegressionNN(input_size, fc1_out_features,
                                          fc2_out_features, output_size)

# Update the adjusted model with SparsePCA l2_50_transformed weights
SparsePCA_l2_50.fc1.weight.data = torch.FloatTensor(l2_50_transformed_fc1_weights).to(sparse_l2_temp.fc1.weight.device)

# Reset biases to zero because they are not specifically adjusted post-PCA
SparsePCA_l2_50.fc1.bias.data.fill_(0)
SparsePCA_l2_50.fc2.bias.data.fill_(0)
SparsePCA_l2_50.fc3.bias.data.fill_(0)

# testing without training
average_loss, average_mae = evaluate_model(SparsePCA_l2_50, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=SparsePCA_l2_50, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

##### no_components = 30

In [ ]:
# create copy of the model
sparse_l230_temp = RegressionNN(input_size, hidden_size1,
                                         hidden_size2, output_size)
transfer_pruned_weights(pruned_local2, sparse_l230_temp)

# Apply SparsePCA to fc1 and fc2 of the pruned model
l2_30_transformed_fc1_weights = apply_sparse_pca(sparse_l230_temp.fc1,
                                           n_components=30, alpha=0.5)

# the adjusted model
fc1_out_features = 30
fc2_out_features = 32
SparsePCA_l2_30 = CompressedRegressionNN(input_size, fc1_out_features,
                                          fc2_out_features, output_size)

# Update the adjusted model with SparsePCA l2_30_transformed weights
SparsePCA_l2_30.fc1.weight.data = torch.FloatTensor(l2_30_transformed_fc1_weights).to(sparse_l230_temp.fc1.weight.device)

# Reset biases to zero because they are not specifically adjusted post-PCA
SparsePCA_l2_30.fc1.bias.data.fill_(0)
SparsePCA_l2_30.fc2.bias.data.fill_(0)
SparsePCA_l2_30.fc3.bias.data.fill_(0)

# testing without training
average_loss, average_mae = evaluate_model(SparsePCA_l2_30, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=SparsePCA_l2_30, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

##### no_components = 10

In [ ]:
# create copy of the model
sparse_l210_temp = RegressionNN(input_size, hidden_size1,
                                         hidden_size2, output_size)
transfer_pruned_weights(pruned_local2, sparse_l210_temp)

# Apply SparsePCA to fc1 and fc2 of the pruned model
l2_10_transformed_fc1_weights = apply_sparse_pca(sparse_l210_temp.fc1,
                                           n_components=10, alpha=0.5)

# the adjusted model
fc1_out_features = 10
fc2_out_features = 32
SparsePCA_l2_10 = CompressedRegressionNN(input_size, fc1_out_features,
                                          fc2_out_features, output_size)

# Update the adjusted model with SparsePCA l2_10_transformed weights
SparsePCA_l2_10.fc1.weight.data = torch.FloatTensor(l2_10_transformed_fc1_weights).to(sparse_l210_temp.fc1.weight.device)

# Reset biases to zero because they are not specifically adjusted post-PCA
SparsePCA_l2_10.fc1.bias.data.fill_(0)
SparsePCA_l2_10.fc2.bias.data.fill_(0)
SparsePCA_l2_10.fc3.bias.data.fill_(0)

# testing without training
average_loss, average_mae = evaluate_model(SparsePCA_l2_10, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=SparsePCA_l2_10, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

#### SparsePCA on the Pruned Model, `pruned_local3`, with Pruning Amount=0.8

##### no_components = 50

In [ ]:
# create copy of the model
sparse_l3_temp = RegressionNN(input_size, hidden_size1, hidden_size2, output_size)
transfer_pruned_weights(pruned_local3, sparse_l3_temp)

In [ ]:
# Apply SparsePCA to fc1 and fc2 of the pruned model
l3_50_transformed_fc1_weights = apply_sparse_pca(sparse_l3_temp.fc1,
                                           n_components=50, alpha=0.5)

# the adjusted model
fc1_out_features = 50
fc2_out_features = 32
SparsePCA_l3_50 = CompressedRegressionNN(input_size, fc1_out_features,
                                          fc2_out_features, output_size)

# Update the adjusted model with SparsePCA transformed weights
SparsePCA_l3_50.fc1.weight.data = torch.FloatTensor(l3_50_transformed_fc1_weights).to(sparse_l3_temp.fc1.weight.device)

# Reset biases to zero because they are not specifically adjusted post-PCA
SparsePCA_l3_50.fc1.bias.data.fill_(0)
SparsePCA_l3_50.fc2.bias.data.fill_(0)
SparsePCA_l3_50.fc3.bias.data.fill_(0)

# testing without training
average_loss, average_mae = evaluate_model(SparsePCA_l3_50, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=SparsePCA_l3_50, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

##### no_components = 30

In [ ]:
# create copy of the model
sparse_l330_temp = RegressionNN(input_size, hidden_size1, hidden_size2, output_size)
transfer_pruned_weights(pruned_local3, sparse_l330_temp)

# Apply SparsePCA to fc1 and fc2 of the pruned model
l3_30_transformed_fc1_weights = apply_sparse_pca(sparse_l330_temp.fc1,
                                           n_components=30, alpha=0.5)

# the adjusted model
fc1_out_features = 30
fc2_out_features = 32
SparsePCA_l3_30 = CompressedRegressionNN(input_size, fc1_out_features,
                                          fc2_out_features, output_size)

# Update the adjusted model with SparsePCA transformed weights
SparsePCA_l3_30.fc1.weight.data = torch.FloatTensor(l3_30_transformed_fc1_weights).to(sparse_l330_temp.fc1.weight.device)

# Reset biases to zero because they are not specifically adjusted post-PCA
SparsePCA_l3_30.fc1.bias.data.fill_(0)
SparsePCA_l3_30.fc2.bias.data.fill_(0)
SparsePCA_l3_30.fc3.bias.data.fill_(0)

# testing without training
average_loss, average_mae = evaluate_model(SparsePCA_l3_30, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=SparsePCA_l3_30, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

##### no_components = 10

In [ ]:
# create copy of the model
sparse_l310_temp = RegressionNN(input_size, hidden_size1, hidden_size2, output_size)
transfer_pruned_weights(pruned_local3, sparse_l310_temp)
# Apply SparsePCA to fc1 and fc2 of the pruned model
l3_10_transformed_fc1_weights = apply_sparse_pca(sparse_l310_temp.fc1,
                                           n_components=10, alpha=0.5)

# the adjusted model
fc1_out_features = 10
fc2_out_features = 32
SparsePCA_l3_10 = CompressedRegressionNN(input_size, fc1_out_features,
                                          fc2_out_features, output_size)

# Update the adjusted model with SparsePCA transformed weights
SparsePCA_l3_10.fc1.weight.data = torch.FloatTensor(l3_10_transformed_fc1_weights).to(sparse_l310_temp.fc1.weight.device)

# Reset biases to zero because they are not specifically adjusted post-PCA
SparsePCA_l3_10.fc1.bias.data.fill_(0)
SparsePCA_l3_10.fc2.bias.data.fill_(0)
SparsePCA_l3_10.fc3.bias.data.fill_(0)

# testing without training
average_loss, average_mae = evaluate_model(SparsePCA_l3_10, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=SparsePCA_l3_10, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

#### SparsePCA on the Pruned Model, `pruned_local4`, with Pruning Amount=0.9

##### no_components = 50

In [ ]:
# create copy of the model
sparse_l4_temp = RegressionNN(input_size, hidden_size1,
                                         hidden_size2, output_size)
transfer_pruned_weights(pruned_local4, sparse_l4_temp)

In [ ]:
# Apply SparsePCA to fc1 and fc2 of the pruned model
l4_50_transformed_fc1_weights = apply_sparse_pca(sparse_l4_temp.fc1,
                                           n_components=50, alpha=0.5)

# the adjusted model
fc1_out_features = 50
fc2_out_features = 32
SparsePCA_l4_50 = CompressedRegressionNN(input_size, fc1_out_features,
                                          fc2_out_features, output_size)

# Update the adjusted model with SparsePCA transformed weights
SparsePCA_l4_50.fc1.weight.data = torch.FloatTensor(l4_50_transformed_fc1_weights).to(sparse_l4_temp.fc1.weight.device)

# Reset biases to zero because they are not specifically adjusted post-PCA
SparsePCA_l4_50.fc1.bias.data.fill_(0)
SparsePCA_l4_50.fc2.bias.data.fill_(0)
SparsePCA_l4_50.fc3.bias.data.fill_(0)

# testing without training
average_loss, average_mae = evaluate_model(SparsePCA_l4_50, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=SparsePCA_l4_50, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

##### no_components = 30

In [ ]:
# create copy of the model
sparse_l430_temp = RegressionNN(input_size, hidden_size1,
                                         hidden_size2, output_size)
transfer_pruned_weights(pruned_local4, sparse_l430_temp)
# Apply SparsePCA to fc1 and fc2 of the pruned model
l4_30_transformed_fc1_weights = apply_sparse_pca(sparse_l430_temp.fc1,
                                           n_components=30, alpha=0.5)

# the adjusted model
fc1_out_features = 30
fc2_out_features = 32
SparsePCA_l4_30 = CompressedRegressionNN(input_size, fc1_out_features,
                                       fc2_out_features, output_size)

# Update the adjusted model with SparsePCA transformed weights
SparsePCA_l4_30.fc1.weight.data = torch.FloatTensor(l4_30_transformed_fc1_weights).to(sparse_l430_temp.fc1.weight.device)

# Reset biases to zero because they are not specifically adjusted post-PCA
SparsePCA_l4_30.fc1.bias.data.fill_(0)
SparsePCA_l4_30.fc2.bias.data.fill_(0)
SparsePCA_l4_30.fc3.bias.data.fill_(0)

# testing without training
average_loss, average_mae = evaluate_model(SparsePCA_l4_30, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=SparsePCA_l4_30, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

##### no_components = 10

In [ ]:
# create copy of the model
sparse_l410_temp = RegressionNN(input_size, hidden_size1,
                                         hidden_size2, output_size)
transfer_pruned_weights(pruned_local4, sparse_l410_temp)

# Apply SparsePCA to fc1 and fc2 of the pruned model
l4_10_transformed_fc1_weights = apply_sparse_pca(sparse_l410_temp.fc1,
                                           n_components=10, alpha=0.5)

# the adjusted model
fc1_out_features = 10
fc2_out_features = 32
SparsePCA_l4_10 = CompressedRegressionNN(input_size, fc1_out_features,
                                          fc2_out_features, output_size)

# Update the adjusted model with SparsePCA transformed weights
SparsePCA_l4_10.fc1.weight.data = torch.FloatTensor(l4_10_transformed_fc1_weights).to(sparse_l410_temp.fc1.weight.device)

# Reset biases to zero because they are not specifically adjusted post-PCA
SparsePCA_l4_10.fc1.bias.data.fill_(0)
SparsePCA_l4_10.fc2.bias.data.fill_(0)
SparsePCA_l4_10.fc3.bias.data.fill_(0)

# testing without training
average_loss, average_mae = evaluate_model(SparsePCA_l4_10, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=SparsePCA_l4_10, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

#### SparsePCA on the Pruned Model, `pruned_local5`, with Pruning Amount=0.95

##### no_components = 50

In [ ]:
# create copy of the model
sparse_l5_temp = RegressionNN(input_size, hidden_size1,
                                         hidden_size2, output_size)
transfer_pruned_weights(pruned_local5, sparse_l5_temp)

In [ ]:
# Apply SparsePCA to fc1 and fc2 of the pruned model
l5_50_transformed_fc1_weights = apply_sparse_pca(sparse_l5_temp.fc1,
                                           n_components=50, alpha=0.5)

# the adjusted model
fc1_out_features = 50
fc2_out_features = 32
SparsePCA_l5_50 = CompressedRegressionNN(input_size, fc1_out_features,
                                          fc2_out_features, output_size)

# Update the adjusted model with SparsePCA transformed weights
SparsePCA_l5_50.fc1.weight.data = torch.FloatTensor(l5_50_transformed_fc1_weights).to(sparse_l5_temp.fc1.weight.device)

# Reset biases to zero because they are not specifically adjusted post-PCA
SparsePCA_l5_50.fc1.bias.data.fill_(0)
SparsePCA_l5_50.fc2.bias.data.fill_(0)
SparsePCA_l5_50.fc3.bias.data.fill_(0)

# testing without training
average_loss, average_mae = evaluate_model(SparsePCA_l5_50, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=SparsePCA_l5_50, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

##### no_components = 30

In [ ]:
# create copy of the model
sparse_l530_temp = RegressionNN(input_size, hidden_size1,
                                         hidden_size2, output_size)
transfer_pruned_weights(pruned_local5, sparse_l530_temp)

# Apply SparsePCA to fc1 and fc2 of the pruned model
l5_30_transform_fc1_weights = apply_sparse_pca(sparse_l530_temp.fc1,
                                           n_components=30, alpha=0.5)

# the adjusted model
fc1_out_features = 30
fc2_out_features = 32
SparsePCA_l5_30 = CompressedRegressionNN(input_size, fc1_out_features,
                                          fc2_out_features, output_size)

# Update the adjusted model with SparsePCA transformed weights
SparsePCA_l5_30.fc1.weight.data = torch.FloatTensor(l5_30_transform_fc1_weights).to(sparse_l530_temp.fc1.weight.device)

# Reset biases to zero because they are not specifically adjusted post-PCA
SparsePCA_l5_30.fc1.bias.data.fill_(0)
SparsePCA_l5_30.fc2.bias.data.fill_(0)
SparsePCA_l5_30.fc3.bias.data.fill_(0)

# testing without training
average_loss, average_mae = evaluate_model(SparsePCA_l5_30, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=SparsePCA_l5_30, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

##### no_components = 10

In [ ]:
# create copy of the model
sparse_l510_temp = RegressionNN(input_size, hidden_size1,
                                         hidden_size2, output_size)
transfer_pruned_weights(pruned_local5, sparse_l510_temp)

# Apply SparsePCA to fc1 and fc2 of the pruned model
l5_10_transformed_fc1_weights = apply_sparse_pca(sparse_l510_temp.fc1,
                                           n_components=10, alpha=0.5)

# the adjusted model
fc1_out_features = 10
fc2_out_features = 32
SparsePCA_l5_10 = CompressedRegressionNN(input_size, fc1_out_features,
                                          fc2_out_features, output_size)

# Update the adjusted model with SparsePCA transformed weights
SparsePCA_l5_10.fc1.weight.data = torch.FloatTensor(l5_10_transformed_fc1_weights).to(sparse_l510_temp.fc1.weight.device)

# Reset biases to zero because they are not specifically adjusted post-PCA
SparsePCA_l5_10.fc1.bias.data.fill_(0)
SparsePCA_l5_10.fc2.bias.data.fill_(0)
SparsePCA_l5_10.fc3.bias.data.fill_(0)

# testing without training
average_loss, average_mae = evaluate_model(SparsePCA_l5_10, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=SparsePCA_l5_10, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

### SparsePCA on Layer-Wise Locally Pruned Models

#### SparsePCA on the Pruned Model, `pruned_sparse1`, with Pruning Amount=0.6

##### no_components = 50

In [ ]:
# create copy of the model
SparsePCA_sparse1 = RegressionNN(input_size, hidden_size1,
                                         hidden_size2, output_size)
transfer_pruned_weights(pruned_sparse1, SparsePCA_sparse1)

In [ ]:
# Apply SparsePCA to fc1 and fc2 of the pruned model
transformed_fc1_weights = apply_sparse_pca(SparsePCA_sparse1.fc1,
                                           n_components=50, alpha=0.5)

# the adjusted model
fc1_out_features = 50
fc2_out_features = 32
SparsePCA_sparse1_50 = CompressedRegressionNN(input_size, fc1_out_features,
                                          fc2_out_features, output_size)

# Update the adjusted model with SparsePCA transformed weights
SparsePCA_sparse1_50.fc1.weight.data = torch.FloatTensor(transformed_fc1_weights).to(SparsePCA_sparse1.fc1.weight.device)

# Reset biases to zero because they are not specifically adjusted post-PCA
SparsePCA_sparse1_50.fc1.bias.data.fill_(0)
SparsePCA_sparse1_50.fc2.bias.data.fill_(0)
SparsePCA_sparse1_50.fc3.bias.data.fill_(0)

# testing without training
average_loss, average_mae = evaluate_model(SparsePCA_sparse1_50, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=SparsePCA_sparse1_50, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

##### no_components = 30

In [ ]:
# create copy of the model
SparsePCA_sparse130 = RegressionNN(input_size, hidden_size1,
                                         hidden_size2, output_size)
transfer_pruned_weights(pruned_sparse1, SparsePCA_sparse130)

# Apply SparsePCA to fc1 and fc2 of the pruned model
transformed_fc1_weights = apply_sparse_pca(SparsePCA_sparse130.fc1,
                                           n_components=30, alpha=0.5)

# the adjusted model
fc1_out_features = 30
fc2_out_features = 32
SparsePCA_sparse1_30 = CompressedRegressionNN(input_size, fc1_out_features,
                                          fc2_out_features, output_size)

# Update the adjusted model with SparsePCA transformed weights
SparsePCA_sparse1_30.fc1.weight.data = torch.FloatTensor(transformed_fc1_weights).to(SparsePCA_sparse130.fc1.weight.device)

# Reset biases to zero because they are not specifically adjusted post-PCA
SparsePCA_sparse1_30.fc1.bias.data.fill_(0)
SparsePCA_sparse1_30.fc2.bias.data.fill_(0)
SparsePCA_sparse1_30.fc3.bias.data.fill_(0)

# testing without training
average_loss, average_mae = evaluate_model(SparsePCA_sparse1_30, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=SparsePCA_sparse1_30, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

##### no_components = 10

In [ ]:
# create copy of the model
SparsePCA_sparse110 = RegressionNN(input_size, hidden_size1,
                                         hidden_size2, output_size)
transfer_pruned_weights(pruned_sparse1, SparsePCA_sparse110)

# Apply SparsePCA to fc1 and fc2 of the pruned model
transformed_fc1_weights = apply_sparse_pca(SparsePCA_sparse110.fc1,
                                           n_components=10, alpha=0.5)

# the adjusted model
fc1_out_features = 10
fc2_out_features = 32
SparsePCA_sparse1_10 = CompressedRegressionNN(input_size, fc1_out_features,
                                          fc2_out_features, output_size)

# Update the adjusted model with SparsePCA transformed weights
SparsePCA_sparse1_10.fc1.weight.data = torch.FloatTensor(transformed_fc1_weights).to(SparsePCA_sparse110.fc1.weight.device)

# Reset biases to zero because they are not specifically adjusted post-PCA
SparsePCA_sparse1_10.fc1.bias.data.fill_(0)
SparsePCA_sparse1_10.fc2.bias.data.fill_(0)
SparsePCA_sparse1_10.fc3.bias.data.fill_(0)

# testing without training
average_loss, average_mae = evaluate_model(SparsePCA_sparse1_10, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=SparsePCA_sparse1_10, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

#### SparsePCA on the Pruned Model, `pruned_sparse2`, with Pruning Amount=0.7

##### no_components = 50

In [ ]:
# create copy of the model
SparsePCA_sparse2 = RegressionNN(input_size, hidden_size1,
                                         hidden_size2, output_size)
transfer_pruned_weights(pruned_sparse2, SparsePCA_sparse2)

In [ ]:
# Apply SparsePCA to fc1 and fc2 of the pruned model
transformed_fc1_weights = apply_sparse_pca(SparsePCA_sparse2.fc1,
                                           n_components=50, alpha=0.5)

# the adjusted model
fc1_out_features = 50
fc2_out_features = 32
SparsePCA_sparse2_50 = CompressedRegressionNN(input_size, fc1_out_features,
                                          fc2_out_features, output_size)

# Update the adjusted model with SparsePCA transformed weights
SparsePCA_sparse2_50.fc1.weight.data = torch.FloatTensor(transformed_fc1_weights).to(SparsePCA_sparse2.fc1.weight.device)

# Reset biases to zero because they are not specifically adjusted post-PCA
SparsePCA_sparse2_50.fc1.bias.data.fill_(0)
SparsePCA_sparse2_50.fc2.bias.data.fill_(0)
SparsePCA_sparse2_50.fc3.bias.data.fill_(0)

# testing without training
average_loss, average_mae = evaluate_model(SparsePCA_sparse2_50, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=SparsePCA_sparse2_50, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

##### no_components = 30

In [ ]:
# create copy of the model
SparsePCA_sparse230 = RegressionNN(input_size, hidden_size1,
                                         hidden_size2, output_size)
transfer_pruned_weights(pruned_sparse2, SparsePCA_sparse230)

# Apply SparsePCA to fc1 and fc2 of the pruned model
transformed_fc1_weights = apply_sparse_pca(SparsePCA_sparse230.fc1,
                                           n_components=30, alpha=0.5)

# the adjusted model
fc1_out_features = 30
fc2_out_features = 32
SparsePCA_sparse2_30 = CompressedRegressionNN(input_size, fc1_out_features,
                                          fc2_out_features, output_size)

# Update the adjusted model with SparsePCA transformed weights
SparsePCA_sparse2_30.fc1.weight.data = torch.FloatTensor(transformed_fc1_weights).to(SparsePCA_sparse230.fc1.weight.device)

# Reset biases to zero because they are not specifically adjusted post-PCA
SparsePCA_sparse2_30.fc1.bias.data.fill_(0)
SparsePCA_sparse2_30.fc2.bias.data.fill_(0)
SparsePCA_sparse2_30.fc3.bias.data.fill_(0)

# testing without training
average_loss, average_mae = evaluate_model(SparsePCA_sparse2_30, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=SparsePCA_sparse2_30, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

##### no_components = 10

In [ ]:
# create copy of the model
SparsePCA_sparse210 = RegressionNN(input_size, hidden_size1,
                                         hidden_size2, output_size)
transfer_pruned_weights(pruned_sparse2, SparsePCA_sparse210)

# Apply SparsePCA to fc1 and fc2 of the pruned model
transformed_fc1_weights = apply_sparse_pca(SparsePCA_sparse210.fc1,
                                           n_components=10, alpha=0.5)

# the adjusted model
fc1_out_features = 10
fc2_out_features = 32
SparsePCA_sparse2_10 = CompressedRegressionNN(input_size, fc1_out_features,
                                          fc2_out_features, output_size)

# Update the adjusted model with SparsePCA transformed weights
SparsePCA_sparse2_10.fc1.weight.data = torch.FloatTensor(transformed_fc1_weights).to(SparsePCA_sparse210.fc1.weight.device)

# Reset biases to zero because they are not specifically adjusted post-PCA
SparsePCA_sparse2_10.fc1.bias.data.fill_(0)
SparsePCA_sparse2_10.fc2.bias.data.fill_(0)
SparsePCA_sparse2_10.fc3.bias.data.fill_(0)

# testing without training
average_loss, average_mae = evaluate_model(SparsePCA_sparse2_10, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=SparsePCA_sparse2_10, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

#### SparsePCA on the Pruned Model, `pruned_sparse3`, with Pruning Amount=0.8

##### no_components = 50

In [ ]:
# create copy of the model
SparsePCA_sparse3 = RegressionNN(input_size, hidden_size1,
                                         hidden_size2, output_size)
transfer_pruned_weights(pruned_sparse3, SparsePCA_sparse3)

In [ ]:
# Apply SparsePCA to fc1 and fc2 of the pruned model
transformed_fc1_weights = apply_sparse_pca(SparsePCA_sparse3.fc1,
                                           n_components=50, alpha=0.5)

# the adjusted model
fc1_out_features = 50
fc2_out_features = 32
SparsePCA_sparse3_50 = CompressedRegressionNN(input_size, fc1_out_features,
                                          fc2_out_features, output_size)

# Update the adjusted model with SparsePCA transformed weights
SparsePCA_sparse3_50.fc1.weight.data = torch.FloatTensor(transformed_fc1_weights).to(SparsePCA_sparse3.fc1.weight.device)

# Reset biases to zero because they are not specifically adjusted post-PCA
SparsePCA_sparse3_50.fc1.bias.data.fill_(0)
SparsePCA_sparse3_50.fc2.bias.data.fill_(0)
SparsePCA_sparse3_50.fc3.bias.data.fill_(0)

# testing without training
average_loss, average_mae = evaluate_model(SparsePCA_sparse3_50, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=SparsePCA_sparse3_50, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

##### no_components = 30

In [ ]:
# create copy of the model
SparsePCA_sparse330 = RegressionNN(input_size, hidden_size1,
                                         hidden_size2, output_size)
transfer_pruned_weights(pruned_sparse3, SparsePCA_sparse330)

# Apply SparsePCA to fc1 and fc2 of the pruned model
transformed_fc1_weights = apply_sparse_pca(SparsePCA_sparse330.fc1,
                                           n_components=30, alpha=0.5)

# the adjusted model
fc1_out_features = 30
fc2_out_features = 32
SparsePCA_sparse3_30 = CompressedRegressionNN(input_size, fc1_out_features,
                                          fc2_out_features, output_size)

# Update the adjusted model with SparsePCA transformed weights
SparsePCA_sparse3_30.fc1.weight.data = torch.FloatTensor(transformed_fc1_weights).to(SparsePCA_sparse330.fc1.weight.device)

# Reset biases to zero because they are not specifically adjusted post-PCA
SparsePCA_sparse3_30.fc1.bias.data.fill_(0)
SparsePCA_sparse3_30.fc2.bias.data.fill_(0)
SparsePCA_sparse3_30.fc3.bias.data.fill_(0)

# testing without training
average_loss, average_mae = evaluate_model(SparsePCA_sparse3_30, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=SparsePCA_sparse3_30, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

##### no_components = 10

In [ ]:
# create copy of the model
SparsePCA_sparse310 = RegressionNN(input_size, hidden_size1,
                                         hidden_size2, output_size)
transfer_pruned_weights(pruned_sparse3, SparsePCA_sparse310)

# Apply SparsePCA to fc1 and fc2 of the pruned model
transformed_fc1_weights = apply_sparse_pca(SparsePCA_sparse310.fc1,
                                           n_components=10, alpha=0.5)

# the adjusted model
fc1_out_features = 10
fc2_out_features = 32
SparsePCA_sparse3_10 = CompressedRegressionNN(input_size, fc1_out_features,
                                          fc2_out_features, output_size)

# Update the adjusted model with SparsePCA transformed weights
SparsePCA_sparse3_10.fc1.weight.data = torch.FloatTensor(transformed_fc1_weights).to(SparsePCA_sparse310.fc1.weight.device)

# Reset biases to zero because they are not specifically adjusted post-PCA
SparsePCA_sparse3_10.fc1.bias.data.fill_(0)
SparsePCA_sparse3_10.fc2.bias.data.fill_(0)
SparsePCA_sparse3_10.fc3.bias.data.fill_(0)

# testing without training
average_loss, average_mae = evaluate_model(SparsePCA_sparse3_10, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=SparsePCA_sparse3_10, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

#### SparsePCA on the Pruned Model, `pruned_sparse4`, with Pruning Amount=0.9

##### no_components = 50

In [ ]:
# create copy of the model
SparsePCA_sparse4 = RegressionNN(input_size, hidden_size1,
                                         hidden_size2, output_size)
transfer_pruned_weights(pruned_sparse4, SparsePCA_sparse4)

In [ ]:
# Apply SparsePCA to fc1 and fc2 of the pruned model
transformed_fc1_weights = apply_sparse_pca(SparsePCA_sparse4.fc1,
                                           n_components=50, alpha=0.5)

# the adjusted model
fc1_out_features = 50
fc2_out_features = 32
SparsePCA_sparse4_50 = CompressedRegressionNN(input_size, fc1_out_features,
                                          fc2_out_features, output_size)

# Update the adjusted model with SparsePCA transformed weights
SparsePCA_sparse4_50.fc1.weight.data = torch.FloatTensor(transformed_fc1_weights).to(SparsePCA_sparse4.fc1.weight.device)

# Reset biases to zero because they are not specifically adjusted post-PCA
SparsePCA_sparse4_50.fc1.bias.data.fill_(0)
SparsePCA_sparse4_50.fc2.bias.data.fill_(0)
SparsePCA_sparse4_50.fc3.bias.data.fill_(0)

# testing without training
average_loss, average_mae = evaluate_model(SparsePCA_sparse4_50, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=SparsePCA_sparse4_50, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

##### no_components = 30

In [ ]:
# create copy of the model
SparsePCA_sparse430 = RegressionNN(input_size, hidden_size1,
                                         hidden_size2, output_size)
transfer_pruned_weights(pruned_sparse4, SparsePCA_sparse430)

# Apply SparsePCA to fc1 and fc2 of the pruned model
transformed_fc1_weights = apply_sparse_pca(SparsePCA_sparse430.fc1,
                                           n_components=30, alpha=0.5)

# the adjusted model
fc1_out_features = 30
fc2_out_features = 32
SparsePCA_sparse4_30 = CompressedRegressionNN(input_size, fc1_out_features,
                                          fc2_out_features, output_size)

# Update the adjusted model with SparsePCA transformed weights
SparsePCA_sparse4_30.fc1.weight.data = torch.FloatTensor(transformed_fc1_weights).to(SparsePCA_sparse430.fc1.weight.device)

# Reset biases to zero because they are not specifically adjusted post-PCA
SparsePCA_sparse4_30.fc1.bias.data.fill_(0)
SparsePCA_sparse4_30.fc2.bias.data.fill_(0)
SparsePCA_sparse4_30.fc3.bias.data.fill_(0)

# testing without training
average_loss, average_mae = evaluate_model(SparsePCA_sparse4_30, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=SparsePCA_sparse4_30, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

##### no_components = 10

In [ ]:
# create copy of the model
SparsePCA_sparse410 = RegressionNN(input_size, hidden_size1,
                                         hidden_size2, output_size)
transfer_pruned_weights(pruned_sparse4, SparsePCA_sparse410)

# Apply SparsePCA to fc1 and fc2 of the pruned model
transformed_fc1_weights = apply_sparse_pca(SparsePCA_sparse410.fc1,
                                           n_components=10, alpha=0.5)

# the adjusted model
fc1_out_features = 10
fc2_out_features = 32
SparsePCA_sparse4_10 = CompressedRegressionNN(input_size, fc1_out_features,
                                          fc2_out_features, output_size)

# Update the adjusted model with SparsePCA transformed weights
SparsePCA_sparse4_10.fc1.weight.data = torch.FloatTensor(transformed_fc1_weights).to(SparsePCA_sparse410.fc1.weight.device)

# Reset biases to zero because they are not specifically adjusted post-PCA
SparsePCA_sparse4_10.fc1.bias.data.fill_(0)
SparsePCA_sparse4_10.fc2.bias.data.fill_(0)
SparsePCA_sparse4_10.fc3.bias.data.fill_(0)

# testing without training
average_loss, average_mae = evaluate_model(SparsePCA_sparse4_10, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=SparsePCA_sparse4_10, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

#### SparsePCA on the Pruned Model, `pruned_sparse5`, with Pruning Amount=0.95

##### no_components = 50

In [ ]:
# create copy of the model
SparsePCA_sparse5 = RegressionNN(input_size, hidden_size1,
                                         hidden_size2, output_size)
transfer_pruned_weights(pruned_sparse5, SparsePCA_sparse5)

In [ ]:
# Apply SparsePCA to fc1 and fc2 of the pruned model
transformed_fc1_weights = apply_sparse_pca(SparsePCA_sparse5.fc1,
                                           n_components=50, alpha=0.5)

# the adjusted model
fc1_out_features = 50
fc2_out_features = 32
SparsePCA_sparse5_50 = CompressedRegressionNN(input_size, fc1_out_features,
                                          fc2_out_features, output_size)

# Update the adjusted model with SparsePCA transformed weights
SparsePCA_sparse5_50.fc1.weight.data = torch.FloatTensor(transformed_fc1_weights).to(SparsePCA_sparse5.fc1.weight.device)

# Reset biases to zero because they are not specifically adjusted post-PCA
SparsePCA_sparse5_50.fc1.bias.data.fill_(0)
SparsePCA_sparse5_50.fc2.bias.data.fill_(0)
SparsePCA_sparse5_50.fc3.bias.data.fill_(0)

# testing without training
average_loss, average_mae = evaluate_model(SparsePCA_sparse5_50, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=SparsePCA_sparse5_50, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

##### no_components = 30

In [ ]:
# create copy of the model
SparsePCA_sparse530 = RegressionNN(input_size, hidden_size1,
                                         hidden_size2, output_size)
transfer_pruned_weights(pruned_sparse5, SparsePCA_sparse530)

# Apply SparsePCA to fc1 and fc2 of the pruned model
transformed_fc1_weights = apply_sparse_pca(SparsePCA_sparse530.fc1,
                                           n_components=30, alpha=0.5)

# the adjusted model
fc1_out_features = 30
fc2_out_features = 32
SparsePCA_sparse5_30 = CompressedRegressionNN(input_size, fc1_out_features,
                                          fc2_out_features, output_size)

# Update the adjusted model with SparsePCA transformed weights
SparsePCA_sparse5_30.fc1.weight.data = torch.FloatTensor(transformed_fc1_weights).to(SparsePCA_sparse530.fc1.weight.device)

# Reset biases to zero because they are not specifically adjusted post-PCA
SparsePCA_sparse5_30.fc1.bias.data.fill_(0)
SparsePCA_sparse5_30.fc2.bias.data.fill_(0)
SparsePCA_sparse5_30.fc3.bias.data.fill_(0)

# testing without training
average_loss, average_mae = evaluate_model(SparsePCA_sparse5_30, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=SparsePCA_sparse5_30, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

##### no_components = 10

In [ ]:
# create copy of the model
SparsePCA_sparse510 = RegressionNN(input_size, hidden_size1,
                                         hidden_size2, output_size)
transfer_pruned_weights(pruned_sparse5, SparsePCA_sparse510)

# Apply SparsePCA to fc1 and fc2 of the pruned model
transformed_fc1_weights = apply_sparse_pca(SparsePCA_sparse510.fc1,
                                           n_components=10, alpha=0.5)

# the adjusted model
fc1_out_features = 10
fc2_out_features = 32
SparsePCA_sparse5_10 = CompressedRegressionNN(input_size, fc1_out_features,
                                          fc2_out_features, output_size)

# Update the adjusted model with SparsePCA transformed weights
SparsePCA_sparse5_10.fc1.weight.data = torch.FloatTensor(transformed_fc1_weights).to(SparsePCA_sparse510.fc1.weight.device)

# Reset biases to zero because they are not specifically adjusted post-PCA
SparsePCA_sparse5_10.fc1.bias.data.fill_(0)
SparsePCA_sparse5_10.fc2.bias.data.fill_(0)
SparsePCA_sparse5_10.fc3.bias.data.fill_(0)

# testing without training
average_loss, average_mae = evaluate_model(SparsePCA_sparse5_10, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=SparsePCA_sparse5_10, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

## Independent Component Analysis (ICA) for a Pruned Model

ICA can be applied to pruned model weights in a similar manner to PCA. The flattened weights from each linear layer are stacked into a matrix and ICA is applied on it to obtain the transformed weights in ica_result.

Reference:
https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.FastICA.html#sklearn.decomposition.FastICA

In [ ]:
# defining a function to apply ICA on the pruned model
def apply_ica_to_layer(layer, n_components):
    # interested in reducing the dimensionality of in_features
    # Transpose to focus on in_features
    weights = layer.weight.data.cpu().numpy().T
    # reference: https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.FastICA.html#sklearn.decomposition.FastICA
    ica = FastICA(n_components=n_components, random_state=0)
    transformed_weights = ica.fit_transform(weights)
    # Transpose back to match original shape orientation
    return transformed_weights.T

In [ ]:
# Define a new adjusted model with dimensions matching the ICA output
class AdjustedModel(RegressionNN):
    def __init__(self):
        super().__init__(input_size=58,
                         hidden_size1=n_components_fc1,
                         hidden_size2=32,
                         output_size=1)

### ICA on Locally Pruned Model

#### ICA on the Pruned Model, `pruned_local1`, with Pruning Amount=0.6

###### no_components = 50

In [ ]:
# create copy of the model
ICA_pruned_local1= RegressionNN(input_size, hidden_size1,
                                         hidden_size2, output_size)
transfer_pruned_weights(pruned_local1, ICA_pruned_local1)

In [ ]:
# Apply ICA to the first layer as an example
n_components_fc1 = 50  # New dimension for fc1
ical150_transformed_fc1_weights = apply_ica_to_layer(ICA_pruned_local1.fc1,
                                             n_components=n_components_fc1)

# Instantiate the adjusted model
ICA_l1_50 = AdjustedModel()

# Update the adjusted model with ICA transformed weights for fc1
ICA_l1_50.fc1.weight.data = torch.from_numpy(ical150_transformed_fc1_weights).float()

# testing without training
average_loss, average_mae = evaluate_model(ICA_l1_50, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=ICA_l1_50, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

##### no_components = 30

In [ ]:
# create copy of the model
ICA_pruned_local130= RegressionNN(input_size, hidden_size1,
                                         hidden_size2, output_size)
transfer_pruned_weights(pruned_local1, ICA_pruned_local130)


# Apply ICA to the first layer as an example
n_components_fc1 = 30  # New dimension for fc1
ical130_transofrmed_fc1_weights = apply_ica_to_layer(ICA_pruned_local130.fc1,
                                             n_components=n_components_fc1)

# Instantiate the adjusted model
ICA_l1_30 = AdjustedModel()

# Update the adjusted model with ICA transformed weights for fc1
ICA_l1_30.fc1.weight.data = torch.from_numpy(ical130_transofrmed_fc1_weights).float()

# testing without training
average_loss, average_mae = evaluate_model(ICA_l1_30, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=ICA_l1_30, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

##### no_components = 10

In [ ]:
# create copy of the model
ICA_pruned_local110= RegressionNN(input_size, hidden_size1,
                                         hidden_size2, output_size)
transfer_pruned_weights(pruned_local1, ICA_pruned_local110)

# Apply ICA to the first layer as an example
n_components_fc1 = 10  # New dimension for fc1
ical110_transformed_fc1_weights = apply_ica_to_layer(ICA_pruned_local110.fc1,
                                                     n_components=n_components_fc1)

# Instantiate the adjusted model
ICA_l1_10 = AdjustedModel()

# Update the adjusted model with ICA transformed weights for fc1
ICA_l1_10.fc1.weight.data = torch.from_numpy(ical110_transformed_fc1_weights).float()

# testing without training
average_loss, average_mae = evaluate_model(ICA_l1_10, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=ICA_l1_10, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

#### ICA on the Pruned Model, `pruned_local2`, with Pruning Amount=0.7

###### no_components = 50

In [ ]:
# create copy of the model
ICA_pruned_local2 = RegressionNN(input_size, hidden_size1,
                                         hidden_size2, output_size)
transfer_pruned_weights(pruned_local2, ICA_pruned_local2)

In [ ]:
# Apply ICA to the first layer as an example
n_components_fc1 = 50  # New dimension for fc1
ical250_transofrmed_fc1_weights = apply_ica_to_layer(ICA_pruned_local2.fc1,
                                             n_components=n_components_fc1)

# Instantiate the adjusted model
ICA_l2_50 = AdjustedModel()

# Update the adjusted model with ICA transformed weights for fc1
ICA_l2_50.fc1.weight.data = torch.from_numpy(ical250_transofrmed_fc1_weights).float()

# testing without training
average_loss, average_mae = evaluate_model(ICA_l2_50, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=ICA_l2_50, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

##### no_components = 30

In [ ]:
# create copy of the model
ICA_pruned_local230 = RegressionNN(input_size, hidden_size1,
                                         hidden_size2, output_size)
transfer_pruned_weights(pruned_local2, ICA_pruned_local230)

# Apply ICA to the first layer as an example
n_components_fc1 = 30  # New dimension for fc1
ical230_transformed_fc1_weights = apply_ica_to_layer(ICA_pruned_local230.fc1,
                                             n_components=n_components_fc1)

# Instantiate the adjusted model
ICA_l2_30 = AdjustedModel()

# Update the adjusted model with ICA transformed weights for fc1
ICA_l2_30.fc1.weight.data = torch.from_numpy(ical230_transformed_fc1_weights).float()

# testing without training
average_loss, average_mae = evaluate_model(ICA_l2_30, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=ICA_l2_30, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

##### no_components = 10

In [ ]:
# create copy of the model
ICA_local_210 = RegressionNN(input_size, hidden_size1,
                                         hidden_size2, output_size)
transfer_pruned_weights(pruned_local2, ICA_local_210)

# Apply ICA to the first layer as an example
n_components_fc1 = 10  # New dimension for fc1
ical210_transformed_fc1_weights = apply_ica_to_layer(ICA_local_210.fc1,
                                             n_components=n_components_fc1)

# Instantiate the adjusted model
ICA_l2_10 = AdjustedModel()

# Update the adjusted model with ICA transformed weights for fc1
ICA_l2_10.fc1.weight.data = torch.from_numpy(ical210_transformed_fc1_weights).float()

# testing without training
average_loss, average_mae = evaluate_model(ICA_l2_10, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=ICA_l2_10, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

#### ICA on the Pruned Model, `pruned_local3`, with Pruning Amount=0.8

###### no_components = 50

In [ ]:
# create copy of the model
ICA_pruned_local_08 = RegressionNN(input_size, hidden_size1,
                                         hidden_size2, output_size)
transfer_pruned_weights(pruned_local3, ICA_pruned_local_08)

In [ ]:
# Apply ICA to the first layer as an example
n_components_fc1 = 50  # New dimension for fc1
transformed_fc1_weights = apply_ica_to_layer(ICA_pruned_local_08.fc1,
                                             n_components=n_components_fc1)

# Instantiate the adjusted model
ICA_l3_50 = AdjustedModel()

# Update the adjusted model with ICA transformed weights for fc1
ICA_l3_50.fc1.weight.data = torch.from_numpy(transformed_fc1_weights).float()

# testing without training
average_loss, average_mae = evaluate_model(ICA_l3_50, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=ICA_l3_50, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

##### no_components = 30

In [ ]:
# create copy of the model
ICA_local330 = RegressionNN(input_size, hidden_size1,
                                         hidden_size2, output_size)
transfer_pruned_weights(pruned_local3, ICA_local330)

# Apply ICA to the first layer as an example
n_components_fc1 = 30  # New dimension for fc1
transformed_fc1_weights = apply_ica_to_layer(ICA_local330.fc1,
                                             n_components=n_components_fc1)

# Instantiate the adjusted model
ICA_l3_30 = AdjustedModel()

# Update the adjusted model with ICA transformed weights for fc1
ICA_l3_30.fc1.weight.data = torch.from_numpy(transformed_fc1_weights).float()

# testing without training
average_loss, average_mae = evaluate_model(ICA_l3_30, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=ICA_l3_30, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

##### no_components = 10

In [ ]:
# create copy of the model
ICA_local310 = RegressionNN(input_size, hidden_size1,
                                         hidden_size2, output_size)
transfer_pruned_weights(pruned_local3, ICA_local310)

# Apply ICA to the first layer as an example
n_components_fc1 = 10  # New dimension for fc1
transformed_fc1_weights = apply_ica_to_layer(ICA_local310.fc1,
                                             n_components=n_components_fc1)

# Instantiate the adjusted model
ICA_l3_10 = AdjustedModel()

# Update the adjusted model with ICA transformed weights for fc1
ICA_l3_10.fc1.weight.data = torch.from_numpy(transformed_fc1_weights).float()

# testing without training
average_loss, average_mae = evaluate_model(ICA_l3_10, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=ICA_l3_10, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

#### ICA on the Pruned Model, `pruned_local4`, with Pruning Amount=0.9

###### no_components = 50

In [ ]:
# create copy of the model
ICA_pruned_local_09 = RegressionNN(input_size, hidden_size1,
                                         hidden_size2, output_size)
transfer_pruned_weights(pruned_local4, ICA_pruned_local_09)

In [ ]:
# Apply ICA to the first layer as an example
n_components_fc1 = 50  # New dimension for fc1
transformed_fc1_weights = apply_ica_to_layer(ICA_pruned_local_09.fc1,
                                             n_components=n_components_fc1)

# Instantiate the adjusted model
ICA_l4_50 = AdjustedModel()

# Update the adjusted model with ICA transformed weights for fc1
ICA_l4_50.fc1.weight.data = torch.from_numpy(transformed_fc1_weights).float()

# testing without training
average_loss, average_mae = evaluate_model(ICA_l4_50, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=ICA_l4_50, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

##### no_components = 30

In [ ]:
# create copy of the model
ICA_local430 = RegressionNN(input_size, hidden_size1,
                                         hidden_size2, output_size)
transfer_pruned_weights(pruned_local4, ICA_local430)

# Apply ICA to the first layer as an example
n_components_fc1 = 30  # New dimension for fc1
transformed_fc1_weights = apply_ica_to_layer(ICA_local430.fc1,
                                             n_components=n_components_fc1)

# Instantiate the adjusted model
ICA_l4_30 = AdjustedModel()

# Update the adjusted model with ICA transformed weights for fc1
ICA_l4_30.fc1.weight.data = torch.from_numpy(transformed_fc1_weights).float()

# testing without training
average_loss, average_mae = evaluate_model(ICA_l4_30, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=ICA_l4_30, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

##### no_components = 10

In [ ]:
# create copy of the model
ICA_local410 = RegressionNN(input_size, hidden_size1,
                                         hidden_size2, output_size)
transfer_pruned_weights(pruned_local4, ICA_local410)

# Apply ICA to the first layer as an example
n_components_fc1 = 10  # New dimension for fc1
transformed_fc1_weights = apply_ica_to_layer(ICA_local410.fc1,
                                             n_components=n_components_fc1)

# Instantiate the adjusted model
ICA_l4_10 = AdjustedModel()

# Update the adjusted model with ICA transformed weights for fc1
ICA_l4_10.fc1.weight.data = torch.from_numpy(transformed_fc1_weights).float()

# testing without training
average_loss, average_mae = evaluate_model(ICA_l4_10, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=ICA_l4_10, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

#### ICA on the Pruned Model, `pruned_local5`, with Pruning Amount=0.95

###### no_components = 50

In [ ]:
# create copy of the model
ICA_pruned_local_09 = RegressionNN(input_size, hidden_size1,
                                         hidden_size2, output_size)
transfer_pruned_weights(pruned_local5, ICA_pruned_local_09)

In [ ]:
# Apply ICA to the first layer as an example
n_components_fc1 = 50  # New dimension for fc1
transformed_fc1_weights = apply_ica_to_layer(ICA_pruned_local_09.fc1,
                                             n_components=n_components_fc1)

# Instantiate the adjusted model
ICA_l5_50 = AdjustedModel()

# Update the adjusted model with ICA transformed weights for fc1
ICA_l5_50.fc1.weight.data = torch.from_numpy(transformed_fc1_weights).float()

# testing without training
average_loss, average_mae = evaluate_model(ICA_l5_50, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=ICA_l5_50, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

##### no_components = 30

In [ ]:
# create copy of the model
ICA_local530 = RegressionNN(input_size, hidden_size1,
                                         hidden_size2, output_size)
transfer_pruned_weights(pruned_local5, ICA_local530)


# Apply ICA to the first layer as an example
n_components_fc1 = 30  # New dimension for fc1
transformed_fc1_weights = apply_ica_to_layer(ICA_local530.fc1,
                                             n_components=n_components_fc1)

# Instantiate the adjusted model
ICA_l5_30 = AdjustedModel()

# Update the adjusted model with ICA transformed weights for fc1
ICA_l5_30.fc1.weight.data = torch.from_numpy(transformed_fc1_weights).float()

# testing without training
average_loss, average_mae = evaluate_model(ICA_l5_30, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=ICA_l5_30, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

##### no_components = 10

In [ ]:
# create copy of the model
ICA_local530 = RegressionNN(input_size, hidden_size1,
                                         hidden_size2, output_size)
transfer_pruned_weights(pruned_local5, ICA_local530)

# Apply ICA to the first layer as an example
n_components_fc1 = 10  # New dimension for fc1
transformed_fc1_weights = apply_ica_to_layer(ICA_local530.fc1,
                                             n_components=n_components_fc1)

# Instantiate the adjusted model
ICA_l5_10 = AdjustedModel()

# Update the adjusted model with ICA transformed weights for fc1
ICA_l5_10.fc1.weight.data = torch.from_numpy(transformed_fc1_weights).float()

# testing without training
average_loss, average_mae = evaluate_model(ICA_l5_10, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=ICA_l5_10, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

### ICA on Layer-Wise Locally Pruned Models

#### ICA on the Pruned Model, `pruned_sparse1`, with Pruning Amount=0.6

###### no_components = 50

In [ ]:
# create copy of the model
ICA_sparse1 = RegressionNN(input_size, hidden_size1,
                                         hidden_size2, output_size)
transfer_pruned_weights(pruned_sparse1, ICA_sparse1)

In [ ]:
# Apply ICA to the first layer as an example
n_components_fc1 = 50  # New dimension for fc1
transformed_fc1_weights = apply_ica_to_layer(ICA_sparse1.fc1,
                                             n_components=n_components_fc1)

# Instantiate the adjusted model
ICA_sparse1_50 = AdjustedModel()

# Update the adjusted model with ICA transformed weights for fc1
ICA_sparse1_50.fc1.weight.data = torch.from_numpy(transformed_fc1_weights).float()

# testing without training
average_loss, average_mae = evaluate_model(ICA_sparse1_50, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=ICA_sparse1_50, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

##### no_components = 30

In [ ]:
# create copy of the model
ICA_sparse130 = RegressionNN(input_size, hidden_size1,
                                         hidden_size2, output_size)
transfer_pruned_weights(pruned_sparse1, ICA_sparse130)

# Apply ICA to the first layer as an example
n_components_fc1 = 30  # New dimension for fc1
transformed_fc1_weights = apply_ica_to_layer(ICA_sparse130.fc1,
                                             n_components=n_components_fc1)

# Instantiate the adjusted model
ICA_sparse1_30 = AdjustedModel()

# Update the adjusted model with ICA transformed weights for fc1
ICA_sparse1_30.fc1.weight.data = torch.from_numpy(transformed_fc1_weights).float()

# testing without training
average_loss, average_mae = evaluate_model(ICA_sparse1_30, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=ICA_sparse1_30, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

##### no_components = 10

In [ ]:
# create copy of the model
ICA_sparse110 = RegressionNN(input_size, hidden_size1,
                                         hidden_size2, output_size)
transfer_pruned_weights(pruned_sparse1, ICA_sparse110)

# Apply ICA to the first layer as an example
n_components_fc1 = 10  # New dimension for fc1
transformed_fc1_weights = apply_ica_to_layer(ICA_sparse110.fc1,
                                             n_components=n_components_fc1)

# Instantiate the adjusted model
ICA_sparse1_10 = AdjustedModel()

# Update the adjusted model with ICA transformed weights for fc1
ICA_sparse1_10.fc1.weight.data = torch.from_numpy(transformed_fc1_weights).float()

# testing without training
average_loss, average_mae = evaluate_model(ICA_sparse1_10, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=ICA_sparse1_10, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

#### ICA on the Pruned Model, `pruned_sparse2`, with Pruning Amount=0.7

###### no_components = 50

In [ ]:
# create copy of the model
ICA_sparse2 = RegressionNN(input_size, hidden_size1,
                                         hidden_size2, output_size)
transfer_pruned_weights(pruned_sparse2, ICA_sparse2)

In [ ]:
# Apply ICA to the first layer as an example
n_components_fc1 = 50  # New dimension for fc1
transformed_fc1_weights = apply_ica_to_layer(ICA_sparse2.fc1,
                                             n_components=n_components_fc1)

# Instantiate the adjusted model
ICA_sparse2_50 = AdjustedModel()

# Update the adjusted model with ICA transformed weights for fc1
ICA_sparse2_50.fc1.weight.data = torch.from_numpy(transformed_fc1_weights).float()

# testing without training
average_loss, average_mae = evaluate_model(ICA_sparse2_50, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=ICA_sparse2_50, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

##### no_components = 30

In [ ]:
# create copy of the model
ICA_sparse230 = RegressionNN(input_size, hidden_size1,
                                         hidden_size2, output_size)
transfer_pruned_weights(pruned_sparse2, ICA_sparse230)

# Apply ICA to the first layer as an example
n_components_fc1 = 30  # New dimension for fc1
transformed_fc1_weights = apply_ica_to_layer(ICA_sparse230.fc1,
                                             n_components=n_components_fc1)

# Instantiate the adjusted model
ICA_sparse2_30 = AdjustedModel()

# Update the adjusted model with ICA transformed weights for fc1
ICA_sparse2_30.fc1.weight.data = torch.from_numpy(transformed_fc1_weights).float()

# testing without training
average_loss, average_mae = evaluate_model(ICA_sparse2_30, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=ICA_sparse2_30, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

##### no_components = 10

In [ ]:
# create copy of the model
ICA_sparse210 = RegressionNN(input_size, hidden_size1,
                                         hidden_size2, output_size)
transfer_pruned_weights(pruned_sparse2, ICA_sparse210)

# Apply ICA to the first layer as an example
n_components_fc1 = 10  # New dimension for fc1
transformed_fc1_weights = apply_ica_to_layer(ICA_sparse210.fc1,
                                             n_components=n_components_fc1)

# Instantiate the adjusted model
ICA_sparse2_10 = AdjustedModel()

# Update the adjusted model with ICA transformed weights for fc1
ICA_sparse2_10.fc1.weight.data = torch.from_numpy(transformed_fc1_weights).float()

# testing without training
average_loss, average_mae = evaluate_model(ICA_sparse2_10, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=ICA_sparse2_10, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

#### ICA on the Pruned Model, `pruned_sparse3`, with Pruning Amount=0.8

###### no_components = 50

In [ ]:
# create copy of the model
ICA_sparse3 = RegressionNN(input_size, hidden_size1,
                                         hidden_size2, output_size)
transfer_pruned_weights(pruned_sparse3, ICA_sparse3)

In [ ]:
# Apply ICA to the first layer as an example
n_components_fc1 = 50  # New dimension for fc1
transformed_fc1_weights = apply_ica_to_layer(ICA_sparse3.fc1,
                                             n_components=n_components_fc1)

# Instantiate the adjusted model
ICA_sparse3_50 = AdjustedModel()

# Update the adjusted model with ICA transformed weights for fc1
ICA_sparse3_50.fc1.weight.data = torch.from_numpy(transformed_fc1_weights).float()

# testing without training
average_loss, average_mae = evaluate_model(ICA_sparse3_50, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=ICA_sparse3_50, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

##### no_components = 30

In [ ]:
# create copy of the model
ICA_sparse330 = RegressionNN(input_size, hidden_size1,
                                         hidden_size2, output_size)
transfer_pruned_weights(pruned_sparse3, ICA_sparse330)

# Apply ICA to the first layer as an example
n_components_fc1 = 30  # New dimension for fc1
transformed_fc1_weights = apply_ica_to_layer(ICA_sparse330.fc1,
                                             n_components=n_components_fc1)

# Instantiate the adjusted model
ICA_sparse3_30 = AdjustedModel()

# Update the adjusted model with ICA transformed weights for fc1
ICA_sparse3_30.fc1.weight.data = torch.from_numpy(transformed_fc1_weights).float()

# testing without training
average_loss, average_mae = evaluate_model(ICA_sparse3_30, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=ICA_sparse3_30, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

##### no_components = 10

In [ ]:
# create copy of the model
ICA_sparse310 = RegressionNN(input_size, hidden_size1,
                                         hidden_size2, output_size)
transfer_pruned_weights(pruned_sparse3, ICA_sparse310)

# Apply ICA to the first layer as an example
n_components_fc1 = 10  # New dimension for fc1
transformed_fc1_weights = apply_ica_to_layer(ICA_sparse310.fc1,
                                             n_components=n_components_fc1)

# Instantiate the adjusted model
ICA_sparse3_10 = AdjustedModel()

# Update the adjusted model with ICA transformed weights for fc1
ICA_sparse3_10.fc1.weight.data = torch.from_numpy(transformed_fc1_weights).float()

# testing without training
average_loss, average_mae = evaluate_model(ICA_sparse3_10, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=ICA_sparse3_10, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

#### ICA on the Pruned Model, `pruned_sparse4`, with Pruning Amount=0.9

###### no_components = 50

In [ ]:
# create copy of the model
ICA_sparse4 = RegressionNN(input_size, hidden_size1,
                                         hidden_size2, output_size)
transfer_pruned_weights(pruned_sparse4, ICA_sparse4)

In [ ]:
# Apply ICA to the first layer as an example
n_components_fc1 = 50  # New dimension for fc1
transformed_fc1_weights = apply_ica_to_layer(ICA_sparse4.fc1,
                                             n_components=n_components_fc1)

# Instantiate the adjusted model
ICA_sparse4_50 = AdjustedModel()

# Update the adjusted model with ICA transformed weights for fc1
ICA_sparse4_50.fc1.weight.data = torch.from_numpy(transformed_fc1_weights).float()

# testing without training
average_loss, average_mae = evaluate_model(ICA_sparse4_50, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=ICA_sparse4_50, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

##### no_components = 30

In [ ]:
# create copy of the model
ICA_sparse430 = RegressionNN(input_size, hidden_size1,
                                         hidden_size2, output_size)
transfer_pruned_weights(pruned_sparse4, ICA_sparse430)

# Apply ICA to the first layer as an example
n_components_fc1 = 30  # New dimension for fc1
transformed_fc1_weights = apply_ica_to_layer(ICA_sparse430.fc1,
                                             n_components=n_components_fc1)

# Instantiate the adjusted model
ICA_sparse4_30 = AdjustedModel()

# Update the adjusted model with ICA transformed weights for fc1
ICA_sparse4_30.fc1.weight.data = torch.from_numpy(transformed_fc1_weights).float()

# testing without training
average_loss, average_mae = evaluate_model(ICA_sparse4_30, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=ICA_sparse4_30, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

##### no_components = 10

In [ ]:
# create copy of the model
ICA_sparse410 = RegressionNN(input_size, hidden_size1,
                                         hidden_size2, output_size)
transfer_pruned_weights(pruned_sparse4, ICA_sparse410)

# Apply ICA to the first layer as an example
n_components_fc1 = 10  # New dimension for fc1
transformed_fc1_weights = apply_ica_to_layer(ICA_sparse410.fc1,
                                             n_components=n_components_fc1)

# Instantiate the adjusted model
ICA_sparse4_10 = AdjustedModel()

# Update the adjusted model with ICA transformed weights for fc1
ICA_sparse4_10.fc1.weight.data = torch.from_numpy(transformed_fc1_weights).float()

# testing without training
average_loss, average_mae = evaluate_model(ICA_sparse4_10, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=ICA_sparse4_10, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

#### ICA on the Pruned Model, `pruned_sparse5`, with Pruning Amount=0.95

###### no_components = 50

In [ ]:
# create copy of the model
ICA_sparse5 = RegressionNN(input_size, hidden_size1,
                                         hidden_size2, output_size)
transfer_pruned_weights(pruned_sparse5, ICA_sparse5)

In [ ]:
# Apply ICA to the first layer as an example
n_components_fc1 = 50  # New dimension for fc1
transformed_fc1_weights = apply_ica_to_layer(ICA_sparse5.fc1,
                                             n_components=n_components_fc1)

# Instantiate the adjusted model
ICA_sparse5_50 = AdjustedModel()

# Update the adjusted model with ICA transformed weights for fc1
ICA_sparse5_50.fc1.weight.data = torch.from_numpy(transformed_fc1_weights).float()

# testing without training
average_loss, average_mae = evaluate_model(ICA_sparse5_50, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=ICA_sparse5_50, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

##### no_components = 30

In [ ]:
# create copy of the model
ICA_sparse530 = RegressionNN(input_size, hidden_size1,
                                         hidden_size2, output_size)
transfer_pruned_weights(pruned_sparse5, ICA_sparse530)

# Apply ICA to the first layer as an example
n_components_fc1 = 30  # New dimension for fc1
transformed_fc1_weights = apply_ica_to_layer(ICA_sparse530.fc1,
                                             n_components=n_components_fc1)

# Instantiate the adjusted model
ICA_sparse5_30 = AdjustedModel()

# Update the adjusted model with ICA transformed weights for fc1
ICA_sparse5_30.fc1.weight.data = torch.from_numpy(transformed_fc1_weights).float()

# testing without training
average_loss, average_mae = evaluate_model(ICA_sparse5_30, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=ICA_sparse5_30, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

##### no_components = 10

In [ ]:
# create copy of the model
ICA_sparse510 = RegressionNN(input_size, hidden_size1,
                                         hidden_size2, output_size)
transfer_pruned_weights(pruned_sparse5, ICA_sparse510)

# Apply ICA to the first layer as an example
n_components_fc1 = 10  # New dimension for fc1
transformed_fc1_weights = apply_ica_to_layer(ICA_sparse510.fc1,
                                             n_components=n_components_fc1)

# Instantiate the adjusted model
ICA_sparse5_10 = AdjustedModel()

# Update the adjusted model with ICA transformed weights for fc1
ICA_sparse5_10.fc1.weight.data = torch.from_numpy(transformed_fc1_weights).float()

# testing without training
average_loss, average_mae = evaluate_model(ICA_sparse5_10, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# train and validate the model with PCA = 30
trained_model, training_losses, validation_losses = train_and_validate(
   model=ICA_sparse5_10, train_loader=train_loader,
   test_loader=test_loader, learning_rate=0.001, epochs=200
)

## Comparison of All Methods


### Creating and Training Smaller Networks

In this section, we will create smaller networks to match the dimensions of our compressed pruned models (PCA, SparsePCA, and ICA).  We are doing this to compare the performance of the original, smaller, and compressed network.

In [ ]:
# parameters for model
input_size = 58  # Number of input features
output_size = 1
epochs = 200
batch_size = 64
learning_rate = 0.001

In [ ]:
# smaller network with fc1=50 and fc2=32
small_5032 = RegressionNN(input_size, 50, 32, output_size)
# Call train_and_validate function
trained_model_5032, training_losses_5032, validation_losses_5032 = train_and_validate(
   model=small_5032, train_loader=train_loader,test_loader=test_loader,
   learning_rate=learning_rate, epochs=epochs
)
# Average Loss and MAE
average_loss, average_mae = evaluate_model(small_5032, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# smaller network with fc1=50 and fc2=30
small_5030 = RegressionNN(input_size, 50, 30, output_size)
# Call train_and_validate function
trained_model_5030, training_losses_5030, validation_losses_5030 = train_and_validate(
   model=small_5030, train_loader=train_loader,test_loader=test_loader,
   learning_rate=learning_rate, epochs=epochs
)
# Average Loss and MAE
average_loss, average_mae = evaluate_model(small_5030, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# smaller network with fc1=50 and fc2=10
small_5010 = RegressionNN(input_size, 50, 10, output_size)
# Call train_and_validate function
trained_model_5010, training_losses_5010, validation_losses_5010 = train_and_validate(
   model=small_5010, train_loader=train_loader,test_loader=test_loader,
   learning_rate=learning_rate, epochs=epochs
)
# Average Loss and MAE
average_loss, average_mae = evaluate_model(small_5010, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# smaller network with fc1=50 and fc2=5
small_505 = RegressionNN(input_size, 50, 5, output_size)
# Call train_and_validate function
trained_model_505, training_losses_505, validation_losses_505 = train_and_validate(
   model=small_505, train_loader=train_loader,test_loader=test_loader,
   learning_rate=learning_rate, epochs=epochs
)
# Average Loss and MAE
average_loss, average_mae = evaluate_model(small_505, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# smaller network with fc1=30 and fc2=32
small_3032 = RegressionNN(input_size, 30, 32, output_size)
# Call train_and_validate function
trained_model_3032, training_losses_3032, validation_losses_3032 = train_and_validate(
   model=small_3032, train_loader=train_loader,test_loader=test_loader,
   learning_rate=learning_rate, epochs=epochs
)
# Average Loss and MAE
average_loss, average_mae = evaluate_model(small_3032, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# smaller network with fc1=30 and fc2=10
small_3010 = RegressionNN(input_size, 30, 10, output_size)
# Call train_and_validate function
trained_model_3010, training_losses_3010, validation_losses_3010 = train_and_validate(
   model=small_3010, train_loader=train_loader,test_loader=test_loader,
   learning_rate=learning_rate, epochs=epochs
)
# Average Loss and MAE
average_loss, average_mae = evaluate_model(small_3010, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# smaller network with fc1=30 and fc2=5
small_305 = RegressionNN(input_size, 30, 5, output_size)
# Call train_and_validate function
trained_model_305, training_losses_305, validation_losses_305 = train_and_validate(
   model=small_305, train_loader=train_loader,test_loader=test_loader,
   learning_rate=learning_rate, epochs=epochs
)
# Average Loss and MAE
average_loss, average_mae = evaluate_model(small_305, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# smaller network with fc1=10 and fc2=32
small_1032 = RegressionNN(input_size, 10, 32, output_size)
# Call train_and_validate function
trained_model_1032, training_losses_1032, validation_losses_1032 = train_and_validate(
   model=small_1032, train_loader=train_loader,test_loader=test_loader,
   learning_rate=learning_rate, epochs=epochs
)
# Average Loss and MAE
average_loss, average_mae = evaluate_model(small_1032, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

In [ ]:
# smaller network with fc1=10 and fc2=5
small_105 = RegressionNN(input_size, 10, 5, output_size)
# Call train_and_validate function
trained_model_105, training_losses_105, validation_losses_105 = train_and_validate(
   model=small_105, train_loader=train_loader,test_loader=test_loader,
   learning_rate=learning_rate, epochs=epochs
)
# Average Loss and MAE
average_loss, average_mae = evaluate_model(small_105, test_loader, device)
print(f'Average Loss: {average_loss:.4f}, Average MAE: {average_mae:.4f}')

### Calculating the MSE and MAE losses

#### Original, Local, and Locally Compressed (PCA, SparsePCA, ICA) Pruned Models

In [ ]:
# checking local models that have 0.6 pruning amount
local1_models = {
    "Original": model,
    "Smaller Network - fc1_out = 50 and fc2_out = 30": small_5030,
    "Smaller Network - fc1_out = 50 and fc2_out = 32": small_5032,
    "Smaller Network - fc1_out = 50 and fc2_out = 10": small_5010,
    "Smaller Network - fc1_out = 50 and fc2_out = 5": small_505,
    "Smaller Network - fc1_out = 30 and fc2_out = 32": small_3032,
    "Smaller Network - fc1_out = 30 and fc2_out = 10": small_3010,
    "Smaller Network - fc1_out = 30 and fc2_out = 5": small_305,
    "Smaller Network - fc1_out = 10 and fc2_out = 32": small_1032,
    "Smaller Network - fc1_out = 10 and fc2_out = 5": small_105,
    "Local Pruned": pruned_local1,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 30": PCA_local1_5030,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 30": PCA_local1_5030,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 10": PCA_local1_5010,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 5": PCA_local1_505,
    "PCA Local Compressed - fc1_out = 30 and fc2_out = 10": PCA_local1_3010,
    "PCA Local Compressed - fc1_out = 30 and fc2_out = 5": PCA_local1_305,
    "PCA Local Compressed - fc1_out = 10 and fc2_out = 5": PCA_local1_105,
    "SparsePCA Local Compressed - fc1_out = 50 and fc2_out = 32": SparsePCA_l1_50,
    "SparsePCA Local Compressed - fc1_out = 30 and fc2_out = 32": SparsePCA_l1_30,
    "SparsePCA Local Compressed - fc1_out = 10 and fc2_out = 32": SparsePCA_l1_10,
    "ICA Local Model - fc1_out = 50 and fc2_out = 32": ICA_l1_50,
    "ICA Local Compressed - fc1_out = 30 and fc2_out = 32": ICA_l1_30,
    "ICA Local Compressed - fc1_out = 10 and fc2_out = 32": ICA_l1_10,

}

results_local1 = {}

for name, model in local1_models.items():
    loss, mae = evaluate_model(model, test_loader, device)
    results_local1[name] = {"MSE Loss": loss, "MAE": mae}
    print(f"{name} - MSE Loss: {loss:.4f}, MAE: {mae:.4f}")

In [ ]:
# checking local models that have 0.7 pruning amount
local2_models = {
    "Original": model,
    "Smaller Network - fc1_out = 50 and fc2_out = 30": small_5030,
    "Smaller Network - fc1_out = 50 and fc2_out = 32": small_5032,
    "Smaller Network - fc1_out = 50 and fc2_out = 10": small_5010,
    "Smaller Network - fc1_out = 50 and fc2_out = 5": small_505,
    "Smaller Network - fc1_out = 30 and fc2_out = 32": small_3032,
    "Smaller Network - fc1_out = 30 and fc2_out = 10": small_3010,
    "Smaller Network - fc1_out = 30 and fc2_out = 5": small_305,
    "Smaller Network - fc1_out = 10 and fc2_out = 32": small_1032,
    "Smaller Network - fc1_out = 10 and fc2_out = 5": small_105,
    "Local Pruned": pruned_local2,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 30": PCA_local2_5030,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 30": PCA_local2_5030,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 10": PCA_local2_5010,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 5": PCA_local2_505,
    "PCA Local Compressed - fc1_out = 30 and fc2_out = 10": PCA_local2_3010,
    "PCA Local Compressed - fc1_out = 30 and fc2_out = 5": PCA_local2_305,
    "PCA Local Compressed - fc1_out = 10 and fc2_out = 5": PCA_local2_105,
    "SparsePCA Local Compressed - fc1_out = 50 and fc2_out = 32": SparsePCA_l2_50,
    "SparsePCA Local Compressed - fc1_out = 30 and fc2_out = 32": SparsePCA_l2_30,
    "SparsePCA Local Compressed - fc1_out = 10 and fc2_out = 32": SparsePCA_l2_10,
    "ICA Local Model - fc1_out = 50 and fc2_out = 32": ICA_l2_50,
    "ICA Local Compressed - fc1_out = 30 and fc2_out = 32": ICA_l2_30,
    "ICA Local Compressed - fc1_out = 10 and fc2_out = 32": ICA_l2_10,

}

results_local2 = {}

for name, model in local2_models.items():
    loss, mae = evaluate_model(model, test_loader, device)
    results_local2[name] = {"MSE Loss": loss, "MAE": mae}
    print(f"{name} - MSE Loss: {loss:.4f}, MAE: {mae:.4f}")

In [ ]:
# checking local models that have 0.8 pruning amount
local3_models = {
    "Original": model,
    "Smaller Network - fc1_out = 50 and fc2_out = 30": small_5030,
    "Smaller Network - fc1_out = 50 and fc2_out = 32": small_5032,
    "Smaller Network - fc1_out = 50 and fc2_out = 10": small_5010,
    "Smaller Network - fc1_out = 50 and fc2_out = 5": small_505,
    "Smaller Network - fc1_out = 30 and fc2_out = 32": small_3032,
    "Smaller Network - fc1_out = 30 and fc2_out = 10": small_3010,
    "Smaller Network - fc1_out = 30 and fc2_out = 5": small_305,
    "Smaller Network - fc1_out = 10 and fc2_out = 32": small_1032,
    "Smaller Network - fc1_out = 10 and fc2_out = 5": small_105,
    "Local Pruned": pruned_local3,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 30": PCA_local3_5030,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 30": PCA_local3_5030,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 10": PCA_local3_5010,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 5": PCA_local3_505,
    "PCA Local Compressed - fc1_out = 30 and fc2_out = 10": PCA_local3_3010,
    "PCA Local Compressed - fc1_out = 30 and fc2_out = 5": PCA_local3_305,
    "PCA Local Compressed - fc1_out = 10 and fc2_out = 5": PCA_local3_105,
    "SparsePCA Local Compressed - fc1_out = 50 and fc2_out = 32": SparsePCA_l3_50,
    "SparsePCA Local Compressed - fc1_out = 30 and fc2_out = 32": SparsePCA_l3_30,
    "SparsePCA Local Compressed - fc1_out = 10 and fc2_out = 32": SparsePCA_l3_10,
    "ICA Local Model - fc1_out = 50 and fc2_out = 32": ICA_l3_50,
    "ICA Local Compressed - fc1_out = 30 and fc2_out = 32": ICA_l3_30,
    "ICA Local Compressed - fc1_out = 10 and fc2_out = 32": ICA_l3_10,

}

results_local3 = {}

for name, model in local3_models.items():
    loss, mae = evaluate_model(model, test_loader, device)
    results_local3[name] = {"MSE Loss": loss, "MAE": mae}
    print(f"{name} - MSE Loss: {loss:.4f}, MAE: {mae:.4f}")

In [ ]:
# checking local models that have 0.9 pruning amount
local4_models = {
    "Original": model,
    "Smaller Network - fc1_out = 50 and fc2_out = 30": small_5030,
    "Smaller Network - fc1_out = 50 and fc2_out = 32": small_5032,
    "Smaller Network - fc1_out = 50 and fc2_out = 10": small_5010,
    "Smaller Network - fc1_out = 50 and fc2_out = 5": small_505,
    "Smaller Network - fc1_out = 30 and fc2_out = 32": small_3032,
    "Smaller Network - fc1_out = 30 and fc2_out = 10": small_3010,
    "Smaller Network - fc1_out = 30 and fc2_out = 5": small_305,
    "Smaller Network - fc1_out = 10 and fc2_out = 32": small_1032,
    "Smaller Network - fc1_out = 10 and fc2_out = 5": small_105,
    "Local Pruned": pruned_local4,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 30": PCA_local4_5030,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 30": PCA_local4_5030,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 10": PCA_local4_5010,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 5": PCA_local4_505,
    "PCA Local Compressed - fc1_out = 30 and fc2_out = 10": PCA_local4_3010,
    "PCA Local Compressed - fc1_out = 30 and fc2_out = 5": PCA_local4_305,
    "PCA Local Compressed - fc1_out = 10 and fc2_out = 5": PCA_local4_105,
    "SparsePCA Local Compressed - fc1_out = 50 and fc2_out = 32": SparsePCA_l4_50,
    "SparsePCA Local Compressed - fc1_out = 30 and fc2_out = 32": SparsePCA_l4_30,
    "SparsePCA Local Compressed - fc1_out = 10 and fc2_out = 32": SparsePCA_l4_10,
    "ICA Local Model - fc1_out = 50 and fc2_out = 32": ICA_l4_50,
    "ICA Local Compressed - fc1_out = 30 and fc2_out = 32": ICA_l4_30,
    "ICA Local Compressed - fc1_out = 10 and fc2_out = 32": ICA_l4_10,

}

results_local4 = {}

for name, model in local4_models.items():
    loss, mae = evaluate_model(model, test_loader, device)
    results_local4[name] = {"MSE Loss": loss, "MAE": mae}
    print(f"{name} - MSE Loss: {loss:.4f}, MAE: {mae:.4f}")

In [ ]:
# checking local models that have 0.95 pruning amount
local5_models = {
    "Original": model,
    "Smaller Network - fc1_out = 50 and fc2_out = 30": small_5030,
    "Smaller Network - fc1_out = 50 and fc2_out = 32": small_5032,
    "Smaller Network - fc1_out = 50 and fc2_out = 10": small_5010,
    "Smaller Network - fc1_out = 50 and fc2_out = 5": small_505,
    "Smaller Network - fc1_out = 30 and fc2_out = 32": small_3032,
    "Smaller Network - fc1_out = 30 and fc2_out = 10": small_3010,
    "Smaller Network - fc1_out = 30 and fc2_out = 5": small_305,
    "Smaller Network - fc1_out = 10 and fc2_out = 32": small_1032,
    "Smaller Network - fc1_out = 10 and fc2_out = 5": small_105,
    "Local Pruned": pruned_local5,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 30": PCA_local5_5030,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 30": PCA_local5_5030,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 10": PCA_local5_5010,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 5": PCA_local5_505,
    "PCA Local Compressed - fc1_out = 30 and fc2_out = 10": PCA_local5_3010,
    "PCA Local Compressed - fc1_out = 30 and fc2_out = 5": PCA_local5_305,
    "PCA Local Compressed - fc1_out = 10 and fc2_out = 5": PCA_local5_105,
    "SparsePCA Local Compressed - fc1_out = 50 and fc2_out = 32": SparsePCA_l5_50,
    "SparsePCA Local Compressed - fc1_out = 30 and fc2_out = 32": SparsePCA_l5_30,
    "SparsePCA Local Compressed - fc1_out = 10 and fc2_out = 32": SparsePCA_l5_10,
    "ICA Local Model - fc1_out = 50 and fc2_out = 32": ICA_l5_50,
    "ICA Local Compressed - fc1_out = 30 and fc2_out = 32": ICA_l5_30,
    "ICA Local Compressed - fc1_out = 10 and fc2_out = 32": ICA_l5_10,

}

results_local5 = {}

for name, model in local5_models.items():
    loss, mae = evaluate_model(model, test_loader, device)
    results_local5[name] = {"MSE Loss": loss, "MAE": mae}
    print(f"{name} - MSE Loss: {loss:.4f}, MAE: {mae:.4f}")

#### Original, Local, and Layer-wise Locally Compressed (PCA, SparsePCA, ICA) Pruned Models

In [ ]:
# checking layer-wise pruned models that have 0.6 pruning amount
sparse1_models = {
    "Original": model,
    "Smaller Network - fc1_out = 50 and fc2_out = 30": small_5030,
    "Smaller Network - fc1_out = 50 and fc2_out = 32": small_5032,
    "Smaller Network - fc1_out = 50 and fc2_out = 10": small_5010,
    "Smaller Network - fc1_out = 50 and fc2_out = 5": small_505,
    "Smaller Network - fc1_out = 30 and fc2_out = 32": small_3032,
    "Smaller Network - fc1_out = 30 and fc2_out = 10": small_3010,
    "Smaller Network - fc1_out = 30 and fc2_out = 5": small_305,
    "Smaller Network - fc1_out = 10 and fc2_out = 32": small_1032,
    "Smaller Network - fc1_out = 10 and fc2_out = 5": small_105,
    "Local Pruned": pruned_sparse1,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 30": PCA_sparse1_5030,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 30": PCA_sparse1_5030,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 10": PCA_sparse1_5010,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 5": PCA_sparse1_505,
    "PCA Local Compressed - fc1_out = 30 and fc2_out = 10": PCA_sparse1_3010,
    "PCA Local Compressed - fc1_out = 30 and fc2_out = 5": PCA_sparse1_305,
    "PCA Local Compressed - fc1_out = 10 and fc2_out = 5": PCA_sparse1_105,
    "SparsePCA Local Compressed - fc1_out = 50 and fc2_out = 32": SparsePCA_sparse1_50,
    "SparsePCA Local Compressed - fc1_out = 30 and fc2_out = 32": SparsePCA_sparse1_30,
    "SparsePCA Local Compressed - fc1_out = 10 and fc2_out = 32": SparsePCA_sparse1_10,
    "ICA Local Model - fc1_out = 50 and fc2_out = 32": ICA_sparse1_50,
    "ICA Local Compressed - fc1_out = 30 and fc2_out = 32": ICA_sparse1_30,
    "ICA Local Compressed - fc1_out = 10 and fc2_out = 32": ICA_sparse1_10,

}

results_sparse1 = {}

for name, model in sparse1_models.items():
    loss, mae = evaluate_model(model, test_loader, device)
    results_sparse1[name] = {"MSE Loss": loss, "MAE": mae}
    print(f"{name} - MSE Loss: {loss:.4f}, MAE: {mae:.4f}")

In [ ]:
# checking layer-wise pruned models that have 0.6 pruning amount
sparse2_models = {
    "Original": model,
    "Smaller Network - fc1_out = 50 and fc2_out = 30": small_5030,
    "Smaller Network - fc1_out = 50 and fc2_out = 32": small_5032,
    "Smaller Network - fc1_out = 50 and fc2_out = 10": small_5010,
    "Smaller Network - fc1_out = 50 and fc2_out = 5": small_505,
    "Smaller Network - fc1_out = 30 and fc2_out = 32": small_3032,
    "Smaller Network - fc1_out = 30 and fc2_out = 10": small_3010,
    "Smaller Network - fc1_out = 30 and fc2_out = 5": small_305,
    "Smaller Network - fc1_out = 10 and fc2_out = 32": small_1032,
    "Smaller Network - fc1_out = 10 and fc2_out = 5": small_105,
    "Local Pruned": pruned_sparse2,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 30": PCA_sparse2_5030,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 30": PCA_sparse2_5030,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 10": PCA_sparse2_5010,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 5": PCA_sparse2_505,
    "PCA Local Compressed - fc1_out = 30 and fc2_out = 10": PCA_sparse2_3010,
    "PCA Local Compressed - fc1_out = 30 and fc2_out = 5": PCA_sparse2_305,
    "PCA Local Compressed - fc1_out = 10 and fc2_out = 5": PCA_sparse2_105,
    "SparsePCA Local Compressed - fc1_out = 50 and fc2_out = 32": SparsePCA_sparse2_50,
    "SparsePCA Local Compressed - fc1_out = 30 and fc2_out = 32": SparsePCA_sparse2_30,
    "SparsePCA Local Compressed - fc1_out = 10 and fc2_out = 32": SparsePCA_sparse2_10,
    "ICA Local Model - fc1_out = 50 and fc2_out = 32": ICA_sparse2_50,
    "ICA Local Compressed - fc1_out = 30 and fc2_out = 32": ICA_sparse2_30,
    "ICA Local Compressed - fc1_out = 10 and fc2_out = 32": ICA_sparse2_10,

}

results_sparse2 = {}

for name, model in sparse2_models.items():
    loss, mae = evaluate_model(model, test_loader, device)
    results_sparse2[name] = {"MSE Loss": loss, "MAE": mae}
    print(f"{name} - MSE Loss: {loss:.4f}, MAE: {mae:.4f}")

In [ ]:
# checking layer-wise pruned models that have 0.7 pruning amount
sparse2_models = {
    "Original": model,
    "Smaller Network - fc1_out = 50 and fc2_out = 30": small_5030,
    "Smaller Network - fc1_out = 50 and fc2_out = 32": small_5032,
    "Smaller Network - fc1_out = 50 and fc2_out = 10": small_5010,
    "Smaller Network - fc1_out = 50 and fc2_out = 5": small_505,
    "Smaller Network - fc1_out = 30 and fc2_out = 32": small_3032,
    "Smaller Network - fc1_out = 30 and fc2_out = 10": small_3010,
    "Smaller Network - fc1_out = 30 and fc2_out = 5": small_305,
    "Smaller Network - fc1_out = 10 and fc2_out = 32": small_1032,
    "Smaller Network - fc1_out = 10 and fc2_out = 5": small_105,
    "Local Pruned": pruned_sparse2,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 30": PCA_sparse2_5030,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 30": PCA_sparse2_5030,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 10": PCA_sparse2_5010,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 5": PCA_sparse2_505,
    "PCA Local Compressed - fc1_out = 30 and fc2_out = 10": PCA_sparse2_3010,
    "PCA Local Compressed - fc1_out = 30 and fc2_out = 5": PCA_sparse2_305,
    "PCA Local Compressed - fc1_out = 10 and fc2_out = 5": PCA_sparse2_105,
    "SparsePCA Local Compressed - fc1_out = 50 and fc2_out = 32": SparsePCA_sparse2_50,
    "SparsePCA Local Compressed - fc1_out = 30 and fc2_out = 32": SparsePCA_sparse2_30,
    "SparsePCA Local Compressed - fc1_out = 10 and fc2_out = 32": SparsePCA_sparse2_10,
    "ICA Local Model - fc1_out = 50 and fc2_out = 32": ICA_sparse2_50,
    "ICA Local Compressed - fc1_out = 30 and fc2_out = 32": ICA_sparse2_30,
    "ICA Local Compressed - fc1_out = 10 and fc2_out = 32": ICA_sparse2_10,

}

results_sparse2 = {}

for name, model in sparse2_models.items():
    loss, mae = evaluate_model(model, test_loader, device)
    results_sparse2[name] = {"MSE Loss": loss, "MAE": mae}
    print(f"{name} - MSE Loss: {loss:.4f}, MAE: {mae:.4f}")

In [ ]:
# checking layer-wise pruned models that have 0.8 pruning amount
sparse3_models = {
    "Original": model,
    "Smaller Network - fc1_out = 50 and fc2_out = 30": small_5030,
    "Smaller Network - fc1_out = 50 and fc2_out = 32": small_5032,
    "Smaller Network - fc1_out = 50 and fc2_out = 10": small_5010,
    "Smaller Network - fc1_out = 50 and fc2_out = 5": small_505,
    "Smaller Network - fc1_out = 30 and fc2_out = 32": small_3032,
    "Smaller Network - fc1_out = 30 and fc2_out = 10": small_3010,
    "Smaller Network - fc1_out = 30 and fc2_out = 5": small_305,
    "Smaller Network - fc1_out = 10 and fc2_out = 32": small_1032,
    "Smaller Network - fc1_out = 10 and fc2_out = 5": small_105,
    "Local Pruned": pruned_sparse3,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 30": PCA_sparse3_5030,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 30": PCA_sparse3_5030,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 10": PCA_sparse3_5010,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 5": PCA_sparse3_505,
    "PCA Local Compressed - fc1_out = 30 and fc2_out = 10": PCA_sparse3_3010,
    "PCA Local Compressed - fc1_out = 30 and fc2_out = 5": PCA_sparse3_305,
    "PCA Local Compressed - fc1_out = 10 and fc2_out = 5": PCA_sparse3_105,
    "SparsePCA Local Compressed - fc1_out = 50 and fc2_out = 32": SparsePCA_sparse3_50,
    "SparsePCA Local Compressed - fc1_out = 30 and fc2_out = 32": SparsePCA_sparse3_30,
    "SparsePCA Local Compressed - fc1_out = 10 and fc2_out = 32": SparsePCA_sparse3_10,
    "ICA Local Model - fc1_out = 50 and fc2_out = 32": ICA_sparse3_50,
    "ICA Local Compressed - fc1_out = 30 and fc2_out = 32": ICA_sparse3_30,
    "ICA Local Compressed - fc1_out = 10 and fc2_out = 32": ICA_sparse3_10,

}

results_sparse3 = {}

for name, model in sparse3_models.items():
    loss, mae = evaluate_model(model, test_loader, device)
    results_sparse3[name] = {"MSE Loss": loss, "MAE": mae}
    print(f"{name} - MSE Loss: {loss:.4f}, MAE: {mae:.4f}")

In [ ]:
# checking layer-wise pruned models that have 0.9 pruning amount
sparse4_models = {
    "Original": model,
    "Smaller Network - fc1_out = 50 and fc2_out = 30": small_5030,
    "Smaller Network - fc1_out = 50 and fc2_out = 32": small_5032,
    "Smaller Network - fc1_out = 50 and fc2_out = 10": small_5010,
    "Smaller Network - fc1_out = 50 and fc2_out = 5": small_505,
    "Smaller Network - fc1_out = 30 and fc2_out = 32": small_3032,
    "Smaller Network - fc1_out = 30 and fc2_out = 10": small_3010,
    "Smaller Network - fc1_out = 30 and fc2_out = 5": small_305,
    "Smaller Network - fc1_out = 10 and fc2_out = 32": small_1032,
    "Smaller Network - fc1_out = 10 and fc2_out = 5": small_105,
    "Local Pruned": pruned_sparse4,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 30": PCA_sparse4_5030,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 30": PCA_sparse4_5030,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 10": PCA_sparse4_5010,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 5": PCA_sparse4_505,
    "PCA Local Compressed - fc1_out = 30 and fc2_out = 10": PCA_sparse4_3010,
    "PCA Local Compressed - fc1_out = 30 and fc2_out = 5": PCA_sparse4_305,
    "PCA Local Compressed - fc1_out = 10 and fc2_out = 5": PCA_sparse4_105,
    "SparsePCA Local Compressed - fc1_out = 50 and fc2_out = 32": SparsePCA_sparse4_50,
    "SparsePCA Local Compressed - fc1_out = 30 and fc2_out = 32": SparsePCA_sparse4_30,
    "SparsePCA Local Compressed - fc1_out = 10 and fc2_out = 32": SparsePCA_sparse4_10,
    "ICA Local Model - fc1_out = 50 and fc2_out = 32": ICA_sparse4_50,
    "ICA Local Compressed - fc1_out = 30 and fc2_out = 32": ICA_sparse4_30,
    "ICA Local Compressed - fc1_out = 10 and fc2_out = 32": ICA_sparse4_10,

}

results_sparse4 = {}

for name, model in sparse4_models.items():
    loss, mae = evaluate_model(model, test_loader, device)
    results_sparse4[name] = {"MSE Loss": loss, "MAE": mae}
    print(f"{name} - MSE Loss: {loss:.4f}, MAE: {mae:.4f}")

In [ ]:
# checking layer-wise pruned models that have 0.95 pruning amount
sparse5_models = {
    "Original": model,
    "Smaller Network - fc1_out = 50 and fc2_out = 30": small_5030,
    "Smaller Network - fc1_out = 50 and fc2_out = 32": small_5032,
    "Smaller Network - fc1_out = 50 and fc2_out = 10": small_5010,
    "Smaller Network - fc1_out = 50 and fc2_out = 5": small_505,
    "Smaller Network - fc1_out = 30 and fc2_out = 32": small_3032,
    "Smaller Network - fc1_out = 30 and fc2_out = 10": small_3010,
    "Smaller Network - fc1_out = 30 and fc2_out = 5": small_305,
    "Smaller Network - fc1_out = 10 and fc2_out = 32": small_1032,
    "Smaller Network - fc1_out = 10 and fc2_out = 5": small_105,
    "Local Pruned": pruned_sparse5,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 30": PCA_sparse5_5030,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 30": PCA_sparse5_5030,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 10": PCA_sparse5_5010,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 5": PCA_sparse5_505,
    "PCA Local Compressed - fc1_out = 30 and fc2_out = 10": PCA_sparse5_3010,
    "PCA Local Compressed - fc1_out = 30 and fc2_out = 5": PCA_sparse5_305,
    "PCA Local Compressed - fc1_out = 10 and fc2_out = 5": PCA_sparse5_105,
    "SparsePCA Local Compressed - fc1_out = 50 and fc2_out = 32": SparsePCA_sparse5_50,
    "SparsePCA Local Compressed - fc1_out = 30 and fc2_out = 32": SparsePCA_sparse5_30,
    "SparsePCA Local Compressed - fc1_out = 10 and fc2_out = 32": SparsePCA_sparse5_10,
    "ICA Local Model - fc1_out = 50 and fc2_out = 32": ICA_sparse5_50,
    "ICA Local Compressed - fc1_out = 30 and fc2_out = 32": ICA_sparse5_30,
    "ICA Local Compressed - fc1_out = 10 and fc2_out = 32": ICA_sparse5_10,

}

results_sparse5 = {}

for name, model in sparse5_models.items():
    loss, mae = evaluate_model(model, test_loader, device)
    results_sparse5[name] = {"MSE Loss": loss, "MAE": mae}
    print(f"{name} - MSE Loss: {loss:.4f}, MAE: {mae:.4f}")

### Weight Distribution in each Layer of the Models

#### Inspecting the Weight Distribution of the Original and Pruned Models

In [ ]:
# list of local and pruned models (without PCA, SparsePCA, and ICA)
local_pruneList = {
    "Original": model,
    "Local Pruned - 0.60": pruned_local1,
    "Local Pruned - 0.70": pruned_local2,
    "Local Pruned - 0.80": pruned_local3,
    "Local Pruned - 0.90": pruned_local4,
    "Local Pruned - 0.95": pruned_local5,

}

In [ ]:
# Define fc1 layers
layers = ['fc1']

weight_data = {layer: [] for layer in layers}

# Collecting weight data
for layer in layers:
    for model_name, model in local_pruneList.items():
      # reference: https://stackoverflow.com/questions/44938160/saving-layer-weights-at-each-epoch-during-training-into-a-numpy-type-array-conv
        weights = getattr(model, layer).weight.data.reshape(-1).cpu().numpy()
        weight_data[layer].append(weights)

# Create subplots for each layer
fig, ax = plt.subplots(1, 1, figsize=(10, 6))

for weights in weight_data['fc1']:
    ax.hist(weights, bins=100, alpha=0.5)
ax.set_ylim(top=125)
ax.set_title("Weight Distribution of fc1")
ax.set_xlabel("Weight Value")
ax.set_ylabel("Frequency")
ax.legend(local_pruneList.keys())

plt.tight_layout()
plt.show()

In [ ]:
# Define fc2 layers
layers = ['fc2']

weight_data = {layer: [] for layer in layers}

# Collecting weight data
for layer in layers:
    for model_name, model in local_pruneList.items():
        weights = getattr(model, layer).weight.data.reshape(-1).cpu().numpy()
        weight_data[layer].append(weights)

# Create subplots for each layer
fig, ax = plt.subplots(1, 1, figsize=(10, 6))

for weights in weight_data['fc2']:
    ax.hist(weights, bins=100, alpha=0.5)
ax.set_ylim(top=125)
ax.set_title("Weight Distribution of fc2")
ax.set_xlabel("Weight Value")
ax.set_ylabel("Frequency")
ax.legend(local_pruneList.keys())

plt.tight_layout()
plt.show()

In [ ]:
# Define fc3 layers
layers = ['fc3']

weight_data = {layer: [] for layer in layers}

# Collecting weight data
for layer in layers:
    for model_name, model in local_pruneList.items():
        weights = getattr(model, layer).weight.data.reshape(-1).cpu().numpy()
        weight_data[layer].append(weights)

# Create subplots for each layer
fig, ax = plt.subplots(1, 1, figsize=(10, 6))

for weights in weight_data['fc3']:
    ax.hist(weights, bins=100, alpha=0.5)
ax.set_ylim(top=125)
ax.set_title("Weight Distribution of fc3")
ax.set_xlabel("Weight Value")
ax.set_ylabel("Frequency")
ax.legend(local_pruneList.keys())

plt.tight_layout()
plt.show()

#### Inspecting the Weight Distribution of the Original and Smaller Models

In [ ]:
# list of local and pruned models (without PCA, SparsePCA, and ICA)
OG_small_list = {
    "Original": model,
    "Smaller Network - fc1_out = 50 and fc2_out = 30": small_5030,
    "Smaller Network - fc1_out = 50 and fc2_out = 32": small_5032,
    "Smaller Network - fc1_out = 50 and fc2_out = 10": small_5010,
    "Smaller Network - fc1_out = 50 and fc2_out = 5": small_505,
    "Smaller Network - fc1_out = 30 and fc2_out = 32": small_3032,
    "Smaller Network - fc1_out = 30 and fc2_out = 10": small_3010,
    "Smaller Network - fc1_out = 30 and fc2_out = 5": small_305,
    "Smaller Network - fc1_out = 10 and fc2_out = 32": small_1032,
    "Smaller Network - fc1_out = 10 and fc2_out = 5": small_105,

}

In [ ]:
# Define fc1 layers
layers = ['fc1']

weight_data = {layer: [] for layer in layers}

# Collecting weight data
for layer in layers:
    for model_name, model in OG_small_list.items():
        weights = getattr(model, layer).weight.data.reshape(-1).cpu().numpy()
        weight_data[layer].append(weights)

# Create subplots for each layer
fig, ax = plt.subplots(1, 1, figsize=(10, 6))

for weights in weight_data['fc1']:
    ax.hist(weights, bins=100, alpha=0.5)
ax.set_ylim(top=125)
ax.set_title("Weight Distribution of fc1")
ax.set_xlabel("Weight Value")
ax.set_ylabel("Frequency")
ax.legend(OG_small_list.keys())

plt.tight_layout()
plt.show()

In [ ]:
# Define fc2 layers
layers = ['fc2']

weight_data = {layer: [] for layer in layers}

# Collecting weight data
for layer in layers:
    for model_name, model in OG_small_list.items():
        weights = getattr(model, layer).weight.data.reshape(-1).cpu().numpy()
        weight_data[layer].append(weights)

# Create subplots for each layer
fig, ax = plt.subplots(1, 1, figsize=(10, 6))

for weights in weight_data['fc2']:
    ax.hist(weights, bins=100, alpha=0.5)
ax.set_ylim(top=100)
ax.set_title("Weight Distribution of fc2")
ax.set_xlabel("Weight Value")
ax.set_ylabel("Frequency")
ax.legend(OG_small_list.keys())

plt.tight_layout()
plt.show()

In [ ]:
# Define fc3 layers
layers = ['fc3']

weight_data = {layer: [] for layer in layers}

# Collecting weight data
for layer in layers:
    for model_name, model in OG_small_list.items():
        weights = getattr(model, layer).weight.data.reshape(-1).cpu().numpy()
        weight_data[layer].append(weights)

# Create subplots for each layer
fig, ax = plt.subplots(1, 1, figsize=(10, 6))

for weights in weight_data['fc3']:
    ax.hist(weights, bins=100, alpha=0.5)
ax.set_ylim(top=10)
ax.set_title("Weight Distribution of fc3")
ax.set_xlabel("Weight Value")
ax.set_ylabel("Frequency")
ax.legend(OG_small_list.keys())

plt.tight_layout()
plt.show()

#### Inspecting the Weight Distribution of the PCA Pruned Models

##### PCA on Locally Pruned Models

###### Pruning Amount = 0.6

In [ ]:
# list of PCA local models
PCA_pruneList = {
    "Original": model,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 30": PCA_local1_5030,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 30": PCA_local1_5030,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 10": PCA_local1_5010,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 5": PCA_local1_505,
    "PCA Local Compressed - fc1_out = 30 and fc2_out = 10": PCA_local1_3010,
    "PCA Local Compressed - fc1_out = 30 and fc2_out = 5": PCA_local1_305,
    "PCA Local Compressed - fc1_out = 10 and fc2_out = 5": PCA_local1_105

}

In [ ]:
# Define fc1 layers
layers = ['fc1']

weight_data = {layer: [] for layer in layers}

# Collecting weight data
for layer in layers:
    for model_name, model in PCA_pruneList.items():
        weights = getattr(model, layer).weight.data.reshape(-1).cpu().numpy()
        weight_data[layer].append(weights)

# Create subplots for each layer
fig, ax = plt.subplots(1, 1, figsize=(10, 6))

for weights in weight_data['fc1']:
    ax.hist(weights, bins=100, alpha=0.5)
ax.set_ylim(top=200)
ax.set_title("Weight Distribution of fc1")
ax.set_xlabel("Weight Value")
ax.set_ylabel("Frequency")
ax.legend(PCA_pruneList.keys())

plt.tight_layout()
plt.show()

In [ ]:
# Define fc2 layers
layers = ['fc2']

weight_data = {layer: [] for layer in layers}

# Collecting weight data
for layer in layers:
    for model_name, model in PCA_pruneList.items():
        weights = getattr(model, layer).weight.data.reshape(-1).cpu().numpy()
        weight_data[layer].append(weights)

# Create subplots for each layer
fig, ax = plt.subplots(1, 1, figsize=(10, 6))

for weights in weight_data['fc2']:
    ax.hist(weights, bins=100, alpha=0.5)
ax.set_ylim(top=100)
ax.set_title("Weight Distribution of fc2")
ax.set_xlabel("Weight Value")
ax.set_ylabel("Frequency")
ax.legend(PCA_pruneList.keys())

plt.tight_layout()
plt.show()

In [ ]:
# Define fc3 layers
layers = ['fc3']

weight_data = {layer: [] for layer in layers}

# Collecting weight data
for layer in layers:
    for model_name, model in PCA_pruneList.items():
        weights = getattr(model, layer).weight.data.reshape(-1).cpu().numpy()
        weight_data[layer].append(weights)

# Create subplots for each layer
fig, ax = plt.subplots(1, 1, figsize=(10, 6))

for weights in weight_data['fc3']:
    ax.hist(weights, bins=100, alpha=0.5)
ax.set_ylim(top=35)
ax.set_title("Weight Distribution of fc3")
ax.set_xlabel("Weight Value")
ax.set_ylabel("Frequency")
ax.legend(PCA_pruneList.keys())

plt.tight_layout()
plt.show()

###### Pruning Amount = 0.7

In [ ]:
# list of PCA local models
PCA_pruneList2 = {
    "Original": model,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 30": PCA_local2_5030,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 30": PCA_local2_5030,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 10": PCA_local2_5010,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 5": PCA_local2_505,
    "PCA Local Compressed - fc1_out = 30 and fc2_out = 10": PCA_local2_3010,
    "PCA Local Compressed - fc1_out = 30 and fc2_out = 5": PCA_local2_305,
    "PCA Local Compressed - fc1_out = 10 and fc2_out = 5": PCA_local2_105

}

In [ ]:
# Define fc1 layers
layers = ['fc1']

weight_data = {layer: [] for layer in layers}

# Collecting weight data
for layer in layers:
    for model_name, model in PCA_pruneList2.items():
        weights = getattr(model, layer).weight.data.reshape(-1).cpu().numpy()
        weight_data[layer].append(weights)

# Create subplots for each layer
fig, ax = plt.subplots(1, 1, figsize=(10, 6))

for weights in weight_data['fc1']:
    ax.hist(weights, bins=100, alpha=0.5)
ax.set_ylim(top=200)
ax.set_title("Weight Distribution of fc1")
ax.set_xlabel("Weight Value")
ax.set_ylabel("Frequency")
ax.legend(PCA_pruneList2.keys())

plt.tight_layout()
plt.show()

In [ ]:
# Define fc2 layers
layers = ['fc2']

weight_data = {layer: [] for layer in layers}

# Collecting weight data
for layer in layers:
    for model_name, model in PCA_pruneList2.items():
        weights = getattr(model, layer).weight.data.reshape(-1).cpu().numpy()
        weight_data[layer].append(weights)

# Create subplots for each layer
fig, ax = plt.subplots(1, 1, figsize=(10, 6))

for weights in weight_data['fc2']:
    ax.hist(weights, bins=100, alpha=0.5)
ax.set_ylim(top=100)
ax.set_title("Weight Distribution of fc2")
ax.set_xlabel("Weight Value")
ax.set_ylabel("Frequency")
ax.legend(PCA_pruneList2.keys())

plt.tight_layout()
plt.show()

In [ ]:
# Define fc3 layers
layers = ['fc3']

weight_data = {layer: [] for layer in layers}

# Collecting weight data
for layer in layers:
    for model_name, model in PCA_pruneList2.items():
        weights = getattr(model, layer).weight.data.reshape(-1).cpu().numpy()
        weight_data[layer].append(weights)

# Create subplots for each layer
fig, ax = plt.subplots(1, 1, figsize=(10, 6))

for weights in weight_data['fc3']:
    ax.hist(weights, bins=100, alpha=0.5)
ax.set_ylim(top=35)
ax.set_title("Weight Distribution of fc3")
ax.set_xlabel("Weight Value")
ax.set_ylabel("Frequency")
ax.legend(PCA_pruneList2.keys())

plt.tight_layout()
plt.show()

###### Pruning Amount = 0.8

In [ ]:
# list of PCA local models
PCA_pruneList3 = {
    "Original": model,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 30": PCA_local3_5030,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 30": PCA_local3_5030,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 10": PCA_local3_5010,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 5": PCA_local3_505,
    "PCA Local Compressed - fc1_out = 30 and fc2_out = 10": PCA_local3_3010,
    "PCA Local Compressed - fc1_out = 30 and fc2_out = 5": PCA_local3_305,
    "PCA Local Compressed - fc1_out = 10 and fc2_out = 5": PCA_local3_105

}

In [ ]:
# Define fc1 layers
layers = ['fc1']

weight_data = {layer: [] for layer in layers}

# Collecting weight data
for layer in layers:
    for model_name, model in PCA_pruneList3.items():
        weights = getattr(model, layer).weight.data.reshape(-1).cpu().numpy()
        weight_data[layer].append(weights)

# Create subplots for each layer
fig, ax = plt.subplots(1, 1, figsize=(10, 6))

for weights in weight_data['fc1']:
    ax.hist(weights, bins=100, alpha=0.5)
ax.set_ylim(top=200)
ax.set_title("Weight Distribution of fc1")
ax.set_xlabel("Weight Value")
ax.set_ylabel("Frequency")
ax.legend(PCA_pruneList3.keys())

plt.tight_layout()
plt.show()

In [ ]:
# Define fc2 layers
layers = ['fc2']

weight_data = {layer: [] for layer in layers}

# Collecting weight data
for layer in layers:
    for model_name, model in PCA_pruneList3.items():
        weights = getattr(model, layer).weight.data.reshape(-1).cpu().numpy()
        weight_data[layer].append(weights)

# Create subplots for each layer
fig, ax = plt.subplots(1, 1, figsize=(10, 6))

for weights in weight_data['fc2']:
    ax.hist(weights, bins=100, alpha=0.5)
ax.set_ylim(top=100)
ax.set_title("Weight Distribution of fc2")
ax.set_xlabel("Weight Value")
ax.set_ylabel("Frequency")
ax.legend(PCA_pruneList3.keys())

plt.tight_layout()
plt.show()

In [ ]:
# Define fc3 layers
layers = ['fc3']

weight_data = {layer: [] for layer in layers}

# Collecting weight data
for layer in layers:
    for model_name, model in PCA_pruneList3.items():
        weights = getattr(model, layer).weight.data.reshape(-1).cpu().numpy()
        weight_data[layer].append(weights)

# Create subplots for each layer
fig, ax = plt.subplots(1, 1, figsize=(10, 6))

for weights in weight_data['fc3']:
    ax.hist(weights, bins=100, alpha=0.5)
ax.set_ylim(top=35)
ax.set_title("Weight Distribution of fc3")
ax.set_xlabel("Weight Value")
ax.set_ylabel("Frequency")
ax.legend(PCA_pruneList3.keys())

plt.tight_layout()
plt.show()

###### Pruning Amount = 0.9

In [ ]:
# list of PCA local models
PCA_pruneList4 = {
    "Original": model,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 30": PCA_local4_5030,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 30": PCA_local4_5030,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 10": PCA_local4_5010,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 5": PCA_local4_505,
    "PCA Local Compressed - fc1_out = 30 and fc2_out = 10": PCA_local4_3010,
    "PCA Local Compressed - fc1_out = 30 and fc2_out = 5": PCA_local4_305,
    "PCA Local Compressed - fc1_out = 10 and fc2_out = 5": PCA_local4_105

}

In [ ]:
# Define fc1 layers
layers = ['fc1']

weight_data = {layer: [] for layer in layers}

# Collecting weight data
for layer in layers:
    for model_name, model in PCA_pruneList4.items():
        weights = getattr(model, layer).weight.data.reshape(-1).cpu().numpy()
        weight_data[layer].append(weights)

# Create subplots for each layer
fig, ax = plt.subplots(1, 1, figsize=(10, 6))

for weights in weight_data['fc1']:
    ax.hist(weights, bins=100, alpha=0.5)
ax.set_ylim(top=200)
ax.set_title("Weight Distribution of fc1")
ax.set_xlabel("Weight Value")
ax.set_ylabel("Frequency")
ax.legend(PCA_pruneList4.keys())

plt.tight_layout()
plt.show()

In [ ]:
# Define fc2 layers
layers = ['fc2']

weight_data = {layer: [] for layer in layers}

# Collecting weight data
for layer in layers:
    for model_name, model in PCA_pruneList4.items():
        weights = getattr(model, layer).weight.data.reshape(-1).cpu().numpy()
        weight_data[layer].append(weights)

# Create subplots for each layer
fig, ax = plt.subplots(1, 1, figsize=(10, 6))

for weights in weight_data['fc2']:
    ax.hist(weights, bins=100, alpha=0.5)
ax.set_ylim(top=100)
ax.set_title("Weight Distribution of fc2")
ax.set_xlabel("Weight Value")
ax.set_ylabel("Frequency")
ax.legend(PCA_pruneList4.keys())

plt.tight_layout()
plt.show()

In [ ]:
# Define fc3 layers
layers = ['fc3']

weight_data = {layer: [] for layer in layers}

# Collecting weight data
for layer in layers:
    for model_name, model in PCA_pruneList4.items():
        weights = getattr(model, layer).weight.data.reshape(-1).cpu().numpy()
        weight_data[layer].append(weights)

# Create subplots for each layer
fig, ax = plt.subplots(1, 1, figsize=(10, 6))

for weights in weight_data['fc3']:
    ax.hist(weights, bins=100, alpha=0.5)
ax.set_ylim(top=35)
ax.set_title("Weight Distribution of fc3")
ax.set_xlabel("Weight Value")
ax.set_ylabel("Frequency")
ax.legend(PCA_pruneList4.keys())

plt.tight_layout()
plt.show()

###### Pruning Amount = 0.95

In [ ]:
# list of PCA local models
PCA_pruneList5 = {
    "Original": model,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 30": PCA_local5_5030,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 30": PCA_local5_5030,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 10": PCA_local5_5010,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 5": PCA_local5_505,
    "PCA Local Compressed - fc1_out = 30 and fc2_out = 10": PCA_local5_3010,
    "PCA Local Compressed - fc1_out = 30 and fc2_out = 5": PCA_local5_305,
    "PCA Local Compressed - fc1_out = 10 and fc2_out = 5": PCA_local5_105

}

In [ ]:
# Define fc1 layers
layers = ['fc1']

weight_data = {layer: [] for layer in layers}

# Collecting weight data
for layer in layers:
    for model_name, model in PCA_pruneList5.items():
        weights = getattr(model, layer).weight.data.reshape(-1).cpu().numpy()
        weight_data[layer].append(weights)

# Create subplots for each layer
fig, ax = plt.subplots(1, 1, figsize=(10, 6))

for weights in weight_data['fc1']:
    ax.hist(weights, bins=100, alpha=0.5)
ax.set_ylim(top=200)
ax.set_title("Weight Distribution of fc1")
ax.set_xlabel("Weight Value")
ax.set_ylabel("Frequency")
ax.legend(PCA_pruneList5.keys())

plt.tight_layout()
plt.show()

In [ ]:
# Define fc2 layers
layers = ['fc2']

weight_data = {layer: [] for layer in layers}

# Collecting weight data
for layer in layers:
    for model_name, model in PCA_pruneList5.items():
        weights = getattr(model, layer).weight.data.reshape(-1).cpu().numpy()
        weight_data[layer].append(weights)

# Create subplots for each layer
fig, ax = plt.subplots(1, 1, figsize=(10, 6))

for weights in weight_data['fc2']:
    ax.hist(weights, bins=100, alpha=0.5)
ax.set_ylim(top=100)
ax.set_title("Weight Distribution of fc2")
ax.set_xlabel("Weight Value")
ax.set_ylabel("Frequency")
ax.legend(PCA_pruneList5.keys())

plt.tight_layout()
plt.show()

In [ ]:
# Define fc3 layers
layers = ['fc3']

weight_data = {layer: [] for layer in layers}

# Collecting weight data
for layer in layers:
    for model_name, model in PCA_pruneList5.items():
        weights = getattr(model, layer).weight.data.reshape(-1).cpu().numpy()
        weight_data[layer].append(weights)

# Create subplots for each layer
fig, ax = plt.subplots(1, 1, figsize=(10, 6))

for weights in weight_data['fc3']:
    ax.hist(weights, bins=100, alpha=0.5)
ax.set_ylim(top=35)
ax.set_title("Weight Distribution of fc3")
ax.set_xlabel("Weight Value")
ax.set_ylabel("Frequency")
ax.legend(PCA_pruneList5.keys())

plt.tight_layout()
plt.show()

##### PCA on Layer-wise Locally Pruned Models

###### Pruning Amount = 0.6

In [ ]:
# list of PCA local models
PCA_layerwiseList = {
    "Original": model,
    "PCA Layer-Wise Compressed - fc1_out = 50 and fc2_out = 30": PCA_sparse1_5030,
    "PCA Layer-Wise Compressed - fc1_out = 50 and fc2_out = 30": PCA_sparse1_5030,
    "PCA Layer-Wise Compressed - fc1_out = 50 and fc2_out = 10": PCA_sparse1_5010,
    "PCA Layer-Wise Compressed - fc1_out = 50 and fc2_out = 5": PCA_sparse1_505,
    "PCA Layer-Wise Compressed - fc1_out = 30 and fc2_out = 10": PCA_sparse1_3010,
    "PCA Layer-Wise Compressed - fc1_out = 30 and fc2_out = 5": PCA_sparse1_305,
    "PCA Layer-Wise Compressed - fc1_out = 10 and fc2_out = 5": PCA_sparse1_105
}

In [ ]:
# Define fc1 layers
layers = ['fc1']

weight_data = {layer: [] for layer in layers}

# Collecting weight data
for layer in layers:
    for model_name, model in PCA_layerwiseList.items():
        weights = getattr(model, layer).weight.data.reshape(-1).cpu().numpy()
        weight_data[layer].append(weights)

# Create subplots for each layer
fig, ax = plt.subplots(1, 1, figsize=(10, 6))

for weights in weight_data['fc1']:
    ax.hist(weights, bins=100, alpha=0.5)
ax.set_ylim(top=200)
ax.set_title("Weight Distribution of fc1")
ax.set_xlabel("Weight Value")
ax.set_ylabel("Frequency")
ax.legend(PCA_layerwiseList.keys())

plt.tight_layout()
plt.show()

In [ ]:
# Define fc2 layers
layers = ['fc2']

weight_data = {layer: [] for layer in layers}

# Collecting weight data
for layer in layers:
    for model_name, model in PCA_layerwiseList.items():
        weights = getattr(model, layer).weight.data.reshape(-1).cpu().numpy()
        weight_data[layer].append(weights)

# Create subplots for each layer
fig, ax = plt.subplots(1, 1, figsize=(10, 6))

for weights in weight_data['fc2']:
    ax.hist(weights, bins=100, alpha=0.5)
ax.set_ylim(top=100)
ax.set_title("Weight Distribution of fc2")
ax.set_xlabel("Weight Value")
ax.set_ylabel("Frequency")
ax.legend(PCA_layerwiseList.keys())

plt.tight_layout()
plt.show()

In [ ]:
# Define fc3 layers
layers = ['fc3']

weight_data = {layer: [] for layer in layers}

# Collecting weight data
for layer in layers:
    for model_name, model in PCA_layerwiseList.items():
        weights = getattr(model, layer).weight.data.reshape(-1).cpu().numpy()
        weight_data[layer].append(weights)

# Create subplots for each layer
fig, ax = plt.subplots(1, 1, figsize=(10, 6))

for weights in weight_data['fc3']:
    ax.hist(weights, bins=100, alpha=0.5)
ax.set_ylim(top=10)
ax.set_title("Weight Distribution of fc3")
ax.set_xlabel("Weight Value")
ax.set_ylabel("Frequency")
ax.legend(PCA_layerwiseList.keys())

plt.tight_layout()
plt.show()

###### Pruning Amount = 0.7

In [ ]:
# list of PCA local models
PCA_layerwiseList2 = {
    "Original": model,
    "PCA Layer-Wise Compressed - fc1_out = 50 and fc2_out = 30": PCA_sparse2_5030,
    "PCA Layer-Wise Compressed - fc1_out = 50 and fc2_out = 30": PCA_sparse2_5030,
    "PCA Layer-Wise Compressed - fc1_out = 50 and fc2_out = 10": PCA_sparse2_5010,
    "PCA Layer-Wise Compressed - fc1_out = 50 and fc2_out = 5": PCA_sparse2_505,
    "PCA Layer-Wise Compressed - fc1_out = 30 and fc2_out = 10": PCA_sparse2_3010,
    "PCA Layer-Wise Compressed - fc1_out = 30 and fc2_out = 5": PCA_sparse2_305,
    "PCA Layer-Wise Compressed - fc1_out = 10 and fc2_out = 5": PCA_sparse2_105
}

In [ ]:
# Define fc1 layers
layers = ['fc1']

weight_data = {layer: [] for layer in layers}

# Collecting weight data
for layer in layers:
    for model_name, model in PCA_layerwiseList2.items():
        weights = getattr(model, layer).weight.data.reshape(-1).cpu().numpy()
        weight_data[layer].append(weights)

# Create subplots for each layer
fig, ax = plt.subplots(1, 1, figsize=(10, 6))

for weights in weight_data['fc1']:
    ax.hist(weights, bins=100, alpha=0.5)
ax.set_ylim(top=200)
ax.set_title("Weight Distribution of fc1")
ax.set_xlabel("Weight Value")
ax.set_ylabel("Frequency")
ax.legend(PCA_layerwiseList2.keys())

plt.tight_layout()
plt.show()

In [ ]:
# Define fc2 layers
layers = ['fc2']

weight_data = {layer: [] for layer in layers}

# Collecting weight data
for layer in layers:
    for model_name, model in PCA_layerwiseList2.items():
        weights = getattr(model, layer).weight.data.reshape(-1).cpu().numpy()
        weight_data[layer].append(weights)

# Create subplots for each layer
fig, ax = plt.subplots(1, 1, figsize=(10, 6))

for weights in weight_data['fc2']:
    ax.hist(weights, bins=100, alpha=0.5)
ax.set_ylim(top=100)
ax.set_title("Weight Distribution of fc2")
ax.set_xlabel("Weight Value")
ax.set_ylabel("Frequency")
ax.legend(PCA_layerwiseList2.keys())

plt.tight_layout()
plt.show()

In [ ]:
# Define fc3 layers
layers = ['fc3']

weight_data = {layer: [] for layer in layers}

# Collecting weight data
for layer in layers:
    for model_name, model in PCA_layerwiseList2.items():
        weights = getattr(model, layer).weight.data.reshape(-1).cpu().numpy()
        weight_data[layer].append(weights)

# Create subplots for each layer
fig, ax = plt.subplots(1, 1, figsize=(10, 6))

for weights in weight_data['fc3']:
    ax.hist(weights, bins=100, alpha=0.5)
ax.set_ylim(top=10)
ax.set_title("Weight Distribution of fc3")
ax.set_xlabel("Weight Value")
ax.set_ylabel("Frequency")
ax.legend(PCA_layerwiseList2.keys())

plt.tight_layout()
plt.show()

###### Pruning Amount = 0.8

In [ ]:
# list of PCA local models
PCA_layerwiseList3 = {
    "Original": model,
    "PCA Layer-Wise Compressed - fc1_out = 50 and fc2_out = 30": PCA_sparse3_5030,
    "PCA Layer-Wise Compressed - fc1_out = 50 and fc2_out = 30": PCA_sparse3_5030,
    "PCA Layer-Wise Compressed - fc1_out = 50 and fc2_out = 10": PCA_sparse3_5010,
    "PCA Layer-Wise Compressed - fc1_out = 50 and fc2_out = 5": PCA_sparse3_505,
    "PCA Layer-Wise Compressed - fc1_out = 30 and fc2_out = 10": PCA_sparse3_3010,
    "PCA Layer-Wise Compressed - fc1_out = 30 and fc2_out = 5": PCA_sparse3_305,
    "PCA Layer-Wise Compressed - fc1_out = 10 and fc2_out = 5": PCA_sparse3_105
}

In [ ]:
# Define fc1 layers
layers = ['fc1']

weight_data = {layer: [] for layer in layers}

# Collecting weight data
for layer in layers:
    for model_name, model in PCA_layerwiseList3.items():
        weights = getattr(model, layer).weight.data.reshape(-1).cpu().numpy()
        weight_data[layer].append(weights)

# Create subplots for each layer
fig, ax = plt.subplots(1, 1, figsize=(10, 6))

for weights in weight_data['fc1']:
    ax.hist(weights, bins=100, alpha=0.5)
ax.set_ylim(top=200)
ax.set_title("Weight Distribution of fc1")
ax.set_xlabel("Weight Value")
ax.set_ylabel("Frequency")
ax.legend(PCA_layerwiseList3.keys())

plt.tight_layout()
plt.show()

In [ ]:
# Define fc2 layers
layers = ['fc2']

weight_data = {layer: [] for layer in layers}

# Collecting weight data
for layer in layers:
    for model_name, model in PCA_layerwiseList3.items():
        weights = getattr(model, layer).weight.data.reshape(-1).cpu().numpy()
        weight_data[layer].append(weights)

# Create subplots for each layer
fig, ax = plt.subplots(1, 1, figsize=(10, 6))

for weights in weight_data['fc2']:
    ax.hist(weights, bins=100, alpha=0.5)
ax.set_ylim(top=100)
ax.set_title("Weight Distribution of fc2")
ax.set_xlabel("Weight Value")
ax.set_ylabel("Frequency")
ax.legend(PCA_layerwiseList3.keys())

plt.tight_layout()
plt.show()

In [ ]:
# Define fc3 layers
layers = ['fc3']

weight_data = {layer: [] for layer in layers}

# Collecting weight data
for layer in layers:
    for model_name, model in PCA_layerwiseList3.items():
        weights = getattr(model, layer).weight.data.reshape(-1).cpu().numpy()
        weight_data[layer].append(weights)

# Create subplots for each layer
fig, ax = plt.subplots(1, 1, figsize=(10, 6))

for weights in weight_data['fc3']:
    ax.hist(weights, bins=100, alpha=0.5)
ax.set_ylim(top=10)
ax.set_title("Weight Distribution of fc3")
ax.set_xlabel("Weight Value")
ax.set_ylabel("Frequency")
ax.legend(PCA_layerwiseList3.keys())

plt.tight_layout()
plt.show()

###### Pruning Amount = 0.9

In [ ]:
# list of PCA local models
PCA_layerwiseList4 = {
    "Original": model,
    "PCA Layer-Wise Compressed - fc1_out = 50 and fc2_out = 30": PCA_sparse4_5030,
    "PCA Layer-Wise Compressed - fc1_out = 50 and fc2_out = 30": PCA_sparse4_5030,
    "PCA Layer-Wise Compressed - fc1_out = 50 and fc2_out = 10": PCA_sparse4_5010,
    "PCA Layer-Wise Compressed - fc1_out = 50 and fc2_out = 5": PCA_sparse4_505,
    "PCA Layer-Wise Compressed - fc1_out = 30 and fc2_out = 10": PCA_sparse4_3010,
    "PCA Layer-Wise Compressed - fc1_out = 30 and fc2_out = 5": PCA_sparse4_305,
    "PCA Layer-Wise Compressed - fc1_out = 10 and fc2_out = 5": PCA_sparse4_105
}

In [ ]:
# Define fc1 layers
layers = ['fc1']

weight_data = {layer: [] for layer in layers}

# Collecting weight data
for layer in layers:
    for model_name, model in PCA_layerwiseList4.items():
        weights = getattr(model, layer).weight.data.reshape(-1).cpu().numpy()
        weight_data[layer].append(weights)

# Create subplots for each layer
fig, ax = plt.subplots(1, 1, figsize=(10, 6))

for weights in weight_data['fc1']:
    ax.hist(weights, bins=100, alpha=0.5)
ax.set_ylim(top=200)
ax.set_title("Weight Distribution of fc1")
ax.set_xlabel("Weight Value")
ax.set_ylabel("Frequency")
ax.legend(PCA_layerwiseList4.keys())

plt.tight_layout()
plt.show()

In [ ]:
# Define fc2 layers
layers = ['fc2']

weight_data = {layer: [] for layer in layers}

# Collecting weight data
for layer in layers:
    for model_name, model in PCA_layerwiseList4.items():
        weights = getattr(model, layer).weight.data.reshape(-1).cpu().numpy()
        weight_data[layer].append(weights)

# Create subplots for each layer
fig, ax = plt.subplots(1, 1, figsize=(10, 6))

for weights in weight_data['fc2']:
    ax.hist(weights, bins=100, alpha=0.5)
ax.set_ylim(top=100)
ax.set_title("Weight Distribution of fc2")
ax.set_xlabel("Weight Value")
ax.set_ylabel("Frequency")
ax.legend(PCA_layerwiseList4.keys())

plt.tight_layout()
plt.show()

In [ ]:
# Define fc3 layers
layers = ['fc3']

weight_data = {layer: [] for layer in layers}

# Collecting weight data
for layer in layers:
    for model_name, model in PCA_layerwiseList4.items():
        weights = getattr(model, layer).weight.data.reshape(-1).cpu().numpy()
        weight_data[layer].append(weights)

# Create subplots for each layer
fig, ax = plt.subplots(1, 1, figsize=(10, 6))

for weights in weight_data['fc3']:
    ax.hist(weights, bins=100, alpha=0.5)
ax.set_ylim(top=10)
ax.set_title("Weight Distribution of fc3")
ax.set_xlabel("Weight Value")
ax.set_ylabel("Frequency")
ax.legend(PCA_layerwiseList4.keys())

plt.tight_layout()
plt.show()

###### Pruning Amount = 0.95

In [ ]:
# list of PCA local models
PCA_layerwiseList5 = {
    "Original": model,
    "PCA Layer-Wise Compressed - fc1_out = 50 and fc2_out = 30": PCA_sparse5_5030,
    "PCA Layer-Wise Compressed - fc1_out = 50 and fc2_out = 30": PCA_sparse5_5030,
    "PCA Layer-Wise Compressed - fc1_out = 50 and fc2_out = 10": PCA_sparse5_5010,
    "PCA Layer-Wise Compressed - fc1_out = 50 and fc2_out = 5": PCA_sparse5_505,
    "PCA Layer-Wise Compressed - fc1_out = 30 and fc2_out = 10": PCA_sparse5_3010,
    "PCA Layer-Wise Compressed - fc1_out = 30 and fc2_out = 5": PCA_sparse5_305,
    "PCA Layer-Wise Compressed - fc1_out = 10 and fc2_out = 5": PCA_sparse5_105
}

In [ ]:
# Define fc1 layers
layers = ['fc1']

weight_data = {layer: [] for layer in layers}

# Collecting weight data
for layer in layers:
    for model_name, model in PCA_layerwiseList5.items():
        weights = getattr(model, layer).weight.data.reshape(-1).cpu().numpy()
        weight_data[layer].append(weights)

# Create subplots for each layer
fig, ax = plt.subplots(1, 1, figsize=(10, 6))

for weights in weight_data['fc1']:
    ax.hist(weights, bins=100, alpha=0.5)
ax.set_ylim(top=200)
ax.set_title("Weight Distribution of fc1")
ax.set_xlabel("Weight Value")
ax.set_ylabel("Frequency")
ax.legend(PCA_layerwiseList5.keys())

plt.tight_layout()
plt.show()

In [ ]:
# Define fc2 layers
layers = ['fc2']

weight_data = {layer: [] for layer in layers}

# Collecting weight data
for layer in layers:
    for model_name, model in PCA_layerwiseList5.items():
        weights = getattr(model, layer).weight.data.reshape(-1).cpu().numpy()
        weight_data[layer].append(weights)

# Create subplots for each layer
fig, ax = plt.subplots(1, 1, figsize=(10, 6))

for weights in weight_data['fc2']:
    ax.hist(weights, bins=100, alpha=0.5)
ax.set_ylim(top=100)
ax.set_title("Weight Distribution of fc2")
ax.set_xlabel("Weight Value")
ax.set_ylabel("Frequency")
ax.legend(PCA_layerwiseList5.keys())

plt.tight_layout()
plt.show()

In [ ]:
# Define fc3 layers
layers = ['fc3']

weight_data = {layer: [] for layer in layers}

# Collecting weight data
for layer in layers:
    for model_name, model in PCA_layerwiseList5.items():
        weights = getattr(model, layer).weight.data.reshape(-1).cpu().numpy()
        weight_data[layer].append(weights)

# Create subplots for each layer
fig, ax = plt.subplots(1, 1, figsize=(10, 6))

for weights in weight_data['fc3']:
    ax.hist(weights, bins=100, alpha=0.5)
ax.set_ylim(top=10)
ax.set_title("Weight Distribution of fc3")
ax.set_xlabel("Weight Value")
ax.set_ylabel("Frequency")
ax.legend(PCA_layerwiseList5.keys())

plt.tight_layout()
plt.show()

#### Inspecting the Weight Distribution of the SparsePCA Pruned Models

##### SparsePCA on Locally Pruned Models

###### Pruning Amount = 0.6

In [ ]:
# list of SparsePCA local models
SparsePCA_pruneList = {
    "Original": model,
    "SparsePCA Local Compressed - fc1_out = 50 and fc2_out = 32": SparsePCA_l1_50,
    "SparsePCA Local Compressed - fc1_out = 30 and fc2_out = 32": SparsePCA_l1_30,
    "SparsePCA Local Compressed - fc1_out = 10 and fc2_out = 32": SparsePCA_l1_10
}

In [ ]:
# Define fc1 layers
layers = ['fc1']

weight_data = {layer: [] for layer in layers}

# Collecting weight data
for layer in layers:
    for model_name, model in SparsePCA_pruneList.items():
        # Flatten the weight data and convert to CPU numpy array
        weights = getattr(model, layer).weight.data.reshape(-1).cpu().numpy()
        weight_data[layer].append(weights)

# Create subplots for each layer
fig, ax = plt.subplots(1, 1, figsize=(10, 6))

for weights in weight_data['fc1']:
    ax.hist(weights, bins=100, alpha=0.5)
ax.set_ylim(top=200)
ax.set_title("Weight Distribution of fc1")
ax.set_xlabel("Weight Value")
ax.set_ylabel("Frequency")
ax.legend(SparsePCA_pruneList.keys())

plt.tight_layout()
plt.show()

In [ ]:
# Define fc2 layers
layers = ['fc2']

weight_data = {layer: [] for layer in layers}

# Collecting weight data
for layer in layers:
    for model_name, model in SparsePCA_pruneList.items():
        # Flatten the weight data and convert to CPU numpy array
        weights = getattr(model, layer).weight.data.reshape(-1).cpu().numpy()
        weight_data[layer].append(weights)

# Create subplots for each layer
fig, ax = plt.subplots(1, 1, figsize=(10, 6))

for weights in weight_data['fc2']:
    ax.hist(weights, bins=100, alpha=0.5)
ax.set_ylim(top=100)
ax.set_title("Weight Distribution of fc2")
ax.set_xlabel("Weight Value")
ax.set_ylabel("Frequency")
ax.legend(SparsePCA_pruneList.keys())

plt.tight_layout()
plt.show()

In [ ]:
# Define fc3 layers
layers = ['fc3']

weight_data = {layer: [] for layer in layers}

# Collecting weight data
for layer in layers:
    for model_name, model in SparsePCA_pruneList.items():
        # Flatten the weight data and convert to CPU numpy array
        weights = getattr(model, layer).weight.data.reshape(-1).cpu().numpy()
        weight_data[layer].append(weights)

# Create subplots for each layer
fig, ax = plt.subplots(1, 1, figsize=(10, 6))

for weights in weight_data['fc3']:
    ax.hist(weights, bins=100, alpha=0.5)
ax.set_ylim(top=20)
ax.set_title("Weight Distribution of fc3")
ax.set_xlabel("Weight Value")
ax.set_ylabel("Frequency")
ax.legend(SparsePCA_pruneList.keys())

plt.tight_layout()
plt.show()

###### Pruning Amount = 0.7

In [ ]:
# list of SparsePCA local models
SparsePCA_pruneList2 = {
    "Original": model,
    "SparsePCA Local Compressed - fc1_out = 50 and fc2_out = 32": SparsePCA_l2_50,
    "SparsePCA Local Compressed - fc1_out = 30 and fc2_out = 32": SparsePCA_l2_30,
    "SparsePCA Local Compressed - fc1_out = 10 and fc2_out = 32": SparsePCA_l2_10
}

In [ ]:
# Define fc1 layers
layers = ['fc1']

weight_data = {layer: [] for layer in layers}

# Collecting weight data
for layer in layers:
    for model_name, model in SparsePCA_pruneList2.items():
        # Flatten the weight data and convert to CPU numpy array
        weights = getattr(model, layer).weight.data.reshape(-1).cpu().numpy()
        weight_data[layer].append(weights)

# Create subplots for each layer
fig, ax = plt.subplots(1, 1, figsize=(10, 6))

for weights in weight_data['fc1']:
    ax.hist(weights, bins=100, alpha=0.5)
ax.set_ylim(top=200)
ax.set_title("Weight Distribution of fc1")
ax.set_xlabel("Weight Value")
ax.set_ylabel("Frequency")
ax.legend(SparsePCA_pruneList2.keys())

plt.tight_layout()
plt.show()

In [ ]:
# Define fc2 layers
layers = ['fc2']

weight_data = {layer: [] for layer in layers}

# Collecting weight data
for layer in layers:
    for model_name, model in SparsePCA_pruneList2.items():
        # Flatten the weight data and convert to CPU numpy array
        weights = getattr(model, layer).weight.data.reshape(-1).cpu().numpy()
        weight_data[layer].append(weights)

# Create subplots for each layer
fig, ax = plt.subplots(1, 1, figsize=(10, 6))

for weights in weight_data['fc2']:
    ax.hist(weights, bins=100, alpha=0.5)
ax.set_ylim(top=100)
ax.set_title("Weight Distribution of fc2")
ax.set_xlabel("Weight Value")
ax.set_ylabel("Frequency")
ax.legend(SparsePCA_pruneList2.keys())

plt.tight_layout()
plt.show()

In [ ]:
# Define fc3 layers
layers = ['fc3']

weight_data = {layer: [] for layer in layers}

# Collecting weight data
for layer in layers:
    for model_name, model in SparsePCA_pruneList2.items():
        # Flatten the weight data and convert to CPU numpy array
        weights = getattr(model, layer).weight.data.reshape(-1).cpu().numpy()
        weight_data[layer].append(weights)

# Create subplots for each layer
fig, ax = plt.subplots(1, 1, figsize=(10, 6))

for weights in weight_data['fc3']:
    ax.hist(weights, bins=100, alpha=0.5)
ax.set_ylim(top=20)
ax.set_title("Weight Distribution of fc3")
ax.set_xlabel("Weight Value")
ax.set_ylabel("Frequency")
ax.legend(SparsePCA_pruneList2.keys())

plt.tight_layout()
plt.show()

###### Pruning Amount = 0.8

In [ ]:
# list of SparsePCA local models
SparsePCA_pruneList3 = {
    "Original": model,
    "SparsePCA Local Compressed - fc1_out = 50 and fc2_out = 32": SparsePCA_l3_50,
    "SparsePCA Local Compressed - fc1_out = 30 and fc2_out = 32": SparsePCA_l3_30,
    "SparsePCA Local Compressed - fc1_out = 10 and fc2_out = 32": SparsePCA_l3_10
}

In [ ]:
# Define fc1 layers
layers = ['fc1']

weight_data = {layer: [] for layer in layers}

# Collecting weight data
for layer in layers:
    for model_name, model in SparsePCA_pruneList3.items():
        # Flatten the weight data and convert to CPU numpy array
        weights = getattr(model, layer).weight.data.reshape(-1).cpu().numpy()
        weight_data[layer].append(weights)

# Create subplots for each layer
fig, ax = plt.subplots(1, 1, figsize=(10, 6))

for weights in weight_data['fc1']:
    ax.hist(weights, bins=100, alpha=0.5)
ax.set_ylim(top=200)
ax.set_title("Weight Distribution of fc1")
ax.set_xlabel("Weight Value")
ax.set_ylabel("Frequency")
ax.legend(SparsePCA_pruneList3.keys())

plt.tight_layout()
plt.show()

In [ ]:
# Define fc2 layers
layers = ['fc2']

weight_data = {layer: [] for layer in layers}

# Collecting weight data
for layer in layers:
    for model_name, model in SparsePCA_pruneList3.items():
        # Flatten the weight data and convert to CPU numpy array
        weights = getattr(model, layer).weight.data.reshape(-1).cpu().numpy()
        weight_data[layer].append(weights)

# Create subplots for each layer
fig, ax = plt.subplots(1, 1, figsize=(10, 6))

for weights in weight_data['fc2']:
    ax.hist(weights, bins=100, alpha=0.5)
ax.set_ylim(top=100)
ax.set_title("Weight Distribution of fc2")
ax.set_xlabel("Weight Value")
ax.set_ylabel("Frequency")
ax.legend(SparsePCA_pruneList3.keys())

plt.tight_layout()
plt.show()

In [ ]:
# Define fc3 layers
layers = ['fc3']

weight_data = {layer: [] for layer in layers}

# Collecting weight data
for layer in layers:
    for model_name, model in SparsePCA_pruneList3.items():
        # Flatten the weight data and convert to CPU numpy array
        weights = getattr(model, layer).weight.data.reshape(-1).cpu().numpy()
        weight_data[layer].append(weights)

# Create subplots for each layer
fig, ax = plt.subplots(1, 1, figsize=(10, 6))

for weights in weight_data['fc3']:
    ax.hist(weights, bins=100, alpha=0.5)
ax.set_ylim(top=20)
ax.set_title("Weight Distribution of fc3")
ax.set_xlabel("Weight Value")
ax.set_ylabel("Frequency")
ax.legend(SparsePCA_pruneList3.keys())

plt.tight_layout()
plt.show()

###### Pruning Amount = 0.9

In [ ]:
# list of SparsePCA local models
SparsePCA_pruneList4 = {
    "Original": model,
    "SparsePCA Local Compressed - fc1_out = 50 and fc2_out = 32": SparsePCA_l4_50,
    "SparsePCA Local Compressed - fc1_out = 30 and fc2_out = 32": SparsePCA_l4_30,
    "SparsePCA Local Compressed - fc1_out = 10 and fc2_out = 32": SparsePCA_l4_10
}

In [ ]:
# Define fc1 layers
layers = ['fc1']

weight_data = {layer: [] for layer in layers}

# Collecting weight data
for layer in layers:
    for model_name, model in SparsePCA_pruneList4.items():
        # Flatten the weight data and convert to CPU numpy array
        weights = getattr(model, layer).weight.data.reshape(-1).cpu().numpy()
        weight_data[layer].append(weights)

# Create subplots for each layer
fig, ax = plt.subplots(1, 1, figsize=(10, 6))

for weights in weight_data['fc1']:
    ax.hist(weights, bins=100, alpha=0.5)
ax.set_ylim(top=200)
ax.set_title("Weight Distribution of fc1")
ax.set_xlabel("Weight Value")
ax.set_ylabel("Frequency")
ax.legend(SparsePCA_pruneList4.keys())

plt.tight_layout()
plt.show()

In [ ]:
# Define fc2 layers
layers = ['fc2']

weight_data = {layer: [] for layer in layers}

# Collecting weight data
for layer in layers:
    for model_name, model in SparsePCA_pruneList4.items():
        # Flatten the weight data and convert to CPU numpy array
        weights = getattr(model, layer).weight.data.reshape(-1).cpu().numpy()
        weight_data[layer].append(weights)

# Create subplots for each layer
fig, ax = plt.subplots(1, 1, figsize=(10, 6))

for weights in weight_data['fc2']:
    ax.hist(weights, bins=100, alpha=0.5)
ax.set_ylim(top=100)
ax.set_title("Weight Distribution of fc2")
ax.set_xlabel("Weight Value")
ax.set_ylabel("Frequency")
ax.legend(SparsePCA_pruneList4.keys())

plt.tight_layout()
plt.show()

In [ ]:
# Define fc3 layers
layers = ['fc3']

weight_data = {layer: [] for layer in layers}

# Collecting weight data
for layer in layers:
    for model_name, model in SparsePCA_pruneList4.items():
        # Flatten the weight data and convert to CPU numpy array
        weights = getattr(model, layer).weight.data.reshape(-1).cpu().numpy()
        weight_data[layer].append(weights)

# Create subplots for each layer
fig, ax = plt.subplots(1, 1, figsize=(10, 6))

for weights in weight_data['fc3']:
    ax.hist(weights, bins=100, alpha=0.5)
ax.set_ylim(top=20)
ax.set_title("Weight Distribution of fc3")
ax.set_xlabel("Weight Value")
ax.set_ylabel("Frequency")
ax.legend(SparsePCA_pruneList4.keys())

plt.tight_layout()
plt.show()

###### Pruning Amount = 0.95

In [ ]:
# list of SparsePCA local models
SparsePCA_pruneList5 = {
    "Original": model,
    "SparsePCA Local Compressed - fc1_out = 50 and fc2_out = 32": SparsePCA_l5_50,
    "SparsePCA Local Compressed - fc1_out = 30 and fc2_out = 32": SparsePCA_l5_30,
    "SparsePCA Local Compressed - fc1_out = 10 and fc2_out = 32": SparsePCA_l5_10
}

In [ ]:
# Define fc1 layers
layers = ['fc1']

weight_data = {layer: [] for layer in layers}

# Collecting weight data
for layer in layers:
    for model_name, model in SparsePCA_pruneList5.items():
        # Flatten the weight data and convert to CPU numpy array
        weights = getattr(model, layer).weight.data.reshape(-1).cpu().numpy()
        weight_data[layer].append(weights)

# Create subplots for each layer
fig, ax = plt.subplots(1, 1, figsize=(10, 6))

for weights in weight_data['fc1']:
    ax.hist(weights, bins=100, alpha=0.5)
ax.set_ylim(top=200)
ax.set_title("Weight Distribution of fc1")
ax.set_xlabel("Weight Value")
ax.set_ylabel("Frequency")
ax.legend(SparsePCA_pruneList5.keys())

plt.tight_layout()
plt.show()

In [ ]:
# Define fc2 layers
layers = ['fc2']

weight_data = {layer: [] for layer in layers}

# Collecting weight data
for layer in layers:
    for model_name, model in SparsePCA_pruneList5.items():
        # Flatten the weight data and convert to CPU numpy array
        weights = getattr(model, layer).weight.data.reshape(-1).cpu().numpy()
        weight_data[layer].append(weights)

# Create subplots for each layer
fig, ax = plt.subplots(1, 1, figsize=(10, 6))

for weights in weight_data['fc2']:
    ax.hist(weights, bins=100, alpha=0.5)
ax.set_ylim(top=100)
ax.set_title("Weight Distribution of fc2")
ax.set_xlabel("Weight Value")
ax.set_ylabel("Frequency")
ax.legend(SparsePCA_pruneList5.keys())

plt.tight_layout()
plt.show()

In [ ]:
# Define fc3 layers
layers = ['fc3']

weight_data = {layer: [] for layer in layers}

# Collecting weight data
for layer in layers:
    for model_name, model in SparsePCA_pruneList5.items():
        # Flatten the weight data and convert to CPU numpy array
        weights = getattr(model, layer).weight.data.reshape(-1).cpu().numpy()
        weight_data[layer].append(weights)

# Create subplots for each layer
fig, ax = plt.subplots(1, 1, figsize=(10, 6))

for weights in weight_data['fc3']:
    ax.hist(weights, bins=100, alpha=0.5)
ax.set_ylim(top=20)
ax.set_title("Weight Distribution of fc3")
ax.set_xlabel("Weight Value")
ax.set_ylabel("Frequency")
ax.legend(SparsePCA_pruneList5.keys())

plt.tight_layout()
plt.show()

##### SparsePCA on Layer-wise Locally Pruned Models

###### Pruning Amount = 0.6

In [ ]:
# list of PCA local models
SparsePCA_spasreList = {
    "Original": model,
    "SparsePCA Local Compressed - fc1_out = 50 and fc2_out = 32": SparsePCA_sparse1_50,
    "SparsePCA Local Compressed - fc1_out = 30 and fc2_out = 32": SparsePCA_sparse1_30,
    "SparsePCA Local Compressed - fc1_out = 10 and fc2_out = 32": SparsePCA_sparse1_10
}

In [ ]:
# Define fc1 layers
layers = ['fc1']

weight_data = {layer: [] for layer in layers}

# Collecting weight data
for layer in layers:
    for model_name, model in SparsePCA_spasreList.items():
        weights = getattr(model, layer).weight.data.reshape(-1).cpu().numpy()
        weight_data[layer].append(weights)

# Create subplots for each layer
fig, ax = plt.subplots(1, 1, figsize=(10, 6))

for weights in weight_data['fc1']:
    ax.hist(weights, bins=100, alpha=0.5)
ax.set_ylim(top=100)
ax.set_title("Weight Distribution of fc1")
ax.set_xlabel("Weight Value")
ax.set_ylabel("Frequency")
ax.legend(SparsePCA_spasreList.keys())

plt.tight_layout()
plt.show()

In [ ]:
# Define fc2 layers
layers = ['fc2']

weight_data = {layer: [] for layer in layers}

# Collecting weight data
for layer in layers:
    for model_name, model in SparsePCA_spasreList.items():
        weights = getattr(model, layer).weight.data.reshape(-1).cpu().numpy()
        weight_data[layer].append(weights)

# Create subplots for each layer
fig, ax = plt.subplots(1, 1, figsize=(10, 6))

for weights in weight_data['fc2']:
    ax.hist(weights, bins=100, alpha=0.5)
ax.set_ylim(top=80)
ax.set_title("Weight Distribution of fc2")
ax.set_xlabel("Weight Value")
ax.set_ylabel("Frequency")
ax.legend(SparsePCA_spasreList.keys())

plt.tight_layout()
plt.show()

In [ ]:
# Define fc3 layers
layers = ['fc3']

weight_data = {layer: [] for layer in layers}

# Collecting weight data
for layer in layers:
    for model_name, model in SparsePCA_spasreList.items():
        weights = getattr(model, layer).weight.data.reshape(-1).cpu().numpy()
        weight_data[layer].append(weights)

# Create subplots for each layer
fig, ax = plt.subplots(1, 1, figsize=(10, 6))

for weights in weight_data['fc3']:
    ax.hist(weights, bins=100, alpha=0.5)
ax.set_ylim(top=15)
ax.set_title("Weight Distribution of fc3")
ax.set_xlabel("Weight Value")
ax.set_ylabel("Frequency")
ax.legend(SparsePCA_spasreList.keys())

plt.tight_layout()
plt.show()

###### Pruning Amount = 0.7

In [ ]:
# list of PCA local models
SparsePCA_spasreList2 = {
    "Original": model,
    "SparsePCA Local Compressed - fc1_out = 50 and fc2_out = 32": SparsePCA_sparse2_50,
    "SparsePCA Local Compressed - fc1_out = 30 and fc2_out = 32": SparsePCA_sparse2_30,
    "SparsePCA Local Compressed - fc1_out = 10 and fc2_out = 32": SparsePCA_sparse2_10
}

In [ ]:
# Define fc1 layers
layers = ['fc1']

weight_data = {layer: [] for layer in layers}

# Collecting weight data
for layer in layers:
    for model_name, model in SparsePCA_spasreList2.items():
        weights = getattr(model, layer).weight.data.reshape(-1).cpu().numpy()
        weight_data[layer].append(weights)

# Create subplots for each layer
fig, ax = plt.subplots(1, 1, figsize=(10, 6))

for weights in weight_data['fc1']:
    ax.hist(weights, bins=100, alpha=0.5)
ax.set_ylim(top=100)
ax.set_title("Weight Distribution of fc1")
ax.set_xlabel("Weight Value")
ax.set_ylabel("Frequency")
ax.legend(SparsePCA_spasreList2.keys())

plt.tight_layout()
plt.show()

In [ ]:
# Define fc2 layers
layers = ['fc2']

weight_data = {layer: [] for layer in layers}

# Collecting weight data
for layer in layers:
    for model_name, model in SparsePCA_spasreList2.items():
        weights = getattr(model, layer).weight.data.reshape(-1).cpu().numpy()
        weight_data[layer].append(weights)

# Create subplots for each layer
fig, ax = plt.subplots(1, 1, figsize=(10, 6))

for weights in weight_data['fc2']:
    ax.hist(weights, bins=100, alpha=0.5)
ax.set_ylim(top=80)
ax.set_title("Weight Distribution of fc2")
ax.set_xlabel("Weight Value")
ax.set_ylabel("Frequency")
ax.legend(SparsePCA_spasreList2.keys())

plt.tight_layout()
plt.show()

In [ ]:
# Define fc3 layers
layers = ['fc3']

weight_data = {layer: [] for layer in layers}

# Collecting weight data
for layer in layers:
    for model_name, model in SparsePCA_spasreList2.items():
        weights = getattr(model, layer).weight.data.reshape(-1).cpu().numpy()
        weight_data[layer].append(weights)

# Create subplots for each layer
fig, ax = plt.subplots(1, 1, figsize=(10, 6))

for weights in weight_data['fc3']:
    ax.hist(weights, bins=100, alpha=0.5)
ax.set_ylim(top=15)
ax.set_title("Weight Distribution of fc3")
ax.set_xlabel("Weight Value")
ax.set_ylabel("Frequency")
ax.legend(SparsePCA_spasreList2.keys())

plt.tight_layout()
plt.show()

###### Pruning Amount = 0.8

In [ ]:
# list of PCA local models
SparsePCA_spasreList3 = {
    "Original": model,
    "SparsePCA Local Compressed - fc1_out = 50 and fc2_out = 32": SparsePCA_sparse3_50,
    "SparsePCA Local Compressed - fc1_out = 30 and fc2_out = 32": SparsePCA_sparse3_30,
    "SparsePCA Local Compressed - fc1_out = 10 and fc2_out = 32": SparsePCA_sparse3_10
}

In [ ]:
# Define fc1 layers
layers = ['fc1']

weight_data = {layer: [] for layer in layers}

# Collecting weight data
for layer in layers:
    for model_name, model in SparsePCA_spasreList3.items():
        # Flatten the weight data and convert to CPU numpy array
        weights = getattr(model, layer).weight.data.reshape(-1).cpu().numpy()
        weight_data[layer].append(weights)

# Create subplots for each layer
fig, ax = plt.subplots(1, 1, figsize=(10, 6))

for weights in weight_data['fc1']:
    ax.hist(weights, bins=100, alpha=0.5)
ax.set_ylim(top=100)
ax.set_title("Weight Distribution of fc1")
ax.set_xlabel("Weight Value")
ax.set_ylabel("Frequency")
ax.legend(SparsePCA_spasreList3.keys())

plt.tight_layout()
plt.show()

In [ ]:
# Define fc2 layers
layers = ['fc2']

weight_data = {layer: [] for layer in layers}

# Collecting weight data
for layer in layers:
    for model_name, model in SparsePCA_spasreList3.items():
        # Flatten the weight data and convert to CPU numpy array
        weights = getattr(model, layer).weight.data.reshape(-1).cpu().numpy()
        weight_data[layer].append(weights)

# Create subplots for each layer
fig, ax = plt.subplots(1, 1, figsize=(10, 6))

for weights in weight_data['fc2']:
    ax.hist(weights, bins=100, alpha=0.5)
ax.set_ylim(top=80)
ax.set_title("Weight Distribution of fc2")
ax.set_xlabel("Weight Value")
ax.set_ylabel("Frequency")
ax.legend(SparsePCA_spasreList3.keys())

plt.tight_layout()
plt.show()

In [ ]:
# Define fc3 layers
layers = ['fc3']

weight_data = {layer: [] for layer in layers}

# Collecting weight data
for layer in layers:
    for model_name, model in SparsePCA_spasreList3.items():
        # Flatten the weight data and convert to CPU numpy array
        weights = getattr(model, layer).weight.data.reshape(-1).cpu().numpy()
        weight_data[layer].append(weights)

# Create subplots for each layer
fig, ax = plt.subplots(1, 1, figsize=(10, 6))

for weights in weight_data['fc3']:
    ax.hist(weights, bins=100, alpha=0.5)
ax.set_ylim(top=15)
ax.set_title("Weight Distribution of fc3")
ax.set_xlabel("Weight Value")
ax.set_ylabel("Frequency")
ax.legend(SparsePCA_spasreList3.keys())

plt.tight_layout()
plt.show()

###### Pruning Amount = 0.9

In [ ]:
# list of PCA local models
SparsePCA_spasreList4 = {
    "Original": model,
    "SparsePCA Local Compressed - fc1_out = 50 and fc2_out = 32": SparsePCA_sparse4_50,
    "SparsePCA Local Compressed - fc1_out = 30 and fc2_out = 32": SparsePCA_sparse4_30,
    "SparsePCA Local Compressed - fc1_out = 10 and fc2_out = 32": SparsePCA_sparse4_10
}

In [ ]:
# Define fc1 layers
layers = ['fc1']

weight_data = {layer: [] for layer in layers}

# Collecting weight data
for layer in layers:
    for model_name, model in SparsePCA_spasreList4.items():
        # Flatten the weight data and convert to CPU numpy array
        weights = getattr(model, layer).weight.data.reshape(-1).cpu().numpy()
        weight_data[layer].append(weights)

# Create subplots for each layer
fig, ax = plt.subplots(1, 1, figsize=(10, 6))

for weights in weight_data['fc1']:
    ax.hist(weights, bins=100, alpha=0.5)
ax.set_ylim(top=100)
ax.set_title("Weight Distribution of fc1")
ax.set_xlabel("Weight Value")
ax.set_ylabel("Frequency")
ax.legend(SparsePCA_spasreList4.keys())

plt.tight_layout()
plt.show()

In [ ]:
# Define fc2 layers
layers = ['fc2']

weight_data = {layer: [] for layer in layers}

# Collecting weight data
for layer in layers:
    for model_name, model in SparsePCA_spasreList4.items():
        # Flatten the weight data and convert to CPU numpy array
        weights = getattr(model, layer).weight.data.reshape(-1).cpu().numpy()
        weight_data[layer].append(weights)

# Create subplots for each layer
fig, ax = plt.subplots(1, 1, figsize=(10, 6))

for weights in weight_data['fc2']:
    ax.hist(weights, bins=100, alpha=0.5)
ax.set_ylim(top=80)
ax.set_title("Weight Distribution of fc2")
ax.set_xlabel("Weight Value")
ax.set_ylabel("Frequency")
ax.legend(SparsePCA_spasreList4.keys())

plt.tight_layout()
plt.show()

In [ ]:
# Define fc3 layers
layers = ['fc3']

weight_data = {layer: [] for layer in layers}

# Collecting weight data
for layer in layers:
    for model_name, model in SparsePCA_spasreList4.items():
        # Flatten the weight data and convert to CPU numpy array
        weights = getattr(model, layer).weight.data.reshape(-1).cpu().numpy()
        weight_data[layer].append(weights)

# Create subplots for each layer
fig, ax = plt.subplots(1, 1, figsize=(10, 6))

for weights in weight_data['fc3']:
    ax.hist(weights, bins=100, alpha=0.5)
ax.set_ylim(top=15)
ax.set_title("Weight Distribution of fc3")
ax.set_xlabel("Weight Value")
ax.set_ylabel("Frequency")
ax.legend(SparsePCA_spasreList4.keys())

plt.tight_layout()
plt.show()

###### Pruning Amount = 0.95

In [ ]:
# list of PCA local models
SparsePCA_spasreList5 = {
    "Original": model,
    "SparsePCA Local Compressed - fc1_out = 50 and fc2_out = 32": SparsePCA_sparse5_50,
    "SparsePCA Local Compressed - fc1_out = 30 and fc2_out = 32": SparsePCA_sparse5_30,
    "SparsePCA Local Compressed - fc1_out = 10 and fc2_out = 32": SparsePCA_sparse5_10
}

In [ ]:
# Define fc1 layers
layers = ['fc1']

weight_data = {layer: [] for layer in layers}

# Collecting weight data
for layer in layers:
    for model_name, model in SparsePCA_spasreList5.items():
        weights = getattr(model, layer).weight.data.reshape(-1).cpu().numpy()
        weight_data[layer].append(weights)

# Create subplots for each layer
fig, ax = plt.subplots(1, 1, figsize=(10, 6))

for weights in weight_data['fc1']:
    ax.hist(weights, bins=100, alpha=0.5)
ax.set_ylim(top=100)
ax.set_title("Weight Distribution of fc1")
ax.set_xlabel("Weight Value")
ax.set_ylabel("Frequency")
ax.legend(SparsePCA_spasreList5.keys())

plt.tight_layout()
plt.show()

In [ ]:
# Define fc2 layers
layers = ['fc2']

weight_data = {layer: [] for layer in layers}

# Collecting weight data
for layer in layers:
    for model_name, model in SparsePCA_spasreList5.items():
        weights = getattr(model, layer).weight.data.reshape(-1).cpu().numpy()
        weight_data[layer].append(weights)

# Create subplots for each layer
fig, ax = plt.subplots(1, 1, figsize=(10, 6))

for weights in weight_data['fc2']:
    ax.hist(weights, bins=100, alpha=0.5)
ax.set_ylim(top=80)
ax.set_title("Weight Distribution of fc2")
ax.set_xlabel("Weight Value")
ax.set_ylabel("Frequency")
ax.legend(SparsePCA_spasreList5.keys())

plt.tight_layout()
plt.show()

In [ ]:
# Define fc3 layers
layers = ['fc3']

weight_data = {layer: [] for layer in layers}

# Collecting weight data
for layer in layers:
    for model_name, model in SparsePCA_spasreList5.items():
        weights = getattr(model, layer).weight.data.reshape(-1).cpu().numpy()
        weight_data[layer].append(weights)

# Create subplots for each layer
fig, ax = plt.subplots(1, 1, figsize=(10, 6))

for weights in weight_data['fc3']:
    ax.hist(weights, bins=100, alpha=0.5)
ax.set_ylim(top=15)
ax.set_title("Weight Distribution of fc3")
ax.set_xlabel("Weight Value")
ax.set_ylabel("Frequency")
ax.legend(SparsePCA_spasreList5.keys())

plt.tight_layout()
plt.show()

### Predictive Performance Plots across all the Models

#### Original, Local, and Locally Pruned Compressed (PCA, SparsePCA, ICA) Models

##### Pruning Amount = 0.6

In [ ]:
# checking local models that have 0.6 pruning amount
PCA_l1_models = {
    "Original": model,
    "Smaller Network - fc1_out = 50 and fc2_out = 30": small_5030,
    "Smaller Network - fc1_out = 50 and fc2_out = 10": small_5010,
    "Smaller Network - fc1_out = 50 and fc2_out = 5": small_505,
    "Smaller Network - fc1_out = 30 and fc2_out = 10": small_3010,
    "Smaller Network - fc1_out = 30 and fc2_out = 5": small_305,
    "Smaller Network - fc1_out = 10 and fc2_out = 5": small_105,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 30": PCA_local1_5030,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 30": PCA_local1_5030,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 10": PCA_local1_5010,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 5": PCA_local1_505,
    "PCA Local Compressed - fc1_out = 30 and fc2_out = 10": PCA_local1_3010,
    "PCA Local Compressed - fc1_out = 30 and fc2_out = 5": PCA_local1_305,
    "PCA Local Compressed - fc1_out = 10 and fc2_out = 5": PCA_local1_105,
}

plt.figure(figsize=(14, 10))

#plot predictions
min_val, max_val = float('inf'), -float('inf')

for model_name, model in PCA_l1_models.items():
    actuals = []
    predictions = []

    model.eval()

    with torch.no_grad():
        for inputs, targets in test_loader:

            outputs = model(inputs)

            # Store actual and predicted values for plotting
            actuals.extend(targets.cpu().numpy())
            predictions.extend(outputs.cpu().numpy())

    # Update min and max values for the perfect prediction line
    # reference: https://stackoverflow.com/questions/40776069/python-convert-prediction-result-into-one-hot
    max_val = max(max_val, max(actuals), max(predictions))
    min_val = min(min_val, min(actuals), min(predictions))

    # Plot actuals vs predictions for this model
    plt.scatter(actuals, predictions, alpha=0.5, label=model_name)

# perfect prediction line
plt.plot([min_val, max_val], [min_val, max_val], 'k--', lw=2,
         label='Perfect Prediction')

plt.title('Actual vs Predicted - All Models')
plt.xlabel('Actual Target')
plt.ylabel('Predicted Target')
plt.tight_layout(rect=[0, 0.1, 1, 1])
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05), fancybox=True,
           shadow=True, ncol=3)
plt.grid(True)
plt.show()

In [ ]:
# checking local models that have 0.6 pruning amount
SparsePCA_l1_models = {
    "Original": model,
    "Smaller Network - fc1_out = 50 and fc2_out = 32": small_5032,
    "Smaller Network - fc1_out = 30 and fc2_out = 32": small_3032,
    "Smaller Network - fc1_out = 10 and fc2_out = 32": small_1032,
    "SparsePCA Local Compressed - fc1_out = 50 and fc2_out = 32": SparsePCA_l1_50,
    "SparsePCA Local Compressed - fc1_out = 30 and fc2_out = 32": SparsePCA_l1_30,
    "SparsePCA Local Compressed - fc1_out = 10 and fc2_out = 32": SparsePCA_l1_10,
}

plt.figure(figsize=(14, 10))

#plot predictions
min_val, max_val = float('inf'), -float('inf')

for model_name, model in SparsePCA_l1_models.items():
    actuals = []
    predictions = []

    model.eval()

    with torch.no_grad():
        for inputs, targets in test_loader:
            outputs = model(inputs)

            # Store actual and predicted values for plotting
            actuals.extend(targets.cpu().numpy())
            predictions.extend(outputs.cpu().numpy())

    # Update min and max values for the perfect prediction line
    max_val = max(max_val, max(actuals), max(predictions))
    min_val = min(min_val, min(actuals), min(predictions))

    # Plot actuals vs predictions for this model
    plt.scatter(actuals, predictions, alpha=0.5, label=model_name)

# perfect prediction line
plt.plot([min_val, max_val], [min_val, max_val], 'k--', lw=2,
         label='Perfect Prediction')

plt.title('Actual vs Predicted - All Models')
plt.xlabel('Actual Target')
plt.ylabel('Predicted Target')
plt.tight_layout(rect=[0, 0.1, 1, 1])
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05), fancybox=True,
           shadow=True, ncol=3)
plt.grid(True)
plt.show()

In [ ]:
# checking local models that have 0.6 pruning amount
ICA_l1_models = {
    "Original": model,
    "Smaller Network - fc1_out = 50 and fc2_out = 32": small_5032,
    "Smaller Network - fc1_out = 30 and fc2_out = 32": small_3032,
    "Smaller Network - fc1_out = 10 and fc2_out = 32": small_1032,
    "ICA Local Model - fc1_out = 50 and fc2_out = 32": ICA_l1_50,
    "ICA Local Compressed - fc1_out = 30 and fc2_out = 32": ICA_l1_30,
    "ICA Local Compressed - fc1_out = 10 and fc2_out = 32": ICA_l1_10,

}

plt.figure(figsize=(14, 10))

#plot predictions
min_val, max_val = float('inf'), -float('inf')

for model_name, model in ICA_l1_models.items():
    actuals = []
    predictions = []

    model.eval()

    with torch.no_grad():
        for inputs, targets in test_loader:
            outputs = model(inputs)

            # Store actual and predicted values for plotting
            actuals.extend(targets.cpu().numpy())
            predictions.extend(outputs.cpu().numpy())

    # Update min and max values for the perfect prediction line
    max_val = max(max_val, max(actuals), max(predictions))
    min_val = min(min_val, min(actuals), min(predictions))

    # Plot actuals vs predictions for this model
    plt.scatter(actuals, predictions, alpha=0.5, label=model_name)

# perfect prediction line
plt.plot([min_val, max_val], [min_val, max_val], 'k--', lw=2,
         label='Perfect Prediction')

plt.title('Actual vs Predicted - All Models')
plt.xlabel('Actual Target')
plt.ylabel('Predicted Target')
plt.tight_layout(rect=[0, 0.1, 1, 1])
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05), fancybox=True,
           shadow=True, ncol=3)
plt.grid(True)
plt.show()

##### Pruning Amount = 0.7

In [ ]:
# listing local models that have 0.7 pruning amount
PCA_l2_models = {
    "Original": model,
    "Smaller Network - fc1_out = 50 and fc2_out = 30": small_5030,
    "Smaller Network - fc1_out = 50 and fc2_out = 10": small_5010,
    "Smaller Network - fc1_out = 50 and fc2_out = 5": small_505,
    "Smaller Network - fc1_out = 30 and fc2_out = 10": small_3010,
    "Smaller Network - fc1_out = 30 and fc2_out = 5": small_305,
    "Smaller Network - fc1_out = 10 and fc2_out = 5": small_105,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 30": PCA_local2_5030,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 30": PCA_local2_5030,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 10": PCA_local2_5010,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 5": PCA_local2_505,
    "PCA Local Compressed - fc1_out = 30 and fc2_out = 10": PCA_local2_3010,
    "PCA Local Compressed - fc1_out = 30 and fc2_out = 5": PCA_local2_305,
    "PCA Local Compressed - fc1_out = 10 and fc2_out = 5": PCA_local2_105,
}

plt.figure(figsize=(14, 10))

#plot predictions
min_val, max_val = float('inf'), -float('inf')

for model_name, model in PCA_l2_models.items():
    actuals = []
    predictions = []

    model.eval()

    with torch.no_grad():
        for inputs, targets in test_loader:
            outputs = model(inputs)

            # Store actual and predicted values for plotting
            actuals.extend(targets.cpu().numpy())
            predictions.extend(outputs.cpu().numpy())

    # Update min and max values for the perfect prediction line
    max_val = max(max_val, max(actuals), max(predictions))
    min_val = min(min_val, min(actuals), min(predictions))

    # Plot actuals vs predictions for this model
    plt.scatter(actuals, predictions, alpha=0.5, label=model_name)

# perfect prediction line
plt.plot([min_val, max_val], [min_val, max_val], 'k--', lw=2,
         label='Perfect Prediction')

plt.title('Actual vs Predicted - All Models')
plt.xlabel('Actual Target')
plt.ylabel('Predicted Target')
plt.tight_layout(rect=[0, 0.1, 1, 1])
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05), fancybox=True,
           shadow=True, ncol=3)
plt.grid(True)
plt.show()

In [ ]:
# listing local models that have 0.7 pruning amount
SparsePCA_l2_models = {
    "Original": model,
    "Smaller Network - fc1_out = 50 and fc2_out = 32": small_5032,
    "Smaller Network - fc1_out = 30 and fc2_out = 32": small_3032,
    "Smaller Network - fc1_out = 10 and fc2_out = 32": small_1032,
    "SparsePCA Local Compressed - fc1_out = 50 and fc2_out = 32": SparsePCA_l2_50,
    "SparsePCA Local Compressed - fc1_out = 30 and fc2_out = 32": SparsePCA_l2_30,
    "SparsePCA Local Compressed - fc1_out = 10 and fc2_out = 32": SparsePCA_l2_10,
}

plt.figure(figsize=(14, 10))

#plot predictions
min_val, max_val = float('inf'), -float('inf')

for model_name, model in SparsePCA_l2_models.items():
    actuals = []
    predictions = []

    model.eval()

    with torch.no_grad():
        for inputs, targets in test_loader:
            outputs = model(inputs)

            # Store actual and predicted values for plotting
            actuals.extend(targets.cpu().numpy())
            predictions.extend(outputs.cpu().numpy())

    # Update min and max values for the perfect prediction line
    max_val = max(max_val, max(actuals), max(predictions))
    min_val = min(min_val, min(actuals), min(predictions))

    # Plot actuals vs predictions for this model
    plt.scatter(actuals, predictions, alpha=0.5, label=model_name)

# perfect prediction line
plt.plot([min_val, max_val], [min_val, max_val], 'k--', lw=2,
         label='Perfect Prediction')

plt.title('Actual vs Predicted - All Models')
plt.xlabel('Actual Target')
plt.ylabel('Predicted Target')
plt.tight_layout(rect=[0, 0.1, 1, 1])
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05), fancybox=True,
           shadow=True, ncol=3)
plt.grid(True)
plt.show()

In [ ]:
# listing local models that have 0.7 pruning amount
ICA_l2_models = {
    "Original": model,
    "Smaller Network - fc1_out = 50 and fc2_out = 32": small_5032,
    "Smaller Network - fc1_out = 30 and fc2_out = 32": small_3032,
    "Smaller Network - fc1_out = 10 and fc2_out = 32": small_1032,
    "ICA Local Model - fc1_out = 50 and fc2_out = 32": ICA_l2_50,
    "ICA Local Compressed - fc1_out = 30 and fc2_out = 32": ICA_l2_30,
    "ICA Local Compressed - fc1_out = 10 and fc2_out = 32": ICA_l2_10,

}

plt.figure(figsize=(14, 10))

#plot predictions
min_val, max_val = float('inf'), -float('inf')

for model_name, model in ICA_l2_models.items():
    actuals = []
    predictions = []

    model.eval()

    with torch.no_grad():
        for inputs, targets in test_loader:
            outputs = model(inputs)

            # Store actual and predicted values for plotting
            actuals.extend(targets.cpu().numpy())
            predictions.extend(outputs.cpu().numpy())

    # Update min and max values for the perfect prediction line
    max_val = max(max_val, max(actuals), max(predictions))
    min_val = min(min_val, min(actuals), min(predictions))

    # Plot actuals vs predictions for this model
    plt.scatter(actuals, predictions, alpha=0.5, label=model_name)

# perfect prediction line
plt.plot([min_val, max_val], [min_val, max_val], 'k--', lw=2,
         label='Perfect Prediction')

plt.title('Actual vs Predicted - All Models')
plt.xlabel('Actual Target')
plt.ylabel('Predicted Target')
plt.tight_layout(rect=[0, 0.1, 1, 1])
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05), fancybox=True,
           shadow=True, ncol=3)
plt.grid(True)
plt.show()

##### Pruning Amount = 0.8

In [ ]:
# listing local models that have 0.8 pruning amount
PCA_l3_models = {
    "Original": model,
    "Smaller Network - fc1_out = 50 and fc2_out = 30": small_5030,
    "Smaller Network - fc1_out = 50 and fc2_out = 10": small_5010,
    "Smaller Network - fc1_out = 50 and fc2_out = 5": small_505,
    "Smaller Network - fc1_out = 30 and fc2_out = 10": small_3010,
    "Smaller Network - fc1_out = 30 and fc2_out = 5": small_305,
    "Smaller Network - fc1_out = 10 and fc2_out = 5": small_105,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 30": PCA_local3_5030,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 30": PCA_local3_5030,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 10": PCA_local3_5010,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 5": PCA_local3_505,
    "PCA Local Compressed - fc1_out = 30 and fc2_out = 10": PCA_local3_3010,
    "PCA Local Compressed - fc1_out = 30 and fc2_out = 5": PCA_local3_305,
    "PCA Local Compressed - fc1_out = 10 and fc2_out = 5": PCA_local3_105,
}

plt.figure(figsize=(14, 10))

#plot predictions
min_val, max_val = float('inf'), -float('inf')

for model_name, model in PCA_l3_models.items():
    actuals = []
    predictions = []

    model.eval()

    with torch.no_grad():
        for inputs, targets in test_loader:
            outputs = model(inputs)

            # Store actual and predicted values for plotting
            actuals.extend(targets.cpu().numpy())
            predictions.extend(outputs.cpu().numpy())

    # Update min and max values for the perfect prediction line
    max_val = max(max_val, max(actuals), max(predictions))
    min_val = min(min_val, min(actuals), min(predictions))

    # Plot actuals vs predictions for this model
    plt.scatter(actuals, predictions, alpha=0.5, label=model_name)

# perfect prediction line
plt.plot([min_val, max_val], [min_val, max_val], 'k--', lw=2,
         label='Perfect Prediction')

plt.title('Actual vs Predicted - All Models')
plt.xlabel('Actual Target')
plt.ylabel('Predicted Target')
plt.tight_layout(rect=[0, 0.1, 1, 1])
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05), fancybox=True,
           shadow=True, ncol=3)
plt.grid(True)
plt.show()

In [ ]:
# listing local models that have 0.8 pruning amoun
SparsePCA_l3_models = {
    "Original": model,
    "Smaller Network - fc1_out = 50 and fc2_out = 32": small_5032,
    "Smaller Network - fc1_out = 30 and fc2_out = 32": small_3032,
    "Smaller Network - fc1_out = 10 and fc2_out = 32": small_1032,
    "SparsePCA Local Compressed - fc1_out = 50 and fc2_out = 32": SparsePCA_l3_50,
    "SparsePCA Local Compressed - fc1_out = 30 and fc2_out = 32": SparsePCA_l3_30,
    "SparsePCA Local Compressed - fc1_out = 10 and fc2_out = 32": SparsePCA_l3_10,
}

plt.figure(figsize=(14, 10))

#plot predictions
min_val, max_val = float('inf'), -float('inf')

for model_name, model in SparsePCA_l3_models.items():
    actuals = []
    predictions = []

    model.eval()

    with torch.no_grad():
        for inputs, targets in test_loader:
            outputs = model(inputs)

            # Store actual and predicted values for plotting
            actuals.extend(targets.cpu().numpy())
            predictions.extend(outputs.cpu().numpy())

    # Update min and max values for the perfect prediction line
    max_val = max(max_val, max(actuals), max(predictions))
    min_val = min(min_val, min(actuals), min(predictions))

    # Plot actuals vs predictions for this model
    plt.scatter(actuals, predictions, alpha=0.5, label=model_name)

# perfect prediction line
plt.plot([min_val, max_val], [min_val, max_val], 'k--', lw=2,
         label='Perfect Prediction')

plt.title('Actual vs Predicted - All Models')
plt.xlabel('Actual Target')
plt.ylabel('Predicted Target')
plt.tight_layout(rect=[0, 0.1, 1, 1])
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05), fancybox=True,
           shadow=True, ncol=3)
plt.grid(True)
plt.show()

In [ ]:
# listing local models that have 0.8 pruning amoun
ICA_l3_models = {
    "Original": model,
    "Smaller Network - fc1_out = 50 and fc2_out = 32": small_5032,
    "Smaller Network - fc1_out = 30 and fc2_out = 32": small_3032,
    "Smaller Network - fc1_out = 10 and fc2_out = 32": small_1032,
    "ICA Local Model - fc1_out = 50 and fc2_out = 32": ICA_l3_50,
    "ICA Local Compressed - fc1_out = 30 and fc2_out = 32": ICA_l3_30,
    "ICA Local Compressed - fc1_out = 10 and fc2_out = 32": ICA_l3_10,

}

plt.figure(figsize=(14, 10))

#plot predictions
min_val, max_val = float('inf'), -float('inf')

for model_name, model in ICA_l3_models.items():
    actuals = []
    predictions = []

    model.eval()

    with torch.no_grad():
        for inputs, targets in test_loader:
            outputs = model(inputs)

            # Store actual and predicted values for plotting
            actuals.extend(targets.cpu().numpy())
            predictions.extend(outputs.cpu().numpy())

    # Update min and max values for the perfect prediction line
    max_val = max(max_val, max(actuals), max(predictions))
    min_val = min(min_val, min(actuals), min(predictions))

    # Plot actuals vs predictions for this model
    plt.scatter(actuals, predictions, alpha=0.5, label=model_name)

# perfect prediction line
plt.plot([min_val, max_val], [min_val, max_val], 'k--', lw=2,
         label='Perfect Prediction')

plt.title('Actual vs Predicted - All Models')
plt.xlabel('Actual Target')
plt.ylabel('Predicted Target')
plt.tight_layout(rect=[0, 0.1, 1, 1])
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05), fancybox=True,
           shadow=True, ncol=3)
plt.grid(True)
plt.show()

##### Pruning Amount = 0.9

In [ ]:
# listing local models that have 0.9 pruning amoun
PCA_l4_models = {
    "Original": model,
    "Smaller Network - fc1_out = 50 and fc2_out = 30": small_5030,
    "Smaller Network - fc1_out = 50 and fc2_out = 10": small_5010,
    "Smaller Network - fc1_out = 50 and fc2_out = 5": small_505,
    "Smaller Network - fc1_out = 30 and fc2_out = 10": small_3010,
    "Smaller Network - fc1_out = 30 and fc2_out = 5": small_305,
    "Smaller Network - fc1_out = 10 and fc2_out = 5": small_105,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 30": PCA_local4_5030,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 30": PCA_local4_5030,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 10": PCA_local4_5010,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 5": PCA_local4_505,
    "PCA Local Compressed - fc1_out = 30 and fc2_out = 10": PCA_local4_3010,
    "PCA Local Compressed - fc1_out = 30 and fc2_out = 5": PCA_local4_305,
    "PCA Local Compressed - fc1_out = 10 and fc2_out = 5": PCA_local4_105,
}

plt.figure(figsize=(14, 10))

#plot predictions
min_val, max_val = float('inf'), -float('inf')

for model_name, model in PCA_l4_models.items():
    actuals = []
    predictions = []

    model.eval()

    with torch.no_grad():
        for inputs, targets in test_loader:
            outputs = model(inputs)

            # Store actual and predicted values for plotting
            actuals.extend(targets.cpu().numpy())
            predictions.extend(outputs.cpu().numpy())

    # Update min and max values for the perfect prediction line
    max_val = max(max_val, max(actuals), max(predictions))
    min_val = min(min_val, min(actuals), min(predictions))

    # Plot actuals vs predictions for this model
    plt.scatter(actuals, predictions, alpha=0.5, label=model_name)

# perfect prediction line
plt.plot([min_val, max_val], [min_val, max_val], 'k--', lw=2,
         label='Perfect Prediction')

plt.title('Actual vs Predicted - All Models')
plt.xlabel('Actual Target')
plt.ylabel('Predicted Target')
plt.tight_layout(rect=[0, 0.1, 1, 1])
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05), fancybox=True,
           shadow=True, ncol=3)
plt.grid(True)
plt.show()

In [ ]:
# listing local models that have 0.9 pruning amount
SparsePCA_l4_models = {
    "Original": model,
    "Smaller Network - fc1_out = 50 and fc2_out = 32": small_5032,
    "Smaller Network - fc1_out = 30 and fc2_out = 32": small_3032,
    "Smaller Network - fc1_out = 10 and fc2_out = 32": small_1032,
    "SparsePCA Local Compressed - fc1_out = 50 and fc2_out = 32": SparsePCA_l4_50,
    "SparsePCA Local Compressed - fc1_out = 30 and fc2_out = 32": SparsePCA_l4_30,
    "SparsePCA Local Compressed - fc1_out = 10 and fc2_out = 32": SparsePCA_l4_10,
}

plt.figure(figsize=(14, 10))

#plot predictions
min_val, max_val = float('inf'), -float('inf')

for model_name, model in SparsePCA_l4_models.items():
    actuals = []
    predictions = []

    model.eval()

    with torch.no_grad():
        for inputs, targets in test_loader:
            outputs = model(inputs)

            # Store actual and predicted values for plotting
            actuals.extend(targets.cpu().numpy())
            predictions.extend(outputs.cpu().numpy())

    # Update min and max values for the perfect prediction line
    max_val = max(max_val, max(actuals), max(predictions))
    min_val = min(min_val, min(actuals), min(predictions))

    # Plot actuals vs predictions for this model
    plt.scatter(actuals, predictions, alpha=0.5, label=model_name)

# perfect prediction line
plt.plot([min_val, max_val], [min_val, max_val], 'k--', lw=2,
         label='Perfect Prediction')

plt.title('Actual vs Predicted - All Models')
plt.xlabel('Actual Target')
plt.ylabel('Predicted Target')
plt.tight_layout(rect=[0, 0.1, 1, 1])
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05), fancybox=True,
           shadow=True, ncol=3)
plt.grid(True)
plt.show()

In [ ]:
# listing local models that have 0.9 pruning amount
ICA_l4_models = {
    "Original": model,
    "Smaller Network - fc1_out = 50 and fc2_out = 32": small_5032,
    "Smaller Network - fc1_out = 30 and fc2_out = 32": small_3032,
    "Smaller Network - fc1_out = 10 and fc2_out = 32": small_1032,
    "ICA Local Model - fc1_out = 50 and fc2_out = 32": ICA_l4_50,
    "ICA Local Compressed - fc1_out = 30 and fc2_out = 32": ICA_l4_30,
    "ICA Local Compressed - fc1_out = 10 and fc2_out = 32": ICA_l4_10,

}

plt.figure(figsize=(14, 10))

#plot predictions
min_val, max_val = float('inf'), -float('inf')

for model_name, model in ICA_l4_models.items():
    actuals = []
    predictions = []

    model.eval()

    with torch.no_grad():
        for inputs, targets in test_loader:
            outputs = model(inputs)

            # Store actual and predicted values for plotting
            actuals.extend(targets.cpu().numpy())
            predictions.extend(outputs.cpu().numpy())

    # Update min and max values for the perfect prediction line
    max_val = max(max_val, max(actuals), max(predictions))
    min_val = min(min_val, min(actuals), min(predictions))

    # Plot actuals vs predictions for this model
    plt.scatter(actuals, predictions, alpha=0.5, label=model_name)

# perfect prediction line
plt.plot([min_val, max_val], [min_val, max_val], 'k--', lw=2,
         label='Perfect Prediction')

plt.title('Actual vs Predicted - All Models')
plt.xlabel('Actual Target')
plt.ylabel('Predicted Target')
plt.tight_layout(rect=[0, 0.1, 1, 1])
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05), fancybox=True,
           shadow=True, ncol=3)
plt.grid(True)
plt.show()

##### Pruning Amount = 0.95

In [ ]:
# listing local models that have 0.95 pruning amoun
PCA_l5_models = {
    "Original": model,
    "Smaller Network - fc1_out = 50 and fc2_out = 30": small_5030,
    "Smaller Network - fc1_out = 50 and fc2_out = 10": small_5010,
    "Smaller Network - fc1_out = 50 and fc2_out = 5": small_505,
    "Smaller Network - fc1_out = 30 and fc2_out = 10": small_3010,
    "Smaller Network - fc1_out = 30 and fc2_out = 5": small_305,
    "Smaller Network - fc1_out = 10 and fc2_out = 5": small_105,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 30": PCA_local5_5030,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 30": PCA_local5_5030,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 10": PCA_local5_5010,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 5": PCA_local5_505,
    "PCA Local Compressed - fc1_out = 30 and fc2_out = 10": PCA_local5_3010,
    "PCA Local Compressed - fc1_out = 30 and fc2_out = 5": PCA_local5_305,
    "PCA Local Compressed - fc1_out = 10 and fc2_out = 5": PCA_local5_105,
}

plt.figure(figsize=(14, 10))

#plot predictions
min_val, max_val = float('inf'), -float('inf')

for model_name, model in PCA_l5_models.items():
    actuals = []
    predictions = []

    model.eval()

    with torch.no_grad():
        for inputs, targets in test_loader:
            outputs = model(inputs)

            # Store actual and predicted values for plotting
            actuals.extend(targets.cpu().numpy())
            predictions.extend(outputs.cpu().numpy())

    # Update min and max values for the perfect prediction line
    max_val = max(max_val, max(actuals), max(predictions))
    min_val = min(min_val, min(actuals), min(predictions))

    # Plot actuals vs predictions for this model
    plt.scatter(actuals, predictions, alpha=0.5, label=model_name)

# perfect prediction line
plt.plot([min_val, max_val], [min_val, max_val], 'k--', lw=2,
         label='Perfect Prediction')

plt.title('Actual vs Predicted - All Models')
plt.xlabel('Actual Target')
plt.ylabel('Predicted Target')
plt.tight_layout(rect=[0, 0.1, 1, 1])
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05), fancybox=True,
           shadow=True, ncol=3)
plt.grid(True)
plt.show()

In [ ]:
# listing local models that have 0.95 pruning amount
SparsePCA_l5_models = {
    "Original": model,
    "Smaller Network - fc1_out = 50 and fc2_out = 32": small_5032,
    "Smaller Network - fc1_out = 30 and fc2_out = 32": small_3032,
    "Smaller Network - fc1_out = 10 and fc2_out = 32": small_1032,
    "SparsePCA Local Compressed - fc1_out = 50 and fc2_out = 32": SparsePCA_l5_50,
    "SparsePCA Local Compressed - fc1_out = 30 and fc2_out = 32": SparsePCA_l5_30,
    "SparsePCA Local Compressed - fc1_out = 10 and fc2_out = 32": SparsePCA_l5_10,
}

plt.figure(figsize=(14, 10))

#plot predictions
min_val, max_val = float('inf'), -float('inf')

for model_name, model in SparsePCA_l5_models.items():
    actuals = []
    predictions = []

    model.eval()

    with torch.no_grad():
        for inputs, targets in test_loader:
            outputs = model(inputs)

            # Store actual and predicted values for plotting
            actuals.extend(targets.cpu().numpy())
            predictions.extend(outputs.cpu().numpy())

    # Update min and max values for the perfect prediction line
    max_val = max(max_val, max(actuals), max(predictions))
    min_val = min(min_val, min(actuals), min(predictions))

    # Plot actuals vs predictions for this model
    plt.scatter(actuals, predictions, alpha=0.5, label=model_name)

# perfect prediction line
plt.plot([min_val, max_val], [min_val, max_val], 'k--', lw=2,
         label='Perfect Prediction')

plt.title('Actual vs Predicted - All Models')
plt.xlabel('Actual Target')
plt.ylabel('Predicted Target')
plt.tight_layout(rect=[0, 0.1, 1, 1])
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05), fancybox=True,
           shadow=True, ncol=3)
plt.grid(True)
plt.show()

In [ ]:
# listing local models that have 0.95 pruning amount
ICA_l5_models = {
    "Original": model,
    "Smaller Network - fc1_out = 50 and fc2_out = 32": small_5032,
    "Smaller Network - fc1_out = 30 and fc2_out = 32": small_3032,
    "Smaller Network - fc1_out = 10 and fc2_out = 32": small_1032,
    "ICA Local Model - fc1_out = 50 and fc2_out = 32": ICA_l5_50,
    "ICA Local Compressed - fc1_out = 30 and fc2_out = 32": ICA_l5_30,
    "ICA Local Compressed - fc1_out = 10 and fc2_out = 32": ICA_l5_10,

}

plt.figure(figsize=(14, 10))

#plot predictions
min_val, max_val = float('inf'), -float('inf')

for model_name, model in ICA_l5_models.items():
    actuals = []
    predictions = []

    model.eval()

    with torch.no_grad():
        for inputs, targets in test_loader:
            outputs = model(inputs)

            # Store actual and predicted values for plotting
            actuals.extend(targets.cpu().numpy())
            predictions.extend(outputs.cpu().numpy())

    # Update min and max values for the perfect prediction line
    max_val = max(max_val, max(actuals), max(predictions))
    min_val = min(min_val, min(actuals), min(predictions))

    # Plot actuals vs predictions for this model
    plt.scatter(actuals, predictions, alpha=0.5, label=model_name)

# perfect prediction line
plt.plot([min_val, max_val], [min_val, max_val], 'k--', lw=2,
         label='Perfect Prediction')

plt.title('Actual vs Predicted - All Models')
plt.xlabel('Actual Target')
plt.ylabel('Predicted Target')
plt.tight_layout(rect=[0, 0.1, 1, 1])
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05), fancybox=True,
           shadow=True, ncol=3)
plt.grid(True)
plt.show()

#### Original, Local, and Layer-Wise Pruned Compressed (PCA, SparsePCA, ICA) Models

##### Pruning Amount = 0.6

In [ ]:
# listing models that have 0.6 pruning amount
PCA_sparse1_models = {
    "Original": model,
    "Smaller Network - fc1_out = 50 and fc2_out = 30": small_5030,
    "Smaller Network - fc1_out = 50 and fc2_out = 10": small_5010,
    "Smaller Network - fc1_out = 50 and fc2_out = 5": small_505,
    "Smaller Network - fc1_out = 30 and fc2_out = 10": small_3010,
    "Smaller Network - fc1_out = 30 and fc2_out = 5": small_305,
    "Smaller Network - fc1_out = 10 and fc2_out = 5": small_105,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 30": PCA_sparse1_5030,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 30": PCA_sparse1_5030,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 10": PCA_sparse1_5010,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 5": PCA_sparse1_505,
    "PCA Local Compressed - fc1_out = 30 and fc2_out = 10": PCA_sparse1_3010,
    "PCA Local Compressed - fc1_out = 30 and fc2_out = 5": PCA_sparse1_305,
    "PCA Local Compressed - fc1_out = 10 and fc2_out = 5": PCA_sparse1_105,
}

plt.figure(figsize=(14, 10))

#plot predictions
min_val, max_val = float('inf'), -float('inf')

for model_name, model in PCA_sparse1_models.items():
    actuals = []
    predictions = []

    model.eval()

    with torch.no_grad():
        for inputs, targets in test_loader:
            outputs = model(inputs)

            # Store actual and predicted values for plotting
            actuals.extend(targets.cpu().numpy())
            predictions.extend(outputs.cpu().numpy())

    # Update min and max values for the perfect prediction line
    max_val = max(max_val, max(actuals), max(predictions))
    min_val = min(min_val, min(actuals), min(predictions))

    # Plot actuals vs predictions for this model
    plt.scatter(actuals, predictions, alpha=0.5, label=model_name)

# perfect prediction line
plt.plot([min_val, max_val], [min_val, max_val], 'k--', lw=2,
         label='Perfect Prediction')

plt.title('Actual vs Predicted - All Models')
plt.xlabel('Actual Target')
plt.ylabel('Predicted Target')
plt.tight_layout(rect=[0, 0.1, 1, 1])
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05), fancybox=True,
           shadow=True, ncol=3)
plt.grid(True)
plt.show()

In [ ]:
# listing models that have 0.6 pruning amount
SparsePCA_sparse1_models = {
    "Original": model,
    "Smaller Network - fc1_out = 50 and fc2_out = 32": small_5032,
    "Smaller Network - fc1_out = 30 and fc2_out = 32": small_3032,
    "Smaller Network - fc1_out = 10 and fc2_out = 32": small_1032,
    "SparsePCA Local Compressed - fc1_out = 50 and fc2_out = 32": SparsePCA_sparse1_50,
    "SparsePCA Local Compressed - fc1_out = 30 and fc2_out = 32": SparsePCA_sparse1_30,
    "SparsePCA Local Compressed - fc1_out = 10 and fc2_out = 32": SparsePCA_sparse1_10,
}

plt.figure(figsize=(14, 10))

#plot predictions
min_val, max_val = float('inf'), -float('inf')

for model_name, model in SparsePCA_sparse1_models.items():
    actuals = []
    predictions = []

    model.eval()

    with torch.no_grad():
        for inputs, targets in test_loader:
            outputs = model(inputs)

            # Store actual and predicted values for plotting
            actuals.extend(targets.cpu().numpy())
            predictions.extend(outputs.cpu().numpy())

    # Update min and max values for the perfect prediction line
    max_val = max(max_val, max(actuals), max(predictions))
    min_val = min(min_val, min(actuals), min(predictions))

    # Plot actuals vs predictions for this model
    plt.scatter(actuals, predictions, alpha=0.5, label=model_name)

# perfect prediction line
plt.plot([min_val, max_val], [min_val, max_val], 'k--', lw=2,
         label='Perfect Prediction')

plt.title('Actual vs Predicted - All Models')
plt.xlabel('Actual Target')
plt.ylabel('Predicted Target')
plt.tight_layout(rect=[0, 0.1, 1, 1])
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05), fancybox=True,
           shadow=True, ncol=3)
plt.grid(True)
plt.show()

In [ ]:
# listing models that have 0.6 pruning amount
ICA_sparse1_models = {
    "Original": model,
    "Smaller Network - fc1_out = 50 and fc2_out = 32": small_5032,
    "Smaller Network - fc1_out = 30 and fc2_out = 32": small_3032,
    "Smaller Network - fc1_out = 10 and fc2_out = 32": small_1032,
    "ICA Local Model - fc1_out = 50 and fc2_out = 32": ICA_sparse1_50,
    "ICA Local Compressed - fc1_out = 30 and fc2_out = 32": ICA_sparse1_30,
    "ICA Local Compressed - fc1_out = 10 and fc2_out = 32": ICA_sparse1_10,

}

plt.figure(figsize=(14, 10))

#plot predictions
min_val, max_val = float('inf'), -float('inf')

for model_name, model in ICA_sparse1_models.items():
    actuals = []
    predictions = []

    model.eval()

    with torch.no_grad():
        for inputs, targets in test_loader:
            outputs = model(inputs)

            # Store actual and predicted values for plotting
            actuals.extend(targets.cpu().numpy())
            predictions.extend(outputs.cpu().numpy())

    # Update min and max values for the perfect prediction line
    max_val = max(max_val, max(actuals), max(predictions))
    min_val = min(min_val, min(actuals), min(predictions))

    # Plot actuals vs predictions for this model
    plt.scatter(actuals, predictions, alpha=0.5, label=model_name)

# perfect prediction line
plt.plot([min_val, max_val], [min_val, max_val], 'k--', lw=2,
         label='Perfect Prediction')

plt.title('Actual vs Predicted - All Models')
plt.xlabel('Actual Target')
plt.ylabel('Predicted Target')
plt.tight_layout(rect=[0, 0.1, 1, 1])
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05), fancybox=True,
           shadow=True, ncol=3)
plt.grid(True)
plt.show()

##### Pruning Amount = 0.7

In [ ]:
# listing local models that have 0.7 pruning amount
PCA_sparse2_models = {
    "Original": model,
    "Smaller Network - fc1_out = 50 and fc2_out = 30": small_5030,
    "Smaller Network - fc1_out = 50 and fc2_out = 10": small_5010,
    "Smaller Network - fc1_out = 50 and fc2_out = 5": small_505,
    "Smaller Network - fc1_out = 30 and fc2_out = 10": small_3010,
    "Smaller Network - fc1_out = 30 and fc2_out = 5": small_305,
    "Smaller Network - fc1_out = 10 and fc2_out = 5": small_105,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 30": PCA_sparse2_5030,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 30": PCA_sparse2_5030,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 10": PCA_sparse2_5010,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 5": PCA_sparse2_505,
    "PCA Local Compressed - fc1_out = 30 and fc2_out = 10": PCA_sparse2_3010,
    "PCA Local Compressed - fc1_out = 30 and fc2_out = 5": PCA_sparse2_305,
    "PCA Local Compressed - fc1_out = 10 and fc2_out = 5": PCA_sparse2_105,
}

plt.figure(figsize=(14, 10))

#plot predictions
min_val, max_val = float('inf'), -float('inf')

for model_name, model in PCA_sparse2_models.items():
    actuals = []
    predictions = []

    model.eval()

    with torch.no_grad():
        for inputs, targets in test_loader:
            outputs = model(inputs)

            # Store actual and predicted values for plotting
            actuals.extend(targets.cpu().numpy())
            predictions.extend(outputs.cpu().numpy())

    # Update min and max values for the perfect prediction line
    max_val = max(max_val, max(actuals), max(predictions))
    min_val = min(min_val, min(actuals), min(predictions))

    # Plot actuals vs predictions for this model
    plt.scatter(actuals, predictions, alpha=0.5, label=model_name)

# perfect prediction line
plt.plot([min_val, max_val], [min_val, max_val], 'k--', lw=2,
         label='Perfect Prediction')

plt.title('Actual vs Predicted - All Models')
plt.xlabel('Actual Target')
plt.ylabel('Predicted Target')
plt.tight_layout(rect=[0, 0.1, 1, 1])
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05), fancybox=True,
           shadow=True, ncol=3)
plt.grid(True)
plt.show()

In [ ]:
# listing local models that have 0.7 pruning amount
SparsePCA_sparse2_models = {
    "Original": model,
    "Smaller Network - fc1_out = 50 and fc2_out = 32": small_5032,
    "Smaller Network - fc1_out = 30 and fc2_out = 32": small_3032,
    "Smaller Network - fc1_out = 10 and fc2_out = 32": small_1032,
    "SparsePCA Local Compressed - fc1_out = 50 and fc2_out = 32": SparsePCA_sparse2_50,
    "SparsePCA Local Compressed - fc1_out = 30 and fc2_out = 32": SparsePCA_sparse2_30,
    "SparsePCA Local Compressed - fc1_out = 10 and fc2_out = 32": SparsePCA_sparse2_10,
}

plt.figure(figsize=(14, 10))

#plot predictions
min_val, max_val = float('inf'), -float('inf')

for model_name, model in SparsePCA_sparse2_models.items():
    actuals = []
    predictions = []

    model.eval()

    with torch.no_grad():
        for inputs, targets in test_loader:
            outputs = model(inputs)

            # Store actual and predicted values for plotting
            actuals.extend(targets.cpu().numpy())
            predictions.extend(outputs.cpu().numpy())

    # Update min and max values for the perfect prediction line
    max_val = max(max_val, max(actuals), max(predictions))
    min_val = min(min_val, min(actuals), min(predictions))

    # Plot actuals vs predictions for this model
    plt.scatter(actuals, predictions, alpha=0.5, label=model_name)

# perfect prediction line
plt.plot([min_val, max_val], [min_val, max_val], 'k--', lw=2,
         label='Perfect Prediction')

plt.title('Actual vs Predicted - All Models')
plt.xlabel('Actual Target')
plt.ylabel('Predicted Target')
plt.tight_layout(rect=[0, 0.1, 1, 1])
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05), fancybox=True,
           shadow=True, ncol=3)
plt.grid(True)
plt.show()

In [ ]:
# listing local models that have 0.7 pruning amount
ICA_sparse2_models = {
    "Original": model,
    "Smaller Network - fc1_out = 50 and fc2_out = 32": small_5032,
    "Smaller Network - fc1_out = 30 and fc2_out = 32": small_3032,
    "Smaller Network - fc1_out = 10 and fc2_out = 32": small_1032,
    "ICA Local Model - fc1_out = 50 and fc2_out = 32": ICA_sparse2_50,
    "ICA Local Compressed - fc1_out = 30 and fc2_out = 32": ICA_sparse2_30,
    "ICA Local Compressed - fc1_out = 10 and fc2_out = 32": ICA_sparse2_10,

}

plt.figure(figsize=(14, 10))

#plot predictions
min_val, max_val = float('inf'), -float('inf')

for model_name, model in ICA_sparse2_models.items():
    actuals = []
    predictions = []

    model.eval()

    with torch.no_grad():
        for inputs, targets in test_loader:
            outputs = model(inputs)

            # Store actual and predicted values for plotting
            actuals.extend(targets.cpu().numpy())
            predictions.extend(outputs.cpu().numpy())

    # Update min and max values for the perfect prediction line
    max_val = max(max_val, max(actuals), max(predictions))
    min_val = min(min_val, min(actuals), min(predictions))

    # Plot actuals vs predictions for this model
    plt.scatter(actuals, predictions, alpha=0.5, label=model_name)

# perfect prediction line
plt.plot([min_val, max_val], [min_val, max_val], 'k--', lw=2,
         label='Perfect Prediction')

plt.title('Actual vs Predicted - All Models')
plt.xlabel('Actual Target')
plt.ylabel('Predicted Target')
plt.tight_layout(rect=[0, 0.1, 1, 1])
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05), fancybox=True,
           shadow=True, ncol=3)
plt.grid(True)
plt.show()

##### Pruning Amount = 0.8

In [ ]:
# listing local models that have 0.8 pruning amount
PCA_sparse3_models = {
    "Original": model,
    "Smaller Network - fc1_out = 50 and fc2_out = 30": small_5030,
    "Smaller Network - fc1_out = 50 and fc2_out = 10": small_5010,
    "Smaller Network - fc1_out = 50 and fc2_out = 5": small_505,
    "Smaller Network - fc1_out = 30 and fc2_out = 10": small_3010,
    "Smaller Network - fc1_out = 30 and fc2_out = 5": small_305,
    "Smaller Network - fc1_out = 10 and fc2_out = 5": small_105,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 30": PCA_sparse3_5030,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 30": PCA_sparse3_5030,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 10": PCA_sparse3_5010,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 5": PCA_sparse3_505,
    "PCA Local Compressed - fc1_out = 30 and fc2_out = 10": PCA_sparse3_3010,
    "PCA Local Compressed - fc1_out = 30 and fc2_out = 5": PCA_sparse3_305,
    "PCA Local Compressed - fc1_out = 10 and fc2_out = 5": PCA_sparse3_105,
}

plt.figure(figsize=(14, 10))

#plot predictions
min_val, max_val = float('inf'), -float('inf')

for model_name, model in PCA_sparse3_models.items():
    actuals = []
    predictions = []

    model.eval()

    with torch.no_grad():
        for inputs, targets in test_loader:
            outputs = model(inputs)

            # Store actual and predicted values for plotting
            actuals.extend(targets.cpu().numpy())
            predictions.extend(outputs.cpu().numpy())

    # Update min and max values for the perfect prediction line
    max_val = max(max_val, max(actuals), max(predictions))
    min_val = min(min_val, min(actuals), min(predictions))

    # Plot actuals vs predictions for this model
    plt.scatter(actuals, predictions, alpha=0.5, label=model_name)

# perfect prediction line
plt.plot([min_val, max_val], [min_val, max_val], 'k--', lw=2,
         label='Perfect Prediction')

plt.title('Actual vs Predicted - All Models')
plt.xlabel('Actual Target')
plt.ylabel('Predicted Target')
plt.tight_layout(rect=[0, 0.1, 1, 1])
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05), fancybox=True,
           shadow=True, ncol=3)
plt.grid(True)
plt.show()

In [ ]:
# listing local models that have 0.8 pruning amount
SparsePCA_sparse3_models = {
    "Original": model,
    "Smaller Network - fc1_out = 50 and fc2_out = 32": small_5032,
    "Smaller Network - fc1_out = 30 and fc2_out = 32": small_3032,
    "Smaller Network - fc1_out = 10 and fc2_out = 32": small_1032,
    "SparsePCA Local Compressed - fc1_out = 50 and fc2_out = 32": SparsePCA_sparse3_50,
    "SparsePCA Local Compressed - fc1_out = 30 and fc2_out = 32": SparsePCA_sparse3_30,
    "SparsePCA Local Compressed - fc1_out = 10 and fc2_out = 32": SparsePCA_sparse3_10,
}

plt.figure(figsize=(14, 10))

#plot predictions
min_val, max_val = float('inf'), -float('inf')

for model_name, model in SparsePCA_sparse3_models.items():
    actuals = []
    predictions = []

    model.eval()

    with torch.no_grad():
        for inputs, targets in test_loader:
            outputs = model(inputs)

            # Store actual and predicted values for plotting
            actuals.extend(targets.cpu().numpy())
            predictions.extend(outputs.cpu().numpy())

    # Update min and max values for the perfect prediction line
    max_val = max(max_val, max(actuals), max(predictions))
    min_val = min(min_val, min(actuals), min(predictions))

    # Plot actuals vs predictions for this model
    plt.scatter(actuals, predictions, alpha=0.5, label=model_name)

# perfect prediction line
plt.plot([min_val, max_val], [min_val, max_val], 'k--', lw=2,
         label='Perfect Prediction')

plt.title('Actual vs Predicted - All Models')
plt.xlabel('Actual Target')
plt.ylabel('Predicted Target')
plt.tight_layout(rect=[0, 0.1, 1, 1])
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05), fancybox=True,
           shadow=True, ncol=3)
plt.grid(True)
plt.show()

In [ ]:
# listing local models that have 0.8 pruning amount
ICA_sparse3_models = {
    "Original": model,
    "Smaller Network - fc1_out = 50 and fc2_out = 32": small_5032,
    "Smaller Network - fc1_out = 30 and fc2_out = 32": small_3032,
    "Smaller Network - fc1_out = 10 and fc2_out = 32": small_1032,
    "ICA Local Model - fc1_out = 50 and fc2_out = 32": ICA_sparse3_50,
    "ICA Local Compressed - fc1_out = 30 and fc2_out = 32": ICA_sparse3_30,
    "ICA Local Compressed - fc1_out = 10 and fc2_out = 32": ICA_sparse3_10,

}

plt.figure(figsize=(14, 10))

#plot predictions
min_val, max_val = float('inf'), -float('inf')

for model_name, model in ICA_sparse3_models.items():
    actuals = []
    predictions = []

    model.eval()

    with torch.no_grad():
        for inputs, targets in test_loader:
            outputs = model(inputs)

            # Store actual and predicted values for plotting
            actuals.extend(targets.cpu().numpy())
            predictions.extend(outputs.cpu().numpy())

    # Update min and max values for the perfect prediction line
    max_val = max(max_val, max(actuals), max(predictions))
    min_val = min(min_val, min(actuals), min(predictions))

    # Plot actuals vs predictions for this model
    plt.scatter(actuals, predictions, alpha=0.5, label=model_name)

# perfect prediction line
plt.plot([min_val, max_val], [min_val, max_val], 'k--', lw=2,
         label='Perfect Prediction')

plt.title('Actual vs Predicted - All Models')
plt.xlabel('Actual Target')
plt.ylabel('Predicted Target')
plt.tight_layout(rect=[0, 0.1, 1, 1])
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05), fancybox=True,
           shadow=True, ncol=3)
plt.grid(True)
plt.show()

##### Pruning Amount = 0.9

In [ ]:
# listing local models that have 0.9 pruning amount
PCA_sparse4_models = {
    "Original": model,
    "Smaller Network - fc1_out = 50 and fc2_out = 30": small_5030,
    "Smaller Network - fc1_out = 50 and fc2_out = 10": small_5010,
    "Smaller Network - fc1_out = 50 and fc2_out = 5": small_505,
    "Smaller Network - fc1_out = 30 and fc2_out = 10": small_3010,
    "Smaller Network - fc1_out = 30 and fc2_out = 5": small_305,
    "Smaller Network - fc1_out = 10 and fc2_out = 5": small_105,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 30": PCA_sparse4_5030,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 30": PCA_sparse4_5030,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 10": PCA_sparse4_5010,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 5": PCA_sparse4_505,
    "PCA Local Compressed - fc1_out = 30 and fc2_out = 10": PCA_sparse4_3010,
    "PCA Local Compressed - fc1_out = 30 and fc2_out = 5": PCA_sparse4_305,
    "PCA Local Compressed - fc1_out = 10 and fc2_out = 5": PCA_sparse4_105,
}

plt.figure(figsize=(14, 10))

#plot predictions
min_val, max_val = float('inf'), -float('inf')

for model_name, model in PCA_sparse4_models.items():
    actuals = []
    predictions = []

    model.eval()

    with torch.no_grad():
        for inputs, targets in test_loader:
            outputs = model(inputs)

            # Store actual and predicted values for plotting
            actuals.extend(targets.cpu().numpy())
            predictions.extend(outputs.cpu().numpy())

    # Update min and max values for the perfect prediction line
    max_val = max(max_val, max(actuals), max(predictions))
    min_val = min(min_val, min(actuals), min(predictions))

    # Plot actuals vs predictions for this model
    plt.scatter(actuals, predictions, alpha=0.5, label=model_name)

# perfect prediction line
plt.plot([min_val, max_val], [min_val, max_val], 'k--', lw=2,
         label='Perfect Prediction')

plt.title('Actual vs Predicted - All Models')
plt.xlabel('Actual Target')
plt.ylabel('Predicted Target')
plt.tight_layout(rect=[0, 0.1, 1, 1])
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05), fancybox=True,
           shadow=True, ncol=3)
plt.grid(True)
plt.show()

In [ ]:
# listing local models that have 0.9 pruning amount
SparsePCA_sparse4_models = {
    "Original": model,
    "Smaller Network - fc1_out = 50 and fc2_out = 32": small_5032,
    "Smaller Network - fc1_out = 30 and fc2_out = 32": small_3032,
    "Smaller Network - fc1_out = 10 and fc2_out = 32": small_1032,
    "SparsePCA Local Compressed - fc1_out = 50 and fc2_out = 32": SparsePCA_sparse4_50,
    "SparsePCA Local Compressed - fc1_out = 30 and fc2_out = 32": SparsePCA_sparse4_30,
    "SparsePCA Local Compressed - fc1_out = 10 and fc2_out = 32": SparsePCA_sparse4_10,
}

plt.figure(figsize=(14, 10))

#plot predictions
min_val, max_val = float('inf'), -float('inf')

for model_name, model in SparsePCA_sparse4_models.items():
    actuals = []
    predictions = []

    model.eval()

    with torch.no_grad():
        for inputs, targets in test_loader:
            outputs = model(inputs)

            # Store actual and predicted values for plotting
            actuals.extend(targets.cpu().numpy())
            predictions.extend(outputs.cpu().numpy())

    # Update min and max values for the perfect prediction line
    max_val = max(max_val, max(actuals), max(predictions))
    min_val = min(min_val, min(actuals), min(predictions))

    # Plot actuals vs predictions for this model
    plt.scatter(actuals, predictions, alpha=0.5, label=model_name)

# perfect prediction line
plt.plot([min_val, max_val], [min_val, max_val], 'k--', lw=2,
         label='Perfect Prediction')

plt.title('Actual vs Predicted - All Models')
plt.xlabel('Actual Target')
plt.ylabel('Predicted Target')
plt.tight_layout(rect=[0, 0.1, 1, 1])
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05), fancybox=True,
           shadow=True, ncol=3)
plt.grid(True)
plt.show()

In [ ]:
# listing local models that have 0.9 pruning amount
ICA_sparse4_models = {
    "Original": model,
    "Smaller Network - fc1_out = 50 and fc2_out = 32": small_5032,
    "Smaller Network - fc1_out = 30 and fc2_out = 32": small_3032,
    "Smaller Network - fc1_out = 10 and fc2_out = 32": small_1032,
    "ICA Local Model - fc1_out = 50 and fc2_out = 32": ICA_sparse4_50,
    "ICA Local Compressed - fc1_out = 30 and fc2_out = 32": ICA_sparse4_30,
    "ICA Local Compressed - fc1_out = 10 and fc2_out = 32": ICA_sparse4_10,

}

plt.figure(figsize=(14, 10))

#plot predictions
min_val, max_val = float('inf'), -float('inf')

for model_name, model in ICA_sparse4_models.items():
    actuals = []
    predictions = []

    model.eval()

    with torch.no_grad():
        for inputs, targets in test_loader:
            outputs = model(inputs)

            # Store actual and predicted values for plotting
            actuals.extend(targets.cpu().numpy())
            predictions.extend(outputs.cpu().numpy())

    # Update min and max values for the perfect prediction line
    max_val = max(max_val, max(actuals), max(predictions))
    min_val = min(min_val, min(actuals), min(predictions))

    # Plot actuals vs predictions for this model
    plt.scatter(actuals, predictions, alpha=0.5, label=model_name)

# perfect prediction line
plt.plot([min_val, max_val], [min_val, max_val], 'k--', lw=2,
         label='Perfect Prediction')

plt.title('Actual vs Predicted - All Models')
plt.xlabel('Actual Target')
plt.ylabel('Predicted Target')
plt.tight_layout(rect=[0, 0.1, 1, 1])
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05), fancybox=True,
           shadow=True, ncol=3)
plt.grid(True)
plt.show()

##### Pruning Amount = 0.95

In [ ]:
# listing local models that have 0.95 pruning amount
PCA_sparse5_models = {
    "Original": model,
    "Smaller Network - fc1_out = 50 and fc2_out = 30": small_5030,
    "Smaller Network - fc1_out = 50 and fc2_out = 10": small_5010,
    "Smaller Network - fc1_out = 50 and fc2_out = 5": small_505,
    "Smaller Network - fc1_out = 30 and fc2_out = 10": small_3010,
    "Smaller Network - fc1_out = 30 and fc2_out = 5": small_305,
    "Smaller Network - fc1_out = 10 and fc2_out = 5": small_105,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 30": PCA_sparse5_5030,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 30": PCA_sparse5_5030,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 10": PCA_sparse5_5010,
    "PCA Local Compressed - fc1_out = 50 and fc2_out = 5": PCA_sparse5_505,
    "PCA Local Compressed - fc1_out = 30 and fc2_out = 10": PCA_sparse5_3010,
    "PCA Local Compressed - fc1_out = 30 and fc2_out = 5": PCA_sparse5_305,
    "PCA Local Compressed - fc1_out = 10 and fc2_out = 5": PCA_sparse5_105,
}

plt.figure(figsize=(14, 10))

#plot predictions
min_val, max_val = float('inf'), -float('inf')

for model_name, model in PCA_sparse5_models.items():
    actuals = []
    predictions = []

    model.eval()

    with torch.no_grad():
        for inputs, targets in test_loader:
            outputs = model(inputs)

            # Store actual and predicted values for plotting
            actuals.extend(targets.cpu().numpy())
            predictions.extend(outputs.cpu().numpy())

    # Update min and max values for the perfect prediction line
    max_val = max(max_val, max(actuals), max(predictions))
    min_val = min(min_val, min(actuals), min(predictions))

    # Plot actuals vs predictions for this model
    plt.scatter(actuals, predictions, alpha=0.5, label=model_name)

# perfect prediction line
plt.plot([min_val, max_val], [min_val, max_val], 'k--', lw=2,
         label='Perfect Prediction')

plt.title('Actual vs Predicted - All Models')
plt.xlabel('Actual Target')
plt.ylabel('Predicted Target')
plt.tight_layout(rect=[0, 0.1, 1, 1])
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05), fancybox=True,
           shadow=True, ncol=3)
plt.grid(True)
plt.show()

In [ ]:
# listing local models that have 0.95 pruning amount
SparsePCA_l5_models = {
    "Original": model,
    "Smaller Network - fc1_out = 50 and fc2_out = 32": small_5032,
    "Smaller Network - fc1_out = 30 and fc2_out = 32": small_3032,
    "Smaller Network - fc1_out = 10 and fc2_out = 32": small_1032,
    "SparsePCA Local Compressed - fc1_out = 50 and fc2_out = 32": SparsePCA_l5_50,
    "SparsePCA Local Compressed - fc1_out = 30 and fc2_out = 32": SparsePCA_l5_30,
    "SparsePCA Local Compressed - fc1_out = 10 and fc2_out = 32": SparsePCA_l5_10,
}

plt.figure(figsize=(14, 10))

#plot predictions
min_val, max_val = float('inf'), -float('inf')

for model_name, model in SparsePCA_l5_models.items():
    actuals = []
    predictions = []

    model.eval()

    with torch.no_grad():
        for inputs, targets in test_loader:
            outputs = model(inputs)

            # Store actual and predicted values for plotting
            actuals.extend(targets.cpu().numpy())
            predictions.extend(outputs.cpu().numpy())

    # Update min and max values for the perfect prediction line
    max_val = max(max_val, max(actuals), max(predictions))
    min_val = min(min_val, min(actuals), min(predictions))

    # Plot actuals vs predictions for this model
    plt.scatter(actuals, predictions, alpha=0.5, label=model_name)

# perfect prediction line
plt.plot([min_val, max_val], [min_val, max_val], 'k--', lw=2,
         label='Perfect Prediction')

plt.title('Actual vs Predicted - All Models')
plt.xlabel('Actual Target')
plt.ylabel('Predicted Target')
plt.tight_layout(rect=[0, 0.1, 1, 1])
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05), fancybox=True,
           shadow=True, ncol=3)
plt.grid(True)
plt.show()

In [ ]:
# listing local models that have 0.95 pruning amount
ICA_sparse5_models = {
    "Original": model,
    "Smaller Network - fc1_out = 50 and fc2_out = 32": small_5032,
    "Smaller Network - fc1_out = 30 and fc2_out = 32": small_3032,
    "Smaller Network - fc1_out = 10 and fc2_out = 32": small_1032,
    "ICA Local Model - fc1_out = 50 and fc2_out = 32": ICA_sparse5_50,
    "ICA Local Compressed - fc1_out = 30 and fc2_out = 32": ICA_sparse5_30,
    "ICA Local Compressed - fc1_out = 10 and fc2_out = 32": ICA_sparse5_10,

}

plt.figure(figsize=(14, 10))

#plot predictions
min_val, max_val = float('inf'), -float('inf')

for model_name, model in ICA_sparse5_models.items():
    actuals = []
    predictions = []

    model.eval()

    with torch.no_grad():
        for inputs, targets in test_loader:
            outputs = model(inputs)

            # Store actual and predicted values for plotting
            actuals.extend(targets.cpu().numpy())
            predictions.extend(outputs.cpu().numpy())

    # Update min and max values for the perfect prediction line
    max_val = max(max_val, max(actuals), max(predictions))
    min_val = min(min_val, min(actuals), min(predictions))

    # Plot actuals vs predictions for this model
    plt.scatter(actuals, predictions, alpha=0.5, label=model_name)

# perfect prediction line
plt.plot([min_val, max_val], [min_val, max_val], 'k--', lw=2,
         label='Perfect Prediction')

plt.title('Actual vs Predicted - All Models')
plt.xlabel('Actual Target')
plt.ylabel('Predicted Target')
plt.tight_layout(rect=[0, 0.1, 1, 1])
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05), fancybox=True,
           shadow=True, ncol=3)
plt.grid(True)
plt.show()